# <h1><center> New Product Simulator <br>
<center>To measure the impact of introducing new product using the attributes of existing product

In [1]:
# ======================================= CODE HEADER ==============================================================
# CODE DESCRIPTION: Get the New Product sales by changing attributes of existing products
# CONTACT: Mu Sigma Inc. 3400 Dundee Rd, Suite 160 Northbrook, IL – 60062 | www.mu-sigma.com
# LAST MODIFIED DATE: 06-12-2021

# ======================================= INSTRUCTIONS  ==============================================================
# 1. Run below 3 cells to get "Import Data" widget
# 2. Click on "Import Data" widget to import data
# 3. Select required features/filters to be changed using widgets provided in this tool & get the New Products Simulated results

In [2]:
# setting the notebook output to be in the center and creating loading symbol

from IPython.display import HTML, display, Markdown, clear_output
display(HTML("""
<style>
.output_png img {
    display: block;
    margin-left: auto;
    margin-right: auto;
}
 
.loader {
  border: 5px solid #f3f3f3;
  border-radius: 50%;
  border-top: 5px solid teal;
  border-right: 5px solid grey;
  border-bottom: 5px solid maroon;
  border-left: 5px solid tan;
  width: 20px;
  height: 20px;
  -webkit-animation: spin 1s linear infinite;
  animation: spin 1s linear infinite;
  float: left;
}

@-webkit-keyframes spin {
  0% { -webkit-transform: rotate(0deg); }
  100% { -webkit-transform: rotate(360deg); }
}

@keyframes spin {
  0% { transform: rotate(0deg); }
  100% { transform: rotate(360deg); }
}

</style>
"""))

In [3]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 2px  black solid !important;
  color: black !important;
}
</style>

In [217]:
display(Markdown('<div><div class="loader"></div><h2> &nbsp; Importing libraries</h2></div>'))

# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import datetime as datetime
import matplotlib
import matplotlib.ticker as mtick
import io
import joblib

# Importing widgets
try:
  import ipywidgets as widgets
  from ipywidgets import interactive, Layout, Button, Box,VBox,interactive_output
except:
  !pip install ipywidgets
  import ipywidgets as widgets
  from ipywidgets import interactive, Layout, Button, Box,VBox

# Importing IPython
try:
  from IPython.display import Javascript, display
except:
  !pip install ipython
  from IPython.display import Javascript, display, HTML

# Importing sklearn
try:
  from sklearn.ensemble import RandomForestRegressor
  from sklearn.preprocessing import MinMaxScaler
except:
  !pip install -U scikit-learn
  from sklearn.ensemble import RandomForestRegressor
  from sklearn.preprocessing import MinMaxScaler

import random
from termcolor import colored
from collections import Counter

# Importing widgets
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import Javascript, display
def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index(), IPython.notebook.ncells()-32)'))

# Converting into year and week number column
from datetime import datetime
def year_week(y, w):
    return datetime.strptime(f'{y} {w} 7', '%G %V %u')

clear_output()
    
# Import data widget    
button = widgets.Button(description="Import Data")
button.on_click(run_all)
display(button)

In [223]:
# Hide code cells
from IPython.core.display import display, HTML
display(HTML(r"""<style id=hide>div.input{display:none;}</style><button type="button"onclick="var myStyle = document.getElementById('hide').sheet;myStyle.insertRule('div.input{display:inherit !important;}', 0);">Show/hide inputs</button>"""))

In [6]:
display(Markdown('<div><div class="loader"></div><h2> &nbsp; Importing Dataset</h2></div>'))

import warnings
warnings.filterwarnings('ignore')

#Reading the required files | Files processed in "Seasonality Index/seasonality_index_calculation.R" deliverable file
seasonality_table = pd.read_csv("Seasonality_Index_data.csv")
#Reading the required files | Files from "Data Treatment/UPC Data Treatment/UPC_Data_Treatment.R" deliverable file
data_all_UPC = pd.read_csv("Treated_data/Treated_data_UPC.csv") 
#Reading the required files | Files from "Data Treatment/Banner Product Data Treatment/POS_ADCAL_SPC_DataTreatment.R" deliverable file
data_all = pd.read_csv('Treated_data/Treated_data_BP.csv')
#Reading the required files | Files from Coca Cola (present under "New Product Simulator" folder)
covid_stages = pd.read_csv("Covid_Stages_data.csv") 
new_product = pd.read_excel("Innovation_Product_Mapping.xlsx")
product_feat = pd.read_csv("Product_features_data.csv")

# Reading input files | Files processed in Jupyterhub for new product stages (present under "New Product Simulator" folder)
new_prod_stage = pd.read_csv("New_Product_cat_stage.csv")

# Removing EVIAN Products as partnership with EVIAN product is over as informed by the client
data_all = data_all.loc[~data_all['Product'].str.contains('EVIAN')]

# Merging datasets to get product features
data_all = pd.merge(data_all,product_feat,how = 'left',left_on = 'Product' , right_on = 'TPO_PRODUCT_GROUP')

# Grouping UPC data at Product level
data_all_UPC_grouped = data_all_UPC.groupby(['Region','Channel','Store.Chain','Banner','Product','Week']).agg({'UPC.PRODUCT':'nunique','UPC_FLAVOUR':'nunique','UPC_SWEETNER':'nunique','UPC_TYPE':'nunique'}).reset_index()

# Changing column names
data_all_UPC_grouped.rename(columns={'UPC.PRODUCT':'No_of_brands','UPC_FLAVOUR':'No_of_flavors','UPC_SWEETNER':'No_of_sweetners','UPC_TYPE':'No_of_types'}, inplace=True)

# Getting Brand, Flavor, Sweetner, Type information in Product level data
data_all = pd.merge(data_all, data_all_UPC_grouped, how='inner', on=['Region','Channel','Store.Chain','Banner','Product','Week'])

# Files present in FTP server under "New Product Simulator/Processed_data" 
# Training data files
combined_dataset_adcal_temp_EAST = pd.read_csv('Processed_data/combined_dataset_adcal_EAST.csv')
combined_dataset_edv_temp_EAST = pd.read_csv('Processed_data/combined_dataset_edv_EAST.csv')
combined_dataset_other_promo_temp_EAST = pd.read_csv('Processed_data/combined_dataset_other_promo_EAST.csv')
combined_dataset_self_promo_temp_EAST = pd.read_csv('Processed_data/combined_dataset_self_promo_EAST.csv')
combined_dataset_adcal_temp_ONTARIO = pd.read_csv('Processed_data/combined_dataset_adcal_ONTARIO.csv')
combined_dataset_edv_temp_ONTARIO = pd.read_csv('Processed_data/combined_dataset_edv_ONTARIO.csv')
combined_dataset_other_promo_temp_ONTARIO = pd.read_csv('Processed_data/combined_dataset_other_promo_ONTARIO.csv')
combined_dataset_self_promo_temp_ONTARIO = pd.read_csv('Processed_data/combined_dataset_self_promo_ONTARIO.csv')
combined_dataset_adcal_temp_QUEBEC = pd.read_csv('Processed_data/combined_dataset_adcal_QUEBEC.csv')
combined_dataset_edv_temp_QUEBEC = pd.read_csv('Processed_data/combined_dataset_edv_QUEBEC.csv')
combined_dataset_other_promo_temp_QUEBEC = pd.read_csv('Processed_data/combined_dataset_other_promo_QUEBEC.csv')
combined_dataset_self_promo_temp_QUEBEC = pd.read_csv('Processed_data/combined_dataset_self_promo_QUEBEC.csv')
combined_dataset_adcal_temp_WEST = pd.read_csv('Processed_data/combined_dataset_adcal_WEST.csv')
combined_dataset_edv_temp_WEST = pd.read_csv('Processed_data/combined_dataset_edv_WEST.csv')
combined_dataset_other_promo_temp_WEST = pd.read_csv('Processed_data/combined_dataset_other_promo_WEST.csv')
combined_dataset_self_promo_temp_WEST = pd.read_csv('Processed_data/combined_dataset_self_promo_WEST.csv')

# Files present in FTP server under "New Product Simulator/Tool_Model_Objects_Net_Gain" 
model_object_EAST_2019Q1 = joblib.load('Tool_Model_Objects_Net_Gain/RF_EAST_2019Q1.pkl')
model_object_EAST_2019Q2 = joblib.load('Tool_Model_Objects_Net_Gain/RF_EAST_2019Q2.pkl')
model_object_EAST_2019Q3 = joblib.load('Tool_Model_Objects_Net_Gain/RF_EAST_2019Q3.pkl')
model_object_EAST_2019Q4 = joblib.load('Tool_Model_Objects_Net_Gain/RF_EAST_2019Q4.pkl')
model_object_ONTARIO_2019Q1 = joblib.load('Tool_Model_Objects_Net_Gain/RF_ONTARIO_2019Q1.pkl')
model_object_ONTARIO_2019Q2 = joblib.load('Tool_Model_Objects_Net_Gain/RF_ONTARIO_2019Q2.pkl')
model_object_ONTARIO_2019Q3 = joblib.load('Tool_Model_Objects_Net_Gain/RF_ONTARIO_2019Q3.pkl')
model_object_ONTARIO_2019Q4 = joblib.load('Tool_Model_Objects_Net_Gain/RF_ONTARIO_2019Q4.pkl')
model_object_QUEBEC_2019Q1 = joblib.load('Tool_Model_Objects_Net_Gain/RF_QUEBEC_2019Q1.pkl')
model_object_QUEBEC_2019Q2 = joblib.load('Tool_Model_Objects_Net_Gain/RF_QUEBEC_2019Q2.pkl')
model_object_QUEBEC_2019Q3 = joblib.load('Tool_Model_Objects_Net_Gain/RF_QUEBEC_2019Q3.pkl')
model_object_QUEBEC_2019Q4 = joblib.load('Tool_Model_Objects_Net_Gain/RF_QUEBEC_2019Q4.pkl')
model_object_WEST_2019Q1 = joblib.load('Tool_Model_Objects_Net_Gain/RF_WEST_2019Q1.pkl')
model_object_WEST_2019Q2 = joblib.load('Tool_Model_Objects_Net_Gain/RF_WEST_2019Q2.pkl')
model_object_WEST_2019Q3 = joblib.load('Tool_Model_Objects_Net_Gain/RF_WEST_2019Q3.pkl')
model_object_WEST_2019Q4 = joblib.load('Tool_Model_Objects_Net_Gain/RF_WEST_2019Q4.pkl')
# Files present in FTP server under "New Product Simulator/Tool_Model_Objects_Net_Gain" 
model_object_EAST_2019Q1_v2 = joblib.load('Tool_Model_Objects/RF_EAST_2019Q1_v2.pkl')
model_object_EAST_2019Q2_v2 = joblib.load('Tool_Model_Objects/RF_EAST_2019Q2_v2.pkl')
model_object_EAST_2019Q3_v2 = joblib.load('Tool_Model_Objects/RF_EAST_2019Q3_v2.pkl')
model_object_EAST_2019Q4_v2 = joblib.load('Tool_Model_Objects/RF_EAST_2019Q4_v2.pkl')
model_object_ONTARIO_2019Q1_v2 = joblib.load('Tool_Model_Objects/RF_ONTARIO_2019Q1_v2.pkl')
model_object_ONTARIO_2019Q2_v2 = joblib.load('Tool_Model_Objects/RF_ONTARIO_2019Q2_v2.pkl')
model_object_ONTARIO_2019Q3_v2 = joblib.load('Tool_Model_Objects/RF_ONTARIO_2019Q3_v2.pkl')
model_object_ONTARIO_2019Q4_v2 = joblib.load('Tool_Model_Objects/RF_ONTARIO_2019Q4_v2.pkl')
model_object_QUEBEC_2019Q1_v2 = joblib.load('Tool_Model_Objects/RF_QUEBEC_2019Q1_v2.pkl')
model_object_QUEBEC_2019Q2_v2 = joblib.load('Tool_Model_Objects/RF_QUEBEC_2019Q2_v2.pkl')
model_object_QUEBEC_2019Q3_v2 = joblib.load('Tool_Model_Objects/RF_QUEBEC_2019Q3_v2.pkl')
model_object_QUEBEC_2019Q4_v2 = joblib.load('Tool_Model_Objects/RF_QUEBEC_2019Q4_v2.pkl')
model_object_WEST_2019Q1_v2 = joblib.load('Tool_Model_Objects/RF_WEST_2019Q1_v2.pkl')
model_object_WEST_2019Q2_v2 = joblib.load('Tool_Model_Objects/RF_WEST_2019Q2_v2.pkl')
model_object_WEST_2019Q3_v2 = joblib.load('Tool_Model_Objects/RF_WEST_2019Q3_v2.pkl')
model_object_WEST_2019Q4_v2 = joblib.load('Tool_Model_Objects/RF_WEST_2019Q4_v2.pkl')

clear_output()

__Choose New Product Attributes__

In [199]:
display(Markdown('<div><div class="loader"></div><h2> &nbsp; Loading Filters</h2></div>'))

# Defining variables for prediction
Region_List = ['ONTARIO','QUEBEC','EAST','WEST']
Max_POS = "2019-52" # Restricting the analysis for 2019 year
holidayflag = "Yes"

# Updating the dependent column
data_all["Eq.Unit.Sales"] = data_all["Std.Unit.Sales"]

# Selecting variables for training
if holidayflag == "Yes":
    ## Revenue Change    
    cols = ["Dollar.Sales","Region","Week.ID","Week","Year","brand","Banner","Product","Channel","Store.Chain","Category","Pack.Subtype","Adcal_Price","EDV.Price","Adcal_DD","Week.Name","Eq.Unit.Sales","Pantry1","Pantry2","Pantry3","Pre.Holiday.Week","Holiday.Week","Post.Holiday.Week","Front.Page", "Middle.Page","Back.Page","Brand",'PACK_CONTENT','SIZE_ML','COUNT','No_of_brands','No_of_flavors','No_of_sweetners','No_of_types']
    Volume_dataset_all_reg = data_all[cols]

# Getting seasonality and covid information
Volume_dataset_all_reg = Volume_dataset_all_reg.merge(seasonality_table)
Volume_dataset_all_reg = Volume_dataset_all_reg.merge(covid_stages, how="left")

# Replacing nulls in Corona stages
Volume_dataset_all_reg.loc[Volume_dataset_all_reg["Corona_Stages"].isnull(),"Corona_Stages"] = "No Stage"
Volume_dataset_all_reg = Volume_dataset_all_reg.loc[Volume_dataset_all_reg["Adcal_Price"] > 0]

# Defining model train and test periods

Test_week_list = {"2019Q1" : ["2019-01","2019-02","2019-03","2019-04","2019-05","2019-06","2019-07","2019-08","2019-09","2019-10","2019-11","2019-12","2019-13"],
     "2019Q2" : ["2019-14","2019-15","2019-16","2019-17","2019-18","2019-19","2019-20","2019-21","2019-22","2019-23","2019-24","2019-25","2019-26"],
     "2019Q3" : ["2019-27","2019-28","2019-29","2019-30","2019-31","2019-32","2019-33","2019-34","2019-35","2019-36","2019-37","2019-38","2019-39"],
     "2019Q4" : ["2019-40","2019-41","2019-42","2019-43","2019-44","2019-45","2019-46","2019-47","2019-48","2019-49","2019-50","2019-51","2019-52"],
                  
     "2020Q1" : ["2020-01","2020-02","2020-03","2020-04","2020-05","2020-06","2020-07","2020-08","2020-09","2020-10","2020-11","2020-12","2020-13"],
     "2020Q2" : ["2020-14","2020-15","2020-16","2020-17","2020-18","2020-19","2020-20","2020-21","2020-22","2020-23","2020-24","2020-25","2020-26"],
     "2020Q3" : ["2020-27","2020-28","2020-29","2020-30","2020-31","2020-32","2020-33","2020-34","2020-35","2020-36","2020-37","2020-38","2020-39"],
     "2020Q4" : ["2020-40","2020-41","2020-42","2020-43","2020-44","2020-45","2020-46","2020-47","2020-48","2020-49","2020-50","2020-51","2020-52"],
     "fulltrain" : []}

Test_periods = ["2019Q4","2019Q1","2019Q2","2019Q3","2020Q4","2020Q1","2020Q2","2020Q3"]


# Grouping category
Volume_dataset_all_reg["Category"] = ["ALTERNATIVE BEVERAGES" if ((x=="DROPS") or (x=="ENERGY") or (x=="ISOTONICS") or (x=="NON CARBONATED BEVERAGES"))
                                 else x for x in Volume_dataset_all_reg["Category"]]

# Adding Test period column
Volume_dataset_all_reg["Test_period"] = ["2019Q1" if ((x >= '2019-01') & (x <= '2019-13')) else "2019Q2" if ((x >= '2019-14') & (x <= '2019-26')) else "2019Q3" if ((x >= '2019-27') & (x <= '2019-39')) else "2019Q4" if ((x >= '2019-40') & (x <= '2019-52')) else 
                                        "2020Q1" if ((x >= '2020-01') & (x <= '2020-13')) else "2020Q2" if ((x >= '2020-14') & (x <= '2020-26')) else "2020Q3" if ((x >= '2020-27') & (x <= '2020-39')) else "2020Q4" if ((x >= '2020-40') & (x <= '2020-52')) else
                                        "2017Q1" if ((x >= '2017-01') & (x <= '2017-13')) else "2017Q2" if ((x >= '2017-14') & (x <= '2017-26')) else "2017Q3" if ((x >= '2017-27') & (x <= '2017-39')) else "2017Q4" if ((x >= '2017-40') & (x <= '2017-52')) else 
                                        "2018Q1" if ((x >= '2018-01') & (x <= '2018-13')) else "2018Q2" if ((x >= '2018-14') & (x <= '2018-26')) else "2018Q3" if ((x >= '2018-27') & (x <= '2018-39')) else "2018Q4" if ((x >= '2018-40') & (x <= '2018-52')) else 
                                        "2021Q1" if ((x >= '2021-01') & (x <= '2021-13')) else "2021Q2" if ((x >= '2021-14') & (x <= '2021-26')) else "2021Q3" if ((x >= '2021-27') & (x <= '2021-39')) else "2021Q4" if ((x >= '2021-40') & (x <= '2021-52')) else x 
                                        for x in Volume_dataset_all_reg["Week"]]

# CORE POWER 414 ML BTTL has duplicate row entries
Volume_dataset_all_reg_11 = Volume_dataset_all_reg[Volume_dataset_all_reg['Product']=='TCCC CORE POWER 414 ML BTTL'].drop_duplicates().copy()
Volume_dataset_all_reg = Volume_dataset_all_reg[Volume_dataset_all_reg['Product']!='TCCC CORE POWER 414 ML BTTL']
Volume_dataset_all_reg = Volume_dataset_all_reg.append(Volume_dataset_all_reg_11, ignore_index = True)

################# NEW PRODUCT 1 #################

# Filters
layout1 = widgets.Layout(width='280px', height='auto')
layout3 = widgets.Layout(width='240px', height='auto')
layout2 = widgets.Layout(width='106px', height='auto')
layout_auto = widgets.Layout(width='auto', height='auto')

# Dropdown widgets for choosing new product features
region = widgets.Dropdown(options=list(np.sort(Volume_dataset_all_reg.Region.unique())) + ['All'], value='ONTARIO', description='Region', layout = layout3)
channel = widgets.Dropdown(options=list(np.sort(Volume_dataset_all_reg.Channel.unique())) + ['All'], value='All', description='Channel', layout = layout3)
size = widgets.BoundedFloatText(value=340, min = 50, max = 2630, description='New Size(ml)',disabled=False,layout = layout3) # Max & Min values are taken as per existing product data
count = widgets.BoundedFloatText(value=8, min = 1, max = 32, description='New Count',disabled=False,layout = layout3)  # Max & Min values are taken as per existing product data
size2 = widgets.BoundedFloatText(value=340, min = 50, max = 2630, description='New Size(ml)',disabled=False,layout = layout3)  # Max & Min values are taken as per existing product data
count2 = widgets.BoundedFloatText(value=8, min = 1, max = 32, description='New Count',disabled=False,layout = layout3)  # Max & Min values are taken as per existing product data

# New Category dropdown
category_drop = widgets.Dropdown(options=list(np.sort(Volume_dataset_all_reg.Category.unique())), value='CARBONATED SOFT DRINKS',layout = layout_auto)
category_drop2 = widgets.Dropdown(options=list(np.sort(Volume_dataset_all_reg.Category.unique())), value='CARBONATED SOFT DRINKS',layout = layout_auto)

# New pack subtype dropdown
pack_subtype_drop = widgets.Dropdown(options=list(np.sort(Volume_dataset_all_reg['Pack.Subtype'].unique())),layout = layout_auto)
pack_subtype_drop2 = widgets.Dropdown(options=list(np.sort(Volume_dataset_all_reg['Pack.Subtype'].unique())),layout = layout_auto)

# New pack content dropdown
pack_content_drop = widgets.Dropdown(options=list(np.sort(Volume_dataset_all_reg['PACK_CONTENT'].unique())),layout = layout_auto)
pack_content_drop2 = widgets.Dropdown(options=list(np.sort(Volume_dataset_all_reg['PACK_CONTENT'].unique())),layout = layout_auto)

# Reactive filter for product
def product_filter(Region_filter,Channel_filter):
    if Channel_filter == 'All':
        channel_list_prod = Volume_dataset_all_reg.Channel.unique()
    else:
        channel_list_prod = [Channel_filter]

    if Region_filter == 'All':
        Region_List_prod = Volume_dataset_all_reg.Region.unique()
    else:
        Region_List_prod = [Region_filter] 

    # Filtering for chosen region-channel
    try:         
        product = widgets.Dropdown(options = list(np.sort(Volume_dataset_all_reg[(Volume_dataset_all_reg.Region.isin(Region_List_prod)) & (Volume_dataset_all_reg.Channel.isin(channel_list_prod))].Product.unique())))
        display(product)
        return product
    
    except:
        print('Product not found in this Region-Channel')

reg_chan_prod = interactive(product_filter, Region_filter=region, Channel_filter = channel)
reg_chan_prod.update()

# Quarter filters
start_week = widgets.HBox([widgets.Label(value=" "), widgets.Label(value=" "),widgets.Label(value=" "),widgets.Label(value=" "),widgets.Label(value=" "),widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value="Start Quarter"),widgets.Dropdown(options=['2019Q1','2019Q2','2019Q3','2019Q4'],value='2019Q1',layout = layout2)])
end_week = widgets.HBox([widgets.Label(value=" "), widgets.Label(value=" "),widgets.Label(value=" "),widgets.Label(value=" "),widgets.Label(value=" "),widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value="End Quarter"),widgets.Dropdown(options=['2019Q1','2019Q2','2019Q3','2019Q4'],value='2019Q4', layout = layout2)])

# Display filters
# Bordering filters
clear_output()
box_layout = Layout(display='flex',flex_flow='rows',align_items='stretch',border='solid 2px',width='auto')

# Ensemble or normal view
toggle_ensemble_normal = widgets.ToggleButtons(options=['Multiple Product','Single Product'],value='Single Product',description='Do you want to use single product or multiple product as existing model pack?',style= {'description_width': 'initial'})
out = widgets.Output(layout=widgets.Layout(border = 'solid 2px'))

# Refresh widget
def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells()-32)'))
layout_sim = widgets.Layout(width='auto', height='40px')
button_sim = widgets.Button(description="Refresh manual filtering/ similarity score filtering", layout = layout_sim)
button_sim.on_click(run_all)

# Widget for single, multiple product as existing product
def fun(obj):
    if toggle_ensemble_normal.value == 'Single Product':          
        clear_output()
        display(toggle_ensemble_normal)
        display(VBox([widgets.HBox((reg_chan_prod.children[0], reg_chan_prod.children[1],widgets.HBox((widgets.Label(value = ' '), widgets.Label(value = ' '), widgets.Label(value = ' '),widgets.Label(value = ' '),widgets.Label(value = 'Model Pack'), reg_chan_prod.children[-1])))), widgets.Label(value="$New Product 1$"),widgets.HBox((widgets.Label(value="New Category"), category_drop,widgets.Label(value="New Pack Subtype"), pack_subtype_drop,widgets.Label(value="New Pack Content"), pack_content_drop)),widgets.HBox((count, size,start_week,end_week))], layout=box_layout))
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells()-32)'))
    else:
        clear_output()
        display(toggle_ensemble_normal)
        display(VBox([widgets.HBox((reg_chan_prod.children[0], reg_chan_prod.children[1],widgets.HBox((widgets.Label(value = ' '), widgets.Label(value = ' '), widgets.Label(value = ' '),widgets.Label(value = ' '))))), widgets.Label(value="$New Product 1$"),widgets.HBox((widgets.Label(value="New Category"), category_drop,widgets.Label(value="New Pack Subtype"), pack_subtype_drop,widgets.Label(value="New Pack Content"), pack_content_drop)),widgets.HBox((count, size,start_week,end_week))], layout=box_layout))
        display(button_sim)
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells()-32)'))
        
display(toggle_ensemble_normal)
# Normal view
display(VBox([widgets.HBox((reg_chan_prod.children[0], reg_chan_prod.children[1],widgets.HBox((widgets.Label(value = ' '), widgets.Label(value = ' '), widgets.Label(value = ' '),widgets.Label(value = ' '),widgets.Label(value = 'Model Pack'), reg_chan_prod.children[-1])))), widgets.Label(value="$New Product 1$"),widgets.HBox((widgets.Label(value="New Category"), category_drop,widgets.Label(value="New Pack Subtype"), pack_subtype_drop,widgets.Label(value="New Pack Content"), pack_content_drop)),widgets.HBox((count, size,start_week,end_week))], layout=box_layout))
toggle_ensemble_normal.observe(fun, 'value')

In [200]:
# Storing filter output
region_fil = region.value
channel_fil = channel.value
product_fil = reg_chan_prod.result.value
size_fil = size.value
count_fil = count.value
size_fil2 = size2.value
count_fil2 = count2.value

# Initialising list for all/specific channel, region
if channel.value == 'All':
    channel_list = Volume_dataset_all_reg.Channel.unique()
else:
    channel_list = [channel.value]
    
if region.value == 'All':
    Region_List = Volume_dataset_all_reg.Region.unique()
else:
    Region_List = [region.value]   
    
# Run prices cell
def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index() + 1,IPython.notebook.ncells()-27)'))

layout = widgets.Layout(width='auto', height='40px')
button_check = widgets.Button(description="Get Existing Prices & Discounts", layout = layout)
button_check.on_click(run_all)

#######################################################################################################################    
                        # Model Pack selection for ensemble model
#######################################################################################################################    

def similarity_score_calculation():
    # Checking existing product region, channel & its presence in more than 1 channel    
    required_volume_dataset = Volume_dataset_all_reg[(Volume_dataset_all_reg.Region.isin(Region_List)) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value) & (Volume_dataset_all_reg.Category == category_drop.value)][["Product","Channel","Category","Pack.Subtype", 'SIZE_ML', 'COUNT', 'PACK_CONTENT']]
    required_volume_dataset_check = required_volume_dataset.groupby('Product')['Channel'].nunique().reset_index()
    if len(channel_list) != 1:
        existing_prod_list = required_volume_dataset_check[required_volume_dataset_check.Channel > 1].Product.unique()
    else:
        existing_prod_list = required_volume_dataset_check.Product.unique()

    # Existing data features
    data_existing_feature = required_volume_dataset[required_volume_dataset.Product.isin(existing_prod_list)][["Product","Category",
                              "Pack.Subtype", 'SIZE_ML', 'COUNT', 'PACK_CONTENT']].drop_duplicates().reset_index(drop=True)
    data_existing_feature['TOTAL_LITERS'] = data_existing_feature['SIZE_ML']*data_existing_feature['COUNT']

    # New Product Features
    data_existing_feature['New_SIZE_ML'] = size.value
    data_existing_feature['New_COUNT'] = count.value
    data_existing_feature['New_TOTAL_LITERS'] = size.value * count.value
    data_existing_feature['New_PACK_CONTENT'] = pack_content_drop.value
    data_existing_feature['New_Pack.Subtype'] = pack_subtype_drop.value
    data_existing_feature['New_Category'] = category_drop.value

    # Processing numerical scores
    data_existing_feature["size_dif"]=abs(data_existing_feature["New_SIZE_ML"]- data_existing_feature["SIZE_ML"])
    data_existing_feature["count_dif"]=abs(data_existing_feature["New_COUNT"]- data_existing_feature["COUNT"])
    data_existing_feature["total_liters_dif"]=abs(data_existing_feature["New_TOTAL_LITERS"]- data_existing_feature["TOTAL_LITERS"])

    data_existing_feature["size_score"]=1- ((data_existing_feature["size_dif"]-data_existing_feature["size_dif"].min())/
                               (( data_existing_feature["size_dif"].max()-data_existing_feature["size_dif"].min() )))
    data_existing_feature["count_score"]=1- ((data_existing_feature["count_dif"]-data_existing_feature["count_dif"].min())/
                               (( data_existing_feature["count_dif"].max()-data_existing_feature["count_dif"].min() )))
    data_existing_feature["total_liters_score"]=1- ((data_existing_feature["total_liters_dif"]-data_existing_feature["total_liters_dif"].min())/
                               (( data_existing_feature["total_liters_dif"].max()-data_existing_feature["total_liters_dif"].min() )))


    # Processing categorical scores
    data_existing_feature["Category_score"]= np.where(data_existing_feature["New_Category"]==data_existing_feature["Category"],1,0)
    data_existing_feature["Pack_content_score"]= np.where(data_existing_feature["New_PACK_CONTENT"]==data_existing_feature["PACK_CONTENT"],1,0)
    data_existing_feature["Pack_Subtype_score"]= np.where(data_existing_feature["New_Pack.Subtype"]==data_existing_feature["Pack.Subtype"],1,0)

    # Aggregation of Category_score,Pack_content_score,Pack_Subtype_score
    data_existing_feature["Categorical_Score"]= (data_existing_feature["Category_score"]+
                                                 data_existing_feature["Pack_content_score"]+
                                                 data_existing_feature["Pack_Subtype_score"])/3

    score_data=data_existing_feature[["Product","size_score","count_score","total_liters_score","Categorical_Score"]]                    

    score_data = score_data.fillna(0)
    score_data["Similarity_Score"]=(score_data["size_score"]+
                                score_data["count_score"]+  
                                score_data["total_liters_score"] +
                                score_data["Categorical_Score"])/4
    
    return score_data

# Ensemble: Manual or with similarity score calculation
if toggle_ensemble_normal.value == 'Multiple Product':           
    toggle_manual_simscore = widgets.ToggleButtons(options=['Manual','Similarity Score'],value='Similarity Score',description='Select model packs manually or using similarity scores',style= {'description_width': 'initial'})    
    
    # Existing model pack selected to be present in more than 1 channel
    required_volume_dataset = Volume_dataset_all_reg[(Volume_dataset_all_reg.Region.isin(Region_List)) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value) & (Volume_dataset_all_reg.Category == category_drop.value)][["Product","Channel","Category","Pack.Subtype", 'SIZE_ML', 'COUNT', 'PACK_CONTENT']]
    required_volume_dataset_check = required_volume_dataset.groupby('Product')['Channel'].nunique().reset_index()
    if len(channel_list) != 1:        
        existing_prod_list = required_volume_dataset_check[required_volume_dataset_check.Channel > 1].Product.unique()    
    else:
        existing_prod_list = required_volume_dataset_check.Product.unique()    
        
    prod_ensemble = widgets.SelectMultiple(options = list(np.sort(Volume_dataset_all_reg[(Volume_dataset_all_reg.Region.isin(Region_List)) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value) & (Volume_dataset_all_reg.Category == category_drop.value) & (Volume_dataset_all_reg.Product.isin(existing_prod_list))].Product.unique())))
    
# Manual/Similarity score changes functions    
def fun(obj):
    # Display manual model pack selection
    if toggle_manual_simscore.value == 'Manual':          
        clear_output()       
        score_data = similarity_score_calculation()
        display(toggle_manual_simscore)
        display(prod_ensemble)          
        print(colored('Exactly 3 products should be selected', 'red',attrs=['bold']))
        display(button_check)

    # Display similarity score model pack selection
    else:    
        clear_output()        
        score_data = similarity_score_calculation()
        display(toggle_manual_simscore)
        display(score_data.sort_values('Similarity_Score', ascending = False)[['Product','Similarity_Score']][:3]) 
        display(button_check)

if toggle_ensemble_normal.value == 'Multiple Product':        
    display(toggle_manual_simscore)
    score_data = similarity_score_calculation()
    display(score_data.sort_values('Similarity_Score', ascending = False)[['Product','Similarity_Score']][:3])
    toggle_manual_simscore.observe(fun, 'value')
                
# New Product 2 feature not available for ensemble model
if toggle_ensemble_normal.value != 'Multiple Product': 
    # Toggle for new product 2 attribute selection
    toggle_np2 = widgets.ToggleButtons(options=['Yes','No'],value='No',description='Do you want to change attributes for 2nd New Product?',style= {'description_width': 'initial'})

    out = widgets.Output(layout=widgets.Layout(border = 'solid 2px'))

    def fun(obj):
        if toggle_np2.value == 'Yes':          
            with out:        
                display(widgets.Label(value="$New Product 2$"),widgets.HBox((widgets.Label(value="New Category"), category_drop2,widgets.Label(value="New Pack Subtype"), pack_subtype_drop2,widgets.Label(value="New Pack Content"), pack_content_drop2)),widgets.HBox((count2, size2)))
        else:
            out.clear_output()

    display(toggle_np2)

    display(out)
    toggle_np2.observe(fun, 'value')   
    
    
# Run prices cell
def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,IPython.notebook.ncells()-27)'))

layout = widgets.Layout(width='auto', height='40px')
button = widgets.Button(description="Get Existing Prices & Discounts", layout = layout)
button.on_click(run_all)

display(button)

In [201]:
try:
    print(colored('The Category, Pack Subtype, Pack Content of selected Model Pack is', 'red',attrs=['bold']),colored(Volume_dataset_all_reg[Volume_dataset_all_reg.Product == reg_chan_prod.result.value].Category.unique()[0], 'red',attrs=['bold']),colored(',', 'red',attrs=['bold']),colored(Volume_dataset_all_reg[Volume_dataset_all_reg.Product == reg_chan_prod.result.value]['Pack.Subtype'].unique()[0], 'red',attrs=['bold']),colored('&', 'red',attrs=['bold']),colored(Volume_dataset_all_reg[Volume_dataset_all_reg.Product == reg_chan_prod.result.value].PACK_CONTENT.unique()[0], 'red',attrs=['bold']),colored('respectively', 'red',attrs=['bold']))
except:
    print('Product is not present in that Region/Channel for selected period')
    product_fil = []

    
# Storing filter output
region_fil = region.value
channel_fil = channel.value
product_fil = reg_chan_prod.result.value
size_fil = size.value
count_fil = count.value
size_fil2 = size2.value
count_fil2 = count2.value

# Initialising list for all/specific channel, region
if channel.value == 'All':
    channel_list = Volume_dataset_all_reg.Channel.unique()
else:
    channel_list = [channel.value]
    
if region.value == 'All':
    Region_List = Volume_dataset_all_reg.Region.unique()
else:
    Region_List = [region.value]   
    
# Ensemble list of products
if toggle_ensemble_normal.value == 'Multiple Product':
    if toggle_manual_simscore.value == 'Manual':    
        prod_ensemble_list = list(prod_ensemble.value)
    else:    
        prod_ensemble_list = list(score_data.sort_values('Similarity_Score', ascending = False)[['Product','Similarity_Score']][:3].Product.unique())

        
uploader = widgets.FileUpload(multiple=False, description = 'Upload Adcal Data of New Product', layout = widgets.Layout(width='300px'))

#### Change price cell run
def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,IPython.notebook.ncells()-15)'))
layout = widgets.Layout(width='auto', height='40px')
button_proceed = widgets.Button(description="Proceed further",layout = layout)
button_proceed.on_click(run_all)

# Toggle for new product uploading option
toggle_upload = widgets.ToggleButtons(options=['Yes','No'],value='No',description='Do you want to upload adcal data?',style= {'description_width': 'initial'})

out_upload = widgets.Output(layout=widgets.Layout(border = 'solid 2px'))

def fun2(obj):
    if (toggle_upload.value == 'Yes') & (toggle_ensemble_normal.value == 'Single Product'):          
        with out_upload:        
            display(uploader)
            display(button_proceed)

    else:
        out_upload.clear_output()

if toggle_ensemble_normal.value == 'Single Product':
    display(toggle_upload)

    display(out_upload)
    toggle_upload.observe(fun2, 'value')

# Function to find the list of cross products
def find_affecting_brands(brand_id):
        Product_type = brand_mapping.loc[brand_mapping["brand"]==brand_id,"Brand"]
        Intra_brands = brand_mapping.loc[brand_mapping['Banner'].isin(brand_mapping.loc[(brand_mapping['brand']==brand_id),"Banner"]),"brand"]
        brand_mapping_category = brand_mapping.copy()
        alt_beverage = brand_mapping_category.loc[(~brand_mapping_category["Category"].isin(["WATER","CARBONATED SOFT DRINKS"]) ) &
                       (brand_mapping_category["Brand"]=="TCCC"),"Category"].unique()
        brand_mapping_category['Category'] = ['ALTERNATIVE BEVERAGES' if x in alt_beverage else x for x in brand_mapping_category['Category']]
        pack = brand_mapping.loc[brand_mapping['Pack.Subtype'].isin(brand_mapping.loc[(brand_mapping['brand']==brand_id),"Pack.Subtype"]),"brand"]
        cat = brand_mapping.loc[brand_mapping['Category'].isin(brand_mapping.loc[(brand_mapping['brand']==brand_id),"Category"]),"brand"]
        packsubtype = pack[pack.isin(cat)]
        Affecting_brands = Intra_brands.append(packsubtype)
        Affecting_brands = list(set(Affecting_brands))
        mean_sales_10 = 0.1*Volume_dataset.loc[Volume_dataset["brand"]==brand_id,"Eq.Unit.Sales"].mean()
        Affecting_brands_vol = pd.DataFrame(Volume_dataset.loc[Volume_dataset["brand"].isin(Affecting_brands)].groupby("brand")["Eq.Unit.Sales"].mean()).reset_index()
        Affecting_brands = Affecting_brands_vol.loc[Affecting_brands_vol["Eq.Unit.Sales"]>mean_sales_10,"brand"]
        return Affecting_brands

# Changing number format
def human_format(num, precision=2, suffixes=['', 'K', 'M', 'G', 'T', 'P']):
    m = sum([abs(num/1000.0**x) >= 1 for x in range(1, len(suffixes))])
    return f'{num/1000.0**m:.{precision}f}{suffixes[m]}'

# Product stage function of new product
def prod_stage_check(Product):
    # Filtering for new products from data_all
    prod_initial = Volume_dataset_all_reg[Volume_dataset_all_reg.Product.isin(list([Product]))]
    # Sorting by product, week
    prod_initial = prod_initial[['Product','Week','Test_period']].drop_duplicates().sort_values(['Product','Week']) 
    # Year
    prod_initial['Year'] = prod_initial.Week.str.slice(0,4)
    # Week number
    prod_initial['Week_no'] = prod_initial.Week.str.slice(5,7)

    # Converting into year-week-days
    prod_initial['year_week_ts'] = prod_initial.apply(lambda row: year_week(row.Year, row.Week_no), axis=1)

    # Get updated start week
    if start_week.children[8].value < Volume_dataset_all_reg[Volume_dataset_all_reg.Product == product_fil].Test_period.min():    
        start_week_prod_stage = Volume_dataset_all_reg[Volume_dataset_all_reg.Product == product_fil].Week.min()
    else:
        start_week_prod_stage = Volume_dataset_all_reg[Volume_dataset_all_reg.Test_period == start_week.children[8].value].Week.min()

    # Difference of weeks with the first week for each product to calculate initial weeks    
    prod_initial = prod_initial[prod_initial.Week >= start_week_prod_stage]
    prod_initial['week_count'] = (prod_initial['year_week_ts'] - prod_initial.groupby('Product')['year_week_ts'].transform('first'))/7
    # Transforming days into integer
    prod_initial['week_count'] = prod_initial['week_count'].dt.days.astype('int')
    # Adding 1 to week_count column
    prod_initial['week_count'] = prod_initial.week_count + 1  
    # Renaming week_count to intial weeks
    prod_initial = prod_initial.rename(columns = {"week_count" : "Intial_weeks"})
    # Droppping unnecessary columns
    prod_initial = prod_initial.drop(['Year','Week_no','year_week_ts'], axis = 1)
    prod_initial = prod_initial.rename(columns = {"Intial_weeks":"Initial_weeks"})

    # Category of product 
    if i == 1:
        prod_initial['Category'] = category_drop.value
    elif i == 2:
        prod_initial['Category'] = category_drop2.value
    
    # For Alt-bev: Intro:1, Growth: 2-12 rows, Stablisation 13 onwards
    prod_initial.loc[(prod_initial.Category == 'ALTERNATIVE BEVERAGES') & (prod_initial.Initial_weeks == 1),'Intial_weeks'] = 'Intro'
    prod_initial.loc[(prod_initial.Category == 'ALTERNATIVE BEVERAGES') & ((prod_initial.Initial_weeks == 2) |(prod_initial.Initial_weeks == 3)| (prod_initial.Initial_weeks == 4) | (prod_initial.Initial_weeks == 5) | (prod_initial.Initial_weeks == 6) | (prod_initial.Initial_weeks == 7) | (prod_initial.Initial_weeks == 8) | (prod_initial.Initial_weeks == 9) | (prod_initial.Initial_weeks == 10) | (prod_initial.Initial_weeks == 11) | (prod_initial.Initial_weeks == 12)),'Intial_weeks'] = 'Growth'

    # For WATER: Intro:1, Growth: 2-8 rows, Stablisation 9 onwards
    prod_initial.loc[(prod_initial.Category == 'WATER') & (prod_initial.Initial_weeks == 1),'Intial_weeks'] = 'Intro'
    prod_initial.loc[(prod_initial.Category == 'WATER') & ((prod_initial.Initial_weeks == 2) |(prod_initial.Initial_weeks == 3)| (prod_initial.Initial_weeks == 4) | (prod_initial.Initial_weeks == 5) | (prod_initial.Initial_weeks == 6) | (prod_initial.Initial_weeks == 7) | (prod_initial.Initial_weeks == 8) ),'Intial_weeks'] = 'Growth'

    # For CSD # Intro:1, Growth: 2-6 rows, Stablisation 7 onwards
    prod_initial.loc[(prod_initial.Category == 'CARBONATED SOFT DRINKS') & (prod_initial.Initial_weeks == 1),'Intial_weeks'] = 'Intro'
    prod_initial.loc[(prod_initial.Category == 'CARBONATED SOFT DRINKS') & ((prod_initial.Initial_weeks == 2) |(prod_initial.Initial_weeks == 3)| (prod_initial.Initial_weeks == 4) | (prod_initial.Initial_weeks == 5) | (prod_initial.Initial_weeks == 6)),'Intial_weeks'] = 'Growth'

    # Fill na with stabilisation
    prod_initial['Intial_weeks'] = prod_initial['Intial_weeks'].fillna('Stabilization')
    prod_initial = prod_initial.rename(columns = {"Intial_weeks":"Initial_weeks1"})
    prod_initial = prod_initial.drop(['Initial_weeks', 'Category'], axis = 1)
    prod_initial = prod_initial[prod_initial.Week <= Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Test_period <= end_week.children[8].value, 'Week'].max()]
    
    return prod_initial


# Product stages for uplaoded data
def prod_stage_check_upload():
    # Filtering for new products from data_all
    prod_initial_upload = upload_data[['Product','Week']]
    # Sorting by product, week
    prod_initial_upload = prod_initial_upload.sort_values(['Product','Week']) 
    # Year
    prod_initial_upload['Year'] = prod_initial_upload.Week.str.slice(0,4)
    # Week number
    prod_initial_upload['Week_no'] = prod_initial_upload.Week.str.slice(5,7)

    # Converting into year-week-days
    prod_initial_upload['year_week_ts'] = prod_initial_upload.apply(lambda row: year_week(row.Year, row.Week_no), axis=1)

    # Difference of weeks with the first week for each product to calculate initial weeks    
    prod_initial_upload['week_count'] = (prod_initial_upload['year_week_ts'] - prod_initial_upload.groupby('Product')['year_week_ts'].transform('first'))/7
    # Transforming days into integer
    prod_initial_upload['week_count'] = prod_initial_upload['week_count'].dt.days.astype('int')
    # Adding 1 to week_count column
    prod_initial_upload['week_count'] = prod_initial_upload.week_count + 1  
    # Renaming week_count to intial weeks
    prod_initial_upload = prod_initial_upload.rename(columns = {"week_count" : "Intial_weeks"})
    # Droppping unnecessary columns
    prod_initial_upload = prod_initial_upload.drop(['Year','Week_no','year_week_ts'], axis = 1)
    prod_initial_upload = prod_initial_upload.rename(columns = {"Intial_weeks":"Initial_weeks"})
    prod_initial_upload = prod_initial_upload.drop_duplicates()

    # Category of product 
    prod_initial_upload ['Category'] = upload_data.Category.unique()[0]

    # For Alt-bev: Intro:1, Growth: 2-12 rows, Stablisation 13 onwards
    prod_initial_upload.loc[(prod_initial_upload.Category == 'ALTERNATIVE BEVERAGES') & (prod_initial_upload.Initial_weeks == 1),'Intial_weeks'] = 'Intro'
    prod_initial_upload.loc[(prod_initial_upload.Category == 'ALTERNATIVE BEVERAGES') & ((prod_initial_upload.Initial_weeks == 2) |(prod_initial_upload.Initial_weeks == 3)| (prod_initial_upload.Initial_weeks == 4) | (prod_initial_upload.Initial_weeks == 5) | (prod_initial_upload.Initial_weeks == 6) | (prod_initial_upload.Initial_weeks == 7) | (prod_initial_upload.Initial_weeks == 8) | (prod_initial_upload.Initial_weeks == 9) | (prod_initial_upload.Initial_weeks == 10) | (prod_initial_upload.Initial_weeks == 11) | (prod_initial_upload.Initial_weeks == 12)),'Intial_weeks'] = 'Growth'

    # For WATER: Intro:1, Growth: 2-8 rows, Stablisation 9 onwards
    prod_initial_upload.loc[(prod_initial_upload.Category == 'WATER') & (prod_initial_upload.Initial_weeks == 1),'Intial_weeks'] = 'Intro'
    prod_initial_upload.loc[(prod_initial_upload.Category == 'WATER') & ((prod_initial_upload.Initial_weeks == 2) |(prod_initial_upload.Initial_weeks == 3)| (prod_initial_upload.Initial_weeks == 4) | (prod_initial_upload.Initial_weeks == 5) | (prod_initial_upload.Initial_weeks == 6) | (prod_initial_upload.Initial_weeks == 7) | (prod_initial_upload.Initial_weeks == 8) ),'Intial_weeks'] = 'Growth'

    # For CSD # Intro:1, Growth: 2-6 rows, Stablisation 7 onwards
    prod_initial_upload.loc[(prod_initial_upload.Category == 'CARBONATED SOFT DRINKS') & (prod_initial_upload.Initial_weeks == 1),'Intial_weeks'] = 'Intro'
    prod_initial_upload.loc[(prod_initial_upload.Category == 'CARBONATED SOFT DRINKS') & ((prod_initial_upload.Initial_weeks == 2) |(prod_initial_upload.Initial_weeks == 3)| (prod_initial_upload.Initial_weeks == 4) | (prod_initial_upload.Initial_weeks == 5) | (prod_initial_upload.Initial_weeks == 6)),'Intial_weeks'] = 'Growth'

    # Fill na with stabilisation
    prod_initial_upload['Intial_weeks'] = prod_initial_upload['Intial_weeks'].fillna('Stabilization')
    prod_initial_upload = prod_initial_upload.drop(['Initial_weeks', 'Category'], axis = 1)
    return prod_initial_upload

# UDF for data treament for uploaded data #CHANGE 101: Integration of uplaoded data with tool
def upload_data_treat():
    data_mart = upload_data.copy()
    data_mart.dropna(inplace=True)
    # Files present in FTP server under "New Product Simulator"
    #Banner Channel dimension table    
    base_data_geo_dim = pd.read_csv('base_data_geo_dim.csv')
    #Merging Channel data to Adcal data
    data_mart = pd.merge(data_mart,base_data_geo_dim, how='inner', left_on='TPO_BANNER_GROUP', right_on='BN_BANNER_GROUP')
    data_mart.rename(columns={'BN_CHANNEL':'Channel'}, inplace=True)
    del data_mart['BN_BANNER_GROUP']

    # Files present in FTP server under "New Product Simulator"
    #Importing Week data
    week_data = pd.read_csv('Week_data_2022.csv')
    #Merging Week data with Adcal data for Week Name and Week Type columns
    data_mart = pd.merge(data_mart, week_data, how='inner', on=['REGION','TPO_WEEK'])

    #Some columns have spaces bewtween the value,removing the spaces
    data_mart['REGION'] = data_mart['REGION'].str.strip()
    data_mart['Channel'] = data_mart['Channel'].str.strip()
    data_mart['FEATURE'] = data_mart['FEATURE'].str.strip()
    data_mart['CAL_WEEK_NAME'] = data_mart['CAL_WEEK_NAME'].str.strip()
    data_mart['CAL_WEEK_TYPE'] = data_mart['CAL_WEEK_TYPE'].str.strip()

    #Renaming the required columns
    data_mart.rename(columns={'REGION':'Region', 'STORE_CHAIN':'Store.Chain','TPO_BANNER_GROUP':'Banner',
                              'TPO_PRODUCT_GROUP':'Product', 'CATEGORY':'Category', 'PACK_SUBTYPE':'Pack.Subtype',
                              'TPO_WEEK':'Week', 'CAL_WEEK_NAME':'Week.Name', 'CAL_WEEK_TYPE':'Week.Type',
                              'FEATURE':'Adcal_Page.Position', 'PRICE':'Adcal_Price'}, inplace=True)

    # Subsetting required columns
    data_mart = data_mart[['Region','Channel','Store.Chain','Banner','Product','Category','PACK_CONTENT','Pack.Subtype','COUNT','SIZE_ML','Week','Week.Name','Week.Type','WEEK','YEAR','QUARTER','Adcal_Page.Position','Adcal_Price']]

    # Dropping duplicates
    data_mart = data_mart.drop_duplicates()

    ### 2)Changing the banner names for Fresh Co, Maxi and Walmart(East and Quebec are changed to East/Quebec)
    #Banner group name same as banner name. Change banner group name to distinguish between the two
    data_mart['Banner'].replace({'FRESH CO ON':'FRESH CO/PRICE CHOPPER ON'}, inplace=True)
    data_mart['Banner'].replace({'MAXI':'MAXI/CIE'}, inplace=True)
    data_mart['Banner'].replace({'WALMART EAST':'WALMART EAST/QUEBEC'}, inplace=True)
    data_mart['Banner'].replace({'AFFILIATED SOBEYS QC':'AFF SOBEYS QC'}, inplace=True)
    data_mart['Banner'].replace({'RCLS':'LCL LIQUOR WEST'}, inplace=True)
    data_mart['Banner'].replace({'LOBLAW QC':'LOBLAWS QC'}, inplace=True)

    #Sorting the data at Banner Product Week level
    data_treatment = data_mart.sort_values(by=['Banner','Product','Week']).copy()

    #Extract Year from Week
    data_treatment['Year'] = data_treatment['Week'].str.slice(0,4)

    ## Calculating EDV Price: 
    EDV_dataset = data_treatment[['Banner','Product','Year']].drop_duplicates()

    data_treatment['EDV.Price'] = data_treatment.groupby(['Banner','Product','Year'])['Adcal_Price'].transform(lambda x: round(np.quantile(x, q=0.9),2))

    ### Calculating the Discount Depth
    #Discount depth is calculated and the Adcal Prices  that are more than 90% of the EDV Price are replaced by the EDV Price. This is taken to keep a cutoff for promo price and replace all non-promo prices with the EDV price 
    ##Calculate discount depth
    data_treatment['Adcal_DD'] = round(((data_treatment['EDV.Price'] - data_treatment['Adcal_Price']) / data_treatment['EDV.Price']), 4)

    #Make negative values zero
    data_treatment.loc[data_treatment['Adcal_DD'] < 0, 'Adcal_DD'] = 0

    #Making Adcal.Price equals to EDV.Price Wherever the Discount Depth is less than 10%
    data_treatment['Adcal_Price'] = round(data_treatment[['Adcal_DD','Adcal_Price','EDV.Price']].apply(lambda x: x['EDV.Price'] if(x['Adcal_DD'] < 0.1) else x['Adcal_Price'], axis=1), 2)

    #Repeating the same process to calculate the new discount depth
    data_treatment['Adcal_DD'] = round(((data_treatment['EDV.Price'] - data_treatment['Adcal_Price']) / data_treatment['EDV.Price']), 4)

    #Make negative values zero
    data_treatment.loc[data_treatment['Adcal_DD'] < 0, 'Adcal_DD'] = 0

    ### Pantry flag creation
    #Pantry flag is calculated for each Banner-product
    pantry_treatment = data_treatment.copy()

    #Creating Pantry1 and Pantry2 flags
    final_df = pd.DataFrame()
    product_list = sorted(list(pantry_treatment['Product'].unique()))
    for product in product_list:
        banner_list = sorted(list(pantry_treatment['Banner'].unique()))
        for banner in banner_list:
            df = pantry_treatment[(pantry_treatment['Banner']==banner)
                                 &(pantry_treatment['Product']==product)]

            df['Test.Vol'] = df['Adcal_DD'].apply(lambda x: 1 if(x > 0.25) else 0)
            df['Pantry1'] = df['Test.Vol'].shift(1).fillna(0).astype(int)
            df['Pantry2_temp'] = df['Test.Vol'].shift(2).fillna(0).astype(int)
            df['Pantry2'] = df[['Pantry1','Pantry2_temp']].apply(lambda x: 0 if((x[0]==1) and (x[1]==1)) else x[1], axis=1)
            final_df = final_df.append(df)


    pantry_treatment = final_df.copy()
    pantry_treatment.drop(columns=['Test.Vol','Pantry2_temp'], inplace=True)

    ### Feature Vision flag creation
    fv_treated = pantry_treatment.copy()
    fv_treated.replace({'FRONT PAGE':'Front.Page', 'INSIDE PAGE':'Middle.Page', 'BACK PAGE':'Back.Page'}, inplace=True)

    promo_dummies = pd.get_dummies(fv_treated['Adcal_Page.Position'])

    promo_dummies = pd.get_dummies(fv_treated['Adcal_Page.Position'])
    promo_lst = list(promo_dummies.columns)

    # Add promo dummies if not present
    if 'Front.Page' not in promo_lst: 
        promo_dummies['Front.Page'] = 0

    if 'Middle.Page' not in promo_lst: 
        promo_dummies['Middle.Page'] = 0

    if 'Back.Page' not in promo_lst: 
        promo_dummies['Back.Page'] = 0    

    fv_treated = pd.concat([fv_treated, promo_dummies], axis=1)
    del fv_treated['IN STORE']

    # Creating Test period column
    fv_treated['YEAR'] = fv_treated['YEAR'].astype(str)
    fv_treated['QUARTER'] = fv_treated['QUARTER'].astype(str)
    fv_treated['Test_period'] = fv_treated['YEAR'] + fv_treated['QUARTER']

    ### Holiday flag creation
    holiday_treatment = fv_treated.copy()
    holiday_dummies = pd.get_dummies(holiday_treatment['Week.Type'])
    holiday_treatment = pd.concat([holiday_treatment, holiday_dummies], axis=1)

    # Files present in FTP server under "New Product Simulator"
    # Loading seasonality table
    seasonality_table = pd.read_csv("Seasonality_Index_data")
    seasonality_table['Week.Name'] = seasonality_table['Week.Name'].str.strip()

    Region_List = sorted(list(holiday_treatment['Region'].unique()))
    category = str(holiday_treatment['Category'].unique()[0])
    pack_subtype = str(holiday_treatment['Pack.Subtype'].unique()[0])

    final_seasonality_table = pd.DataFrame()

    for Region_key in Region_List:
        #Creating Region Category Pack Subtype variables
        region = Region_key

        # Filtering seasonality table
        seasonality_table_filtered = seasonality_table[(seasonality_table['Region']==Region_key)
                                                      &(seasonality_table['Category']==category)
                                                      &(seasonality_table['Pack.Subtype']==pack_subtype)]

        # Filtering Week data
        week_data_filtered = week_data[week_data['REGION']==Region_key]
        week_data_filtered['CAL_WEEK_NAME'] = week_data_filtered['CAL_WEEK_NAME'].str.strip()

        # Updated seasonality table
        seasonality_table_updated = pd.merge(week_data_filtered[['REGION','TPO_WEEK','CAL_WEEK_NAME']], seasonality_table_filtered[['Region','Week.Name','seasonality_index']], how='left', left_on=['REGION','CAL_WEEK_NAME'], right_on=['Region','Week.Name']).fillna(method='ffill')
        seasonality_table_updated['CAL_WEEK_NAME'] = seasonality_table_updated['CAL_WEEK_NAME'].str.strip()

        final_seasonality_table = final_seasonality_table.append(seasonality_table_updated)

    # Merging holiday treatment data with seasonality data
    Volume_dataset_all_reg_upload = pd.merge(holiday_treatment,final_seasonality_table[['REGION','CAL_WEEK_NAME','seasonality_index']], how='left', left_on = ['Week.Name','Region'], right_on=['CAL_WEEK_NAME','REGION'])
    del Volume_dataset_all_reg_upload['CAL_WEEK_NAME']
    Volume_dataset_all_reg_upload.rename(columns={'Post-holiday':'Post.Holiday.Week', 'Pre-holiday':'Pre.Holiday.Week', 'Holiday':'Holiday.Week'}, inplace=True)
    Volume_dataset_all_reg_upload.drop(columns = {'REGION'}, inplace = True)
    return Volume_dataset_all_reg_upload
    

__Prices, Discounts for existing product/ uploaded product__

In [202]:
# In case the user doesn't want to upload adcal data
if toggle_upload.value != 'Yes':

    display(Markdown('<div><div class="loader"></div><h2> &nbsp; Getting Existing Prices & Discounts</h2></div>'))

    ######################### FILTERS & LAYOUTS #########################
    layout1 = widgets.Layout(width='200px', height='auto')
    layout4 = widgets.Layout(width='100px', height='auto')
    test_period = widgets.SelectMultiple(options=list(np.sort(Volume_dataset_all_reg[(Volume_dataset_all_reg.Test_period >= Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Product==product_fil,'Test_period'].min()) & (Volume_dataset_all_reg.Test_period <= Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Product==product_fil,'Test_period'].max()) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)].Test_period.unique())) + ['All'], value=['All'], description='Year Quarter', layout = layout1)
    test_period2 = widgets.SelectMultiple(options=list(np.sort(Volume_dataset_all_reg[(Volume_dataset_all_reg.Test_period >= Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Product==product_fil,'Test_period'].min()) & (Volume_dataset_all_reg.Test_period <= Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Product==product_fil,'Test_period'].max()) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)].Test_period.unique())) + ['All'], value=['All'], description='Year Quarter', layout = layout1)
    test_period3 = widgets.SelectMultiple(options=list(np.sort(Volume_dataset_all_reg[(Volume_dataset_all_reg.Test_period >= Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Product==product_fil,'Test_period'].min()) & (Volume_dataset_all_reg.Test_period <= Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Product==product_fil,'Test_period'].max()) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)].Test_period.unique())) + ['All'], value=['All'], description='Year Quarter', layout = layout1)    

    # Filter
    layout = widgets.Layout(width='250px', height='auto')
    base_price_change = widgets.HBox([widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "),widgets.Label(value=" "), widgets.Label(value=" "),widgets.Label(value="Baseline Price Change (%)"), widgets.BoundedFloatText(value=0, min = -99, max = 1000, disabled=False,layout = layout2)])   
    base_price_change2 = widgets.HBox([widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "),widgets.Label(value=" "), widgets.Label(value=" "),widgets.Label(value="Baseline Price Change (%)"), widgets.BoundedFloatText(value=0, min = -99, max = 1000, disabled=False,layout = layout2)])   
    base_price_change3 = widgets.HBox([widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "),widgets.Label(value=" "), widgets.Label(value=" "),widgets.Label(value="Baseline Price Change (%)"), widgets.BoundedFloatText(value=0, min = -99, max = 1000, disabled=False,layout = layout2)])
    disc_change = widgets.HBox([widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "),widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value="Discount Change (%)"), widgets.BoundedFloatText(value=0, min = -99, max = 1000, disabled=False,layout = layout2)])
    disc_change2 = widgets.HBox([widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "),widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value="Discount Change (%)"), widgets.BoundedFloatText(value=0, min = -99, max = 1000, disabled=False,layout = layout2)])
    disc_change3 = widgets.HBox([widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value=" "),widgets.Label(value=" "), widgets.Label(value=" "), widgets.Label(value="Discount Change (%)"), widgets.BoundedFloatText(value=0, min = -99, max = 1000, disabled=False,layout = layout2)])

    # Bordering filters
    box_layout = Layout(display='flex',flex_flow='rows',align_items='stretch',border='solid 2px',width='auto')    
            
    ############## IN CASE OF ENSEMBLE MODEL ################

    if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
        if len(prod_ensemble_list) != 3:
            clear_output()
            print('Select 3 products as existing model pack')
        else:

            ######################### Quarterly Adcal prices for selected combination #########################
            qtr_edv_baseline = Volume_dataset_all_reg[(Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value) & (Volume_dataset_all_reg.Region.isin(Region_List)) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Product.isin(prod_ensemble_list))].groupby(['Product','Test_period'])['EDV.Price'].mean().reset_index()

            if len(qtr_edv_baseline) != 0 : # If product is present in selected test period 

                # Renaming required column
                qtr_edv_baseline = qtr_edv_baseline.rename(columns = {'Product':'Existing Model Pack','Test_period':'Year Quarter','EDV.Price':'Baseline Pricing'})

                # Baseline & Discount pricing merge
                qtr_edv = qtr_edv_baseline
                qtr_edv_prev = qtr_edv.copy()

                ########################## Quarterly Adcal DD ##########################
                qtr_disc = Volume_dataset_all_reg[(Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value) & (Volume_dataset_all_reg.Adcal_DD >= 0.1) & (Volume_dataset_all_reg.Region.isin(Region_List)) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Product.isin(prod_ensemble_list))].groupby(['Product','Test_period']).agg({'Adcal_DD':'mean'}).reset_index()
                qtr_disc = qtr_disc.rename(columns = {'Product':'Existing Model Pack','Test_period':'Year Quarter','Adcal_DD':'Discount (%)'})
                qtr_disc['Discount (%)'] = qtr_disc['Discount (%)']*100
                qtr_disc_prev = qtr_disc.copy()

                ######################### Tabulating results #########################
                from tabulate import tabulate
                edv_disc_join = qtr_edv_prev.merge(qtr_disc_prev, on = ['Existing Model Pack','Year Quarter'], how = 'left')
                clear_output()
                edv_disc_join = edv_disc_join[['Existing Model Pack','Year Quarter','Baseline Pricing','Discount (%)']].sort_values('Existing Model Pack')
                display(edv_disc_join.round(2).astype(str).style.set_table_styles([{'selector' : '','props' : [('border','1px solid black')]}]))

            else:     # If product is not present in selected test period 
                clear_output()
                print('Product is not present in that Region/Channel for selected period')            

    else: # In case of normal approach
        
        ######################### Quarterly Adcal prices for selected combination #########################
        qtr_edv_baseline = Volume_dataset_all_reg[(Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value) & (Volume_dataset_all_reg.Region.isin(Region_List)) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Product.isin([product_fil]))].groupby('Test_period')['EDV.Price'].mean().reset_index()

        if len(qtr_edv_baseline) != 0 : # If product is present in selected test period 

            # Renaming required column
            qtr_edv_baseline = qtr_edv_baseline.rename(columns = {'Test_period':'Year Quarter','EDV.Price':'Baseline Pricing'})

            # Baseline & Discount pricing merge
            qtr_edv = qtr_edv_baseline
            qtr_edv_prev = qtr_edv.copy()

            ########################## Quarterly Adcal DD ##########################
            qtr_disc = Volume_dataset_all_reg[(Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value) & (Volume_dataset_all_reg.Adcal_DD >= 0.1) & (Volume_dataset_all_reg.Region.isin(Region_List)) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Product.isin([product_fil]))].groupby('Test_period').agg({'Adcal_DD':'mean'}).reset_index()
            qtr_disc = qtr_disc.rename(columns = {'Test_period':'Year Quarter','Adcal_DD':'Discount (%)'})
            qtr_disc['Discount (%)'] = qtr_disc['Discount (%)']*100
            qtr_disc_prev = qtr_disc.copy()

            ######################### Tabulating results #########################
            from tabulate import tabulate
            edv_disc_join = qtr_edv_prev.merge(qtr_disc_prev, on = 'Year Quarter', how = 'left')
            clear_output()
            edv_disc_join.fillna(0, inplace=True)
            edv_disc_join['Existing Model Pack'] = product_fil
            edv_disc_join = edv_disc_join[['Existing Model Pack','Year Quarter','Baseline Pricing','Discount (%)']].sort_values('Existing Model Pack')
            display(edv_disc_join.round(2).astype(str).style.set_table_styles([{'selector' : '','props' : [('border','1px solid black')]}]))

        else:    
            clear_output()
            print('Product is not present in that Region/Channel for selected period')

else: # Integration of uploaded data with tool
    if len(uploader.value) != 0:
        try:
            qtr_edv_baseline = Volume_dataset_all_reg[(Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value) & (Volume_dataset_all_reg.Region.isin(Region_List)) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Product.isin([product_fil]))].groupby('Test_period')['EDV.Price'].mean().reset_index()
            input_file = list(uploader.value.values())[0]
            content = input_file['content']
            content = io.StringIO(content.decode('utf-8'))
            upload_data = pd.read_csv(content)
            pd.set_option('display.max_columns',50)
            print('Uploaded Data : ')
            display(upload_data.head())        
            ## Data Treatment Steps
            upload_data_1 = upload_data_treat()
            upload_data = upload_data_1.copy()                
        except:
            print(colored('Please upload the data in proper format with all required columns', 'red',attrs=['bold']))
        
    else: # Integration of uplaoded data with tool
        print(colored('Please upload the file to proceed', 'red',attrs=['bold']))
        upload_data = pd.DataFrame()

In [203]:
# In case the user doesn't want to upload adcal data
if toggle_upload.value != 'Yes':

    if len(qtr_edv_baseline) != 0:  # If product is present in selected test period 
        
        if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
            if len(prod_ensemble_list) != 3:
                print('Select 3 products as existing model pack')
            else:
                display(VBox([widgets.Label(value="$Model Pack 1$" + ': ' + edv_disc_join['Existing Model Pack'].unique()[0] ),widgets.HBox((test_period,base_price_change,disc_change))], layout=box_layout))
                display(VBox([widgets.Label(value="$Model Pack 2$" + ': ' + edv_disc_join['Existing Model Pack'].unique()[1] ),widgets.HBox((test_period2,base_price_change2,disc_change2))], layout=box_layout))
                display(VBox([widgets.Label(value="$Model Pack 3$" + ': ' + edv_disc_join['Existing Model Pack'].unique()[2] ),widgets.HBox((test_period3,base_price_change3,disc_change3))], layout=box_layout))
          
        else: # In case of normal approach
            # Display filters
            if toggle_np2.value == 'Yes':
                display(VBox([widgets.Label(value="$New Product 1$"),widgets.HBox((test_period,base_price_change,disc_change))], layout=box_layout))
                display(VBox([widgets.Label(value="$New Product 2$"),widgets.HBox((test_period2,base_price_change2,disc_change2))], layout=box_layout))
            elif toggle_np2.value == 'No':
                display(VBox([widgets.HBox((test_period,base_price_change,disc_change))], layout=box_layout))
    else:
        clear_output()
        print('Product is not present in that Region/Channel for selected period')

In [204]:
# In case the user doesn't want to upload adcal data
if toggle_upload.value != 'Yes':

    #### Change price cell run
    def run_all(ev):
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,IPython.notebook.ncells()-21)'))
    layout = widgets.Layout(width='auto', height='40px')
    button = widgets.Button(description="Change Prices & Discounts",layout = layout)
    button.on_click(run_all)
    display(button)

__Updated Prices & Discounts__

In [205]:
# In case the user doesn't want to upload adcal data
if toggle_upload.value != 'Yes':

    ##################################### PRICE & DISCOUNT #####################################
    qtr_edv_new = pd.DataFrame()
    qtr_disc_new = pd.DataFrame()
    qtr_price_disc_new_merge = pd.DataFrame()
    
    # Storing filter output
    # Product 1
    test_period_fil = list(test_period.value)
    base_price_change_fil = base_price_change.children[10].value/100
    disc_change_fil = disc_change.children[10].value/100
    # Product 2
    test_period_fil2 = list(test_period2.value)
    base_price_change_fil2 = base_price_change2.children[10].value/100
    disc_change_fil2 = disc_change2.children[10].value/100
    # Product 3
    test_period_fil3 = list(test_period3.value)
    base_price_change_fil3 = base_price_change3.children[10].value/100
    disc_change_fil3 = disc_change3.children[10].value/100

    if list(test_period.value) == ['All']:
        test_period_fil = qtr_edv['Year Quarter'].unique()
    else:
        test_period_fil = list(test_period.value)
    if list(test_period2.value) == ['All']:
        test_period_fil2 = qtr_edv['Year Quarter'].unique()
    else:
        test_period_fil2 = list(test_period2.value)
    if list(test_period3.value) == ['All']:            
        test_period_fil3 = qtr_edv['Year Quarter'].unique()    
    else:
        test_period_fil3 = list(test_period3.value)
    

    if len(qtr_edv_baseline) != 0 :  # If product is present in selected test period 

        if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
            if len(prod_ensemble_list) != 3: # if 3 products are not selected
                clear_output()
                print('Select 3 products as existing model pack')
            else:

                for exist_pack in prod_ensemble_list:
                    # Changing prices/discounts for each model pack
                    qtr_edv_copy = qtr_edv[qtr_edv['Existing Model Pack'] == exist_pack].copy()

                    # Price change
                    if exist_pack == edv_disc_join['Existing Model Pack'].unique()[0]: # Model Pack 1
                        qtr_edv_copy.loc[qtr_edv_copy['Year Quarter'].isin(test_period_fil),'Baseline Pricing'] = (1+base_price_change_fil)*qtr_edv.loc[qtr_edv['Year Quarter'].isin(test_period_fil),'Baseline Pricing']
                    elif exist_pack == edv_disc_join['Existing Model Pack'].unique()[1]: # Model Pack 2
                        qtr_edv_copy.loc[qtr_edv_copy['Year Quarter'].isin(test_period_fil2),'Baseline Pricing'] = (1+base_price_change_fil2)*qtr_edv.loc[qtr_edv['Year Quarter'].isin(test_period_fil2),'Baseline Pricing']
                    elif exist_pack == edv_disc_join['Existing Model Pack'].unique()[2]: # Model Pack 3
                        qtr_edv_copy.loc[qtr_edv_copy['Year Quarter'].isin(test_period_fil3),'Baseline Pricing'] = (1+base_price_change_fil3)*qtr_edv.loc[qtr_edv['Year Quarter'].isin(test_period_fil3),'Baseline Pricing']

                    # Discount change
                    qtr_disc_copy = qtr_disc[qtr_disc['Existing Model Pack'] == exist_pack].copy()

                    if exist_pack == edv_disc_join['Existing Model Pack'].unique()[0]: # Model Pack 1
                        qtr_disc_copy.loc[qtr_disc_copy['Year Quarter'].isin(test_period_fil),'Discount (%)'] = (1+disc_change_fil)*qtr_disc.loc[qtr_disc['Year Quarter'].isin(test_period_fil),'Discount (%)']
                    elif exist_pack == edv_disc_join['Existing Model Pack'].unique()[1]: # Model Pack 2
                        qtr_disc_copy.loc[qtr_disc_copy['Year Quarter'].isin(test_period_fil2),'Discount (%)'] = (1+disc_change_fil2)*qtr_disc.loc[qtr_disc['Year Quarter'].isin(test_period_fil2),'Discount (%)']
                    elif exist_pack == edv_disc_join['Existing Model Pack'].unique()[2]: # Model Pack 3
                        qtr_disc_copy.loc[qtr_disc_copy['Year Quarter'].isin(test_period_fil3),'Discount (%)'] = (1+disc_change_fil3)*qtr_disc.loc[qtr_disc['Year Quarter'].isin(test_period_fil3),'Discount (%)']

                    # Updated adcal prices
                    qtr_edv_new = qtr_edv_prev[qtr_edv_prev['Existing Model Pack'] == exist_pack].merge(qtr_edv_copy, on = 'Year Quarter', how = 'left')
                    qtr_edv_new.drop(columns = 'Existing Model Pack_y', inplace = True)
                    qtr_edv_new = qtr_edv_new.rename(columns = {'Existing Model Pack_x':'Existing Model Pack','Baseline Pricing_x':'Previous Baseline Pricing','Baseline Pricing_y':'Current Baseline Pricing'})
                    # Updated adcal discounts
                    qtr_disc_new = qtr_disc_prev[qtr_disc_prev['Existing Model Pack'] == exist_pack].merge(qtr_disc_copy, on = 'Year Quarter', how = 'left')
                    qtr_disc_new.drop(columns = 'Existing Model Pack_y', inplace = True)
                    qtr_disc_new = qtr_disc_new.rename(columns = {'Existing Model Pack_x':'Existing Model Pack','Discount (%)_x':'Previous Discount (%)', 'Discount (%)_y':'Current Discount (%)'})


                    # ##################################### MERGING DATAFRAMES #####################################
                    qtr_price_disc_new = qtr_edv_new.merge(qtr_disc_new, on = ['Year Quarter','Existing Model Pack'], how = 'left')       
                    qtr_price_disc_new = qtr_price_disc_new[['Existing Model Pack','Year Quarter','Previous Baseline Pricing', 'Current Baseline Pricing', 'Previous Discount (%)','Current Discount (%)']]       
                    qtr_price_disc_new.fillna(0, inplace=True)       

                    qtr_price_disc_new_merge = qtr_price_disc_new_merge.append(qtr_price_disc_new, ignore_index = True)
                    qtr_price_disc_new_merge = qtr_price_disc_new_merge[['Existing Model Pack','Year Quarter','Previous Baseline Pricing','Current Baseline Pricing','Previous Discount (%)','Current Discount (%)']]

                qtr_price_disc_new_merge = qtr_price_disc_new_merge.sort_values('Existing Model Pack')
                display(widgets.Label(value="Existing Model Packs Price/Discount Changes"),HTML(qtr_price_disc_new_merge.round(2).to_html(index=False)))

            
        else: # In case of normal approach
            
            for i in [1,2]:

                qtr_edv_copy = qtr_edv.copy()
                qtr_edv_copy['iter'] = i
                if i == 1:                
                    qtr_edv_copy.loc[qtr_edv_copy['Year Quarter'].isin(test_period_fil),'Baseline Pricing'] = (1+base_price_change_fil)*qtr_edv.loc[qtr_edv['Year Quarter'].isin(test_period_fil),'Baseline Pricing']
                elif i == 2:
                    qtr_edv_copy.loc[qtr_edv_copy['Year Quarter'].isin(test_period_fil2),'Baseline Pricing'] = (1+base_price_change_fil2)*qtr_edv.loc[qtr_edv['Year Quarter'].isin(test_period_fil2),'Baseline Pricing']
                qtr_disc_copy = qtr_disc.copy()            
                qtr_disc_copy['iter'] = i
                if i == 1:                
                    qtr_disc_copy.loc[qtr_disc_copy['Year Quarter'].isin(test_period_fil),'Discount (%)'] = (1+disc_change_fil)*qtr_disc.loc[qtr_disc['Year Quarter'].isin(test_period_fil),'Discount (%)']
                elif i == 2:
                    qtr_disc_copy.loc[qtr_disc_copy['Year Quarter'].isin(test_period_fil2),'Discount (%)'] = (1+disc_change_fil2)*qtr_disc.loc[qtr_disc['Year Quarter'].isin(test_period_fil2),'Discount (%)']

                # Updated adcal prices
                if i == 1:
                    qtr_edv_new = qtr_edv_prev.merge(qtr_edv_copy, on = 'Year Quarter', how = 'left')
                    qtr_edv_new = qtr_edv_new.rename(columns = {'Baseline Pricing_x':'Previous Baseline Pricing','Baseline Pricing_y':'Current Baseline Pricing'})
                    # Updated adcal discounts
                    qtr_disc_new = qtr_disc_prev.merge(qtr_disc_copy, on = 'Year Quarter', how = 'left')
                    qtr_disc_new = qtr_disc_new.rename(columns = {'Discount (%)_x':'Previous Discount (%)', 'Discount (%)_y':'Current Discount (%)'})

                elif i == 2:
                    qtr_edv_new2 = qtr_edv_prev.merge(qtr_edv_copy, on = 'Year Quarter', how = 'left')
                    qtr_edv_new2 = qtr_edv_new2.rename(columns = {'Baseline Pricing_x':'Previous Baseline Pricing','Baseline Pricing_y':'Current Baseline Pricing'})
                    # Updated adcal discounts
                    qtr_disc_new2 = qtr_disc_prev.merge(qtr_disc_copy, on = 'Year Quarter', how = 'left')
                    qtr_disc_new2= qtr_disc_new2.rename(columns = {'Discount (%)_x':'Previous Discount (%)', 'Discount (%)_y':'Current Discount (%)'})

            # ##################################### MERGING DATAFRAMES #####################################
            qtr_price_disc_new = qtr_edv_new.merge(qtr_disc_new, on = 'Year Quarter', how = 'left')
            qtr_price_disc_new2 = qtr_edv_new2.merge(qtr_disc_new2, on = 'Year Quarter', how = 'left')
            qtr_price_disc_new = qtr_price_disc_new[['Year Quarter','Previous Baseline Pricing', 'Current Baseline Pricing', 'Previous Discount (%)','Current Discount (%)']]
            qtr_price_disc_new2 = qtr_price_disc_new2[['Year Quarter','Previous Baseline Pricing', 'Current Baseline Pricing', 'Previous Discount (%)','Current Discount (%)']]
            qtr_price_disc_new.fillna(0, inplace=True)
            qtr_price_disc_new2.fillna(0, inplace=True)

            qtr_price_disc_new['New Product'] = '1'
            qtr_price_disc_new2['New Product'] = '2'

            qtr_price_disc_new_merge = qtr_price_disc_new.append(qtr_price_disc_new2, ignore_index = True)
            qtr_price_disc_new_merge = qtr_price_disc_new_merge[['New Product','Year Quarter','Previous Baseline Pricing','Current Baseline Pricing','Previous Discount (%)','Current Discount (%)']]            
            
            if toggle_np2.value == 'Yes':
                display(widgets.Label(value="New Product 1 vs New Product 2"),HTML(qtr_price_disc_new_merge.round(2).to_html(index=False)))        
            else:
                display(qtr_price_disc_new.round(2).astype(str).style.set_table_styles([{'selector' : '','props' : [('border','1px solid black')]}]))

    else:        
        print('Product is not present in that Region/Channel for selected period')    

__Existing Product Promotions__

In [206]:
# In case the user doesn't want to upload adcal data
if toggle_upload.value != 'Yes':
    try:
        if toggle_ensemble_normal.value != 'Multiple Product': # In case of normal approach

            Volume_dataset_all_reg_for_promo = Volume_dataset_all_reg.groupby(['Region','Channel','Store.Chain','Banner','Product','Test_period','Week'])['Front.Page','Middle.Page','Back.Page'].mean().reset_index()
            Volume_dataset_all_reg_for_promo.rename(columns={'Front.Page':'Front Page','Middle.Page':'Middle Page','Back.Page':'Back Page'}, inplace=True)

            qtr_promo = Volume_dataset_all_reg_for_promo[(Volume_dataset_all_reg_for_promo.Test_period >= start_week.children[8].value)
                                              &(Volume_dataset_all_reg_for_promo.Test_period <= end_week.children[8].value)
                                              &(Volume_dataset_all_reg_for_promo.Region.isin(Region_List))
                                              &(Volume_dataset_all_reg_for_promo.Channel.isin(channel_list))
                                              &(Volume_dataset_all_reg_for_promo.Product.isin([product_fil]))]

            qtr_df = qtr_promo.groupby('Banner').agg({'Front Page':'sum','Middle Page':'sum','Back Page':'sum'}).reset_index() 

            banner_list = list(qtr_promo['Banner'].unique())
            banner_max_df = pd.DataFrame()
            for banner_var in banner_list:
                max_limit = qtr_promo[qtr_promo['Banner']==banner_var].Week.nunique()
                promo_given = (qtr_df.loc[qtr_df.Banner == banner_var,'Front Page'] + qtr_df.loc[qtr_df.Banner == banner_var,'Middle Page']  + qtr_df.loc[qtr_df.Banner == banner_var,'Back Page']).values[0]
                avail_promo = max_limit - promo_given

                banner_max_df = banner_max_df.append({'Banner':banner_var,'Total Promotions Available':avail_promo}, ignore_index = True)

            qtr_df_final = pd.merge(qtr_df, banner_max_df, how='inner',on='Banner')
            qtr_df_final['Existing Model Pack'] = product_fil
            qtr_df_final = qtr_df_final[['Existing Model Pack','Banner','Front Page','Middle Page','Back Page','Total Promotions Available']]

            # pd.options.display.precision = 0
            display(HTML(qtr_df_final.to_html(index=False)))
            print(colored('NOTE: The Total Promotions Available is the sum of promotions available for Front, Middle & Back Page promotions', 'red',attrs=['bold']))

        elif toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
            if len(prod_ensemble_list) != 3: # if 3 products are not selected
                clear_output()
                print('Select 3 products as existing model pack')
            else:
                Volume_dataset_all_reg_for_promo = Volume_dataset_all_reg.groupby(['Region','Channel','Store.Chain','Banner','Product','Test_period','Week'])['Front.Page','Middle.Page','Back.Page'].mean().reset_index()
                Volume_dataset_all_reg_for_promo.rename(columns={'Front.Page':'Front Page','Middle.Page':'Middle Page','Back.Page':'Back Page'}, inplace=True)

                qtr_promo = Volume_dataset_all_reg_for_promo[(Volume_dataset_all_reg_for_promo.Test_period >= start_week.children[8].value)
                                                  &(Volume_dataset_all_reg_for_promo.Test_period <= end_week.children[8].value)
                                                  &(Volume_dataset_all_reg_for_promo.Region.isin(Region_List))
                                                  &(Volume_dataset_all_reg_for_promo.Channel.isin(channel_list))
                                                  &(Volume_dataset_all_reg_for_promo.Product.isin(prod_ensemble_list))]

                qtr_df = qtr_promo.groupby(['Product','Banner']).agg({'Front Page':'sum','Middle Page':'sum','Back Page':'sum'}).reset_index() 

                banner_max_df = pd.DataFrame()
                for exist_pack in prod_ensemble_list:
                    banner_list = list(qtr_promo[qtr_promo.Product == exist_pack]['Banner'].unique())            
                    qtr_promo_pack = qtr_promo[qtr_promo.Product == exist_pack]
                    for banner_var in banner_list:

                        max_limit = qtr_promo_pack[(qtr_promo_pack['Banner']==banner_var)].Week.nunique()
                        promo_given = (qtr_df.loc[(qtr_df.Banner == banner_var) & (qtr_df.Product == exist_pack),'Front Page'] + qtr_df.loc[(qtr_df.Banner == banner_var)& (qtr_df.Product == exist_pack),'Middle Page']  + qtr_df.loc[(qtr_df.Banner == banner_var) & (qtr_df.Product == exist_pack),'Back Page']).values[0]
                        avail_promo = max_limit - promo_given

                        banner_max_df = banner_max_df.append({'Product':exist_pack,'Banner':banner_var,'Total Promotions Available':avail_promo}, ignore_index = True)

                    qtr_df_final = pd.merge(qtr_df, banner_max_df, how='inner',on=['Banner','Product'])
                    qtr_df_final = qtr_df_final[['Product','Banner','Front Page','Middle Page','Back Page','Total Promotions Available']]
                    qtr_df_final.rename(columns = {'Product':'Existing Model Pack',}, inplace = True)
                    qtr_df_final = qtr_df_final.sort_values('Existing Model Pack')
            
                display(HTML(qtr_df_final.to_html(index=False)))
                print(colored('NOTE: The Total Promotions Available is the sum of promotions available for Front, Middle & Back Page promotions', 'red',attrs=['bold']))                        
            
    except:
        print('Product is not present in that Region/Channel for selected period')    

In [219]:
# In case the user doesn't want to upload adcal data
if toggle_upload.value != 'Yes':

    ############# Toggle for Promo change #############
    promo_change_toggle = widgets.ToggleButtons(options=['Yes (All Banners)', 'Yes (Specific Banners)','No'],value='No',description='Do you want to change promotions?',
        disabled=False,
        display='flex',
        flex_flow='column',
        align_items='stretch',
        style= {'description_width': 'initial'})

    promo_change_toggle.style.button_width='auto'
    promo_change_toggle.style.button_height='auto'

    display(promo_change_toggle)

In [207]:
# In case the user doesn't want to upload adcal data
if toggle_upload.value != 'Yes':

    if (promo_change_toggle.value == 'Yes (All Banners)') | (promo_change_toggle.value == 'Yes (Specific Banners)'):
        def run_all(ev):
            display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index(),IPython.notebook.ncells()-19)'))

        layout = widgets.Layout(width='auto', height='40px')
        button = widgets.Button(description="Proceed further", layout = layout)
        button.on_click(run_all)
        display(button)
    else:
        def run_all(ev):
            display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index(),IPython.notebook.ncells()-15)'))

        layout = widgets.Layout(width='auto', height='40px')
        button = widgets.Button(description="Proceed further", layout = layout)
        button.on_click(run_all)
        display(button)

In [208]:
# In case the user doesn't want to upload adcal data
if toggle_upload.value != 'Yes':

    if (promo_change_toggle.value == 'Yes (All Banners)') | (promo_change_toggle.value == 'Yes (Specific Banners)'):
        # In case no product is present in selected region-channel
        if len(qtr_edv_baseline) != 0 :

            if (promo_change_toggle.value == 'Yes (Specific Banners)'):
                # Banner selection
                specific_banner = sorted(list(qtr_df_final['Banner'].unique()))
                specific_banner_list = specific_banner
                banner_select = widgets.SelectMultiple(options=list(np.sort(specific_banner_list)), value = [specific_banner_list[0]],layout = layout3)

            # Type-in filters for promotion increase
            layout_promo = widgets.Layout(width='auto', height='auto')

            # Product 1: Providing front,middle,back promotion edits
            front_change = widgets.HBox([widgets.Label(value="Front Page Change"), widgets.BoundedFloatText(value=0, min = -13, max = 13, disabled=False,layout = layout_promo)])
            middle_change = widgets.HBox([widgets.Label(value="Middle Page Change"), widgets.BoundedFloatText(value=0, min = -13, max = 13,  disabled=False,layout = layout_promo)])
            back_change = widgets.HBox([widgets.Label(value="Back Page Change"),widgets.BoundedFloatText(value=0, min = -13, max = 13, disabled=False,layout = layout_promo)])

            # Product 2: Providing front,middle,back promotion edits
            front_change2 = widgets.HBox([widgets.Label(value="Front Page Change"), widgets.BoundedFloatText(value=0, min = -13, max = 13, disabled=False,layout = layout_promo)])
            middle_change2 = widgets.HBox([widgets.Label(value="Middle Page Change"), widgets.BoundedFloatText(value=0, min = -13, max = 13,  disabled=False,layout = layout_promo)])
            back_change2 = widgets.HBox([widgets.Label(value="Back Page Change"),widgets.BoundedFloatText(value=0, min = -13, max = 13, disabled=False,layout = layout_promo)])

            # Product 3: Providing front,middle,back promotion edits
            front_change3 = widgets.HBox([widgets.Label(value="Front Page Change"), widgets.BoundedFloatText(value=0, min = -13, max = 13, disabled=False,layout = layout_promo)])
            middle_change3 = widgets.HBox([widgets.Label(value="Middle Page Change"), widgets.BoundedFloatText(value=0, min = -13, max = 13,  disabled=False,layout = layout_promo)])
            back_change3 = widgets.HBox([widgets.Label(value="Back Page Change"),widgets.BoundedFloatText(value=0, min = -13, max = 13, disabled=False,layout = layout_promo)])

            if toggle_ensemble_normal.value != 'Multiple Product': # In case of normal approach                
                # Show selections as per specific banner
                if (promo_change_toggle.value == 'Yes (Specific Banners)'):
                    display(widgets.HBox([widgets.Label(value="Select Banners"),banner_select]))
                    # Show New product 1, new product 2 promo change
                    if toggle_np2.value == 'Yes':
                        display(widgets.Label(value="$New Product 1$"),widgets.HBox([front_change,middle_change,back_change]))
                        display(widgets.Label(value="$New Product 2$"),widgets.HBox([front_change2,middle_change2,back_change2]))
                    else:
                        display(widgets.HBox([front_change,middle_change,back_change]))

                if (promo_change_toggle.value == 'Yes (All Banners)'):
                    # Show New product 1, new product 2 promo change
                    if toggle_np2.value == 'Yes':
                        display(widgets.Label(value="$New Product 1$"),widgets.HBox([front_change,middle_change,back_change]))
                        display(widgets.Label(value="$New Product 2$"),widgets.HBox([front_change2,middle_change2,back_change2]))
                    else:
                        display(widgets.HBox([front_change,middle_change,back_change]))
                        
            elif toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
                if len(prod_ensemble_list) != 3: # if 3 products are not selected
                    clear_output()
                    print('Select 3 products as existing model pack')
                else:

                    # Show selections as per specific banner
                    if (promo_change_toggle.value == 'Yes (Specific Banners)'):
                        display(widgets.HBox([widgets.Label(value="Select Banners"),banner_select]))
                        # Show model pack 1,modle pack 2, model pack 3 promo change
                        display(widgets.Label(value="$Model Pack 1$" + ': ' + edv_disc_join['Existing Model Pack'].unique()[0]),widgets.HBox([front_change,middle_change,back_change]))
                        display(widgets.Label(value="$Model Pack 2$" + ': ' + edv_disc_join['Existing Model Pack'].unique()[1]),widgets.HBox([front_change2,middle_change2,back_change2]))
                        display(widgets.Label(value="$Model Pack 3$" + ': ' + edv_disc_join['Existing Model Pack'].unique()[2]),widgets.HBox([front_change3,middle_change3,back_change3]))


                    if (promo_change_toggle.value == 'Yes (All Banners)'):
                        # Show model pack 1,modle pack 2, model pack 3 promo change
                        display(widgets.Label(value="$Model Pack 1$" + ': ' + edv_disc_join['Existing Model Pack'].unique()[0]),widgets.HBox([front_change,middle_change,back_change]))
                        display(widgets.Label(value="$Model Pack 2$" + ': ' + edv_disc_join['Existing Model Pack'].unique()[1]),widgets.HBox([front_change2,middle_change2,back_change2]))
                        display(widgets.Label(value="$Model Pack 3$" + ': ' + edv_disc_join['Existing Model Pack'].unique()[2]),widgets.HBox([front_change3,middle_change3,back_change3]))                

            # Banners with 0 promotions
            banner_promos = Volume_dataset_all_reg.groupby('Banner')['Front.Page','Middle.Page','Back.Page'].sum().reset_index()
            fp_remove_banner = list(banner_promos[banner_promos['Front.Page'] == 0].Banner.unique())
            mp_remove_banner = list(banner_promos[banner_promos['Middle.Page'] == 0].Banner.unique())
            bp_remove_banner = list(banner_promos[banner_promos['Back.Page'] == 0].Banner.unique())

            # In case zero banner exist for existing product then pop up the question
            if len(qtr_df_final[(qtr_df_final.Banner.isin(fp_remove_banner)) | (qtr_df_final.Banner.isin(mp_remove_banner)) | (qtr_df_final.Banner.isin(bp_remove_banner))]) != 0:
                # Increase for zero banner promos?
                zero_banner_promo_inc = widgets.ToggleButtons(options=['Yes','No'],value='No',description='Do you want to increase for banners which have never given promotions?',
                disabled=False,
                display='flex',
                flex_flow='column',
                align_items='stretch',
                style= {'description_width': 'initial'})

                zero_banner_promo_inc.style.button_width='auto'
                zero_banner_promo_inc.style.button_height='auto'

                display(zero_banner_promo_inc)

        else:
            print("Product is not present in that Region/Channel for selected period")

In [209]:
# In case the user doesn't want to upload adcal data
if toggle_upload.value != 'Yes':

    if (promo_change_toggle.value == 'Yes (All Banners)') | (promo_change_toggle.value == 'Yes (Specific Banners)'):
        def run_all(ev):
            display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,IPython.notebook.ncells()-15)'))

        layout = widgets.Layout(width='auto', height='40px')
        button = widgets.Button(description="Apply changes", layout = layout)
        button.on_click(run_all)
        display(button)

In [172]:
# In case the user doesn't want to upload adcal data
if toggle_upload.value != 'Yes':

    Test_week_list = {"2019Q1" : ["2019-01","2019-02","2019-03","2019-04","2019-05","2019-06","2019-07","2019-08","2019-09","2019-10","2019-11","2019-12","2019-13"],
     "2019Q2" : ["2019-14","2019-15","2019-16","2019-17","2019-18","2019-19","2019-20","2019-21","2019-22","2019-23","2019-24","2019-25","2019-26"],
     "2019Q3" : ["2019-27","2019-28","2019-29","2019-30","2019-31","2019-32","2019-33","2019-34","2019-35","2019-36","2019-37","2019-38","2019-39"],
     "2019Q4" : ["2019-40","2019-41","2019-42","2019-43","2019-44","2019-45","2019-46","2019-47","2019-48","2019-49","2019-50","2019-51","2019-52"],
                     
     "2020Q1" : ["2020-01","2020-02","2020-03","2020-04","2020-05","2020-06","2020-07","2020-08","2020-09","2020-10","2020-11","2020-12","2020-13"],
     "2020Q2" : ["2020-14","2020-15","2020-16","2020-17","2020-18","2020-19","2020-20","2020-21","2020-22","2020-23","2020-24","2020-25","2020-26"],
     "2020Q3" : ["2020-27","2020-28","2020-29","2020-30","2020-31","2020-32","2020-33","2020-34","2020-35","2020-36","2020-37","2020-38","2020-39"],
     "2020Q4" : ["2020-40","2020-41","2020-42","2020-43","2020-44","2020-45","2020-46","2020-47","2020-48","2020-49","2020-50","2020-51","2020-52"],
     "fulltrain" : []}

    Test_periods = ["2019Q4","2019Q1","2019Q2","2019Q3","2020Q4","2020Q1","2020Q2","2020Q3"]

    # Initializing dataframe
    Test_results = pd.DataFrame()
    
    ban_week_fp_copy = pd.DataFrame()    
    ban_week_mp_copy = pd.DataFrame()    
    ban_week_bp_copy = pd.DataFrame()

    ban_week_fp2 = pd.DataFrame({'Banner': pd.Series([], dtype='object'),'Week': pd.Series([], dtype='object'),'ban_week_fp_check': pd.Series([], dtype='object')})
    ban_week_mp2 = pd.DataFrame({'Banner': pd.Series([], dtype='object'),'Week': pd.Series([], dtype='object'),'ban_week_mp_check': pd.Series([], dtype='object')})
    ban_week_bp2 = pd.DataFrame({'Banner': pd.Series([], dtype='object'),'Week': pd.Series([], dtype='object')})

    front_page_promo_weeks_all = pd.DataFrame()
    middle_page_promo_weeks_all = pd.DataFrame() 
    back_page_promo_weeks_all = pd.DataFrame()
    Test_results_sim = pd.DataFrame()
    
    random.seed(0)
   
    #########################################################################################################
    # Normal Simulation at banner-week-product level
    #########################################################################################################
    
    if len(qtr_edv_baseline) != 0 :  # In case there is not product present or product not present in selected region-channel & test period
        
        if toggle_ensemble_normal.value != 'Multiple Product': # In case of normal approach
            prod_ensemble_list = [product_fil]
        
        for product_fil in sorted(prod_ensemble_list): # Loop over product_fil
            ban_week_fp = pd.DataFrame({'Banner': pd.Series([], dtype='object'),'Week': pd.Series([], dtype='object'),'ban_week_fp_check': pd.Series([], dtype='object')})
            ban_week_mp = pd.DataFrame({'Banner': pd.Series([], dtype='object'),'Week': pd.Series([], dtype='object'),'ban_week_mp_check': pd.Series([], dtype='object')})
            ban_week_bp = pd.DataFrame({'Banner': pd.Series([], dtype='object'),'Week': pd.Series([], dtype='object')})
            
            if (promo_change_toggle.value == 'Yes (All Banners)') | (promo_change_toggle.value == 'Yes (Specific Banners)'):
                if (front_change.children[1].value != 0) | (middle_change.children[1].value != 0) | (back_change.children[1].value != 0) | (front_change2.children[1].value != 0) | (middle_change2.children[1].value != 0) | (back_change2.children[1].value != 0) | (front_change3.children[1].value != 0) | (middle_change3.children[1].value != 0) | (back_change3.children[1].value != 0) : 

                    display(Markdown('<div><div class="loader"></div><h2> &nbsp;Updating Promotions </h2></div>'))

                    if toggle_ensemble_normal.value != 'Multiple Product': # In case of normal approach
                        if toggle_np2.value == 'Yes':
                            iter_change_list = [1,2]
                        elif toggle_np2.value == 'No':
                            iter_change_list = [1]
                    else:
                        iter_change_list = [1]

                    for i in iter_change_list:
                        #Defining empty dataframes for model training
                        combined_dataset = pd.DataFrame({'Week': pd.Series([], dtype='object')})
                        Test_results = pd.DataFrame({'Region': pd.Series([], dtype='object')})
                        Test_Data = pd.DataFrame()

                        #Model training and prediction
                        for Region_key in Region_List:
                            # In case the product is not present at that region
                            if len(Volume_dataset_all_reg[(Volume_dataset_all_reg.Product == product_fil) & (Volume_dataset_all_reg.Region == Region_key)]) == 0:
                                continue

                            Volume_dataset = Volume_dataset_all_reg.loc[(Volume_dataset_all_reg["Region"] == Region_key)
                                                                       &(Volume_dataset_all_reg["Channel"].isin(channel_list))]
                            Volume_dataset['Pantry2'] = Volume_dataset['Pantry2'].fillna(0)
                            required_columns = ['Week','Week.Name','Banner','Product','Channel','Pack.Subtype','PACK_CONTENT','Adcal_Price','EDV.Price','Adcal_DD','Eq.Unit.Sales','Front.Page','Middle.Page','Back.Page','seasonality_index','SIZE_ML','COUNT','No_of_brands','No_of_flavors','No_of_sweetners','No_of_types','Category','Pantry1','Pantry2','Holiday.Week','Pre.Holiday.Week','Post.Holiday.Week']
                            combined_dataset = Volume_dataset[required_columns]

                            combined_dataset = combined_dataset.loc[((combined_dataset["Week"] >= "2017-01") & 
                                                                     (combined_dataset["Week"] <= "2019-52"))]

                            combined_dataset["Christmas_flag"] = [1 if x=="Christmas" else 0 for x in combined_dataset["Week.Name"]]
                            combined_dataset["Easter_flag"] = [1 if x=="Easter" else 0 for x in combined_dataset["Week.Name"]]

                            ## CHECKING if banner dummies are not present in required region, channel then introduce banner dummies
                            banner_dummies = pd.get_dummies(combined_dataset.Banner)              

                            # Uploaded data banner dummies
                            banner_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key].Banner)                                 
                            # If all banners are not present for the training data
                            c = [i for i in list(banner_dummies_check.columns) if i not in list(banner_dummies.columns)]
                            if len(c) != 0:
                                banner_dummies[c] = 0

                            # Reordering columns                
                            banner_dummies = banner_dummies.reindex(sorted(banner_dummies.columns), axis=1)                
                            combined_dataset = pd.concat([combined_dataset, banner_dummies], axis=1)

                            ## Not adding Product dummies since for New Product Simulator other Product dummies are not significant
                            combined_dataset = combined_dataset.loc[combined_dataset["Eq.Unit.Sales"].notnull()]

                            combined_dataset.sort_values('Week',inplace = True)
                            combined_dataset = pd.merge(combined_dataset,new_prod_stage,on = ['Product','Week'],how = 'left')
                            combined_dataset['Intial_weeks'] = combined_dataset['Intial_weeks'].fillna('Stabilization')
                            combined_dataset.drop(columns='Unnamed: 0', inplace=True)

                            #Adding dummies for category
                            category_dummies = pd.get_dummies(combined_dataset['Category'])

                            # Adding dummies for product attribute - Category
                            pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
                            # Uploaded data category dummies
                            category_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['Category'])                                 
                            # If all pack subtypes are not present for the training data
                            c = [i for i in list(category_dummies_check.columns) if i not in list(category_dummies.columns)]
                            if len(c) != 0:
                                category_dummies[c] = 0

                            # Reordering columns                
                            category_dummies = category_dummies.reindex(sorted(category_dummies.columns), axis=1)                
                            combined_dataset = pd.concat([combined_dataset, category_dummies], axis=1)
                            combined_dataset.drop(["Category"], inplace=True, axis=1)

                            # Adding dummies for product attribute - pack subtype
                            pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
                            # Uploaded data pack subtype dummies
                            pack_subtypes_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['Pack.Subtype'])                                 
                            # If all pack subtypes are not present for the training data
                            c = [i for i in list(pack_subtypes_dummies_check.columns) if i not in list(pack_subtypes_dummies.columns)]
                            if len(c) != 0:
                                pack_subtypes_dummies[c] = 0

                            # Reordering columns                
                            pack_subtypes_dummies = pack_subtypes_dummies.reindex(sorted(pack_subtypes_dummies.columns), axis=1)                
                            combined_dataset = pd.concat([combined_dataset, pack_subtypes_dummies], axis=1)
                            combined_dataset.drop(["Pack.Subtype"], inplace=True, axis=1)

                            # Adding pack content dummies
                            pack_content_dummies = pd.get_dummies(combined_dataset['PACK_CONTENT'])
                            # Uploaded data pack content dummies
                            pack_content_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['PACK_CONTENT'])                                 
                            # If all pack contents are not present for the training data
                            c = [i for i in list(pack_content_dummies_check.columns) if i not in list(pack_content_dummies.columns)]
                            if len(c) != 0:
                                pack_content_dummies[c] = 0

                            # Reordering columns                
                            pack_content_dummies = pack_content_dummies.reindex(sorted(pack_content_dummies.columns), axis=1)                
                            combined_dataset = pd.concat([combined_dataset, pack_content_dummies], axis=1)
                            combined_dataset.drop(["PACK_CONTENT"], inplace=True, axis=1)

                            combined_dataset_test = combined_dataset.copy()

                            # Product stage dummies
                            stage_dummies = pd.get_dummies(combined_dataset['Intial_weeks'])
                            combined_dataset = pd.concat([combined_dataset, stage_dummies], axis=1)
                            combined_dataset.drop(["Intial_weeks"], inplace=True, axis=1)

                            # Changing product stage as per start date provided by the user
                            prod_stage_data = prod_stage_check(product_fil)            

                            # Getting updated product stages
                            combined_dataset_test_check = combined_dataset_test[combined_dataset_test.Product == product_fil].merge(prod_stage_data[['Week','Initial_weeks1']], how = 'left')
                            combined_dataset_test_check['Initial_weeks1'] = combined_dataset_test_check['Initial_weeks1'].fillna(combined_dataset_test_check['Intial_weeks'])
                            combined_dataset_test_check.drop(columns = {'Intial_weeks'}, inplace = True)
                            combined_dataset_test_check.rename(columns = {'Initial_weeks1':'Intial_weeks'}, inplace = True)

                            combined_dataset_test = combined_dataset_test_check.copy()

                            # Product stage dummies
                            stage_dummies_test = pd.get_dummies(combined_dataset_test['Intial_weeks'])

                            # If all product stages are not present for the uploaded data
                            c = [i for i in list(stage_dummies.columns) if i not in list(stage_dummies_test.columns)]
                            if len(c) != 0 :
                                stage_dummies_test[c] = 0

                            # Reordering columns                
                            stage_dummies_test = stage_dummies_test.reindex(sorted(stage_dummies_test.columns), axis=1)
                            combined_dataset_test = pd.concat([combined_dataset_test, stage_dummies_test], axis=1)
                            combined_dataset_test.drop(["Intial_weeks"], inplace=True, axis=1)

                            # CHANGES : For adcal discounts
                            if 'All' in list(test_period.value):
                                # Discount change
                                if i == 1:                
                                    combined_dataset_test['Adcal_DD'] = (1+disc_change_fil)*combined_dataset_test['Adcal_DD']
                                    combined_dataset_test.loc[:,'EDV.Price'] = (1+base_price_change_fil)*combined_dataset_test.loc[:,'EDV.Price']
                                elif i == 2:
                                    combined_dataset_test['Adcal_DD'] = (1+disc_change_fil2)*combined_dataset_test['Adcal_DD']
                                    combined_dataset_test.loc[:,'EDV.Price'] = (1+base_price_change_fil2)*combined_dataset_test.loc[:,'EDV.Price']          
                            else:                
                                if i == 1:                
                                    # Discount change
                                    combined_dataset_test.loc[combined_dataset_test['Test_period'].isin(test_period_fil),'Adcal_DD'] = (1+disc_change_fil)*combined_dataset_test[combined_dataset_test['Test_period'].isin(test_period_fil)]['Adcal_DD']
                                    # Baseline pricing
                                    combined_dataset_test.loc[(combined_dataset_test['Test_period'].isin(test_period_fil)),'EDV.Price'] = (1+base_price_change_fil)*combined_dataset_test[(combined_dataset_test['Test_period'].isin(test_period_fil))]['EDV.Price']
                                elif i == 2:
                                    # Discount change
                                    combined_dataset_test.loc[combined_dataset_test['Test_period'].isin(test_period_fil2),'Adcal_DD'] = (1+disc_change_fil2)*combined_dataset_test[combined_dataset_test['Test_period'].isin(test_period_fil2)]['Adcal_DD']             
                                    # Baseline pricing
                                    combined_dataset_test.loc[(combined_dataset_test['Test_period'].isin(test_period_fil2)),'EDV.Price'] = (1+base_price_change_fil2)*combined_dataset_test[(combined_dataset_test['Test_period'].isin(test_period_fil2))]['EDV.Price']


                            #Adding discount depth columns
                            combined_dataset["DD_1"] = [1 if (0.25> x >0.1)
                                                  else 0 for x in combined_dataset["Adcal_DD"]]
                            combined_dataset["DD_2"] = [1 if (x>=0.25)
                                                             else 0 for x in combined_dataset["Adcal_DD"]]

                            combined_dataset_test["DD_1"] = [1 if (0.25> x >0.1)
                                                  else 0 for x in combined_dataset_test["Adcal_DD"]]
                            combined_dataset_test["DD_2"] = [1 if (x>=0.25)
                                                             else 0 for x in combined_dataset_test["Adcal_DD"]]            

                            # Adcal Price calculation
                            # In case the discounts go over 100%, limit to 100%
                            combined_dataset_test.loc[combined_dataset_test.Adcal_DD >= 1, 'Adcal_DD'] = 0.99

                            combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset_test.loc[combined_dataset_test['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset_test.loc[combined_dataset_test['Adcal_DD']>=0.1,'EDV.Price']
                            # Adcal_DD < 10 then Adcal Price = EDV Price            
                            combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] < 0.1, 'EDV.Price']

                            # Adcal Price calculation
                            combined_dataset.loc[combined_dataset['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'EDV.Price']
                            # Adcal_DD < 10 then Adcal Price = EDV Price            
                            combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'EDV.Price']

                            # Dropping adcal DD as DD1 & DD2 are added
                            combined_dataset.drop(["Adcal_DD"], inplace=True, axis=1)
                            combined_dataset_test.drop(["Adcal_DD"], inplace=True, axis=1)

                            # Dropping null volumes
                            complete_dataset = combined_dataset.copy()
                            complete_dataset_test = combined_dataset_test.copy()

                            complete_dataset = complete_dataset.loc[complete_dataset["Eq.Unit.Sales"].notnull()]
                            complete_dataset_test = complete_dataset_test.loc[complete_dataset_test["Eq.Unit.Sales"].notnull()]

                            complete_dataset['Adcal_Price'] = np.log(complete_dataset['Adcal_Price'])
                            complete_dataset['Eq.Unit.Sales'] = np.log(complete_dataset['Eq.Unit.Sales'])

                            complete_dataset_test['Adcal_Price'] = np.log(complete_dataset_test['Adcal_Price'])
                            complete_dataset_test['Eq.Unit.Sales'] = np.log(complete_dataset_test['Eq.Unit.Sales'])

                            # TEST PERIOD LOOP
                            Test_periods = ['2019Q1','2019Q2','2019Q3','2019Q4']

                            for Test_period in Test_periods:
                                if (Test_period>end_week.children[8].value) | (Test_period > Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Product == product_fil, 'Test_period'].max()):
                                    continue
                                if (Test_period<start_week.children[8].value) | (Test_period < Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Product == product_fil, 'Test_period'].min()):
                                    continue

                                #Filtering test weeks
                                Test_weeks = Test_week_list[Test_period]

                                # Dividing train & test data
                                complete_train_data_set = complete_dataset.loc[~complete_dataset["Week"].isin(Test_weeks)].reset_index(drop=True)
                                complete_test_data_set = complete_dataset_test.loc[complete_dataset_test["Week"].isin(Test_weeks)].reset_index(drop=True)

                                # Dropping unneccessary columns from train dataset
                                train_data_set = complete_train_data_set.copy()

                                train_data_brand = train_data_set.copy()
                                train_data_brand.drop(["Week","Week.Name","Banner","Product","Channel","EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)

                                # Filtering for existing product
                                test_data_set = complete_test_data_set[complete_test_data_set.Product == product_fil].copy()

                                # New Category
                                if i == 1:
                                    for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)].Category.unique():
                                        if (cols == category_drop.value):                      
                                            test_data_set[cols] = 1
                                        else:
                                            test_data_set[cols] = 0
                                if i == 2:
                                    for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)].Category.unique():
                                        if (cols == category_drop2.value):                      
                                            test_data_set[cols] = 1
                                        else:
                                            test_data_set[cols] = 0

                                # New Pack Subtype
                                if i == 1:
                                    for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['Pack.Subtype'].unique():
                                        if (cols == pack_subtype_drop.value):                      
                                            test_data_set[cols] = 1
                                        else:
                                            test_data_set[cols] = 0
                                if i == 2:
                                    for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['Pack.Subtype'].unique():
                                        if (cols == pack_subtype_drop2.value):                      
                                            test_data_set[cols] = 1
                                        else:
                                            test_data_set[cols] = 0

                                # New Pack Content
                                if i == 1:
                                    for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['PACK_CONTENT'].unique():
                                        if (cols == pack_content_drop.value):                      
                                            test_data_set[cols] = 1
                                        else:
                                            test_data_set[cols] = 0
                                if i == 2:
                                    for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['PACK_CONTENT'].unique():
                                        if (cols == pack_content_drop2.value):                      
                                            test_data_set[cols] = 1
                                        else:
                                            test_data_set[cols] = 0

                                test_data_brand = test_data_set.copy()

                                if i == 1 :
                                    # For New SIZE
                                    test_data_brand['SIZE_ML'] = size.value

                                    # For New Count
                                    test_data_brand['COUNT'] = count.value
                                elif i == 2:
                                    # For New SIZE
                                    test_data_brand['SIZE_ML'] = size2.value

                                    # For New Count
                                    test_data_brand['COUNT'] = count2.value

                                # Dropping unneccessary columns from test dataset
                                test_data_brand.drop(["Week","Week.Name","Banner","Product","Channel","EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)

                                #Creating test and train data - independent & dependent columns
                                X_train = train_data_brand
                                y_train = train_data_set["Eq.Unit.Sales"]
                                X_test = test_data_brand
                                y_test = test_data_set["Eq.Unit.Sales"]

                                # Appending test data
                                test_data_set["Region"] = Region_key
                                test_data_set["Test_period"] = Test_period
                                # In case the product is not present in certain region-quarter
                                if len(test_data_set) == 0:
                                    continue

                                Test_Data = Test_Data.append(test_data_set, ignore_index = True)

                                if((Region_key == 'EAST') and (Test_period == '2019Q1')):
                                        rf = model_object_EAST_2019Q1_v2
                                elif((Region_key == 'EAST') and (Test_period == '2019Q2')):
                                        rf = model_object_EAST_2019Q2_v2
                                elif((Region_key == 'EAST') and (Test_period == '2019Q3')):
                                        rf = model_object_EAST_2019Q3_v2
                                elif((Region_key == 'EAST') and (Test_period == '2019Q4')):
                                        rf = model_object_EAST_2019Q4_v2
                                elif((Region_key == 'ONTARIO') and (Test_period == '2019Q1')):
                                        rf = model_object_ONTARIO_2019Q1_v2
                                elif((Region_key == 'ONTARIO') and (Test_period == '2019Q2')):
                                        rf = model_object_ONTARIO_2019Q2_v2
                                elif((Region_key == 'ONTARIO') and (Test_period == '2019Q3')):
                                        rf = model_object_ONTARIO_2019Q3_v2
                                elif((Region_key == 'ONTARIO') and (Test_period == '2019Q4')):
                                        rf = model_object_ONTARIO_2019Q4_v2
                                elif((Region_key == 'QUEBEC') and (Test_period == '2019Q1')):
                                        rf = model_object_QUEBEC_2019Q1_v2
                                elif((Region_key == 'QUEBEC') and (Test_period == '2019Q2')):
                                        rf = model_object_QUEBEC_2019Q2_v2
                                elif((Region_key == 'QUEBEC') and (Test_period == '2019Q3')):
                                        rf = model_object_QUEBEC_2019Q3_v2
                                elif((Region_key == 'QUEBEC') and (Test_period == '2019Q4')):
                                        rf = model_object_QUEBEC_2019Q4_v2
                                elif((Region_key == 'WEST') and (Test_period == '2019Q1')):
                                        rf = model_object_WEST_2019Q1_v2
                                elif((Region_key == 'WEST') and (Test_period == '2019Q2')):
                                        rf = model_object_WEST_2019Q2_v2
                                elif((Region_key == 'WEST') and (Test_period == '2019Q3')):
                                        rf = model_object_WEST_2019Q3_v2
                                else:
                                        rf = model_object_WEST_2019Q4_v2

                                #Model predictions
                                predictions_rf = rf.predict(X_test)                

                                #Storing test results
                                result = X_test.copy()
                                result["Predictions_rf"] = predictions_rf
                                result["Actuals"] = y_test

                                result['Predictions_rf'] = np.exp(result['Predictions_rf'])                    
                                result['Actuals'] = np.exp(result['Actuals'])                                                    

                                #Calculating APE for the models
                                result["ape_rf"] = (result["Predictions_rf"]-result["Actuals"]).abs()

                                result["Product"] = test_data_set["Product"]
                                result["Region"] = Region_key
                                result["Test_period"] = Test_period
                                result["Banner"] = test_data_set["Banner"]
                                result["Week"] = test_data_set["Week"]
                                result["DD_1"] = test_data_set['DD_1']
                                result["DD_2"] = test_data_set['DD_2']
                                result["Front.Page"] = test_data_set["Front.Page"]
                                result["Middle.Page"] = test_data_set["Middle.Page"]
                                result["Back.Page"] = test_data_set["Back.Page"]
                                result["Channel"] = test_data_set['Channel']
                                result["iter"] = i
                                ##Columns for Revenue                            
                                result["Adcal_Price"] = test_data_set["Adcal_Price"]

                                result = result[["Region","Banner","Product",'Channel',"Test_period","Week",'Front.Page','Middle.Page','Back.Page',"Actuals","ape_rf","Predictions_rf","DD_1","DD_2","Adcal_Price",'iter']]

                                #Appending results to the final results dataframe
                                Test_results = Test_results.append(result, ignore_index=True)

                            Test_results_sim = Test_results.rename(columns = {'Predictions_rf':'Baseline Prediction'})

                ##########################################################################################################   
                                                #  FRONT PAGE INCREASE SIMULATION 
                ##########################################################################################################    
                        if i == 1:
                            change_fp = front_change.children[1].value
                        elif i == 2:
                            change_fp = front_change2.children[1].value
                       
                        if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
                            if product_fil == edv_disc_join['Existing Model Pack'].unique()[0]:
                                change_fp = front_change.children[1].value
                            elif product_fil == edv_disc_join['Existing Model Pack'].unique()[1]:
                                change_fp = front_change2.children[1].value
                            elif product_fil == edv_disc_join['Existing Model Pack'].unique()[2]:
                                change_fp = front_change3.children[1].value

                        if change_fp > 0:    

                            ################## FRONT PAGE SIMULATION ##################
                            import warnings
                            warnings.filterwarnings('ignore')

                            # Defining empty dataframes for model training
                            All_coefs = pd.DataFrame({'Region': pd.Series([], dtype='object')})
                            Train_results = pd.DataFrame({'Region': pd.Series([], dtype='object')})
                            Test_results = pd.DataFrame({'Region': pd.Series([], dtype='object')})
                            train_data_brand = pd.DataFrame({'Week': pd.Series([], dtype='object')})
                            combined_dataset = pd.DataFrame({'Week': pd.Series([], dtype='object')})
                            Test_Data = pd.DataFrame()    

                            #Model training and prediction
                            for Region_key in Region_List:
                                # In case the product is not present at that region
                                if len(Volume_dataset_all_reg[(Volume_dataset_all_reg.Product == product_fil) & (Volume_dataset_all_reg.Region == Region_key)]) == 0:
                                    continue

                                Volume_dataset = Volume_dataset_all_reg.loc[(Volume_dataset_all_reg["Region"] == Region_key)
                                                                           &(Volume_dataset_all_reg["Channel"].isin(channel_list))]
                                Volume_dataset['Pantry2'] = Volume_dataset['Pantry2'].fillna(0)
                                required_columns = ['Week','Week.Name','Banner','Product','Channel','Pack.Subtype','PACK_CONTENT','Adcal_Price','EDV.Price','Adcal_DD','Eq.Unit.Sales','Front.Page','Middle.Page','Back.Page','seasonality_index','SIZE_ML','COUNT','No_of_brands','No_of_flavors','No_of_sweetners','No_of_types','Category','Pantry1','Pantry2','Holiday.Week','Pre.Holiday.Week','Post.Holiday.Week']
                                combined_dataset = Volume_dataset[required_columns]

                                combined_dataset = combined_dataset.loc[((combined_dataset["Week"] >= "2017-01") & 
                                                                         (combined_dataset["Week"] <= "2019-52"))]

                                combined_dataset["Christmas_flag"] = [1 if x=="Christmas" else 0 for x in combined_dataset["Week.Name"]]
                                combined_dataset["Easter_flag"] = [1 if x=="Easter" else 0 for x in combined_dataset["Week.Name"]]

                                ## CHECKING if banner dummies are not present in required region, channel then introduce banner dummies
                                banner_dummies = pd.get_dummies(combined_dataset.Banner)              

                                # Uploaded data banner dummies
                                banner_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key].Banner)                                 
                                # If all banners are not present for the training data
                                c = [i for i in list(banner_dummies_check.columns) if i not in list(banner_dummies.columns)]
                                if len(c) != 0:
                                    banner_dummies[c] = 0

                                # Reordering columns                
                                banner_dummies = banner_dummies.reindex(sorted(banner_dummies.columns), axis=1)                
                                combined_dataset = pd.concat([combined_dataset, banner_dummies], axis=1)

                                ## Not adding Product dummies since for New Product Simulator other Product dummies are not significant
                                combined_dataset = combined_dataset.loc[combined_dataset["Eq.Unit.Sales"].notnull()]

                                combined_dataset.sort_values('Week',inplace = True)
                                combined_dataset = pd.merge(combined_dataset,new_prod_stage,on = ['Product','Week'],how = 'left')
                                combined_dataset['Intial_weeks'] = combined_dataset['Intial_weeks'].fillna('Stabilization')
                                combined_dataset.drop(columns='Unnamed: 0', inplace=True)

                                #Adding dummies for category
                                category_dummies = pd.get_dummies(combined_dataset['Category'])

                                # Adding dummies for product attribute - Category
                                pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
                                # Uploaded data category dummies
                                category_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['Category'])                                 
                                # If all pack subtypes are not present for the training data
                                c = [i for i in list(category_dummies_check.columns) if i not in list(category_dummies.columns)]
                                if len(c) != 0:
                                    category_dummies[c] = 0

                                # Reordering columns                
                                category_dummies = category_dummies.reindex(sorted(category_dummies.columns), axis=1)                
                                combined_dataset = pd.concat([combined_dataset, category_dummies], axis=1)
                                combined_dataset.drop(["Category"], inplace=True, axis=1)

                                # Adding dummies for product attribute - pack subtype
                                pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
                                # Uploaded data pack subtype dummies
                                pack_subtypes_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['Pack.Subtype'])                                 
                                # If all pack subtypes are not present for the training data
                                c = [i for i in list(pack_subtypes_dummies_check.columns) if i not in list(pack_subtypes_dummies.columns)]
                                if len(c) != 0:
                                    pack_subtypes_dummies[c] = 0

                                # Reordering columns                
                                pack_subtypes_dummies = pack_subtypes_dummies.reindex(sorted(pack_subtypes_dummies.columns), axis=1)                
                                combined_dataset = pd.concat([combined_dataset, pack_subtypes_dummies], axis=1)
                                combined_dataset.drop(["Pack.Subtype"], inplace=True, axis=1)

                                # Adding pack content dummies
                                pack_content_dummies = pd.get_dummies(combined_dataset['PACK_CONTENT'])
                                # Uploaded data pack content dummies
                                pack_content_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['PACK_CONTENT'])                                 
                                # If all pack contents are not present for the training data
                                c = [i for i in list(pack_content_dummies_check.columns) if i not in list(pack_content_dummies.columns)]
                                if len(c) != 0:
                                    pack_content_dummies[c] = 0

                                # Reordering columns                
                                pack_content_dummies = pack_content_dummies.reindex(sorted(pack_content_dummies.columns), axis=1)                
                                combined_dataset = pd.concat([combined_dataset, pack_content_dummies], axis=1)
                                combined_dataset.drop(["PACK_CONTENT"], inplace=True, axis=1)

                                combined_dataset_test = combined_dataset.copy()

                                # Product stage dummies
                                stage_dummies = pd.get_dummies(combined_dataset['Intial_weeks'])
                                combined_dataset = pd.concat([combined_dataset, stage_dummies], axis=1)
                                combined_dataset.drop(["Intial_weeks"], inplace=True, axis=1)

                                # Changing product stage as per start date provided by the user
                                prod_stage_data = prod_stage_check(product_fil)            

                                # Getting updated product stages
                                combined_dataset_test_check = combined_dataset_test[combined_dataset_test.Product == product_fil].merge(prod_stage_data[['Week','Initial_weeks1']], how = 'left')
                                combined_dataset_test_check['Initial_weeks1'] = combined_dataset_test_check['Initial_weeks1'].fillna(combined_dataset_test_check['Intial_weeks'])
                                combined_dataset_test_check.drop(columns = {'Intial_weeks'}, inplace = True)
                                combined_dataset_test_check.rename(columns = {'Initial_weeks1':'Intial_weeks'}, inplace = True)

                                combined_dataset_test = combined_dataset_test_check.copy()

                                # Product stage dummies
                                stage_dummies_test = pd.get_dummies(combined_dataset_test['Intial_weeks'])

                                # If all product stages are not present for the uploaded data
                                c = [i for i in list(stage_dummies.columns) if i not in list(stage_dummies_test.columns)]
                                if len(c) != 0 :
                                    stage_dummies_test[c] = 0

                                # Reordering columns                
                                stage_dummies_test = stage_dummies_test.reindex(sorted(stage_dummies_test.columns), axis=1)
                                combined_dataset_test = pd.concat([combined_dataset_test, stage_dummies_test], axis=1)
                                combined_dataset_test.drop(["Intial_weeks"], inplace=True, axis=1)

                                # CHANGES : For adcal discounts
                                if 'All' in list(test_period.value):
                                    # Discount change
                                    if i == 1:                
                                        combined_dataset_test['Adcal_DD'] = (1+disc_change_fil)*combined_dataset_test['Adcal_DD']
                                        combined_dataset_test.loc[:,'EDV.Price'] = (1+base_price_change_fil)*combined_dataset_test.loc[:,'EDV.Price']
                                    elif i == 2:
                                        combined_dataset_test['Adcal_DD'] = (1+disc_change_fil2)*combined_dataset_test['Adcal_DD']
                                        combined_dataset_test.loc[:,'EDV.Price'] = (1+base_price_change_fil2)*combined_dataset_test.loc[:,'EDV.Price']          
                                else:                
                                    if i == 1:                
                                        # Discount change
                                        combined_dataset_test.loc[combined_dataset_test['Test_period'].isin(test_period_fil),'Adcal_DD'] = (1+disc_change_fil)*combined_dataset_test[combined_dataset_test['Test_period'].isin(test_period_fil)]['Adcal_DD'] 
                                        # Baseline pricing
                                        combined_dataset_test.loc[(combined_dataset_test['Test_period'].isin(test_period_fil)),'EDV.Price'] = (1+base_price_change_fil)*combined_dataset_test[(combined_dataset_test['Test_period'].isin(test_period_fil))]['EDV.Price']
                                    elif i == 2:
                                        # Discount change
                                        combined_dataset_test.loc[combined_dataset_test['Test_period'].isin(test_period_fil2),'Adcal_DD'] = (1+disc_change_fil2)*combined_dataset_test[combined_dataset_test['Test_period'].isin(test_period_fil2)]['Adcal_DD']             
                                        # Baseline pricing
                                        combined_dataset_test.loc[(combined_dataset_test['Test_period'].isin(test_period_fil2)),'EDV.Price'] = (1+base_price_change_fil2)*combined_dataset_test[(combined_dataset_test['Test_period'].isin(test_period_fil2))]['EDV.Price']


                                #Adding discount depth columns
                                combined_dataset["DD_1"] = [1 if (0.25> x >0.1)
                                                      else 0 for x in combined_dataset["Adcal_DD"]]
                                combined_dataset["DD_2"] = [1 if (x>=0.25)
                                                                 else 0 for x in combined_dataset["Adcal_DD"]]

                                combined_dataset_test["DD_1"] = [1 if (0.25> x >0.1)
                                                      else 0 for x in combined_dataset_test["Adcal_DD"]]
                                combined_dataset_test["DD_2"] = [1 if (x>=0.25)
                                                                 else 0 for x in combined_dataset_test["Adcal_DD"]]            

                                # Adcal Price calculation
                                # In case the discounts go over 100%, limit to 100%
                                combined_dataset_test.loc[combined_dataset_test.Adcal_DD >= 1, 'Adcal_DD'] = 0.99

                                combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset_test.loc[combined_dataset_test['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset_test.loc[combined_dataset_test['Adcal_DD']>=0.1,'EDV.Price']
                                # Adcal_DD < 10 then Adcal Price = EDV Price            
                                combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] < 0.1, 'EDV.Price']

                                # Adcal Price calculation
                                combined_dataset.loc[combined_dataset['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'EDV.Price']
                                # Adcal_DD < 10 then Adcal Price = EDV Price            
                                combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'EDV.Price']

                                # Dropping adcal DD as DD1 & DD2 are added
                                combined_dataset.drop(["Adcal_DD"], inplace=True, axis=1)
                                combined_dataset_test.drop(["Adcal_DD"], inplace=True, axis=1)

                                # Dropping null volumes
                                complete_dataset = combined_dataset.copy()
                                complete_dataset_test = combined_dataset_test.copy()

                                complete_dataset = complete_dataset.loc[complete_dataset["Eq.Unit.Sales"].notnull()]
                                complete_dataset_test = complete_dataset_test.loc[complete_dataset_test["Eq.Unit.Sales"].notnull()]

                                complete_dataset['Adcal_Price'] = np.log(complete_dataset['Adcal_Price'])
                                complete_dataset['Eq.Unit.Sales'] = np.log(complete_dataset['Eq.Unit.Sales'])

                                complete_dataset_test['Adcal_Price'] = np.log(complete_dataset_test['Adcal_Price'])
                                complete_dataset_test['Eq.Unit.Sales'] = np.log(complete_dataset_test['Eq.Unit.Sales'])

                                # TEST PERIOD LOOP
                                Test_periods = ['2019Q1','2019Q2','2019Q3','2019Q4']

                                for Test_period in Test_periods:
                                    if (Test_period>end_week.children[8].value) | (Test_period > Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Product == product_fil, 'Test_period'].max()):
                                        continue
                                    if (Test_period<start_week.children[8].value) | (Test_period < Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Product == product_fil, 'Test_period'].min()):
                                        continue

                                    #Filtering test weeks
                                    Test_weeks = Test_week_list[Test_period]

                                    # Dividing train & test data
                                    complete_train_data_set = complete_dataset.loc[~complete_dataset["Week"].isin(Test_weeks)].reset_index(drop=True)
                                    complete_test_data_set = complete_dataset_test.loc[complete_dataset_test["Week"].isin(Test_weeks)].reset_index(drop=True)

                                    # Dropping unneccessary columns from train dataset
                                    train_data_set = complete_train_data_set.copy()

                                    train_data_brand = train_data_set.copy()
                                    train_data_brand.drop(["Week","Week.Name","Banner","Product","Channel","EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)

                                    # Filtering for existing product
                                    test_data_set = complete_test_data_set[complete_test_data_set.Product == product_fil].copy()

                                    #################### INCREASING PROMOTIONS FOR BANNER WEEKS COMBINATIONS ####################
                                    # Get banner week promos
                                    banner_week_promos_combo = test_data_set[['Banner','Week','Front.Page','Middle.Page','Back.Page']].drop_duplicates()             
                                    banner_week_upc = banner_week_promos_combo.copy()
                                    # Get rows where there is availability of increase in prmotions
                                    avail_promo_weeks = banner_week_upc[(banner_week_upc['Front.Page'] == 0) & (banner_week_upc['Middle.Page'] == 0) & (banner_week_upc['Back.Page'] == 0)]
                                    # Increasing 1 front page promotions for every week
                                    avail_promo_weeks['Front.Page'] = 1
                                    # Merge to get required increase in banner-week combintation
                                    test_data_set_promo_check = test_data_set.merge(avail_promo_weeks[['Banner','Week','Front.Page']], how = 'left', on = ['Banner','Week'])
                                    # Filling na with existing values in Front.Page_y
                                    test_data_set_promo_check['Front.Page_y'].fillna(test_data_set_promo_check['Front.Page_x'], inplace = True)
                                    # Replacing values of 'Front.Page_x' with 'Front.Page_y' wherever it was zero previously
                                    test_data_set_promo_check['Front.Page_x'] = np.where(test_data_set_promo_check['Front.Page_x'] == 0, test_data_set_promo_check['Front.Page_y'], test_data_set_promo_check['Front.Page_x'])
                                    # Drop 'Front.Page_y' 
                                    test_data_set_promo_check.drop('Front.Page_y', axis = 1, inplace =True)
                                    # Renaming 'Front.Page_x'
                                    test_data_set_promo_check.rename(columns = {'Front.Page_x':'Front.Page'}, inplace = True)

                                    test_data_set = test_data_set_promo_check.copy()   

                                    # New Category
                                    if i == 1:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)].Category.unique():
                                            if (cols == category_drop.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0
                                    if i == 2:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)].Category.unique():
                                            if (cols == category_drop2.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0                    

                                    # New Pack Subtype
                                    if i == 1:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['Pack.Subtype'].unique():
                                            if (cols == pack_subtype_drop.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0
                                    if i == 2:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['Pack.Subtype'].unique():
                                            if (cols == pack_subtype_drop2.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0

                                    # New Pack Content
                                    if i == 1:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['PACK_CONTENT'].unique():
                                            if (cols == pack_content_drop.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0
                                    if i == 2:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['PACK_CONTENT'].unique():
                                            if (cols == pack_content_drop2.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0

                                    test_data_brand = test_data_set.copy()

                                    if i == 1 :
                                        # For New SIZE
                                        test_data_brand['SIZE_ML'] = size.value

                                        # For New Count
                                        test_data_brand['COUNT'] = count.value
                                    elif i == 2:
                                        # For New SIZE
                                        test_data_brand['SIZE_ML'] = size2.value

                                        # For New Count
                                        test_data_brand['COUNT'] = count2.value

                                    # Dropping unneccessary columns from test dataset
                                    test_data_brand.drop(["Week","Week.Name","Banner","Product","Channel","EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)

                                    #Creating test and train data - independent & dependent columns
                                    X_train = train_data_brand
                                    y_train = train_data_set["Eq.Unit.Sales"]
                                    X_test = test_data_brand
                                    y_test = test_data_set["Eq.Unit.Sales"]

                                    # Appending test data
                                    test_data_set["Region"] = Region_key
                                    test_data_set["Test_period"] = Test_period
                                    # In case the product is not present in certain region-quarter
                                    if len(test_data_set) == 0:
                                        continue

                                    Test_Data = Test_Data.append(test_data_set, ignore_index = True)

                                    if((Region_key == 'EAST') and (Test_period == '2019Q1')):
                                            rf = model_object_EAST_2019Q1_v2
                                    elif((Region_key == 'EAST') and (Test_period == '2019Q2')):
                                            rf = model_object_EAST_2019Q2_v2
                                    elif((Region_key == 'EAST') and (Test_period == '2019Q3')):
                                            rf = model_object_EAST_2019Q3_v2
                                    elif((Region_key == 'EAST') and (Test_period == '2019Q4')):
                                            rf = model_object_EAST_2019Q4_v2
                                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q1')):
                                            rf = model_object_ONTARIO_2019Q1_v2
                                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q2')):
                                            rf = model_object_ONTARIO_2019Q2_v2
                                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q3')):
                                            rf = model_object_ONTARIO_2019Q3_v2
                                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q4')):
                                            rf = model_object_ONTARIO_2019Q4_v2
                                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q1')):
                                            rf = model_object_QUEBEC_2019Q1_v2
                                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q2')):
                                            rf = model_object_QUEBEC_2019Q2_v2
                                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q3')):
                                            rf = model_object_QUEBEC_2019Q3_v2
                                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q4')):
                                            rf = model_object_QUEBEC_2019Q4_v2
                                    elif((Region_key == 'WEST') and (Test_period == '2019Q1')):
                                            rf = model_object_WEST_2019Q1_v2
                                    elif((Region_key == 'WEST') and (Test_period == '2019Q2')):
                                            rf = model_object_WEST_2019Q2_v2
                                    elif((Region_key == 'WEST') and (Test_period == '2019Q3')):
                                            rf = model_object_WEST_2019Q3_v2
                                    else:
                                            rf = model_object_WEST_2019Q4_v2

                                    #Model predictions
                                    predictions_rf = rf.predict(X_test)                

                                    #Storing test results
                                    result = X_test.copy()
                                    result["Predictions_rf"] = predictions_rf
                                    result["Actuals"] = y_test

                                    result['Predictions_rf'] = np.exp(result['Predictions_rf'])                    
                                    result['Actuals'] = np.exp(result['Actuals'])                                                    

                                    result["Product"] = test_data_set["Product"]
                                    result["Region"] = Region_key
                                    result["Test_period"] = Test_period
                                    result["Banner"] = test_data_set["Banner"]
                                    result["Channel"] = test_data_set['Channel']
                                    result["Week"] = test_data_set["Week"]
                                    result['iter'] = i

                                    result = result[["Region","Banner","Product",'Channel',"Test_period","Week","Actuals","Predictions_rf",'iter']]

                                    #Appending results to the final results dataframe
                                    Test_results = Test_results.append(result, ignore_index=True)

                            Test_results = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)]
                            # Removing 0 promotion banners
                            if len(qtr_df_final[(qtr_df_final.Banner.isin(fp_remove_banner)) | (qtr_df_final.Banner.isin(mp_remove_banner)) | (qtr_df_final.Banner.isin(bp_remove_banner))]) != 0:
                                if zero_banner_promo_inc.value == 'No':
                                    Test_results = Test_results[~Test_results.Banner.isin(fp_remove_banner)]

                            # Merge & get prediction difference column
                            Test_results = Test_results[Test_results.iter == i].merge(Test_results_sim[Test_results_sim.iter == i][['Banner','Week','Baseline Prediction']], on = ['Banner','Week'], how = 'left')        
                            Test_results['Diff'] = Test_results[Test_results.iter == i]['Predictions_rf'] - Test_results[Test_results.iter == i]['Baseline Prediction']
                            # Consider positive difference only
                            Test_results = Test_results[Test_results.iter == i][Test_results[Test_results.iter == i].Diff > 0]
                            Test_results = Test_results[Test_results.iter == i].sort_values('Diff', ascending = False)

                            # Select required number of banner-weeks where promotion increase would occur
                            if (promo_change_toggle.value == 'Yes (Specific Banners)'):         
                                Test_results = Test_results[Test_results.Banner.isin(list(banner_select.value))]
                                for banner_loop in banner_select.value:    
                                    # Selecting banner-weeks
                                    ban_week_fp0 = Test_results[Test_results.Banner == banner_loop][:abs(int(change_fp))][['Banner','Week']]
                                    # if iter = 1, iter = 2 loop
                                    if i == 1:
                                        ban_week_fp = ban_week_fp.append(ban_week_fp0, ignore_index = True)
                                        # Banner-week concat column    
                                        ban_week_fp['ban_week_fp_check'] = ban_week_fp['Banner'] + ban_week_fp['Week']                            

                                    elif i == 2:
                                        ban_week_fp2 = ban_week_fp2.append(ban_week_fp0, ignore_index = True)
                                        # Banner-week concat column    
                                        ban_week_fp2['ban_week_fp_check'] = ban_week_fp2['Banner'] + ban_week_fp2['Week']                             

                            elif (promo_change_toggle.value == 'Yes (All Banners)'):
                                for banner_loop in qtr_df_final.Banner.unique():    
                                    # Selecting banner-weeks
                                    ban_week_fp0 = Test_results[Test_results.Banner == banner_loop][:abs(int(change_fp))][['Banner','Week']]
                                    # if iter = 1, iter = 2 loop
                                    if i == 1:
                                        ban_week_fp = ban_week_fp.append(ban_week_fp0, ignore_index = True)
                                        # Banner-week concat column    
                                        ban_week_fp['ban_week_fp_check'] = ban_week_fp['Banner'] + ban_week_fp['Week']                            
                                    elif i == 2:
                                        ban_week_fp2 = ban_week_fp2.append(ban_week_fp0, ignore_index = True)
                                        # Banner-week concat column    
                                        ban_week_fp2['ban_week_fp_check'] = ban_week_fp2['Banner'] + ban_week_fp2['Week']
                           
                            if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach                        
                                ban_week_fp['Product'] = product_fil
                            else:
                                if i == 1:
                                    ban_week_fp['Product'] = product_fil
                                elif i == 2:
                                    ban_week_fp2['Product'] = product_fil

                        ##########################################################################################################   
                                                        #  FRONT PAGE DECREASE SIMULATION 
                        ##########################################################################################################    

                        if i == 1:
                            change_fp = front_change.children[1].value
                        elif i == 2:
                            change_fp = front_change2.children[1].value

                        if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
                            if product_fil == edv_disc_join['Existing Model Pack'].unique()[0]:
                                change_fp = front_change.children[1].value
                            elif product_fil == edv_disc_join['Existing Model Pack'].unique()[1]:
                                change_fp = front_change2.children[1].value
                            elif product_fil == edv_disc_join['Existing Model Pack'].unique()[2]:
                                change_fp = front_change3.children[1].value

                        if (change_fp < 0) & ((qtr_df_final['Front Page']).sum() != 0):

                            import warnings
                            warnings.filterwarnings('ignore')

                            # Defining empty dataframes for model training
                            All_coefs = pd.DataFrame({'Region': pd.Series([], dtype='object')})
                            Train_results = pd.DataFrame({'Region': pd.Series([], dtype='object')})
                            Test_results = pd.DataFrame({'Region': pd.Series([], dtype='object')})
                            train_data_brand = pd.DataFrame({'Week': pd.Series([], dtype='object')})
                            combined_dataset = pd.DataFrame({'Week': pd.Series([], dtype='object')})
                            Test_Data = pd.DataFrame()    

                            #Model training and prediction
                            for Region_key in Region_List:
                                # In case the product is not present at that region
                                if len(Volume_dataset_all_reg[(Volume_dataset_all_reg.Product == product_fil) & (Volume_dataset_all_reg.Region == Region_key)]) == 0:
                                    continue

                                Volume_dataset = Volume_dataset_all_reg.loc[(Volume_dataset_all_reg["Region"] == Region_key)
                                                                           &(Volume_dataset_all_reg["Channel"].isin(channel_list))]
                                Volume_dataset['Pantry2'] = Volume_dataset['Pantry2'].fillna(0)
                                required_columns = ['Week','Week.Name','Banner','Product','Channel','Pack.Subtype','PACK_CONTENT','Adcal_Price','EDV.Price','Adcal_DD','Eq.Unit.Sales','Front.Page','Middle.Page','Back.Page','seasonality_index','SIZE_ML','COUNT','No_of_brands','No_of_flavors','No_of_sweetners','No_of_types','Category','Pantry1','Pantry2','Holiday.Week','Pre.Holiday.Week','Post.Holiday.Week']
                                combined_dataset = Volume_dataset[required_columns]

                                combined_dataset = combined_dataset.loc[((combined_dataset["Week"] >= "2017-01") & 
                                                                         (combined_dataset["Week"] <= "2019-52"))]

                                combined_dataset["Christmas_flag"] = [1 if x=="Christmas" else 0 for x in combined_dataset["Week.Name"]]
                                combined_dataset["Easter_flag"] = [1 if x=="Easter" else 0 for x in combined_dataset["Week.Name"]]

                                ## CHECKING if banner dummies are not present in required region, channel then introduce banner dummies
                                banner_dummies = pd.get_dummies(combined_dataset.Banner)              

                                # Uploaded data banner dummies
                                banner_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key].Banner)                                 
                                # If all banners are not present for the training data
                                c = [i for i in list(banner_dummies_check.columns) if i not in list(banner_dummies.columns)]
                                if len(c) != 0:
                                    banner_dummies[c] = 0

                                # Reordering columns                
                                banner_dummies = banner_dummies.reindex(sorted(banner_dummies.columns), axis=1)                
                                combined_dataset = pd.concat([combined_dataset, banner_dummies], axis=1)

                                ## Not adding Product dummies since for New Product Simulator other Product dummies are not significant
                                combined_dataset = combined_dataset.loc[combined_dataset["Eq.Unit.Sales"].notnull()]

                                combined_dataset.sort_values('Week',inplace = True)
                                combined_dataset = pd.merge(combined_dataset,new_prod_stage,on = ['Product','Week'],how = 'left')
                                combined_dataset['Intial_weeks'] = combined_dataset['Intial_weeks'].fillna('Stabilization')
                                combined_dataset.drop(columns='Unnamed: 0', inplace=True)

                                #Adding dummies for category
                                category_dummies = pd.get_dummies(combined_dataset['Category'])

                                # Adding dummies for product attribute - Category
                                pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
                                # Uploaded data category dummies
                                category_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['Category'])                                 
                                # If all pack subtypes are not present for the training data
                                c = [i for i in list(category_dummies_check.columns) if i not in list(category_dummies.columns)]
                                if len(c) != 0:
                                    category_dummies[c] = 0

                                # Reordering columns                
                                category_dummies = category_dummies.reindex(sorted(category_dummies.columns), axis=1)                
                                combined_dataset = pd.concat([combined_dataset, category_dummies], axis=1)
                                combined_dataset.drop(["Category"], inplace=True, axis=1)

                                # Adding dummies for product attribute - pack subtype
                                pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
                                # Uploaded data pack subtype dummies
                                pack_subtypes_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['Pack.Subtype'])                                 
                                # If all pack subtypes are not present for the training data
                                c = [i for i in list(pack_subtypes_dummies_check.columns) if i not in list(pack_subtypes_dummies.columns)]
                                if len(c) != 0:
                                    pack_subtypes_dummies[c] = 0

                                # Reordering columns                
                                pack_subtypes_dummies = pack_subtypes_dummies.reindex(sorted(pack_subtypes_dummies.columns), axis=1)                
                                combined_dataset = pd.concat([combined_dataset, pack_subtypes_dummies], axis=1)
                                combined_dataset.drop(["Pack.Subtype"], inplace=True, axis=1)

                                # Adding pack content dummies
                                pack_content_dummies = pd.get_dummies(combined_dataset['PACK_CONTENT'])
                                # Uploaded data pack content dummies
                                pack_content_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['PACK_CONTENT'])                                 
                                # If all pack contents are not present for the training data
                                c = [i for i in list(pack_content_dummies_check.columns) if i not in list(pack_content_dummies.columns)]
                                if len(c) != 0:
                                    pack_content_dummies[c] = 0

                                # Reordering columns                
                                pack_content_dummies = pack_content_dummies.reindex(sorted(pack_content_dummies.columns), axis=1)                
                                combined_dataset = pd.concat([combined_dataset, pack_content_dummies], axis=1)
                                combined_dataset.drop(["PACK_CONTENT"], inplace=True, axis=1)

                                combined_dataset_test = combined_dataset.copy()

                                # Product stage dummies
                                stage_dummies = pd.get_dummies(combined_dataset['Intial_weeks'])
                                combined_dataset = pd.concat([combined_dataset, stage_dummies], axis=1)
                                combined_dataset.drop(["Intial_weeks"], inplace=True, axis=1)

                                # Changing product stage as per start date provided by the user
                                prod_stage_data = prod_stage_check(product_fil)            

                                # Getting updated product stages
                                combined_dataset_test_check = combined_dataset_test[combined_dataset_test.Product == product_fil].merge(prod_stage_data[['Week','Initial_weeks1']], how = 'left')
                                combined_dataset_test_check['Initial_weeks1'] = combined_dataset_test_check['Initial_weeks1'].fillna(combined_dataset_test_check['Intial_weeks'])
                                combined_dataset_test_check.drop(columns = {'Intial_weeks'}, inplace = True)
                                combined_dataset_test_check.rename(columns = {'Initial_weeks1':'Intial_weeks'}, inplace = True)

                                combined_dataset_test = combined_dataset_test_check.copy()

                                # Product stage dummies
                                stage_dummies_test = pd.get_dummies(combined_dataset_test['Intial_weeks'])

                                # If all product stages are not present for the uploaded data
                                c = [i for i in list(stage_dummies.columns) if i not in list(stage_dummies_test.columns)]
                                if len(c) != 0 :
                                    stage_dummies_test[c] = 0

                                # Reordering columns                
                                stage_dummies_test = stage_dummies_test.reindex(sorted(stage_dummies_test.columns), axis=1)
                                combined_dataset_test = pd.concat([combined_dataset_test, stage_dummies_test], axis=1)
                                combined_dataset_test.drop(["Intial_weeks"], inplace=True, axis=1)

                                # CHANGES : For adcal discounts
                                if 'All' in list(test_period.value):
                                    # Discount change
                                    if i == 1:                
                                        combined_dataset_test['Adcal_DD'] = (1+disc_change_fil)*combined_dataset_test['Adcal_DD']
                                        combined_dataset_test.loc[:,'EDV.Price'] = (1+base_price_change_fil)*combined_dataset_test.loc[:,'EDV.Price']
                                    elif i == 2:
                                        combined_dataset_test['Adcal_DD'] = (1+disc_change_fil2)*combined_dataset_test['Adcal_DD']
                                        combined_dataset_test.loc[:,'EDV.Price'] = (1+base_price_change_fil2)*combined_dataset_test.loc[:,'EDV.Price']          
                                else:                
                                    if i == 1:                
                                        # Discount change
                                        combined_dataset_test.loc[combined_dataset_test['Test_period'].isin(test_period_fil),'Adcal_DD'] = (1+disc_change_fil)*combined_dataset_test[combined_dataset_test['Test_period'].isin(test_period_fil)]['Adcal_DD'] 
                                        # Baseline pricing
                                        combined_dataset_test.loc[(combined_dataset_test['Test_period'].isin(test_period_fil)),'EDV.Price'] = (1+base_price_change_fil)*combined_dataset_test[(combined_dataset_test['Test_period'].isin(test_period_fil))]['EDV.Price']
                                    elif i == 2:
                                        # Discount change
                                        combined_dataset_test.loc[combined_dataset_test['Test_period'].isin(test_period_fil2),'Adcal_DD'] = (1+disc_change_fil2)*combined_dataset_test[combined_dataset_test['Test_period'].isin(test_period_fil2)]['Adcal_DD']             
                                        # Baseline pricing
                                        combined_dataset_test.loc[(combined_dataset_test['Test_period'].isin(test_period_fil2)),'EDV.Price'] = (1+base_price_change_fil2)*combined_dataset_test[(combined_dataset_test['Test_period'].isin(test_period_fil2))]['EDV.Price']


                                #Adding discount depth columns
                                combined_dataset["DD_1"] = [1 if (0.25> x >0.1)
                                                      else 0 for x in combined_dataset["Adcal_DD"]]
                                combined_dataset["DD_2"] = [1 if (x>=0.25)
                                                                 else 0 for x in combined_dataset["Adcal_DD"]]

                                combined_dataset_test["DD_1"] = [1 if (0.25> x >0.1)
                                                      else 0 for x in combined_dataset_test["Adcal_DD"]]
                                combined_dataset_test["DD_2"] = [1 if (x>=0.25)
                                                                 else 0 for x in combined_dataset_test["Adcal_DD"]]            

                                # Adcal Price calculation
                                # In case the discounts go over 100%, limit to 100%
                                combined_dataset_test.loc[combined_dataset_test.Adcal_DD >= 1, 'Adcal_DD'] = 0.99

                                combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset_test.loc[combined_dataset_test['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset_test.loc[combined_dataset_test['Adcal_DD']>=0.1,'EDV.Price']
                                # Adcal_DD < 10 then Adcal Price = EDV Price            
                                combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] < 0.1, 'EDV.Price']

                                # Adcal Price calculation
                                combined_dataset.loc[combined_dataset['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'EDV.Price']
                                # Adcal_DD < 10 then Adcal Price = EDV Price            
                                combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'EDV.Price']

                                # Dropping adcal DD as DD1 & DD2 are added
                                combined_dataset.drop(["Adcal_DD"], inplace=True, axis=1)
                                combined_dataset_test.drop(["Adcal_DD"], inplace=True, axis=1)

                                # Dropping null volumes
                                complete_dataset = combined_dataset.copy()
                                complete_dataset_test = combined_dataset_test.copy()

                                complete_dataset = complete_dataset.loc[complete_dataset["Eq.Unit.Sales"].notnull()]
                                complete_dataset_test = complete_dataset_test.loc[complete_dataset_test["Eq.Unit.Sales"].notnull()]

                                complete_dataset['Adcal_Price'] = np.log(complete_dataset['Adcal_Price'])
                                complete_dataset['Eq.Unit.Sales'] = np.log(complete_dataset['Eq.Unit.Sales'])

                                complete_dataset_test['Adcal_Price'] = np.log(complete_dataset_test['Adcal_Price'])
                                complete_dataset_test['Eq.Unit.Sales'] = np.log(complete_dataset_test['Eq.Unit.Sales'])

                                # TEST PERIOD LOOP
                                Test_periods = ['2019Q1','2019Q2','2019Q3','2019Q4']

                                for Test_period in Test_periods:
                                    if (Test_period>end_week.children[8].value) | (Test_period > Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Product == product_fil, 'Test_period'].max()):
                                        continue
                                    if (Test_period<start_week.children[8].value) | (Test_period < Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Product == product_fil, 'Test_period'].min()):
                                        continue

                                    #Filtering test weeks
                                    Test_weeks = Test_week_list[Test_period]

                                    # Dividing train & test data
                                    complete_train_data_set = complete_dataset.loc[~complete_dataset["Week"].isin(Test_weeks)].reset_index(drop=True)
                                    complete_test_data_set = complete_dataset_test.loc[complete_dataset_test["Week"].isin(Test_weeks)].reset_index(drop=True)

                                    # Dropping unneccessary columns from train dataset
                                    train_data_set = complete_train_data_set.copy()

                                    train_data_brand = train_data_set.copy()
                                    train_data_brand.drop(["Week","Week.Name","Banner","Product","Channel","EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)

                                    # Filtering for existing product
                                    test_data_set = complete_test_data_set[complete_test_data_set.Product == product_fil].copy()

                                    #################### DECREASING PROMOTIONS FOR BANNER WEEKS COMBINATIONS ####################
                                    # Allotting all front page promos as 0
                                    front_page_promo_weeks = pd.DataFrame(test_data_set[test_data_set['Front.Page'] != 0][['Banner','Week']])
                                    front_page_promo_weeks_all = front_page_promo_weeks_all.append(front_page_promo_weeks, ignore_index = True)
                                    test_data_set['Front.Page'] = 0

                                    # New Category
                                    if i == 1:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)].Category.unique():
                                            if (cols == category_drop.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0
                                    if i == 2:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)].Category.unique():
                                            if (cols == category_drop2.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0


                                    # New Pack Subtype
                                    if i == 1:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['Pack.Subtype'].unique():
                                            if (cols == pack_subtype_drop.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0
                                    if i == 2:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['Pack.Subtype'].unique():
                                            if (cols == pack_subtype_drop2.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0

                                    # New Pack Content
                                    if i == 1:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['PACK_CONTENT'].unique():
                                            if (cols == pack_content_drop.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0
                                    if i == 2:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['PACK_CONTENT'].unique():
                                            if (cols == pack_content_drop2.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0

                                    test_data_brand = test_data_set.copy()                        

                                    if i == 1 :
                                        # For New SIZE
                                        test_data_brand['SIZE_ML'] = size.value

                                        # For New Count
                                        test_data_brand['COUNT'] = count.value
                                    elif i == 2:
                                        # For New SIZE
                                        test_data_brand['SIZE_ML'] = size2.value

                                        # For New Count
                                        test_data_brand['COUNT'] = count2.value

                                    # Dropping unneccessary columns from test dataset
                                    test_data_brand.drop(["Week","Week.Name","Banner","Product","Channel","EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)

                                    #Creating test and train data - independent & dependent columns
                                    X_train = train_data_brand
                                    y_train = train_data_set["Eq.Unit.Sales"]
                                    X_test = test_data_brand
                                    y_test = test_data_set["Eq.Unit.Sales"]

                                    # Appending test data
                                    test_data_set["Region"] = Region_key
                                    test_data_set["Test_period"] = Test_period

                                    Test_Data = Test_Data.append(test_data_set, ignore_index = True)
                                    # In case the product is not present in certain region-quarter
                                    if len(test_data_set) == 0:
                                        continue

                                    if((Region_key == 'EAST') and (Test_period == '2019Q1')):
                                            rf = model_object_EAST_2019Q1_v2
                                    elif((Region_key == 'EAST') and (Test_period == '2019Q2')):
                                            rf = model_object_EAST_2019Q2_v2
                                    elif((Region_key == 'EAST') and (Test_period == '2019Q3')):
                                            rf = model_object_EAST_2019Q3_v2
                                    elif((Region_key == 'EAST') and (Test_period == '2019Q4')):
                                            rf = model_object_EAST_2019Q4_v2
                                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q1')):
                                            rf = model_object_ONTARIO_2019Q1_v2
                                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q2')):
                                            rf = model_object_ONTARIO_2019Q2_v2
                                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q3')):
                                            rf = model_object_ONTARIO_2019Q3_v2
                                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q4')):
                                            rf = model_object_ONTARIO_2019Q4_v2
                                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q1')):
                                            rf = model_object_QUEBEC_2019Q1_v2
                                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q2')):
                                            rf = model_object_QUEBEC_2019Q2_v2
                                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q3')):
                                            rf = model_object_QUEBEC_2019Q3_v2
                                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q4')):
                                            rf = model_object_QUEBEC_2019Q4_v2
                                    elif((Region_key == 'WEST') and (Test_period == '2019Q1')):
                                            rf = model_object_WEST_2019Q1_v2
                                    elif((Region_key == 'WEST') and (Test_period == '2019Q2')):
                                            rf = model_object_WEST_2019Q2_v2
                                    elif((Region_key == 'WEST') and (Test_period == '2019Q3')):
                                            rf = model_object_WEST_2019Q3_v2
                                    else:
                                            rf = model_object_WEST_2019Q4_v2

                                    #Model predictions
                                    predictions_rf = rf.predict(X_test)                

                                    #Storing test results
                                    result = X_test.copy()
                                    result["Predictions_rf"] = predictions_rf
                                    result["Actuals"] = y_test

                                    result['Predictions_rf'] = np.exp(result['Predictions_rf'])                    
                                    result['Actuals'] = np.exp(result['Actuals'])                                                    

                                    result["Banner"] = test_data_set['Banner']
                                    result["Channel"] = test_data_set['Channel']
                                    result["Product"] = test_data_set["Product"]
                                    result["Region"] = Region_key
                                    result["Test_period"] = Test_period
                                    result["Week"] = test_data_set["Week"]
                                    result["Front.Page"] = test_data_set["Front.Page"]
                                    result["Middle.Page"] = test_data_set["Middle.Page"]
                                    result["Back.Page"] = test_data_set["Back.Page"]
                                    result['iter'] = i

                                    result = result[["Region","Banner","Product",'Channel',"Test_period","Week",'Front.Page','Middle.Page','Back.Page',"Actuals","Predictions_rf",'iter']]

                                    #Appending results to the final results dataframe
                                    Test_results = Test_results.append(result, ignore_index=True)    

                            Test_results = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)]        
                            # Merge & get prediction difference column
                            Test_results = Test_results[Test_results.iter == i].merge(Test_results_sim[Test_results_sim.iter == i][['Banner','Week','Baseline Prediction']], on = ['Banner','Week'], how = 'left')        
                            Test_results['Diff'] = Test_results[Test_results.iter == i]['Predictions_rf'] - Test_results[Test_results.iter == i]['Baseline Prediction']
                            # Consider positive difference only
                            Test_results = Test_results[Test_results.iter == i][Test_results[Test_results.iter == i].Diff > 0]
                            Test_results = Test_results[Test_results.iter == i].sort_values('Diff', ascending = False)

                            # Filtering out banner weeks where back & middle page promotion were given 
                            Test_results = Test_results[(Test_results['Back.Page'] == 0) & (Test_results['Middle.Page'] == 0)]    

                            # Filtering banner weeks where front page promotion was given previously
                            Test_results['Ban_week_concat'] = Test_results['Banner'] + Test_results['Week']
                            front_page_promo_weeks_all['Ban_week_concat'] = front_page_promo_weeks_all['Banner'] + front_page_promo_weeks_all['Week']    
                            Test_results = Test_results[Test_results['Ban_week_concat'].isin(front_page_promo_weeks_all['Ban_week_concat'].unique())]

                            # Select required number of banner-weeks where promotion decrease would occur
                            if (promo_change_toggle.value == 'Yes (Specific Banners)'):         
                                Test_results = Test_results[Test_results.Banner.isin(list(banner_select.value))]
                                for banner_loop in banner_select.value:    
                                    # Selecting banner-weeks
                                    ban_week_fp0 = Test_results[Test_results.Banner == banner_loop][:abs(int(change_fp))][['Banner','Week']]
                                    # if iter = 1, iter = 2 loop
                                    if i == 1:
                                        ban_week_fp = ban_week_fp.append(ban_week_fp0, ignore_index = True)
                                    elif i == 2:
                                        ban_week_fp2 = ban_week_fp2.append(ban_week_fp0, ignore_index = True)

                            elif (promo_change_toggle.value == 'Yes (All Banners)'):
                                for banner_loop in qtr_df_final.Banner.unique():    
                                    # Selecting banner-weeks
                                    ban_week_fp0 = Test_results[Test_results.Banner == banner_loop][:abs(int(change_fp))][['Banner','Week']]
                                    # if iter = 1, iter = 2 loop
                                    if i == 1:
                                        ban_week_fp = ban_week_fp.append(ban_week_fp0, ignore_index = True)
                                    elif i == 2:
                                        ban_week_fp2 = ban_week_fp2.append(ban_week_fp0, ignore_index = True)
                            
                            if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach                        
                                ban_week_fp['Product'] = product_fil
                            else:
                                if i == 1:
                                    ban_week_fp['Product'] = product_fil
                                elif i == 2:
                                    ban_week_fp2['Product'] = product_fil

                        ##########################################################################################################   
                                                        #  MIDDLE PAGE INCREASE SIMULATION 
                        ##########################################################################################################    


                        if i == 1:
                            change_mp = middle_change.children[1].value
                        elif i == 2:
                            change_mp = middle_change2.children[1].value

                        if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
                            if product_fil == edv_disc_join['Existing Model Pack'].unique()[0]:
                                change_mp = middle_change.children[1].value
                            elif product_fil == edv_disc_join['Existing Model Pack'].unique()[1]:
                                change_mp = middle_change2.children[1].value
                            elif product_fil == edv_disc_join['Existing Model Pack'].unique()[2]:
                                change_mp = middle_change3.children[1].value
                           
                        
                        if change_mp > 0:    

                            import warnings
                            warnings.filterwarnings('ignore')

                            # Defining empty dataframes for model training
                            All_coefs = pd.DataFrame({'Region': pd.Series([], dtype='object')})
                            Train_results = pd.DataFrame({'Region': pd.Series([], dtype='object')})
                            Test_results = pd.DataFrame({'Region': pd.Series([], dtype='object')})
                            train_data_brand = pd.DataFrame({'Week': pd.Series([], dtype='object')})
                            combined_dataset = pd.DataFrame({'Week': pd.Series([], dtype='object')})
                            Test_Data = pd.DataFrame()    

                            #Model training and prediction
                            for Region_key in Region_List:
                                # In case the product is not present at that region
                                if len(Volume_dataset_all_reg[(Volume_dataset_all_reg.Product == product_fil) & (Volume_dataset_all_reg.Region == Region_key)]) == 0:
                                    continue

                                Volume_dataset = Volume_dataset_all_reg.loc[(Volume_dataset_all_reg["Region"] == Region_key)
                                                                           &(Volume_dataset_all_reg["Channel"].isin(channel_list))]
                                Volume_dataset['Pantry2'] = Volume_dataset['Pantry2'].fillna(0)
                                required_columns = ['Week','Week.Name','Banner','Product','Channel','Pack.Subtype','PACK_CONTENT','Adcal_Price','EDV.Price','Adcal_DD','Eq.Unit.Sales','Front.Page','Middle.Page','Back.Page','seasonality_index','SIZE_ML','COUNT','No_of_brands','No_of_flavors','No_of_sweetners','No_of_types','Category','Pantry1','Pantry2','Holiday.Week','Pre.Holiday.Week','Post.Holiday.Week']
                                combined_dataset = Volume_dataset[required_columns]

                                combined_dataset = combined_dataset.loc[((combined_dataset["Week"] >= "2017-01") & 
                                                                         (combined_dataset["Week"] <= "2019-52"))]

                                combined_dataset["Christmas_flag"] = [1 if x=="Christmas" else 0 for x in combined_dataset["Week.Name"]]
                                combined_dataset["Easter_flag"] = [1 if x=="Easter" else 0 for x in combined_dataset["Week.Name"]]

                                ## CHECKING if banner dummies are not present in required region, channel then introduce banner dummies
                                banner_dummies = pd.get_dummies(combined_dataset.Banner)              

                                # Uploaded data banner dummies
                                banner_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key].Banner)                                 
                                # If all banners are not present for the training data
                                c = [i for i in list(banner_dummies_check.columns) if i not in list(banner_dummies.columns)]
                                if len(c) != 0:
                                    banner_dummies[c] = 0

                                # Reordering columns                
                                banner_dummies = banner_dummies.reindex(sorted(banner_dummies.columns), axis=1)                
                                combined_dataset = pd.concat([combined_dataset, banner_dummies], axis=1)

                                ## Not adding Product dummies since for New Product Simulator other Product dummies are not significant
                                combined_dataset = combined_dataset.loc[combined_dataset["Eq.Unit.Sales"].notnull()]

                                combined_dataset.sort_values('Week',inplace = True)
                                combined_dataset = pd.merge(combined_dataset,new_prod_stage,on = ['Product','Week'],how = 'left')
                                combined_dataset['Intial_weeks'] = combined_dataset['Intial_weeks'].fillna('Stabilization')
                                combined_dataset.drop(columns='Unnamed: 0', inplace=True)

                                #Adding dummies for category
                                category_dummies = pd.get_dummies(combined_dataset['Category'])

                                # Adding dummies for product attribute - Category
                                pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
                                # Uploaded data category dummies
                                category_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['Category'])                                 
                                # If all pack subtypes are not present for the training data
                                c = [i for i in list(category_dummies_check.columns) if i not in list(category_dummies.columns)]
                                if len(c) != 0:
                                    category_dummies[c] = 0

                                # Reordering columns                
                                category_dummies = category_dummies.reindex(sorted(category_dummies.columns), axis=1)                
                                combined_dataset = pd.concat([combined_dataset, category_dummies], axis=1)
                                combined_dataset.drop(["Category"], inplace=True, axis=1)

                                # Adding dummies for product attribute - pack subtype
                                pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
                                # Uploaded data pack subtype dummies
                                pack_subtypes_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['Pack.Subtype'])                                 
                                # If all pack subtypes are not present for the training data
                                c = [i for i in list(pack_subtypes_dummies_check.columns) if i not in list(pack_subtypes_dummies.columns)]
                                if len(c) != 0:
                                    pack_subtypes_dummies[c] = 0

                                # Reordering columns                
                                pack_subtypes_dummies = pack_subtypes_dummies.reindex(sorted(pack_subtypes_dummies.columns), axis=1)                
                                combined_dataset = pd.concat([combined_dataset, pack_subtypes_dummies], axis=1)
                                combined_dataset.drop(["Pack.Subtype"], inplace=True, axis=1)

                                # Adding pack content dummies
                                pack_content_dummies = pd.get_dummies(combined_dataset['PACK_CONTENT'])
                                # Uploaded data pack content dummies
                                pack_content_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['PACK_CONTENT'])                                 
                                # If all pack contents are not present for the training data
                                c = [i for i in list(pack_content_dummies_check.columns) if i not in list(pack_content_dummies.columns)]
                                if len(c) != 0:
                                    pack_content_dummies[c] = 0

                                # Reordering columns                
                                pack_content_dummies = pack_content_dummies.reindex(sorted(pack_content_dummies.columns), axis=1)                
                                combined_dataset = pd.concat([combined_dataset, pack_content_dummies], axis=1)
                                combined_dataset.drop(["PACK_CONTENT"], inplace=True, axis=1)

                                combined_dataset_test = combined_dataset.copy()

                                # Product stage dummies
                                stage_dummies = pd.get_dummies(combined_dataset['Intial_weeks'])
                                combined_dataset = pd.concat([combined_dataset, stage_dummies], axis=1)
                                combined_dataset.drop(["Intial_weeks"], inplace=True, axis=1)

                                # Changing product stage as per start date provided by the user
                                prod_stage_data = prod_stage_check(product_fil)            

                                # Getting updated product stages
                                combined_dataset_test_check = combined_dataset_test[combined_dataset_test.Product == product_fil].merge(prod_stage_data[['Week','Initial_weeks1']], how = 'left')
                                combined_dataset_test_check['Initial_weeks1'] = combined_dataset_test_check['Initial_weeks1'].fillna(combined_dataset_test_check['Intial_weeks'])
                                combined_dataset_test_check.drop(columns = {'Intial_weeks'}, inplace = True)
                                combined_dataset_test_check.rename(columns = {'Initial_weeks1':'Intial_weeks'}, inplace = True)

                                combined_dataset_test = combined_dataset_test_check.copy()

                                # Product stage dummies
                                stage_dummies_test = pd.get_dummies(combined_dataset_test['Intial_weeks'])

                                # If all product stages are not present for the uploaded data
                                c = [i for i in list(stage_dummies.columns) if i not in list(stage_dummies_test.columns)]
                                if len(c) != 0 :
                                    stage_dummies_test[c] = 0

                                # Reordering columns                
                                stage_dummies_test = stage_dummies_test.reindex(sorted(stage_dummies_test.columns), axis=1)
                                combined_dataset_test = pd.concat([combined_dataset_test, stage_dummies_test], axis=1)
                                combined_dataset_test.drop(["Intial_weeks"], inplace=True, axis=1)

                                # CHANGES : For adcal discounts
                                if 'All' in list(test_period.value):
                                    # Discount change
                                    if i == 1:                
                                        combined_dataset_test['Adcal_DD'] = (1+disc_change_fil)*combined_dataset_test['Adcal_DD']
                                        combined_dataset_test.loc[:,'EDV.Price'] = (1+base_price_change_fil)*combined_dataset_test.loc[:,'EDV.Price']
                                    elif i == 2:
                                        combined_dataset_test['Adcal_DD'] = (1+disc_change_fil2)*combined_dataset_test['Adcal_DD']
                                        combined_dataset_test.loc[:,'EDV.Price'] = (1+base_price_change_fil2)*combined_dataset_test.loc[:,'EDV.Price']          
                                else:                
                                    if i == 1:                
                                        # Discount change
                                        combined_dataset_test.loc[combined_dataset_test['Test_period'].isin(test_period_fil),'Adcal_DD'] = (1+disc_change_fil)*combined_dataset_test[combined_dataset_test['Test_period'].isin(test_period_fil)]['Adcal_DD'] 
                                        # Baseline pricing
                                        combined_dataset_test.loc[(combined_dataset_test['Test_period'].isin(test_period_fil)),'EDV.Price'] = (1+base_price_change_fil)*combined_dataset_test[(combined_dataset_test['Test_period'].isin(test_period_fil))]['EDV.Price']
                                    elif i == 2:
                                        # Discount change
                                        combined_dataset_test.loc[combined_dataset_test['Test_period'].isin(test_period_fil2),'Adcal_DD'] = (1+disc_change_fil2)*combined_dataset_test[combined_dataset_test['Test_period'].isin(test_period_fil2)]['Adcal_DD']             
                                        # Baseline pricing
                                        combined_dataset_test.loc[(combined_dataset_test['Test_period'].isin(test_period_fil2)),'EDV.Price'] = (1+base_price_change_fil2)*combined_dataset_test[(combined_dataset_test['Test_period'].isin(test_period_fil2))]['EDV.Price']


                                #Adding discount depth columns
                                combined_dataset["DD_1"] = [1 if (0.25> x >0.1)
                                                      else 0 for x in combined_dataset["Adcal_DD"]]
                                combined_dataset["DD_2"] = [1 if (x>=0.25)
                                                                 else 0 for x in combined_dataset["Adcal_DD"]]

                                combined_dataset_test["DD_1"] = [1 if (0.25> x >0.1)
                                                      else 0 for x in combined_dataset_test["Adcal_DD"]]
                                combined_dataset_test["DD_2"] = [1 if (x>=0.25)
                                                                 else 0 for x in combined_dataset_test["Adcal_DD"]]            

                                # Adcal Price calculation
                                # In case the discounts go over 100%, limit to 100%
                                combined_dataset_test.loc[combined_dataset_test.Adcal_DD >= 1, 'Adcal_DD'] = 0.99

                                combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset_test.loc[combined_dataset_test['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset_test.loc[combined_dataset_test['Adcal_DD']>=0.1,'EDV.Price']
                                # Adcal_DD < 10 then Adcal Price = EDV Price            
                                combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] < 0.1, 'EDV.Price']

                                # Adcal Price calculation
                                combined_dataset.loc[combined_dataset['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'EDV.Price']
                                # Adcal_DD < 10 then Adcal Price = EDV Price            
                                combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'EDV.Price']

                                # Dropping adcal DD as DD1 & DD2 are added
                                combined_dataset.drop(["Adcal_DD"], inplace=True, axis=1)
                                combined_dataset_test.drop(["Adcal_DD"], inplace=True, axis=1)

                                # Dropping null volumes
                                complete_dataset = combined_dataset.copy()
                                complete_dataset_test = combined_dataset_test.copy()

                                complete_dataset = complete_dataset.loc[complete_dataset["Eq.Unit.Sales"].notnull()]
                                complete_dataset_test = complete_dataset_test.loc[complete_dataset_test["Eq.Unit.Sales"].notnull()]

                                complete_dataset['Adcal_Price'] = np.log(complete_dataset['Adcal_Price'])
                                complete_dataset['Eq.Unit.Sales'] = np.log(complete_dataset['Eq.Unit.Sales'])

                                complete_dataset_test['Adcal_Price'] = np.log(complete_dataset_test['Adcal_Price'])
                                complete_dataset_test['Eq.Unit.Sales'] = np.log(complete_dataset_test['Eq.Unit.Sales'])

                                # TEST PERIOD LOOP
                                Test_periods = ['2019Q1','2019Q2','2019Q3','2019Q4']

                                for Test_period in Test_periods:
                                    if (Test_period>end_week.children[8].value) | (Test_period > Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Product == product_fil, 'Test_period'].max()):
                                        continue
                                    if (Test_period<start_week.children[8].value) | (Test_period < Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Product == product_fil, 'Test_period'].min()):
                                        continue

                                    #Filtering test weeks
                                    Test_weeks = Test_week_list[Test_period]

                                    # Dividing train & test data
                                    complete_train_data_set = complete_dataset.loc[~complete_dataset["Week"].isin(Test_weeks)].reset_index(drop=True)
                                    complete_test_data_set = complete_dataset_test.loc[complete_dataset_test["Week"].isin(Test_weeks)].reset_index(drop=True)

                                    # Dropping unneccessary columns from train dataset
                                    train_data_set = complete_train_data_set.copy()

                                    train_data_brand = train_data_set.copy()
                                    train_data_brand.drop(["Week","Week.Name","Banner","Product","Channel","EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)

                                    # Filtering for existing product
                                    test_data_set = complete_test_data_set[complete_test_data_set.Product == product_fil].copy()

                                    #################### INCREASING PROMOTIONS FOR BANNER WEEKS COMBINATIONS ####################

                                    # Get banner week promos
                                    banner_week_promos_combo = test_data_set[['Banner','Week','Front.Page','Middle.Page','Back.Page']].drop_duplicates()            
                                    banner_week_upc = banner_week_promos_combo.copy()
                                    # Get rows where there is availability of increase in prmotions
                                    avail_promo_weeks = banner_week_upc[(banner_week_upc['Front.Page'] == 0) & (banner_week_upc['Middle.Page'] == 0) & (banner_week_upc['Back.Page'] == 0)]
                                    # Increasing 1 Middle page promotions for every week
                                    avail_promo_weeks['Middle.Page'] = 1
                                    # Merge to get required increase in banner-week combintation
                                    test_data_set_promo_check = test_data_set.merge(avail_promo_weeks[['Banner','Week','Middle.Page']], how = 'left', on = ['Banner','Week'])

                                    # Filling na with existing values in Middle.Page_y
                                    test_data_set_promo_check['Middle.Page_y'].fillna(test_data_set_promo_check['Middle.Page_x'], inplace = True)
                                    # Replacing values of 'Middle.Page_x' with 'Middle.Page_y' wherever it was zero previously
                                    test_data_set_promo_check['Middle.Page_x'] = np.where(test_data_set_promo_check['Middle.Page_x'] == 0, test_data_set_promo_check['Middle.Page_y'], test_data_set_promo_check['Middle.Page_x'])
                                    # Drop 'Middle.Page_y' 
                                    test_data_set_promo_check.drop('Middle.Page_y', axis = 1, inplace =True)
                                    # Renaming 'Middle.Page_x'
                                    test_data_set_promo_check.rename(columns = {'Middle.Page_x':'Middle.Page'}, inplace = True)

                                    test_data_set = test_data_set_promo_check.copy()

                                    # New Category
                                    if i == 1:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)].Category.unique():
                                            if (cols == category_drop.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0
                                    if i == 2:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)].Category.unique():
                                            if (cols == category_drop2.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0
                                    # New Pack Subtype
                                    if i == 1:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['Pack.Subtype'].unique():
                                            if (cols == pack_subtype_drop.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0
                                    if i == 2:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['Pack.Subtype'].unique():
                                            if (cols == pack_subtype_drop2.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0

                                    # New Pack Content
                                    if i == 1:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['PACK_CONTENT'].unique():
                                            if (cols == pack_content_drop.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0
                                    if i == 2:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['PACK_CONTENT'].unique():
                                            if (cols == pack_content_drop2.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0

                                    test_data_brand = test_data_set.copy()

                                    if i == 1 :
                                        # For New SIZE
                                        test_data_brand['SIZE_ML'] = size.value

                                        # For New Count
                                        test_data_brand['COUNT'] = count.value
                                    elif i == 2:
                                        # For New SIZE
                                        test_data_brand['SIZE_ML'] = size2.value

                                        # For New Count
                                        test_data_brand['COUNT'] = count2.value

                                    # Dropping unneccessary columns from test dataset
                                    test_data_brand.drop(["Week","Week.Name","Banner","Product","Channel","EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)

                                    #Creating test and train data - independent & dependent columns
                                    X_train = train_data_brand
                                    y_train = train_data_set["Eq.Unit.Sales"]
                                    X_test = test_data_brand
                                    y_test = test_data_set["Eq.Unit.Sales"]

                                    # Appending test data
                                    test_data_set["Region"] = Region_key
                                    test_data_set["Test_period"] = Test_period
                                    # In case the product is not present in certain region-quarter
                                    if len(test_data_set) == 0:
                                        continue

                                    Test_Data = Test_Data.append(test_data_set, ignore_index = True)

                                    if((Region_key == 'EAST') and (Test_period == '2019Q1')):
                                            rf = model_object_EAST_2019Q1_v2
                                    elif((Region_key == 'EAST') and (Test_period == '2019Q2')):
                                            rf = model_object_EAST_2019Q2_v2
                                    elif((Region_key == 'EAST') and (Test_period == '2019Q3')):
                                            rf = model_object_EAST_2019Q3_v2
                                    elif((Region_key == 'EAST') and (Test_period == '2019Q4')):
                                            rf = model_object_EAST_2019Q4_v2
                                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q1')):
                                            rf = model_object_ONTARIO_2019Q1_v2
                                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q2')):
                                            rf = model_object_ONTARIO_2019Q2_v2
                                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q3')):
                                            rf = model_object_ONTARIO_2019Q3_v2
                                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q4')):
                                            rf = model_object_ONTARIO_2019Q4_v2
                                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q1')):
                                            rf = model_object_QUEBEC_2019Q1_v2
                                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q2')):
                                            rf = model_object_QUEBEC_2019Q2_v2
                                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q3')):
                                            rf = model_object_QUEBEC_2019Q3_v2
                                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q4')):
                                            rf = model_object_QUEBEC_2019Q4_v2
                                    elif((Region_key == 'WEST') and (Test_period == '2019Q1')):
                                            rf = model_object_WEST_2019Q1_v2
                                    elif((Region_key == 'WEST') and (Test_period == '2019Q2')):
                                            rf = model_object_WEST_2019Q2_v2
                                    elif((Region_key == 'WEST') and (Test_period == '2019Q3')):
                                            rf = model_object_WEST_2019Q3_v2
                                    else:
                                            rf = model_object_WEST_2019Q4_v2

                                    #Model predictions
                                    predictions_rf = rf.predict(X_test)                

                                    #Storing test results
                                    result = X_test.copy()
                                    result["Predictions_rf"] = predictions_rf
                                    result["Actuals"] = y_test

                                    result['Predictions_rf'] = np.exp(result['Predictions_rf'])                    
                                    result['Actuals'] = np.exp(result['Actuals'])                                                    

                                    #Calculating APE for the models
                                    result["ape_rf"] = (result["Predictions_rf"]-result["Actuals"]).abs()

                                    result["Product"] = test_data_set["Product"]
                                    result["Region"] = Region_key
                                    result["Test_period"] = Test_period
                                    result["Banner"] = test_data_set["Banner"]
                                    result["Channel"] = test_data_set['Channel']
                                    result["Week"] = test_data_set["Week"]
                                    result["DD_1"] = test_data_set['DD_1']
                                    result["DD_2"] = test_data_set['DD_2']
                                    ##Columns for Revenue                                
                                    result["Adcal_Price"] = test_data_set["Adcal_Price"]
                                    result['iter'] = i

                                    result = result[["Region","Banner","Product",'Channel',"Test_period","Week","Actuals","ape_rf","Predictions_rf","DD_1","DD_2","Adcal_Price",'iter']]

                                    #Appending results to the final results dataframe
                                    Test_results = Test_results.append(result, ignore_index=True)                

                            Test_results = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)]
                            # Removing 0 promotion banners
                            if len(qtr_df_final[(qtr_df_final.Banner.isin(fp_remove_banner)) | (qtr_df_final.Banner.isin(mp_remove_banner)) | (qtr_df_final.Banner.isin(bp_remove_banner))]) != 0:
                                if zero_banner_promo_inc.value == 'No':
                                    Test_results = Test_results[~Test_results.Banner.isin(mp_remove_banner)]

                            # Merge & get prediction difference column
                            Test_results = Test_results[Test_results.iter == i].merge(Test_results_sim[Test_results_sim.iter == i][['Banner','Week','Baseline Prediction']], on = ['Banner','Week'], how = 'left')        
                            Test_results['Diff'] = Test_results[Test_results.iter == i]['Predictions_rf'] - Test_results[Test_results.iter == i]['Baseline Prediction']
                            # Consider positive difference only
                            Test_results = Test_results[Test_results.iter == i][Test_results[Test_results.iter == i].Diff > 0]
                            Test_results = Test_results[Test_results.iter == i].sort_values('Diff', ascending = False)

                            # Filtering out banner weeks where front page promotions were given for priority
                            if change_fp != 0:
                                Test_results['ban_week_mp_check'] = Test_results['Banner'] + Test_results['Week']
                                # if-else loop for iter == 1, iter == 2
                                if i == 1:
                                    Test_results = Test_results[~Test_results['ban_week_mp_check'].isin(ban_week_fp['ban_week_fp_check'].unique())]
                                elif i == 2:
                                    Test_results = Test_results[~Test_results['ban_week_mp_check'].isin(ban_week_fp2['ban_week_fp_check'].unique())]

                            # Select required number of banner-weeks where promotion increase would occur
                            if (promo_change_toggle.value == 'Yes (Specific Banners)'):         
                                Test_results = Test_results[Test_results.Banner.isin(list(banner_select.value))]
                                for banner_loop in banner_select.value:    
                                    # Selecting banner-weeks
                                    ban_week_mp0 = Test_results[Test_results.Banner == banner_loop][:abs(int(change_mp))][['Banner','Week']]
                                    # if iter = 1, iter = 2 loop
                                    if i == 1:
                                        ban_week_mp = ban_week_mp.append(ban_week_mp0, ignore_index = True)
                                        # Banner-week concat column    
                                        ban_week_mp['ban_week_mp_check'] = ban_week_mp['Banner'] + ban_week_mp['Week']                            

                                    elif i == 2:
                                        ban_week_mp2 = ban_week_mp2.append(ban_week_mp0, ignore_index = True)
                                        # Banner-week concat column    
                                        ban_week_mp2['ban_week_mp_check'] = ban_week_mp2['Banner'] + ban_week_mp2['Week']                             

                            elif (promo_change_toggle.value == 'Yes (All Banners)'):
                                for banner_loop in qtr_df_final.Banner.unique():    
                                    # Selecting banner-weeks
                                    ban_week_mp0 = Test_results[Test_results.Banner == banner_loop][:abs(int(change_mp))][['Banner','Week']]
                                    # if iter = 1, iter = 2 loop
                                    if i == 1:
                                        ban_week_mp = ban_week_mp.append(ban_week_mp0, ignore_index = True)
                                        # Banner-week concat column    
                                        ban_week_mp['ban_week_mp_check'] = ban_week_mp['Banner'] + ban_week_mp['Week']                            
                                    elif i == 2:
                                        ban_week_mp2 = ban_week_mp2.append(ban_week_mp0, ignore_index = True)
                                        # Banner-week concat column    
                                        ban_week_mp2['ban_week_mp_check'] = ban_week_mp2['Banner'] + ban_week_mp2['Week'] 

                            if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach                        
                                ban_week_mp['Product'] = product_fil
                            else:
                                if i == 1:
                                    ban_week_mp['Product'] = product_fil
                                elif i == 2:
                                    ban_week_mp2['Product'] = product_fil

                            
                        ##########################################################################################################   
                                                        #  MIDDLE PAGE DECREASE SIMULATION 
                        ##########################################################################################################    

                        if i == 1:
                            change_mp = middle_change.children[1].value
                        elif i == 2:
                            change_mp = middle_change2.children[1].value

                        if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
                            if product_fil == edv_disc_join['Existing Model Pack'].unique()[0]:
                                change_mp = middle_change.children[1].value
                            elif product_fil == edv_disc_join['Existing Model Pack'].unique()[1]:
                                change_mp = middle_change2.children[1].value
                            elif product_fil == edv_disc_join['Existing Model Pack'].unique()[2]:
                                change_mp = middle_change3.children[1].value
                            
                        if (change_mp < 0) & ((qtr_df_final['Middle Page']).sum() != 0):

                            ################## MIDDLE PAGE SIMULATION ##################
                            import warnings
                            warnings.filterwarnings('ignore')

                            # Defining empty dataframes for model training
                            All_coefs = pd.DataFrame({'Region': pd.Series([], dtype='object')})
                            Train_results = pd.DataFrame({'Region': pd.Series([], dtype='object')})
                            Test_results = pd.DataFrame({'Region': pd.Series([], dtype='object')})
                            train_data_brand = pd.DataFrame({'Week': pd.Series([], dtype='object')})
                            combined_dataset = pd.DataFrame({'Week': pd.Series([], dtype='object')})
                            Test_Data = pd.DataFrame()    

                            #Model training and prediction
                            for Region_key in Region_List:
                                # In case the product is not present at that region
                                if len(Volume_dataset_all_reg[(Volume_dataset_all_reg.Product == product_fil) & (Volume_dataset_all_reg.Region == Region_key)]) == 0:
                                    continue

                                Volume_dataset = Volume_dataset_all_reg.loc[(Volume_dataset_all_reg["Region"] == Region_key)
                                                                           &(Volume_dataset_all_reg["Channel"].isin(channel_list))]
                                Volume_dataset['Pantry2'] = Volume_dataset['Pantry2'].fillna(0)
                                required_columns = ['Week','Week.Name','Banner','Product','Channel','Pack.Subtype','PACK_CONTENT','Adcal_Price','EDV.Price','Adcal_DD','Eq.Unit.Sales','Front.Page','Middle.Page','Back.Page','seasonality_index','SIZE_ML','COUNT','No_of_brands','No_of_flavors','No_of_sweetners','No_of_types','Category','Pantry1','Pantry2','Holiday.Week','Pre.Holiday.Week','Post.Holiday.Week']
                                combined_dataset = Volume_dataset[required_columns]

                                combined_dataset = combined_dataset.loc[((combined_dataset["Week"] >= "2017-01") & 
                                                                         (combined_dataset["Week"] <= "2019-52"))]

                                combined_dataset["Christmas_flag"] = [1 if x=="Christmas" else 0 for x in combined_dataset["Week.Name"]]
                                combined_dataset["Easter_flag"] = [1 if x=="Easter" else 0 for x in combined_dataset["Week.Name"]]

                                ## CHECKING if banner dummies are not present in required region, channel then introduce banner dummies
                                banner_dummies = pd.get_dummies(combined_dataset.Banner)              

                                # Uploaded data banner dummies
                                banner_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key].Banner)                                 
                                # If all banners are not present for the training data
                                c = [i for i in list(banner_dummies_check.columns) if i not in list(banner_dummies.columns)]
                                if len(c) != 0:
                                    banner_dummies[c] = 0

                                # Reordering columns                
                                banner_dummies = banner_dummies.reindex(sorted(banner_dummies.columns), axis=1)                
                                combined_dataset = pd.concat([combined_dataset, banner_dummies], axis=1)

                                ## Not adding Product dummies since for New Product Simulator other Product dummies are not significant
                                combined_dataset = combined_dataset.loc[combined_dataset["Eq.Unit.Sales"].notnull()]

                                combined_dataset.sort_values('Week',inplace = True)
                                combined_dataset = pd.merge(combined_dataset,new_prod_stage,on = ['Product','Week'],how = 'left')
                                combined_dataset['Intial_weeks'] = combined_dataset['Intial_weeks'].fillna('Stabilization')
                                combined_dataset.drop(columns='Unnamed: 0', inplace=True)

                                #Adding dummies for category
                                category_dummies = pd.get_dummies(combined_dataset['Category'])

                                # Adding dummies for product attribute - Category
                                pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
                                # Uploaded data category dummies
                                category_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['Category'])                                 
                                # If all pack subtypes are not present for the training data
                                c = [i for i in list(category_dummies_check.columns) if i not in list(category_dummies.columns)]
                                if len(c) != 0:
                                    category_dummies[c] = 0

                                # Reordering columns                
                                category_dummies = category_dummies.reindex(sorted(category_dummies.columns), axis=1)                
                                combined_dataset = pd.concat([combined_dataset, category_dummies], axis=1)
                                combined_dataset.drop(["Category"], inplace=True, axis=1)

                                # Adding dummies for product attribute - pack subtype
                                pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
                                # Uploaded data pack subtype dummies
                                pack_subtypes_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['Pack.Subtype'])                                 
                                # If all pack subtypes are not present for the training data
                                c = [i for i in list(pack_subtypes_dummies_check.columns) if i not in list(pack_subtypes_dummies.columns)]
                                if len(c) != 0:
                                    pack_subtypes_dummies[c] = 0

                                # Reordering columns                
                                pack_subtypes_dummies = pack_subtypes_dummies.reindex(sorted(pack_subtypes_dummies.columns), axis=1)                
                                combined_dataset = pd.concat([combined_dataset, pack_subtypes_dummies], axis=1)
                                combined_dataset.drop(["Pack.Subtype"], inplace=True, axis=1)

                                # Adding pack content dummies
                                pack_content_dummies = pd.get_dummies(combined_dataset['PACK_CONTENT'])
                                # Uploaded data pack content dummies
                                pack_content_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['PACK_CONTENT'])                                 
                                # If all pack contents are not present for the training data
                                c = [i for i in list(pack_content_dummies_check.columns) if i not in list(pack_content_dummies.columns)]
                                if len(c) != 0:
                                    pack_content_dummies[c] = 0

                                # Reordering columns                
                                pack_content_dummies = pack_content_dummies.reindex(sorted(pack_content_dummies.columns), axis=1)                
                                combined_dataset = pd.concat([combined_dataset, pack_content_dummies], axis=1)
                                combined_dataset.drop(["PACK_CONTENT"], inplace=True, axis=1)

                                combined_dataset_test = combined_dataset.copy()

                                # Product stage dummies
                                stage_dummies = pd.get_dummies(combined_dataset['Intial_weeks'])
                                combined_dataset = pd.concat([combined_dataset, stage_dummies], axis=1)
                                combined_dataset.drop(["Intial_weeks"], inplace=True, axis=1)

                                # Changing product stage as per start date provided by the user
                                prod_stage_data = prod_stage_check(product_fil)            

                                # Getting updated product stages
                                combined_dataset_test_check = combined_dataset_test[combined_dataset_test.Product == product_fil].merge(prod_stage_data[['Week','Initial_weeks1']], how = 'left')
                                combined_dataset_test_check['Initial_weeks1'] = combined_dataset_test_check['Initial_weeks1'].fillna(combined_dataset_test_check['Intial_weeks'])
                                combined_dataset_test_check.drop(columns = {'Intial_weeks'}, inplace = True)
                                combined_dataset_test_check.rename(columns = {'Initial_weeks1':'Intial_weeks'}, inplace = True)

                                combined_dataset_test = combined_dataset_test_check.copy()

                                # Product stage dummies
                                stage_dummies_test = pd.get_dummies(combined_dataset_test['Intial_weeks'])

                                # If all product stages are not present for the uploaded data
                                c = [i for i in list(stage_dummies.columns) if i not in list(stage_dummies_test.columns)]
                                if len(c) != 0 :
                                    stage_dummies_test[c] = 0

                                # Reordering columns                
                                stage_dummies_test = stage_dummies_test.reindex(sorted(stage_dummies_test.columns), axis=1)
                                combined_dataset_test = pd.concat([combined_dataset_test, stage_dummies_test], axis=1)
                                combined_dataset_test.drop(["Intial_weeks"], inplace=True, axis=1)

                                # CHANGES : For adcal discounts
                                if 'All' in list(test_period.value):
                                    # Discount change
                                    if i == 1:                
                                        combined_dataset_test['Adcal_DD'] = (1+disc_change_fil)*combined_dataset_test['Adcal_DD']
                                        combined_dataset_test.loc[:,'EDV.Price'] = (1+base_price_change_fil)*combined_dataset_test.loc[:,'EDV.Price']
                                    elif i == 2:
                                        combined_dataset_test['Adcal_DD'] = (1+disc_change_fil2)*combined_dataset_test['Adcal_DD']
                                        combined_dataset_test.loc[:,'EDV.Price'] = (1+base_price_change_fil2)*combined_dataset_test.loc[:,'EDV.Price']          
                                else:                
                                    if i == 1:                
                                        # Discount change
                                        combined_dataset_test.loc[combined_dataset_test['Test_period'].isin(test_period_fil),'Adcal_DD'] = (1+disc_change_fil)*combined_dataset_test[combined_dataset_test['Test_period'].isin(test_period_fil)]['Adcal_DD'] 
                                        # Baseline pricing
                                        combined_dataset_test.loc[(combined_dataset_test['Test_period'].isin(test_period_fil)),'EDV.Price'] = (1+base_price_change_fil)*combined_dataset_test[(combined_dataset_test['Test_period'].isin(test_period_fil))]['EDV.Price']
                                    elif i == 2:
                                        # Discount change
                                        combined_dataset_test.loc[combined_dataset_test['Test_period'].isin(test_period_fil2),'Adcal_DD'] = (1+disc_change_fil2)*combined_dataset_test[combined_dataset_test['Test_period'].isin(test_period_fil2)]['Adcal_DD']             
                                        # Baseline pricing
                                        combined_dataset_test.loc[(combined_dataset_test['Test_period'].isin(test_period_fil2)),'EDV.Price'] = (1+base_price_change_fil2)*combined_dataset_test[(combined_dataset_test['Test_period'].isin(test_period_fil2))]['EDV.Price']

                                #Adding discount depth columns
                                combined_dataset["DD_1"] = [1 if (0.25> x >0.1)
                                                      else 0 for x in combined_dataset["Adcal_DD"]]
                                combined_dataset["DD_2"] = [1 if (x>=0.25)
                                                                 else 0 for x in combined_dataset["Adcal_DD"]]

                                combined_dataset_test["DD_1"] = [1 if (0.25> x >0.1)
                                                      else 0 for x in combined_dataset_test["Adcal_DD"]]
                                combined_dataset_test["DD_2"] = [1 if (x>=0.25)
                                                                 else 0 for x in combined_dataset_test["Adcal_DD"]]            

                                # Adcal Price calculation
                                # In case the discounts go over 100%, limit to 100%
                                combined_dataset_test.loc[combined_dataset_test.Adcal_DD >= 1, 'Adcal_DD'] = 0.99

                                combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset_test.loc[combined_dataset_test['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset_test.loc[combined_dataset_test['Adcal_DD']>=0.1,'EDV.Price']
                                # Adcal_DD < 10 then Adcal Price = EDV Price            
                                combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] < 0.1, 'EDV.Price']

                                # Adcal Price calculation
                                combined_dataset.loc[combined_dataset['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'EDV.Price']
                                # Adcal_DD < 10 then Adcal Price = EDV Price            
                                combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'EDV.Price']

                                # Dropping adcal DD as DD1 & DD2 are added
                                combined_dataset.drop(["Adcal_DD"], inplace=True, axis=1)
                                combined_dataset_test.drop(["Adcal_DD"], inplace=True, axis=1)

                                # Dropping null volumes
                                complete_dataset = combined_dataset.copy()
                                complete_dataset_test = combined_dataset_test.copy()

                                complete_dataset = complete_dataset.loc[complete_dataset["Eq.Unit.Sales"].notnull()]
                                complete_dataset_test = complete_dataset_test.loc[complete_dataset_test["Eq.Unit.Sales"].notnull()]

                                complete_dataset['Adcal_Price'] = np.log(complete_dataset['Adcal_Price'])
                                complete_dataset['Eq.Unit.Sales'] = np.log(complete_dataset['Eq.Unit.Sales'])

                                complete_dataset_test['Adcal_Price'] = np.log(complete_dataset_test['Adcal_Price'])
                                complete_dataset_test['Eq.Unit.Sales'] = np.log(complete_dataset_test['Eq.Unit.Sales'])

                                # TEST PERIOD LOOP
                                Test_periods = ['2019Q1','2019Q2','2019Q3','2019Q4']

                                for Test_period in Test_periods:
                                    if (Test_period>end_week.children[8].value) | (Test_period > Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Product == product_fil, 'Test_period'].max()):
                                        continue
                                    if (Test_period<start_week.children[8].value) | (Test_period < Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Product == product_fil, 'Test_period'].min()):
                                        continue

                                    #Filtering test weeks
                                    Test_weeks = Test_week_list[Test_period]

                                    # Dividing train & test data
                                    complete_train_data_set = complete_dataset.loc[~complete_dataset["Week"].isin(Test_weeks)].reset_index(drop=True)
                                    complete_test_data_set = complete_dataset_test.loc[complete_dataset_test["Week"].isin(Test_weeks)].reset_index(drop=True)

                                    # Dropping unneccessary columns from train dataset
                                    train_data_set = complete_train_data_set.copy()

                                    train_data_brand = train_data_set.copy()
                                    train_data_brand.drop(["Week","Week.Name","Banner","Product","Channel","EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)

                                    # Filtering for existing product
                                    test_data_set = complete_test_data_set[complete_test_data_set.Product == product_fil].copy()

                                    #################### DECREASING PROMOTIONS FOR BANNER WEEKS COMBINATIONS ####################
                                    # Allotting all middle page promos as 0
                                    middle_page_promo_weeks = pd.DataFrame(test_data_set[test_data_set['Middle.Page'] != 0][['Banner','Week']])
                                    middle_page_promo_weeks_all = middle_page_promo_weeks_all.append(middle_page_promo_weeks, ignore_index = True)
                                    test_data_set['Middle.Page'] = 0

                                    # New Category
                                    if i == 1:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)].Category.unique():
                                            if (cols == category_drop.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0
                                    if i == 2:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)].Category.unique():
                                            if (cols == category_drop2.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0

                                    # New Pack Subtype
                                    if i == 1:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['Pack.Subtype'].unique():
                                            if (cols == pack_subtype_drop.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0
                                    if i == 2:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['Pack.Subtype'].unique():
                                            if (cols == pack_subtype_drop2.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0
                                    # New Pack Content
                                    if i == 1:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['PACK_CONTENT'].unique():
                                            if (cols == pack_content_drop.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0
                                    if i == 2:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['PACK_CONTENT'].unique():
                                            if (cols == pack_content_drop2.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0

                                    test_data_brand = test_data_set.copy()

                                    if i == 1 :
                                        # For New SIZE
                                        test_data_brand['SIZE_ML'] = size.value

                                        # For New Count
                                        test_data_brand['COUNT'] = count.value
                                    elif i == 2:
                                        # For New SIZE
                                        test_data_brand['SIZE_ML'] = size2.value

                                        # For New Count
                                        test_data_brand['COUNT'] = count2.value

                                    # Dropping unneccessary columns from test dataset
                                    test_data_brand.drop(["Week","Week.Name","Banner","Product","Channel","EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)

                                    #Creating test and train data - independent & dependent columns
                                    X_train = train_data_brand
                                    y_train = train_data_set["Eq.Unit.Sales"]
                                    X_test = test_data_brand
                                    y_test = test_data_set["Eq.Unit.Sales"]

                                    # Appending test data
                                    test_data_set["Region"] = Region_key
                                    test_data_set["Test_period"] = Test_period
                                    # In case the product is not present in certain region-quarter
                                    if len(test_data_set) == 0:
                                        continue

                                    Test_Data = Test_Data.append(test_data_set, ignore_index = True)

                                    if((Region_key == 'EAST') and (Test_period == '2019Q1')):
                                            rf = model_object_EAST_2019Q1_v2
                                    elif((Region_key == 'EAST') and (Test_period == '2019Q2')):
                                            rf = model_object_EAST_2019Q2_v2
                                    elif((Region_key == 'EAST') and (Test_period == '2019Q3')):
                                            rf = model_object_EAST_2019Q3_v2
                                    elif((Region_key == 'EAST') and (Test_period == '2019Q4')):
                                            rf = model_object_EAST_2019Q4_v2
                                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q1')):
                                            rf = model_object_ONTARIO_2019Q1_v2
                                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q2')):
                                            rf = model_object_ONTARIO_2019Q2_v2
                                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q3')):
                                            rf = model_object_ONTARIO_2019Q3_v2
                                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q4')):
                                            rf = model_object_ONTARIO_2019Q4_v2
                                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q1')):
                                            rf = model_object_QUEBEC_2019Q1_v2
                                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q2')):
                                            rf = model_object_QUEBEC_2019Q2_v2
                                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q3')):
                                            rf = model_object_QUEBEC_2019Q3_v2
                                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q4')):
                                            rf = model_object_QUEBEC_2019Q4_v2
                                    elif((Region_key == 'WEST') and (Test_period == '2019Q1')):
                                            rf = model_object_WEST_2019Q1_v2
                                    elif((Region_key == 'WEST') and (Test_period == '2019Q2')):
                                            rf = model_object_WEST_2019Q2_v2
                                    elif((Region_key == 'WEST') and (Test_period == '2019Q3')):
                                            rf = model_object_WEST_2019Q3_v2
                                    else:
                                            rf = model_object_WEST_2019Q4_v2

                                    #Model predictions
                                    predictions_rf = rf.predict(X_test)                

                                    #Storing test results
                                    result = X_test.copy()
                                    result["Predictions_rf"] = predictions_rf
                                    result["Actuals"] = y_test

                                    result['Predictions_rf'] = np.exp(result['Predictions_rf'])                    
                                    result['Actuals'] = np.exp(result['Actuals'])                                                    

                                    result["Banner"] = test_data_set['Banner']
                                    result["Channel"] = test_data_set['Channel']
                                    result["Product"] = test_data_set["Product"]
                                    result["Region"] = Region_key
                                    result["Test_period"] = Test_period
                                    result["Week"] = test_data_set["Week"]  
                                    result["Front.Page"] = test_data_set["Front.Page"]
                                    result["Middle.Page"] = test_data_set["Middle.Page"]
                                    result["Back.Page"] = test_data_set["Back.Page"]
                                    result['iter'] = i

                                    result = result[["Region","Banner","Product",'Channel',"Test_period","Week",'Front.Page','Middle.Page','Back.Page',"Actuals","Predictions_rf",'iter']]

                                    #Appending results to the final results dataframe
                                    Test_results = Test_results.append(result, ignore_index=True)                

                            Test_results = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)]        
                            # Merge & get prediction difference column
                            Test_results = Test_results[Test_results.iter == i].merge(Test_results_sim[Test_results_sim.iter == i][['Banner','Week','Baseline Prediction']], on = ['Banner','Week'], how = 'left')        
                            Test_results['Diff'] = Test_results[Test_results.iter == i]['Predictions_rf'] - Test_results[Test_results.iter == i]['Baseline Prediction']
                            # Consider positive difference only
                            Test_results = Test_results[Test_results.iter == i][Test_results[Test_results.iter == i].Diff > 0]
                            Test_results = Test_results[Test_results.iter == i].sort_values('Diff', ascending = False)

                            # Filtering out banner weeks where back & middle page promotion were given 
                            Test_results = Test_results[(Test_results['Back.Page'] == 0) & (Test_results['Front.Page'] == 0)]    

                            # Filtering banner weeks where middle page promotion was given previously
                            Test_results['Ban_week_concat'] = Test_results['Banner'] + Test_results['Week']
                            middle_page_promo_weeks_all['Ban_week_concat'] = middle_page_promo_weeks_all['Banner'] + middle_page_promo_weeks_all['Week']    
                            Test_results = Test_results[Test_results['Ban_week_concat'].isin(middle_page_promo_weeks_all['Ban_week_concat'].unique())]

                            # Select required number of banner-weeks where promotion decrease would occur
                            if (promo_change_toggle.value == 'Yes (Specific Banners)'):         
                                Test_results = Test_results[Test_results.Banner.isin(list(banner_select.value))]
                                for banner_loop in banner_select.value:    
                                    # Selecting banner-weeks
                                    ban_week_mp0 = Test_results[Test_results.Banner == banner_loop][:abs(int(change_mp))][['Banner','Week']]
                                    # if iter = 1, iter = 2 loop
                                    if i == 1:
                                        ban_week_mp = ban_week_mp.append(ban_week_mp0, ignore_index = True)
                                    elif i == 2:
                                        ban_week_mp2 = ban_week_mp2.append(ban_week_mp0, ignore_index = True)

                            elif (promo_change_toggle.value == 'Yes (All Banners)'):
                                for banner_loop in qtr_df_final.Banner.unique():    
                                    # Selecting banner-weeks
                                    ban_week_mp0 = Test_results[Test_results.Banner == banner_loop][:abs(int(change_mp))][['Banner','Week']]
                                    # if iter = 1, iter = 2 loop
                                    if i == 1:
                                        ban_week_mp = ban_week_mp.append(ban_week_mp0, ignore_index = True)
                                    elif i == 2:
                                        ban_week_mp2 = ban_week_mp2.append(ban_week_mp0, ignore_index = True)

                            if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach                        
                                ban_week_mp['Product'] = product_fil
                            else:
                                if i == 1:
                                    ban_week_mp['Product'] = product_fil
                                elif i == 2:
                                    ban_week_mp2['Product'] = product_fil

                        ##########################################################################################################   
                                                        #  BACK PAGE INCREASE SIMULATION 
                        ##########################################################################################################    

                        if i == 1:
                            change_bp = back_change.children[1].value
                        elif i == 2:
                            change_bp = back_change2.children[1].value

                        if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
                            if product_fil == edv_disc_join['Existing Model Pack'].unique()[0]:
                                change_bp = back_change.children[1].value
                            elif product_fil == edv_disc_join['Existing Model Pack'].unique()[1]:
                                change_bp = back_change2.children[1].value
                            elif product_fil == edv_disc_join['Existing Model Pack'].unique()[2]:
                                change_bp = back_change3.children[1].value
                            
                        if change_bp > 0:    

                            import warnings
                            warnings.filterwarnings('ignore')

                            # Defining empty dataframes for model training
                            All_coefs = pd.DataFrame({'Region': pd.Series([], dtype='object')})
                            Train_results = pd.DataFrame({'Region': pd.Series([], dtype='object')})
                            Test_results = pd.DataFrame({'Region': pd.Series([], dtype='object')})
                            train_data_brand = pd.DataFrame({'Week': pd.Series([], dtype='object')})
                            combined_dataset = pd.DataFrame({'Week': pd.Series([], dtype='object')})
                            Test_Data = pd.DataFrame()    

                            #Model training and prediction
                            for Region_key in Region_List:
                                # In case the product is not present at that region
                                if len(Volume_dataset_all_reg[(Volume_dataset_all_reg.Product == product_fil) & (Volume_dataset_all_reg.Region == Region_key)]) == 0:
                                    continue

                                Volume_dataset = Volume_dataset_all_reg.loc[(Volume_dataset_all_reg["Region"] == Region_key)
                                                                           &(Volume_dataset_all_reg["Channel"].isin(channel_list))]
                                Volume_dataset['Pantry2'] = Volume_dataset['Pantry2'].fillna(0)
                                required_columns = ['Week','Week.Name','Banner','Product','Channel','Pack.Subtype','PACK_CONTENT','Adcal_Price','EDV.Price','Adcal_DD','Eq.Unit.Sales','Front.Page','Middle.Page','Back.Page','seasonality_index','SIZE_ML','COUNT','No_of_brands','No_of_flavors','No_of_sweetners','No_of_types','Category','Pantry1','Pantry2','Holiday.Week','Pre.Holiday.Week','Post.Holiday.Week']
                                combined_dataset = Volume_dataset[required_columns]

                                combined_dataset = combined_dataset.loc[((combined_dataset["Week"] >= "2017-01") & 
                                                                         (combined_dataset["Week"] <= "2019-52"))]

                                combined_dataset["Christmas_flag"] = [1 if x=="Christmas" else 0 for x in combined_dataset["Week.Name"]]
                                combined_dataset["Easter_flag"] = [1 if x=="Easter" else 0 for x in combined_dataset["Week.Name"]]

                                ## CHECKING if banner dummies are not present in required region, channel then introduce banner dummies
                                banner_dummies = pd.get_dummies(combined_dataset.Banner)              

                                # Uploaded data banner dummies
                                banner_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key].Banner)                                 
                                # If all banners are not present for the training data
                                c = [i for i in list(banner_dummies_check.columns) if i not in list(banner_dummies.columns)]
                                if len(c) != 0:
                                    banner_dummies[c] = 0

                                # Reordering columns                
                                banner_dummies = banner_dummies.reindex(sorted(banner_dummies.columns), axis=1)                
                                combined_dataset = pd.concat([combined_dataset, banner_dummies], axis=1)

                                ## Not adding Product dummies since for New Product Simulator other Product dummies are not significant
                                combined_dataset = combined_dataset.loc[combined_dataset["Eq.Unit.Sales"].notnull()]

                                combined_dataset.sort_values('Week',inplace = True)
                                combined_dataset = pd.merge(combined_dataset,new_prod_stage,on = ['Product','Week'],how = 'left')
                                combined_dataset['Intial_weeks'] = combined_dataset['Intial_weeks'].fillna('Stabilization')
                                combined_dataset.drop(columns='Unnamed: 0', inplace=True)

                                #Adding dummies for category
                                category_dummies = pd.get_dummies(combined_dataset['Category'])

                                # Adding dummies for product attribute - Category
                                pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
                                # Uploaded data category dummies
                                category_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['Category'])                                 
                                # If all pack subtypes are not present for the training data
                                c = [i for i in list(category_dummies_check.columns) if i not in list(category_dummies.columns)]
                                if len(c) != 0:
                                    category_dummies[c] = 0

                                # Reordering columns                
                                category_dummies = category_dummies.reindex(sorted(category_dummies.columns), axis=1)                
                                combined_dataset = pd.concat([combined_dataset, category_dummies], axis=1)
                                combined_dataset.drop(["Category"], inplace=True, axis=1)

                                # Adding dummies for product attribute - pack subtype
                                pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
                                # Uploaded data pack subtype dummies
                                pack_subtypes_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['Pack.Subtype'])                                 
                                # If all pack subtypes are not present for the training data
                                c = [i for i in list(pack_subtypes_dummies_check.columns) if i not in list(pack_subtypes_dummies.columns)]
                                if len(c) != 0:
                                    pack_subtypes_dummies[c] = 0

                                # Reordering columns                
                                pack_subtypes_dummies = pack_subtypes_dummies.reindex(sorted(pack_subtypes_dummies.columns), axis=1)                
                                combined_dataset = pd.concat([combined_dataset, pack_subtypes_dummies], axis=1)
                                combined_dataset.drop(["Pack.Subtype"], inplace=True, axis=1)

                                # Adding pack content dummies
                                pack_content_dummies = pd.get_dummies(combined_dataset['PACK_CONTENT'])
                                # Uploaded data pack content dummies
                                pack_content_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['PACK_CONTENT'])                                 
                                # If all pack contents are not present for the training data
                                c = [i for i in list(pack_content_dummies_check.columns) if i not in list(pack_content_dummies.columns)]
                                if len(c) != 0:
                                    pack_content_dummies[c] = 0

                                # Reordering columns                
                                pack_content_dummies = pack_content_dummies.reindex(sorted(pack_content_dummies.columns), axis=1)                
                                combined_dataset = pd.concat([combined_dataset, pack_content_dummies], axis=1)
                                combined_dataset.drop(["PACK_CONTENT"], inplace=True, axis=1)

                                combined_dataset_test = combined_dataset.copy()

                                # Product stage dummies
                                stage_dummies = pd.get_dummies(combined_dataset['Intial_weeks'])
                                combined_dataset = pd.concat([combined_dataset, stage_dummies], axis=1)
                                combined_dataset.drop(["Intial_weeks"], inplace=True, axis=1)

                                # Changing product stage as per start date provided by the user
                                prod_stage_data = prod_stage_check(product_fil)            

                                # Getting updated product stages
                                combined_dataset_test_check = combined_dataset_test[combined_dataset_test.Product == product_fil].merge(prod_stage_data[['Week','Initial_weeks1']], how = 'left')
                                combined_dataset_test_check['Initial_weeks1'] = combined_dataset_test_check['Initial_weeks1'].fillna(combined_dataset_test_check['Intial_weeks'])
                                combined_dataset_test_check.drop(columns = {'Intial_weeks'}, inplace = True)
                                combined_dataset_test_check.rename(columns = {'Initial_weeks1':'Intial_weeks'}, inplace = True)

                                combined_dataset_test = combined_dataset_test_check.copy()

                                # Product stage dummies
                                stage_dummies_test = pd.get_dummies(combined_dataset_test['Intial_weeks'])

                                # If all product stages are not present for the uploaded data
                                c = [i for i in list(stage_dummies.columns) if i not in list(stage_dummies_test.columns)]
                                if len(c) != 0 :
                                    stage_dummies_test[c] = 0

                                # Reordering columns                
                                stage_dummies_test = stage_dummies_test.reindex(sorted(stage_dummies_test.columns), axis=1)
                                combined_dataset_test = pd.concat([combined_dataset_test, stage_dummies_test], axis=1)
                                combined_dataset_test.drop(["Intial_weeks"], inplace=True, axis=1)

                                # CHANGES : For adcal discounts
                                if 'All' in list(test_period.value):
                                    # Discount change
                                    if i == 1:                
                                        combined_dataset_test['Adcal_DD'] = (1+disc_change_fil)*combined_dataset_test['Adcal_DD']
                                        combined_dataset_test.loc[:,'EDV.Price'] = (1+base_price_change_fil)*combined_dataset_test.loc[:,'EDV.Price']
                                    elif i == 2:
                                        combined_dataset_test['Adcal_DD'] = (1+disc_change_fil2)*combined_dataset_test['Adcal_DD']
                                        combined_dataset_test.loc[:,'EDV.Price'] = (1+base_price_change_fil2)*combined_dataset_test.loc[:,'EDV.Price']          
                                else:                
                                    if i == 1:                
                                        # Discount change
                                        combined_dataset_test.loc[combined_dataset_test['Test_period'].isin(test_period_fil),'Adcal_DD'] = (1+disc_change_fil)*combined_dataset_test[combined_dataset_test['Test_period'].isin(test_period_fil)]['Adcal_DD'] 
                                        # Baseline pricing
                                        combined_dataset_test.loc[(combined_dataset_test['Test_period'].isin(test_period_fil)),'EDV.Price'] = (1+base_price_change_fil)*combined_dataset_test[(combined_dataset_test['Test_period'].isin(test_period_fil))]['EDV.Price']
                                    elif i == 2:
                                        # Discount change
                                        combined_dataset_test.loc[combined_dataset_test['Test_period'].isin(test_period_fil2),'Adcal_DD'] = (1+disc_change_fil2)*combined_dataset_test[combined_dataset_test['Test_period'].isin(test_period_fil2)]['Adcal_DD']             
                                        # Baseline pricing
                                        combined_dataset_test.loc[(combined_dataset_test['Test_period'].isin(test_period_fil2)),'EDV.Price'] = (1+base_price_change_fil2)*combined_dataset_test[(combined_dataset_test['Test_period'].isin(test_period_fil2))]['EDV.Price']

                                #Adding discount depth columns
                                combined_dataset["DD_1"] = [1 if (0.25> x >0.1)
                                                      else 0 for x in combined_dataset["Adcal_DD"]]
                                combined_dataset["DD_2"] = [1 if (x>=0.25)
                                                                 else 0 for x in combined_dataset["Adcal_DD"]]

                                combined_dataset_test["DD_1"] = [1 if (0.25> x >0.1)
                                                      else 0 for x in combined_dataset_test["Adcal_DD"]]
                                combined_dataset_test["DD_2"] = [1 if (x>=0.25)
                                                                 else 0 for x in combined_dataset_test["Adcal_DD"]]            

                                # Adcal Price calculation
                                # In case the discounts go over 100%, limit to 100%
                                combined_dataset_test.loc[combined_dataset_test.Adcal_DD >= 1, 'Adcal_DD'] = 0.99

                                combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset_test.loc[combined_dataset_test['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset_test.loc[combined_dataset_test['Adcal_DD']>=0.1,'EDV.Price']
                                # Adcal_DD < 10 then Adcal Price = EDV Price            
                                combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] < 0.1, 'EDV.Price']

                                # Adcal Price calculation
                                combined_dataset.loc[combined_dataset['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'EDV.Price']
                                # Adcal_DD < 10 then Adcal Price = EDV Price            
                                combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'EDV.Price']

                                # Dropping adcal DD as DD1 & DD2 are added
                                combined_dataset.drop(["Adcal_DD"], inplace=True, axis=1)
                                combined_dataset_test.drop(["Adcal_DD"], inplace=True, axis=1)

                                # Dropping null volumes
                                complete_dataset = combined_dataset.copy()
                                complete_dataset_test = combined_dataset_test.copy()

                                complete_dataset = complete_dataset.loc[complete_dataset["Eq.Unit.Sales"].notnull()]
                                complete_dataset_test = complete_dataset_test.loc[complete_dataset_test["Eq.Unit.Sales"].notnull()]

                                complete_dataset['Adcal_Price'] = np.log(complete_dataset['Adcal_Price'])
                                complete_dataset['Eq.Unit.Sales'] = np.log(complete_dataset['Eq.Unit.Sales'])

                                complete_dataset_test['Adcal_Price'] = np.log(complete_dataset_test['Adcal_Price'])
                                complete_dataset_test['Eq.Unit.Sales'] = np.log(complete_dataset_test['Eq.Unit.Sales'])

                                # TEST PERIOD LOOP
                                Test_periods = ['2019Q1','2019Q2','2019Q3','2019Q4']

                                for Test_period in Test_periods:
                                    if (Test_period>end_week.children[8].value) | (Test_period > Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Product == product_fil, 'Test_period'].max()):
                                        continue
                                    if (Test_period<start_week.children[8].value) | (Test_period < Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Product == product_fil, 'Test_period'].min()):
                                        continue

                                    #Filtering test weeks
                                    Test_weeks = Test_week_list[Test_period]

                                    # Dividing train & test data
                                    complete_train_data_set = complete_dataset.loc[~complete_dataset["Week"].isin(Test_weeks)].reset_index(drop=True)
                                    complete_test_data_set = complete_dataset_test.loc[complete_dataset_test["Week"].isin(Test_weeks)].reset_index(drop=True)

                                    # Dropping unneccessary columns from train dataset
                                    train_data_set = complete_train_data_set.copy()

                                    train_data_brand = train_data_set.copy()
                                    train_data_brand.drop(["Week","Week.Name","Banner","Product","Channel","EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)

                                    # Filtering for existing product
                                    test_data_set = complete_test_data_set[complete_test_data_set.Product == product_fil].copy()

                                    #################### INCREASING PROMOTIONS FOR BANNER WEEKS COMBINATIONS ####################
                                    # Get banner week promos
                                    banner_week_promos_combo = test_data_set[['Banner','Week','Front.Page','Middle.Page','Back.Page']].drop_duplicates()
                                    banner_week_upc = banner_week_promos_combo.copy()
                                    # Get rows where there is availability of increase in prmotions
                                    avail_promo_weeks = banner_week_upc[(banner_week_upc['Front.Page'] == 0) & (banner_week_upc['Middle.Page'] == 0) & (banner_week_upc['Back.Page'] == 0)]
                                    # Increasing 1 Back page promotions for every week
                                    avail_promo_weeks['Back.Page'] = 1

                                    # Merge to get required increase in banner-week combintation
                                    test_data_set_promo_check = test_data_set.merge(avail_promo_weeks[['Banner','Week','Back.Page']], how = 'left', on = ['Banner','Week'])

                                    # Filling na with existing values in Back.Page_y
                                    test_data_set_promo_check['Back.Page_y'].fillna(test_data_set_promo_check['Back.Page_x'], inplace = True)
                                    # Replacing values of 'Back.Page_x' with 'Back.Page_y' wherever it was zero previously
                                    test_data_set_promo_check['Back.Page_x'] = np.where(test_data_set_promo_check['Back.Page_x'] == 0, test_data_set_promo_check['Back.Page_y'], test_data_set_promo_check['Back.Page_x'])
                                    # Drop 'Back.Page_y' 
                                    test_data_set_promo_check.drop('Back.Page_y', axis = 1, inplace =True)
                                    # Renaming 'Back.Page_x'
                                    test_data_set_promo_check.rename(columns = {'Back.Page_x':'Back.Page'}, inplace = True)

                                    test_data_set = test_data_set_promo_check.copy()

                                    # New Category
                                    if i == 1:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)].Category.unique():
                                            if (cols == category_drop.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0
                                    if i == 2:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)].Category.unique():
                                            if (cols == category_drop2.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0

                                    # New Pack Subtype
                                    if i == 1:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['Pack.Subtype'].unique():
                                            if (cols == pack_subtype_drop.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0
                                    if i == 2:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['Pack.Subtype'].unique():
                                            if (cols == pack_subtype_drop2.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0
                                    # New Pack Content
                                    if i == 1:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['PACK_CONTENT'].unique():
                                            if (cols == pack_content_drop.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0
                                    if i == 2:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['PACK_CONTENT'].unique():
                                            if (cols == pack_content_drop2.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0

                                    test_data_brand = test_data_set.copy()

                                    if i == 1 :
                                        # For New SIZE
                                        test_data_brand['SIZE_ML'] = size.value

                                        # For New Count
                                        test_data_brand['COUNT'] = count.value
                                    elif i == 2:
                                        # For New SIZE
                                        test_data_brand['SIZE_ML'] = size2.value

                                        # For New Count
                                        test_data_brand['COUNT'] = count2.value

                                    # Dropping unneccessary columns from test dataset
                                    test_data_brand.drop(["Week","Week.Name","Banner","Product","Channel","EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)

                                    #Creating test and train data - independent & dependent columns
                                    X_train = train_data_brand
                                    y_train = train_data_set["Eq.Unit.Sales"]
                                    X_test = test_data_brand
                                    y_test = test_data_set["Eq.Unit.Sales"]

                                    # Appending test data
                                    test_data_set["Region"] = Region_key
                                    test_data_set["Test_period"] = Test_period
                                    # In case the product is not present in certain region-quarter
                                    if len(test_data_set) == 0:
                                        continue

                                    Test_Data = Test_Data.append(test_data_set, ignore_index = True)

                                    if((Region_key == 'EAST') and (Test_period == '2019Q1')):
                                            rf = model_object_EAST_2019Q1_v2
                                    elif((Region_key == 'EAST') and (Test_period == '2019Q2')):
                                            rf = model_object_EAST_2019Q2_v2
                                    elif((Region_key == 'EAST') and (Test_period == '2019Q3')):
                                            rf = model_object_EAST_2019Q3_v2
                                    elif((Region_key == 'EAST') and (Test_period == '2019Q4')):
                                            rf = model_object_EAST_2019Q4_v2
                                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q1')):
                                            rf = model_object_ONTARIO_2019Q1_v2
                                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q2')):
                                            rf = model_object_ONTARIO_2019Q2_v2
                                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q3')):
                                            rf = model_object_ONTARIO_2019Q3_v2
                                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q4')):
                                            rf = model_object_ONTARIO_2019Q4_v2
                                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q1')):
                                            rf = model_object_QUEBEC_2019Q1_v2
                                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q2')):
                                            rf = model_object_QUEBEC_2019Q2_v2
                                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q3')):
                                            rf = model_object_QUEBEC_2019Q3_v2
                                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q4')):
                                            rf = model_object_QUEBEC_2019Q4_v2
                                    elif((Region_key == 'WEST') and (Test_period == '2019Q1')):
                                            rf = model_object_WEST_2019Q1_v2
                                    elif((Region_key == 'WEST') and (Test_period == '2019Q2')):
                                            rf = model_object_WEST_2019Q2_v2
                                    elif((Region_key == 'WEST') and (Test_period == '2019Q3')):
                                            rf = model_object_WEST_2019Q3_v2
                                    else:
                                            rf = model_object_WEST_2019Q4_v2
                                            
                                    #Model predictions
                                    predictions_rf = rf.predict(X_test)                

                                    #Storing test results
                                    result = X_test.copy()
                                    result["Predictions_rf"] = predictions_rf
                                    result["Actuals"] = y_test

                                    result['Predictions_rf'] = np.exp(result['Predictions_rf'])                    
                                    result['Actuals'] = np.exp(result['Actuals'])                                                    

                                    result["Product"] = test_data_set["Product"]
                                    result["Region"] = Region_key
                                    result["Test_period"] = Test_period
                                    result["Banner"] = test_data_set["Banner"]
                                    result["Channel"] = test_data_set['Channel']
                                    result["Week"] = test_data_set["Week"]
                                    result['iter'] = i

                                    result = result[["Region","Banner","Product",'Channel',"Test_period","Week","Actuals","Predictions_rf",'iter']]

                                    #Appending results to the final results dataframe
                                    Test_results = Test_results.append(result, ignore_index=True)                

                            Test_results = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)]
                            # Removing 0 promotion banners
                            if len(qtr_df_final[(qtr_df_final.Banner.isin(fp_remove_banner)) | (qtr_df_final.Banner.isin(mp_remove_banner)) | (qtr_df_final.Banner.isin(bp_remove_banner))]) != 0:
                                if zero_banner_promo_inc.value == 'No':
                                    Test_results = Test_results[~Test_results.Banner.isin(bp_remove_banner)]

                            # Merge & get prediction difference column
                            Test_results = Test_results[Test_results.iter == i].merge(Test_results_sim[Test_results_sim.iter == i][['Banner','Week','Baseline Prediction']], on = ['Banner','Week'], how = 'left')        
                            Test_results['Diff'] = Test_results[Test_results.iter == i]['Predictions_rf'] - Test_results[Test_results.iter == i]['Baseline Prediction']
                            # Consider positive difference only
                            Test_results = Test_results[Test_results.iter == i][Test_results[Test_results.iter == i].Diff > 0]
                            Test_results = Test_results[Test_results.iter == i].sort_values('Diff', ascending = False)

                            # Filtering out front & middle page promotion banner weeks selected for priority
                            if change_fp != 0:
                                if i == 1:
                                    Test_results['ban_week_bp_check'] = Test_results['Banner'] + Test_results['Week']       
                                    Test_results = Test_results[~Test_results['ban_week_bp_check'].isin(ban_week_fp['ban_week_fp_check'].unique())]
                                elif i == 2:
                                    Test_results['ban_week_bp_check'] = Test_results['Banner'] + Test_results['Week']       
                                    Test_results = Test_results[~Test_results['ban_week_bp_check'].isin(ban_week_fp2['ban_week_fp_check'].unique())]

                            if change_mp != 0:
                                if i == 1:
                                    Test_results['ban_week_bp_check'] = Test_results['Banner'] + Test_results['Week']       
                                    Test_results = Test_results[~Test_results['ban_week_bp_check'].isin(ban_week_mp['ban_week_mp_check'].unique())]
                                elif i == 2:
                                    Test_results['ban_week_bp_check'] = Test_results['Banner'] + Test_results['Week']       
                                    Test_results = Test_results[~Test_results['ban_week_bp_check'].isin(ban_week_mp2['ban_week_mp_check'].unique())]                        

                            # Select required number of banner-weeks where promotion increase would occur
                            if (promo_change_toggle.value == 'Yes (Specific Banners)'):         
                                Test_results = Test_results[Test_results.Banner.isin(list(banner_select.value))]
                                for banner_loop in banner_select.value:    
                                    # Selecting banner-weeks
                                    ban_week_bp0 = Test_results[Test_results.Banner == banner_loop][:abs(int(change_bp))][['Banner','Week']]
                                    # if iter = 1, iter = 2 loop
                                    if i == 1:
                                        ban_week_bp = ban_week_bp.append(ban_week_bp0, ignore_index = True)
                                        # Banner-week concat column    
                                        ban_week_bp['ban_week_bp_check'] = ban_week_bp['Banner'] + ban_week_bp['Week']                            

                                    elif i == 2:
                                        ban_week_bp2 = ban_week_bp2.append(ban_week_bp0, ignore_index = True)
                                        # Banner-week concat column    
                                        ban_week_bp2['ban_week_bp_check'] = ban_week_bp2['Banner'] + ban_week_bp2['Week']                             

                            elif (promo_change_toggle.value == 'Yes (All Banners)'):
                                for banner_loop in qtr_df_final.Banner.unique():    
                                    # Selecting banner-weeks
                                    ban_week_bp0 = Test_results[Test_results.Banner == banner_loop][:abs(int(change_bp))][['Banner','Week']]
                                    # if iter = 1, iter = 2 loop
                                    if i == 1:
                                        ban_week_bp = ban_week_bp.append(ban_week_bp0, ignore_index = True)
                                        # Banner-week concat column    
                                        ban_week_bp['ban_week_bp_check'] = ban_week_bp['Banner'] + ban_week_bp['Week']                            
                                    elif i == 2:
                                        ban_week_bp2 = ban_week_bp2.append(ban_week_bp0, ignore_index = True)
                                        # Banner-week concat column    
                                        ban_week_bp2['ban_week_bp_check'] = ban_week_bp2['Banner'] + ban_week_bp2['Week'] 

                            if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach                        
                                ban_week_bp['Product'] = product_fil
                            else:
                                if i == 1:
                                    ban_week_bp['Product'] = product_fil
                                elif i == 2:
                                    ban_week_bp2['Product'] = product_fil

                        ##########################################################################################################
                                                        # BACK PAGE DECREASE SIMULATION 
                        ##########################################################################################################    

                        if i == 1:
                            change_bp = back_change.children[1].value
                        elif i == 2:
                            change_bp = back_change2.children[1].value

                        if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
                            if product_fil == edv_disc_join['Existing Model Pack'].unique()[0]:
                                change_bp = back_change.children[1].value
                            elif product_fil == edv_disc_join['Existing Model Pack'].unique()[1]:
                                change_bp = back_change2.children[1].value
                            elif product_fil == edv_disc_join['Existing Model Pack'].unique()[2]:
                                change_bp = back_change3.children[1].value
                            
                        if (change_bp < 0) & ((qtr_df_final['Back Page']).sum() != 0):

                            ################## BACK PAGE SIMULATION ##################
                            import warnings
                            warnings.filterwarnings('ignore')

                            # Defining empty dataframes for model training
                            All_coefs = pd.DataFrame({'Region': pd.Series([], dtype='object')})
                            Train_results = pd.DataFrame({'Region': pd.Series([], dtype='object')})
                            Test_results = pd.DataFrame({'Region': pd.Series([], dtype='object')})
                            train_data_brand = pd.DataFrame({'Week': pd.Series([], dtype='object')})
                            combined_dataset = pd.DataFrame({'Week': pd.Series([], dtype='object')})
                            Test_Data = pd.DataFrame()    

                            #Model training and prediction
                            for Region_key in Region_List:
                                # In case the product is not present at that region
                                if len(Volume_dataset_all_reg[(Volume_dataset_all_reg.Product == product_fil) & (Volume_dataset_all_reg.Region == Region_key)]) == 0:
                                    continue

                                Volume_dataset = Volume_dataset_all_reg.loc[(Volume_dataset_all_reg["Region"] == Region_key)
                                                                           &(Volume_dataset_all_reg["Channel"].isin(channel_list))]
                                Volume_dataset['Pantry2'] = Volume_dataset['Pantry2'].fillna(0)
                                required_columns = ['Week','Week.Name','Banner','Product','Channel','Pack.Subtype','PACK_CONTENT','Adcal_Price','EDV.Price','Adcal_DD','Eq.Unit.Sales','Front.Page','Middle.Page','Back.Page','seasonality_index','SIZE_ML','COUNT','No_of_brands','No_of_flavors','No_of_sweetners','No_of_types','Category','Pantry1','Pantry2','Holiday.Week','Pre.Holiday.Week','Post.Holiday.Week']
                                combined_dataset = Volume_dataset[required_columns]

                                combined_dataset = combined_dataset.loc[((combined_dataset["Week"] >= "2017-01") & 
                                                                         (combined_dataset["Week"] <= "2019-52"))]

                                combined_dataset["Christmas_flag"] = [1 if x=="Christmas" else 0 for x in combined_dataset["Week.Name"]]
                                combined_dataset["Easter_flag"] = [1 if x=="Easter" else 0 for x in combined_dataset["Week.Name"]]

                                ## CHECKING if banner dummies are not present in required region, channel then introduce banner dummies
                                banner_dummies = pd.get_dummies(combined_dataset.Banner)              

                                # Uploaded data banner dummies
                                banner_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key].Banner)                                 
                                # If all banners are not present for the training data
                                c = [i for i in list(banner_dummies_check.columns) if i not in list(banner_dummies.columns)]
                                if len(c) != 0:
                                    banner_dummies[c] = 0

                                # Reordering columns                
                                banner_dummies = banner_dummies.reindex(sorted(banner_dummies.columns), axis=1)                
                                combined_dataset = pd.concat([combined_dataset, banner_dummies], axis=1)

                                ## Not adding Product dummies since for New Product Simulator other Product dummies are not significant
                                combined_dataset = combined_dataset.loc[combined_dataset["Eq.Unit.Sales"].notnull()]

                                combined_dataset.sort_values('Week',inplace = True)
                                combined_dataset = pd.merge(combined_dataset,new_prod_stage,on = ['Product','Week'],how = 'left')
                                combined_dataset['Intial_weeks'] = combined_dataset['Intial_weeks'].fillna('Stabilization')
                                combined_dataset.drop(columns='Unnamed: 0', inplace=True)

                                #Adding dummies for category
                                category_dummies = pd.get_dummies(combined_dataset['Category'])

                                # Adding dummies for product attribute - Category
                                pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
                                # Uploaded data category dummies
                                category_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['Category'])                                 
                                # If all pack subtypes are not present for the training data
                                c = [i for i in list(category_dummies_check.columns) if i not in list(category_dummies.columns)]
                                if len(c) != 0:
                                    category_dummies[c] = 0

                                # Reordering columns                
                                category_dummies = category_dummies.reindex(sorted(category_dummies.columns), axis=1)                
                                combined_dataset = pd.concat([combined_dataset, category_dummies], axis=1)
                                combined_dataset.drop(["Category"], inplace=True, axis=1)

                                # Adding dummies for product attribute - pack subtype
                                pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
                                # Uploaded data pack subtype dummies
                                pack_subtypes_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['Pack.Subtype'])                                 
                                # If all pack subtypes are not present for the training data
                                c = [i for i in list(pack_subtypes_dummies_check.columns) if i not in list(pack_subtypes_dummies.columns)]
                                if len(c) != 0:
                                    pack_subtypes_dummies[c] = 0

                                # Reordering columns                
                                pack_subtypes_dummies = pack_subtypes_dummies.reindex(sorted(pack_subtypes_dummies.columns), axis=1)                
                                combined_dataset = pd.concat([combined_dataset, pack_subtypes_dummies], axis=1)
                                combined_dataset.drop(["Pack.Subtype"], inplace=True, axis=1)

                                # Adding pack content dummies
                                pack_content_dummies = pd.get_dummies(combined_dataset['PACK_CONTENT'])
                                # Uploaded data pack content dummies
                                pack_content_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['PACK_CONTENT'])                                 
                                # If all pack contents are not present for the training data
                                c = [i for i in list(pack_content_dummies_check.columns) if i not in list(pack_content_dummies.columns)]
                                if len(c) != 0:
                                    pack_content_dummies[c] = 0

                                # Reordering columns                
                                pack_content_dummies = pack_content_dummies.reindex(sorted(pack_content_dummies.columns), axis=1)                
                                combined_dataset = pd.concat([combined_dataset, pack_content_dummies], axis=1)
                                combined_dataset.drop(["PACK_CONTENT"], inplace=True, axis=1)

                                combined_dataset_test = combined_dataset.copy()

                                # Product stage dummies
                                stage_dummies = pd.get_dummies(combined_dataset['Intial_weeks'])
                                combined_dataset = pd.concat([combined_dataset, stage_dummies], axis=1)
                                combined_dataset.drop(["Intial_weeks"], inplace=True, axis=1)

                                # Changing product stage as per start date provided by the user
                                prod_stage_data = prod_stage_check(product_fil)            

                                # Getting updated product stages
                                combined_dataset_test_check = combined_dataset_test[combined_dataset_test.Product == product_fil].merge(prod_stage_data[['Week','Initial_weeks1']], how = 'left')
                                combined_dataset_test_check['Initial_weeks1'] = combined_dataset_test_check['Initial_weeks1'].fillna(combined_dataset_test_check['Intial_weeks'])
                                combined_dataset_test_check.drop(columns = {'Intial_weeks'}, inplace = True)
                                combined_dataset_test_check.rename(columns = {'Initial_weeks1':'Intial_weeks'}, inplace = True)

                                combined_dataset_test = combined_dataset_test_check.copy()

                                # Product stage dummies
                                stage_dummies_test = pd.get_dummies(combined_dataset_test['Intial_weeks'])

                                # If all product stages are not present for the uploaded data
                                c = [i for i in list(stage_dummies.columns) if i not in list(stage_dummies_test.columns)]
                                if len(c) != 0 :
                                    stage_dummies_test[c] = 0

                                # Reordering columns                
                                stage_dummies_test = stage_dummies_test.reindex(sorted(stage_dummies_test.columns), axis=1)
                                combined_dataset_test = pd.concat([combined_dataset_test, stage_dummies_test], axis=1)
                                combined_dataset_test.drop(["Intial_weeks"], inplace=True, axis=1)

                                # CHANGES : For adcal discounts
                                if 'All' in list(test_period.value):
                                    # Discount change
                                    if i == 1:                
                                        combined_dataset_test['Adcal_DD'] = (1+disc_change_fil)*combined_dataset_test['Adcal_DD']
                                        combined_dataset_test.loc[:,'EDV.Price'] = (1+base_price_change_fil)*combined_dataset_test.loc[:,'EDV.Price']
                                    elif i == 2:
                                        combined_dataset_test['Adcal_DD'] = (1+disc_change_fil2)*combined_dataset_test['Adcal_DD']
                                        combined_dataset_test.loc[:,'EDV.Price'] = (1+base_price_change_fil2)*combined_dataset_test.loc[:,'EDV.Price']          
                                else:                
                                    if i == 1:                
                                        # Discount change
                                        combined_dataset_test.loc[combined_dataset_test['Test_period'].isin(test_period_fil),'Adcal_DD'] = (1+disc_change_fil)*combined_dataset_test[combined_dataset_test['Test_period'].isin(test_period_fil)]['Adcal_DD'] 
                                        # Baseline pricing
                                        combined_dataset_test.loc[(combined_dataset_test['Test_period'].isin(test_period_fil)),'EDV.Price'] = (1+base_price_change_fil)*combined_dataset_test[(combined_dataset_test['Test_period'].isin(test_period_fil))]['EDV.Price']
                                    elif i == 2:
                                        # Discount change
                                        combined_dataset_test.loc[combined_dataset_test['Test_period'].isin(test_period_fil2),'Adcal_DD'] = (1+disc_change_fil2)*combined_dataset_test[combined_dataset_test['Test_period'].isin(test_period_fil2)]['Adcal_DD']             
                                        # Baseline pricing
                                        combined_dataset_test.loc[(combined_dataset_test['Test_period'].isin(test_period_fil2)),'EDV.Price'] = (1+base_price_change_fil2)*combined_dataset_test[(combined_dataset_test['Test_period'].isin(test_period_fil2))]['EDV.Price']

                                #Adding discount depth columns
                                combined_dataset["DD_1"] = [1 if (0.25> x >0.1)
                                                      else 0 for x in combined_dataset["Adcal_DD"]]
                                combined_dataset["DD_2"] = [1 if (x>=0.25)
                                                                 else 0 for x in combined_dataset["Adcal_DD"]]

                                combined_dataset_test["DD_1"] = [1 if (0.25> x >0.1)
                                                      else 0 for x in combined_dataset_test["Adcal_DD"]]
                                combined_dataset_test["DD_2"] = [1 if (x>=0.25)
                                                                 else 0 for x in combined_dataset_test["Adcal_DD"]]            

                                # Adcal Price calculation
                                # In case the discounts go over 100%, limit to 100%
                                combined_dataset_test.loc[combined_dataset_test.Adcal_DD >= 1, 'Adcal_DD'] = 0.99

                                combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset_test.loc[combined_dataset_test['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset_test.loc[combined_dataset_test['Adcal_DD']>=0.1,'EDV.Price']
                                # Adcal_DD < 10 then Adcal Price = EDV Price            
                                combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] < 0.1, 'EDV.Price']

                                # Adcal Price calculation
                                combined_dataset.loc[combined_dataset['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'EDV.Price']
                                # Adcal_DD < 10 then Adcal Price = EDV Price            
                                combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'EDV.Price']

                                # Dropping adcal DD as DD1 & DD2 are added
                                combined_dataset.drop(["Adcal_DD"], inplace=True, axis=1)
                                combined_dataset_test.drop(["Adcal_DD"], inplace=True, axis=1)

                                # Dropping null volumes
                                complete_dataset = combined_dataset.copy()
                                complete_dataset_test = combined_dataset_test.copy()

                                complete_dataset = complete_dataset.loc[complete_dataset["Eq.Unit.Sales"].notnull()]
                                complete_dataset_test = complete_dataset_test.loc[complete_dataset_test["Eq.Unit.Sales"].notnull()]

                                complete_dataset['Adcal_Price'] = np.log(complete_dataset['Adcal_Price'])
                                complete_dataset['Eq.Unit.Sales'] = np.log(complete_dataset['Eq.Unit.Sales'])

                                complete_dataset_test['Adcal_Price'] = np.log(complete_dataset_test['Adcal_Price'])
                                complete_dataset_test['Eq.Unit.Sales'] = np.log(complete_dataset_test['Eq.Unit.Sales'])

                                # TEST PERIOD LOOP
                                Test_periods = ['2019Q1','2019Q2','2019Q3','2019Q4']

                                for Test_period in Test_periods:
                                    if (Test_period>end_week.children[8].value) | (Test_period > Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Product == product_fil, 'Test_period'].max()):
                                        continue
                                    if (Test_period<start_week.children[8].value) | (Test_period < Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Product == product_fil, 'Test_period'].min()):
                                        continue

                                    #Filtering test weeks
                                    Test_weeks = Test_week_list[Test_period]

                                    # Dividing train & test data
                                    complete_train_data_set = complete_dataset.loc[~complete_dataset["Week"].isin(Test_weeks)].reset_index(drop=True)
                                    complete_test_data_set = complete_dataset_test.loc[complete_dataset_test["Week"].isin(Test_weeks)].reset_index(drop=True)

                                    # Dropping unneccessary columns from train dataset
                                    train_data_set = complete_train_data_set.copy()

                                    train_data_brand = train_data_set.copy()
                                    train_data_brand.drop(["Week","Week.Name","Banner","Product","Channel","EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)

                                    # Filtering for existing product
                                    test_data_set = complete_test_data_set[complete_test_data_set.Product == product_fil].copy()

                                    #################### DECREASING PROMOTIONS FOR BANNER WEEKS COMBINATIONS ####################
                                    # Allotting all back page promos as 0
                                    back_page_promo_weeks = pd.DataFrame(test_data_set[test_data_set['Back.Page'] != 0][['Banner','Week']])
                                    back_page_promo_weeks_all = back_page_promo_weeks_all.append(back_page_promo_weeks, ignore_index = True)            
                                    test_data_set['Back.Page'] = 0

                                    # New Category
                                    if i == 1:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)].Category.unique():
                                            if (cols == category_drop.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0
                                    if i == 2:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)].Category.unique():
                                            if (cols == category_drop2.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0

                                    # New Pack Subtype
                                    if i == 1:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['Pack.Subtype'].unique():
                                            if (cols == pack_subtype_drop.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0
                                    if i == 2:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['Pack.Subtype'].unique():
                                            if (cols == pack_subtype_drop2.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0
                                    # New Pack Content
                                    if i == 1:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['PACK_CONTENT'].unique():
                                            if (cols == pack_content_drop.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0
                                    if i == 2:
                                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['PACK_CONTENT'].unique():
                                            if (cols == pack_content_drop2.value):                      
                                                test_data_set[cols] = 1
                                            else:
                                                test_data_set[cols] = 0

                                    test_data_brand = test_data_set.copy()

                                    if i == 1 :
                                        # For New SIZE
                                        test_data_brand['SIZE_ML'] = size.value

                                        # For New Count
                                        test_data_brand['COUNT'] = count.value
                                    elif i == 2:
                                        # For New SIZE
                                        test_data_brand['SIZE_ML'] = size2.value

                                        # For New Count
                                        test_data_brand['COUNT'] = count2.value

                                    # Dropping unneccessary columns from test dataset
                                    test_data_brand.drop(["Week","Week.Name","Banner","Product","Channel","EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)

                                    #Creating test and train data - independent & dependent columns
                                    X_train = train_data_brand
                                    y_train = train_data_set["Eq.Unit.Sales"]
                                    X_test = test_data_brand
                                    y_test = test_data_set["Eq.Unit.Sales"]

                                    # Appending test data
                                    test_data_set["Region"] = Region_key
                                    test_data_set["Test_period"] = Test_period

                                    Test_Data = Test_Data.append(test_data_set, ignore_index = True)
                                    # In case the product is not present in certain region-quarter
                                    if len(test_data_set) == 0:
                                        continue

                                    if((Region_key == 'EAST') and (Test_period == '2019Q1')):
                                            rf = model_object_EAST_2019Q1_v2
                                    elif((Region_key == 'EAST') and (Test_period == '2019Q2')):
                                            rf = model_object_EAST_2019Q2_v2
                                    elif((Region_key == 'EAST') and (Test_period == '2019Q3')):
                                            rf = model_object_EAST_2019Q3_v2
                                    elif((Region_key == 'EAST') and (Test_period == '2019Q4')):
                                            rf = model_object_EAST_2019Q4_v2
                                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q1')):
                                            rf = model_object_ONTARIO_2019Q1_v2
                                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q2')):
                                            rf = model_object_ONTARIO_2019Q2_v2
                                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q3')):
                                            rf = model_object_ONTARIO_2019Q3_v2
                                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q4')):
                                            rf = model_object_ONTARIO_2019Q4_v2
                                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q1')):
                                            rf = model_object_QUEBEC_2019Q1_v2
                                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q2')):
                                            rf = model_object_QUEBEC_2019Q2_v2
                                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q3')):
                                            rf = model_object_QUEBEC_2019Q3_v2
                                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q4')):
                                            rf = model_object_QUEBEC_2019Q4_v2
                                    elif((Region_key == 'WEST') and (Test_period == '2019Q1')):
                                            rf = model_object_WEST_2019Q1_v2
                                    elif((Region_key == 'WEST') and (Test_period == '2019Q2')):
                                            rf = model_object_WEST_2019Q2_v2
                                    elif((Region_key == 'WEST') and (Test_period == '2019Q3')):
                                            rf = model_object_WEST_2019Q3_v2
                                    else:
                                            rf = model_object_WEST_2019Q4_v2

                                    # Model predictions
                                    predictions_rf = rf.predict(X_test)                

                                    # Storing test results
                                    result = X_test.copy()
                                    result["Predictions_rf"] = predictions_rf
                                    result["Actuals"] = y_test

                                    result['Predictions_rf'] = np.exp(result['Predictions_rf'])                    
                                    result['Actuals'] = np.exp(result['Actuals'])                                                    

                                    result["Banner"] = test_data_set['Banner']
                                    result["Channel"] = test_data_set['Channel']
                                    result["Product"] = test_data_set["Product"]
                                    result["Region"] = Region_key
                                    result["Test_period"] = Test_period
                                    result["Week"] = test_data_set["Week"]
                                    result["Front.Page"] = test_data_set["Front.Page"]
                                    result["Middle.Page"] = test_data_set["Middle.Page"]
                                    result["Back.Page"] = test_data_set["Back.Page"]
                                    result['iter'] = i

                                    result = result[["Region","Banner","Product",'Channel',"Test_period","Week",'Front.Page','Middle.Page','Back.Page',"Actuals","Predictions_rf",'iter']]

                                    # Appending results to the final results dataframe
                                    Test_results = Test_results.append(result, ignore_index=True)                

                            Test_results = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)]                            
                            # Merge & get prediction difference column
                            Test_results = Test_results[Test_results.iter == i].merge(Test_results_sim[Test_results_sim.iter == i][['Banner','Week','Baseline Prediction']], on = ['Banner','Week'], how = 'left')        
                            Test_results['Diff'] = Test_results[Test_results.iter == i]['Predictions_rf'] - Test_results[Test_results.iter == i]['Baseline Prediction']
                            # Consider positive difference only
                            Test_results = Test_results[Test_results.iter == i][Test_results[Test_results.iter == i].Diff > 0]
                            Test_results = Test_results[Test_results.iter == i].sort_values('Diff', ascending = False)

                            # Filtering out banner weeks where middle & front page promotion were given 
                            Test_results = Test_results[(Test_results['Middle.Page'] == 0) & (Test_results['Front.Page'] == 0)]    

                            # Filtering banner weeks where back page promotion was given previously
                            Test_results['Ban_week_concat'] = Test_results['Banner'] + Test_results['Week']
                            back_page_promo_weeks_all['Ban_week_concat'] = back_page_promo_weeks_all['Banner'] + back_page_promo_weeks_all['Week']    
                            Test_results = Test_results[Test_results['Ban_week_concat'].isin(back_page_promo_weeks_all['Ban_week_concat'].unique())]

                            # Select required number of banner-weeks where promotion decrease would occur
                            if (promo_change_toggle.value == 'Yes (Specific Banners)'):         
                                Test_results = Test_results[Test_results.Banner.isin(list(banner_select.value))]
                                for banner_loop in banner_select.value:    
                                    # Selecting banner-weeks
                                    ban_week_bp0 = Test_results[Test_results.Banner == banner_loop][:abs(int(change_bp))][['Banner','Week']]
                                    # if iter = 1, iter = 2 loop
                                    if i == 1:
                                        ban_week_bp = ban_week_bp.append(ban_week_bp0, ignore_index = True)
                                    elif i == 2:
                                        ban_week_bp2 = ban_week_bp2.append(ban_week_bp0, ignore_index = True)

                            elif (promo_change_toggle.value == 'Yes (All Banners)'):
                                for banner_loop in qtr_df_final.Banner.unique():    
                                    # Selecting banner-weeks
                                    ban_week_bp0 = Test_results[Test_results.Banner == banner_loop][:abs(int(change_bp))][['Banner','Week']]
                                    # if iter = 1, iter = 2 loop
                                    if i == 1:
                                        ban_week_bp = ban_week_bp.append(ban_week_bp0, ignore_index = True)
                                    elif i == 2:
                                        ban_week_bp2 = ban_week_bp2.append(ban_week_bp0, ignore_index = True)

                            if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach                        
                                ban_week_bp['Product'] = product_fil
                            else:
                                if i == 1:
                                    ban_week_bp['Product'] = product_fil
                                elif i == 2:
                                    ban_week_bp2['Product'] = product_fil    

                        if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach                        
                            ban_week_fp_copy = ban_week_fp_copy.append(ban_week_fp, ignore_index = True)
                            ban_week_mp_copy = ban_week_mp_copy.append(ban_week_mp, ignore_index = True)
                            ban_week_bp_copy = ban_week_bp_copy.append(ban_week_bp, ignore_index = True)

                    clear_output()    


    else:
        print("Product is not present in that Region/Channel for selected period")

#### Updated Promotions

In [210]:
# In case the user doesn't want to upload adcal data
if toggle_upload.value != 'Yes':
    
    if len(qtr_edv_baseline) != 0 : # In case there is not product present or product not present in selected region-channel & test period

        if (promo_change_toggle.value == 'Yes (All Banners)') | (promo_change_toggle.value == 'Yes (Specific Banners)'):
            if (front_change.children[1].value != 0) | (middle_change.children[1].value != 0) | (back_change.children[1].value != 0) | (front_change2.children[1].value != 0) | (middle_change2.children[1].value != 0) | (back_change2.children[1].value != 0) | (front_change3.children[1].value != 0) | (middle_change3.children[1].value != 0) | (back_change3.children[1].value != 0) : 

                display(Markdown('<div><div class="loader"></div><h2> &nbsp; Updating promotions </h2></div>'))
                qtr_df = qtr_df_final.groupby(['Existing Model Pack','Banner']).agg({'Front Page':'sum','Middle Page':'sum','Back Page':'sum'}).reset_index()
                qtr_df2 = qtr_df.copy()

                for exist_pack in sorted(qtr_df_final['Existing Model Pack'].unique()): # Looping through each existing product

                    # Increase/decrease promotions for required banners
                    # Front increase
                    if exist_pack == edv_disc_join['Existing Model Pack'].unique()[0]:
                        if front_change.children[1].value > 0:
                            if toggle_ensemble_normal.value != 'Multiple Product': # In case of normal approach
                                ban_week_fp_copy = ban_week_fp.copy()
                                
                            for banner_change in ban_week_fp_copy[ban_week_fp_copy.Product == exist_pack]['Banner'].unique(): 
                                qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Front Page'] = qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Front Page'].values[0] + ban_week_fp_copy[(ban_week_fp_copy.Product == exist_pack) & (ban_week_fp_copy['Banner'] == banner_change)].Banner.count()
                    
                    # Increasing promotions of 2nd exist product in ensemble approach
                    if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
                        if exist_pack == edv_disc_join['Existing Model Pack'].unique()[1]:
                            if front_change2.children[1].value > 0:
                                for banner_change in ban_week_fp_copy[ban_week_fp_copy.Product == exist_pack]['Banner'].unique(): 
                                    qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Front Page'] = qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Front Page'].values[0] + ban_week_fp_copy[(ban_week_fp_copy.Product == exist_pack) & (ban_week_fp_copy['Banner'] == banner_change)].Banner.count()
                    # Increasing promotions of 3rd exist product in ensemble approach
                        if exist_pack == edv_disc_join['Existing Model Pack'].unique()[2]:
                            if front_change3.children[1].value > 0:
                                for banner_change in ban_week_fp_copy[ban_week_fp_copy.Product == exist_pack]['Banner'].unique(): 
                                    qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Front Page'] = qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Front Page'].values[0] + ban_week_fp_copy[(ban_week_fp_copy.Product == exist_pack) & (ban_week_fp_copy['Banner'] == banner_change)].Banner.count()
                            
                    
                    # In case the user selects to simulate New Product 2
                    if toggle_ensemble_normal.value != 'Multiple Product': # In case of normal approach
                        if toggle_np2.value == 'Yes':
                            if front_change2.children[1].value > 0:
                                for banner_change in ban_week_fp2[ban_week_fp2.Product == exist_pack]['Banner'].unique(): 
                                    qtr_df2.loc[(qtr_df2['Existing Model Pack'] == exist_pack) & (qtr_df2['Banner'] == banner_change),'Front Page'] = qtr_df2.loc[(qtr_df2['Existing Model Pack'] == exist_pack) & (qtr_df2['Banner'] == banner_change),'Front Page'].values[0] + ban_week_fp2[(ban_week_fp2['Banner'] == banner_change) & (ban_week_fp2.Product == exist_pack)].Banner.count()
                                    
                    # Front decrease    
                    if exist_pack == edv_disc_join['Existing Model Pack'].unique()[0]:
                        if front_change.children[1].value < 0:
                            if toggle_ensemble_normal.value != 'Multiple Product': # In case of normal approach
                                ban_week_fp_copy = ban_week_fp.copy()
                            
                            for banner_change in ban_week_fp_copy[ban_week_fp_copy.Product == exist_pack]['Banner'].unique():
                                qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Front Page'] = qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Front Page'].values[0] - ban_week_fp_copy[(ban_week_fp_copy.Product == exist_pack) & (ban_week_fp_copy['Banner'] == banner_change)].Banner.count()

                    # Decreasing promotions of 2nd exist product in ensemble approach
                    if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
                        if exist_pack == edv_disc_join['Existing Model Pack'].unique()[1]:
                            if front_change2.children[1].value < 0:
                                for banner_change in ban_week_fp_copy[ban_week_fp_copy.Product == exist_pack]['Banner'].unique():
                                    qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Front Page'] = qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Front Page'].values[0] - ban_week_fp_copy[(ban_week_fp_copy.Product == exist_pack) & (ban_week_fp_copy['Banner'] == banner_change)].Banner.count()

                    # Decreasing promotions of 3rd exist product in ensemble approach
                    if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
                        if exist_pack == edv_disc_join['Existing Model Pack'].unique()[2]:
                            if front_change3.children[1].value < 0:
                                for banner_change in ban_week_fp_copy[ban_week_fp_copy.Product == exist_pack]['Banner'].unique():
                                    qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Front Page'] = qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Front Page'].values[0] - ban_week_fp_copy[(ban_week_fp_copy.Product == exist_pack) & (ban_week_fp_copy['Banner'] == banner_change)].Banner.count()


                    # In case the user selects to simulate New Product 2
                    if toggle_ensemble_normal.value != 'Multiple Product': # In case of normal approach
                        # In case the user selects to simulate New Product 2
                        if toggle_np2.value == 'Yes':                
                            if front_change2.children[1].value < 0:
                                for banner_change in ban_week_fp2[ban_week_fp2.Product == exist_pack]['Banner'].unique(): 
                                    qtr_df2.loc[(qtr_df2['Existing Model Pack'] == exist_pack) & (qtr_df2['Banner'] == banner_change),'Front Page'] = qtr_df2.loc[(qtr_df2['Existing Model Pack'] == exist_pack) & (qtr_df2['Banner'] == banner_change),'Front Page'].values[0] - ban_week_fp2[(ban_week_fp2['Banner'] == banner_change) & (ban_week_fp2.Product == exist_pack)].Banner.count()

                    # Middle increase
                    if exist_pack == edv_disc_join['Existing Model Pack'].unique()[0]:
                        if middle_change.children[1].value > 0:
                            if toggle_ensemble_normal.value != 'Multiple Product': # In case of normal approach
                                ban_week_mp_copy = ban_week_mp.copy()
                            
                            for banner_change in ban_week_mp_copy[ban_week_mp_copy.Product == exist_pack]['Banner'].unique(): 
                                qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Middle Page'] = qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Middle Page'].values[0] + ban_week_mp_copy[(ban_week_mp_copy.Product == exist_pack) & (ban_week_mp_copy['Banner'] == banner_change)].Banner.count()
                    
                    # Increasing promotions of 2nd exist product in ensemble approach
                    if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
                        if exist_pack == edv_disc_join['Existing Model Pack'].unique()[1]:
                            if middle_change2.children[1].value > 0:
                                for banner_change in ban_week_mp_copy[ban_week_mp_copy.Product == exist_pack]['Banner'].unique(): 
                                    qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Middle Page'] = qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Middle Page'].values[0] + ban_week_mp_copy[(ban_week_mp_copy.Product == exist_pack) & (ban_week_mp_copy['Banner'] == banner_change)].Banner.count()
                    # Increasing promotions of 3rd exist product in ensemble approach
                        if exist_pack == edv_disc_join['Existing Model Pack'].unique()[2]:
                            if middle_change3.children[1].value > 0:
                                for banner_change in ban_week_mp_copy[ban_week_mp_copy.Product == exist_pack]['Banner'].unique(): 
                                    qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Middle Page'] = qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Middle Page'].values[0] + ban_week_mp_copy[(ban_week_mp_copy.Product == exist_pack) & (ban_week_mp_copy['Banner'] == banner_change)].Banner.count()
                            
                    
                    # In case the user selects to simulate New Product 2
                    if toggle_ensemble_normal.value != 'Multiple Product': # In case of normal approach
                        if toggle_np2.value == 'Yes':
                            if middle_change2.children[1].value > 0:
                                for banner_change in ban_week_mp2[ban_week_mp2.Product == exist_pack]['Banner'].unique(): 
                                    qtr_df2.loc[(qtr_df2['Existing Model Pack'] == exist_pack) & (qtr_df2['Banner'] == banner_change),'Middle Page'] = qtr_df2.loc[(qtr_df2['Existing Model Pack'] == exist_pack) & (qtr_df2['Banner'] == banner_change),'Middle Page'].values[0] + ban_week_mp2[(ban_week_mp2['Banner'] == banner_change) & (ban_week_mp2.Product == exist_pack)].Banner.count()
                                    
                    # Middle decrease    
                    if exist_pack == edv_disc_join['Existing Model Pack'].unique()[0]:
                        if middle_change.children[1].value < 0:
                            if toggle_ensemble_normal.value != 'Multiple Product': # In case of normal approach
                                ban_week_mp_copy = ban_week_mp.copy()
                            
                            for banner_change in ban_week_mp_copy[ban_week_mp_copy.Product == exist_pack]['Banner'].unique():
                                qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Middle Page'] = qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Middle Page'].values[0] - ban_week_mp_copy[(ban_week_mp_copy.Product == exist_pack) & (ban_week_mp_copy['Banner'] == banner_change)].Banner.count()

                    # Decreasing promotions of 2nd exist product in ensemble approach
                    if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
                        if exist_pack == edv_disc_join['Existing Model Pack'].unique()[1]:
                            if middle_change2.children[1].value < 0:
                                for banner_change in ban_week_mp_copy[ban_week_mp_copy.Product == exist_pack]['Banner'].unique():
                                    qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Middle Page'] = qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Middle Page'].values[0] - ban_week_mp_copy[(ban_week_mp_copy.Product == exist_pack) & (ban_week_mp_copy['Banner'] == banner_change)].Banner.count()

                    # Decreasing promotions of 3rd exist product in ensemble approach
                    if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
                        if exist_pack == edv_disc_join['Existing Model Pack'].unique()[2]:
                            if middle_change3.children[1].value < 0:
                                for banner_change in ban_week_mp_copy[ban_week_mp_copy.Product == exist_pack]['Banner'].unique():
                                    qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Middle Page'] = qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Middle Page'].values[0] - ban_week_mp_copy[(ban_week_mp_copy.Product == exist_pack) & (ban_week_mp_copy['Banner'] == banner_change)].Banner.count()

                    # In case the user selects to simulate New Product 2
                    if toggle_ensemble_normal.value != 'Multiple Product': # In case of normal approach
                        # In case the user selects to simulate New Product 2
                        if toggle_np2.value == 'Yes':                
                            if middle_change2.children[1].value < 0:
                                for banner_change in ban_week_mp2[ban_week_mp2.Product == exist_pack]['Banner'].unique(): 
                                    qtr_df2.loc[(qtr_df2['Existing Model Pack'] == exist_pack) & (qtr_df2['Banner'] == banner_change),'Middle Page'] = qtr_df2.loc[(qtr_df2['Existing Model Pack'] == exist_pack) & (qtr_df2['Banner'] == banner_change),'Middle Page'].values[0] - ban_week_mp2[(ban_week_mp2['Banner'] == banner_change) & (ban_week_mp2.Product == exist_pack)].Banner.count()

                    # Back increase
                    if exist_pack == edv_disc_join['Existing Model Pack'].unique()[0]:
                        if back_change.children[1].value > 0:
                            if toggle_ensemble_normal.value != 'Multiple Product': # In case of normal approach
                                ban_week_bp_copy = ban_week_bp.copy()
                            
                            for banner_change in ban_week_bp_copy[ban_week_bp_copy.Product == exist_pack]['Banner'].unique(): 
                                qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Back Page'] = qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Back Page'].values[0] + ban_week_bp_copy[(ban_week_bp_copy.Product == exist_pack) & (ban_week_bp_copy['Banner'] == banner_change)].Banner.count()
                    
                    # Increasing promotions of 2nd exist product in ensemble approach
                    if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
                        if exist_pack == edv_disc_join['Existing Model Pack'].unique()[1]:
                            if back_change2.children[1].value > 0:
                                for banner_change in ban_week_bp_copy[ban_week_bp_copy.Product == exist_pack]['Banner'].unique(): 
                                    qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Back Page'] = qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Back Page'].values[0] + ban_week_bp_copy[(ban_week_bp_copy.Product == exist_pack) & (ban_week_bp_copy['Banner'] == banner_change)].Banner.count()
                    # Increasing promotions of 3rd exist product in ensemble approach
                        if exist_pack == edv_disc_join['Existing Model Pack'].unique()[2]:
                            if back_change3.children[1].value > 0:
                                for banner_change in ban_week_bp_copy[ban_week_bp_copy.Product == exist_pack]['Banner'].unique(): 
                                    qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Back Page'] = qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Back Page'].values[0] + ban_week_bp_copy[(ban_week_bp_copy.Product == exist_pack) & (ban_week_bp_copy['Banner'] == banner_change)].Banner.count()
                            
                    
                    # In case the user selects to simulate New Product 2
                    if toggle_ensemble_normal.value != 'Multiple Product': # In case of normal approach
                        if toggle_np2.value == 'Yes':
                            if back_change2.children[1].value > 0:
                                for banner_change in ban_week_bp2[ban_week_bp2.Product == exist_pack]['Banner'].unique(): 
                                    qtr_df2.loc[(qtr_df2['Existing Model Pack'] == exist_pack) & (qtr_df2['Banner'] == banner_change),'Back Page'] = qtr_df2.loc[(qtr_df2['Existing Model Pack'] == exist_pack) & (qtr_df2['Banner'] == banner_change),'Back Page'].values[0] + ban_week_bp2[(ban_week_bp2['Banner'] == banner_change) & (ban_week_bp2.Product == exist_pack)].Banner.count()
                                    
                    # Back decrease    
                    if exist_pack == edv_disc_join['Existing Model Pack'].unique()[0]:
                        if back_change.children[1].value < 0:
                            if toggle_ensemble_normal.value != 'Multiple Product': # In case of normal approach
                                ban_week_bp_copy = ban_week_bp.copy()
                            
                            for banner_change in ban_week_bp_copy[ban_week_bp_copy.Product == exist_pack]['Banner'].unique():
                                qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Back Page'] = qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Back Page'].values[0] - ban_week_bp_copy[(ban_week_bp_copy.Product == exist_pack) & (ban_week_bp_copy['Banner'] == banner_change)].Banner.count()

                    # Decreasing promotions of 2nd exist product in ensemble approach
                    if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
                        if exist_pack == edv_disc_join['Existing Model Pack'].unique()[1]:
                            if back_change2.children[1].value < 0:
                                for banner_change in ban_week_bp_copy[ban_week_bp_copy.Product == exist_pack]['Banner'].unique():
                                    qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Back Page'] = qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Back Page'].values[0] - ban_week_bp_copy[(ban_week_bp_copy.Product == exist_pack) & (ban_week_bp_copy['Banner'] == banner_change)].Banner.count()

                    # Decreasing promotions of 3rd exist product in ensemble approach
                    if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
                        if exist_pack == edv_disc_join['Existing Model Pack'].unique()[2]:
                            if back_change3.children[1].value < 0:
                                for banner_change in ban_week_bp_copy[ban_week_bp_copy.Product == exist_pack]['Banner'].unique():
                                    qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Back Page'] = qtr_df.loc[(qtr_df['Existing Model Pack'] == exist_pack) & (qtr_df['Banner'] == banner_change),'Back Page'].values[0] - ban_week_bp_copy[(ban_week_bp_copy.Product == exist_pack) & (ban_week_bp_copy['Banner'] == banner_change)].Banner.count()

                    # In case the user selects to simulate New Product 2
                    if toggle_ensemble_normal.value != 'Multiple Product': # In case of normal approach
                        # In case the user selects to simulate New Product 2
                        if toggle_np2.value == 'Yes':                
                            if back_change2.children[1].value < 0:
                                for banner_change in ban_week_bp2[ban_week_bp2.Product == exist_pack]['Banner'].unique(): 
                                    qtr_df2.loc[(qtr_df2['Existing Model Pack'] == exist_pack) & (qtr_df2['Banner'] == banner_change),'Back Page'] = qtr_df2.loc[(qtr_df2['Existing Model Pack'] == exist_pack) & (qtr_df2['Banner'] == banner_change),'Back Page'].values[0] - ban_week_bp2[(ban_week_bp2['Banner'] == banner_change) & (ban_week_bp2.Product == exist_pack)].Banner.count()

                clear_output()  

                # In case the user selects to simulate New Product 2
                if toggle_ensemble_normal.value != 'Multiple Product': # In case of normal approach                                                                                                                                                      
                    if toggle_np2.value == 'Yes':
                        # Rename columns
                        qtr_df.rename(columns = {'Front Page':'Front Page (New Product 1)','Middle Page':'Middle Page (New Product 1)','Back Page':'Back Page (New Product 1)'}, inplace = True)
                        qtr_df2.rename(columns = {'Front Page':'Front Page (New Product 2)','Middle Page':'Middle Page (New Product 2)','Back Page':'Back Page (New Product 2)'}, inplace = True)
                        qtr_df2.drop(columns = {'Existing Model Pack'}, inplace = True)
                        qtr_df_all = qtr_df.merge(qtr_df2, on = 'Banner', how = 'left')
                        display(widgets.Label(value="Updated Promotions of $New Product 1$ vs $New Product 2$"),HTML(qtr_df_all.to_html(index=False)))        
                    else:
                        display(HTML(qtr_df.to_html(index=False)))                                
                else:
                    display(HTML(qtr_df.to_html(index=False)))        
    else:
        print("Product is not present in that Region/Channel for selected period")

In [211]:
# Simulate new product cell run
def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index(), IPython.notebook.ncells())'))

layout = widgets.Layout(width='auto', height='40px')
button = widgets.Button(description="Simulate New Product Results",layout = layout, button_style='danger')
button.on_click(run_all)

display(button)

In [176]:
## Defining empty dataframes for model training
combined_dataset = pd.DataFrame({'Week': pd.Series([], dtype='object')})
Test_results = pd.DataFrame({'Region': pd.Series([], dtype='object')})
Test_Data = pd.DataFrame()
Test_results_exist2 = pd.DataFrame()

# Checking category, pack subtype, pack content of existing product matches with new product
cat_exist = Volume_dataset_all_reg[Volume_dataset_all_reg.Product == product_fil]['Category'].unique()[0]
packsubtype_exist = Volume_dataset_all_reg[Volume_dataset_all_reg.Product == product_fil]['Pack.Subtype'].unique()[0]
packcontent_exist = Volume_dataset_all_reg[Volume_dataset_all_reg.Product == product_fil]['PACK_CONTENT'].unique()[0]

# Select same quarter for existing data

Test_week_list_qtr = {"Q1" : ["2019-01","2019-02","2019-03","2019-04","2019-05","2019-06","2019-07","2019-08","2019-09","2019-10","2019-11","2019-12","2019-13"],
     "Q2" : ["2019-14","2019-15","2019-16","2019-17","2019-18","2019-19","2019-20","2019-21","2019-22","2019-23","2019-24","2019-25","2019-26"],
     "Q3" : ["2019-27","2019-28","2019-29","2019-30","2019-31","2019-32","2019-33","2019-34","2019-35","2019-36","2019-37","2019-38","2019-39"],
     "Q4" : ["2019-40","2019-41","2019-42","2019-43","2019-44","2019-45","2019-46","2019-47","2019-48","2019-49","2019-50","2019-51","2019-52"],
      }

# Defining test week list
Test_week_list = {"2019Q1" : ["2019-01","2019-02","2019-03","2019-04","2019-05","2019-06","2019-07","2019-08","2019-09","2019-10","2019-11","2019-12","2019-13"],
     "2019Q2" : ["2019-14","2019-15","2019-16","2019-17","2019-18","2019-19","2019-20","2019-21","2019-22","2019-23","2019-24","2019-25","2019-26"],
     "2019Q3" : ["2019-27","2019-28","2019-29","2019-30","2019-31","2019-32","2019-33","2019-34","2019-35","2019-36","2019-37","2019-38","2019-39"],
     "2019Q4" : ["2019-40","2019-41","2019-42","2019-43","2019-44","2019-45","2019-46","2019-47","2019-48","2019-49","2019-50","2019-51","2019-52"],
                  
# CHECK : ADDING 2020 weeks                  
     "2020Q1" : ["2020-01","2020-02","2020-03","2020-04","2020-05","2020-06","2020-07","2020-08","2020-09","2020-10","2020-11","2020-12","2020-13"],
     "2020Q2" : ["2020-14","2020-15","2020-16","2020-17","2020-18","2020-19","2020-20","2020-21","2020-22","2020-23","2020-24","2020-25","2020-26"],
     "2020Q3" : ["2020-27","2020-28","2020-29","2020-30","2020-31","2020-32","2020-33","2020-34","2020-35","2020-36","2020-37","2020-38","2020-39"],
     "2020Q4" : ["2020-40","2020-41","2020-42","2020-43","2020-44","2020-45","2020-46","2020-47","2020-48","2020-49","2020-50","2020-51","2020-52"],
     "fulltrain" : []}

#Model training and prediction
category_check_list=[]

########################################################################################################################
                                    # SIMULATION (WITH UPLOADED DATA)
########################################################################################################################

# In case the user uploads the data
if (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:

    # In case no product is present in selected region-channel
    if len(qtr_edv_baseline) != 0 :
        display(Markdown('<div><div class="loader"></div><h2> &nbsp;Simulating New Product Results</h2></div>'))

        # Run for 1 iteration
        iter_change_list = [1]

        for i in iter_change_list:
            channel_list = upload_data.Channel.unique()
            #Model training and prediction
            for Region_key in upload_data.Region.unique(): 
                # In case the product is not present at that region
                if len(upload_data[(upload_data.Region == Region_key)]) == 0:
                    continue

                Volume_dataset = Volume_dataset_all_reg.loc[(Volume_dataset_all_reg["Region"] == Region_key)
                                                           &(Volume_dataset_all_reg["Channel"].isin(channel_list))]
    
                Volume_dataset_upload = upload_data.loc[(upload_data["Region"] == Region_key)
                                           &(upload_data["Channel"].isin(channel_list))]

    
                Volume_dataset['Pantry2'] = Volume_dataset['Pantry2'].fillna(0)
                Volume_dataset_upload['Pantry2'] = Volume_dataset_upload['Pantry2'].fillna(0)
            
                required_columns_upload = ['Week','Week.Name','Banner','Product','Channel','Pack.Subtype','PACK_CONTENT','Adcal_Price','EDV.Price','Adcal_DD','Front.Page','Middle.Page','Back.Page','seasonality_index','SIZE_ML','COUNT','Category','Pantry1','Pantry2','Holiday.Week','Pre.Holiday.Week','Post.Holiday.Week'] #CHANGE 101: Integration of uplaoded data with tool
                required_columns = ['Week','Week.Name','Banner','Product','Channel','Pack.Subtype','PACK_CONTENT','Adcal_Price','EDV.Price','Adcal_DD','Eq.Unit.Sales','Front.Page','Middle.Page','Back.Page','seasonality_index','SIZE_ML','COUNT','Category','Pantry1','Pantry2','Holiday.Week','Pre.Holiday.Week','Post.Holiday.Week'] #CHANGE 101: Integration of uplaoded data with tool
                combined_dataset = Volume_dataset[required_columns] 
                combined_dataset_upload = Volume_dataset_upload[required_columns_upload] 

                combined_dataset = combined_dataset.loc[((combined_dataset["Week"] >= "2017-01") & 
                                                         (combined_dataset["Week"] <= "2019-52"))]
                
                combined_dataset["Christmas_flag"] = [1 if x=="Christmas" else 0 for x in combined_dataset["Week.Name"]]
                combined_dataset["Easter_flag"] = [1 if x=="Easter" else 0 for x in combined_dataset["Week.Name"]]
                
                combined_dataset_upload["Christmas_flag"] = [1 if x=="Christmas" else 0 for x in combined_dataset_upload["Week.Name"]]
                combined_dataset_upload["Easter_flag"] = [1 if x=="Easter" else 0 for x in combined_dataset_upload["Week.Name"]]                

                banner_dummies = pd.get_dummies(combined_dataset.Banner) 
                banner_dummies = banner_dummies.reindex(sorted(banner_dummies.columns), axis=1) #sorting banner columns
                combined_dataset = pd.concat([combined_dataset, banner_dummies], axis=1)
                
                # Uploaded data banner dummies
                banner_dummies_upload = pd.get_dummies(combined_dataset_upload.Banner)                                 
                # If all banners are not present for the uploaded data
                c = [i for i in list(banner_dummies.columns) if i not in list(banner_dummies_upload.columns)]
                if len(c) != 0:
                    banner_dummies_upload[c] = 0
                # Reordering columns                
                banner_dummies_upload = banner_dummies_upload.reindex(sorted(banner_dummies_upload.columns), axis=1)                
                combined_dataset_upload = pd.concat([combined_dataset_upload, banner_dummies_upload], axis=1)
                
                ## Not adding Product dummies since for New Product Simulator other Product dummies are not significant

                combined_dataset = combined_dataset.loc[combined_dataset["Eq.Unit.Sales"].notnull()]

                combined_dataset.sort_values('Week',inplace = True)
                combined_dataset = pd.merge(combined_dataset,new_prod_stage,on = ['Product','Week'],how = 'left')
                combined_dataset['Intial_weeks'] = combined_dataset['Intial_weeks'].fillna('Stabilization')
                combined_dataset.drop(columns='Unnamed: 0', inplace=True)

                combined_dataset_upload.sort_values('Week',inplace = True)
                combined_dataset_upload = pd.merge(combined_dataset_upload,prod_stage_check_upload(),on = ['Product','Week'],how = 'left')
                combined_dataset_upload['Intial_weeks'] = combined_dataset_upload['Intial_weeks'].fillna('Stabilization')                
                
                #Adding dummies for category
                category_dummies = pd.get_dummies(combined_dataset['Category'])
                combined_dataset = pd.concat([combined_dataset, category_dummies], axis=1) 
                combined_dataset.drop(["Category"], inplace=True, axis=1)

                cat_col = category_dummies.columns #need all category dummy in the test data
                
                #Adding dummies for category UPLOADED DDATA
                combined_dataset_upload[cat_col] = 0
                
                #Adding dummies for product attribute - pack subtype
                pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
                combined_dataset = pd.concat([combined_dataset, pack_subtypes_dummies], axis=1)
                combined_dataset.drop(["Pack.Subtype"], inplace=True, axis=1)

                pack_subtypes_col = pack_subtypes_dummies.columns
                
                #Adding dummies for packsubtype UPLOADED DDATA
                combined_dataset_upload[pack_subtypes_col] = 0
                
                # Adding pack content dummies
                pack_content_dummies = pd.get_dummies(combined_dataset['PACK_CONTENT'])
                combined_dataset = pd.concat([combined_dataset, pack_content_dummies], axis=1)
                combined_dataset.drop(["PACK_CONTENT"], inplace=True, axis=1)

                pack_content_col = pack_content_dummies.columns
                
                #Adding dummies for packsubtype UPLOADED DDATA
                combined_dataset_upload[pack_content_col] = 0
                
                combined_dataset_upload.drop(["Pack.Subtype"], inplace=True, axis=1)
                combined_dataset_upload.drop(["PACK_CONTENT"], inplace=True, axis=1)
                combined_dataset_upload.drop(["Category"], inplace=True, axis=1)
                
                # Product stage dummies
                stage_dummies = pd.get_dummies(combined_dataset['Intial_weeks'])
                combined_dataset = pd.concat([combined_dataset, stage_dummies], axis=1)
                combined_dataset.drop(["Intial_weeks"], inplace=True, axis=1)

                # Product stage dummies
                stage_dummies_upload = pd.get_dummies(combined_dataset_upload['Intial_weeks'])
                
                # If all product stages are not present for the uploaded data
                c = [i for i in list(stage_dummies.columns) if i not in list(stage_dummies_upload.columns)]
                if len(c) != 0 :
                    stage_dummies_upload[c] = 0
                # Reordering columns                
                stage_dummies_upload = stage_dummies_upload.reindex(sorted(stage_dummies_upload.columns), axis=1)
                combined_dataset_upload = pd.concat([combined_dataset_upload, stage_dummies_upload], axis=1)
                combined_dataset_upload.drop(["Intial_weeks"], inplace=True, axis=1)

                #Adding discount depth columns
                combined_dataset["DD_1"] = [1 if (0.25> x >0.1)
                                      else 0 for x in combined_dataset["Adcal_DD"]]
                combined_dataset["DD_2"] = [1 if (x>=0.25)
                                                 else 0 for x in combined_dataset["Adcal_DD"]]

                combined_dataset_upload["DD_1"] = [1 if (0.25> x >0.1)
                                      else 0 for x in combined_dataset_upload["Adcal_DD"]]
                combined_dataset_upload["DD_2"] = [1 if (x>=0.25)
                                                 else 0 for x in combined_dataset_upload["Adcal_DD"]]            

                # Adcal Price calculation
                combined_dataset_upload.loc[combined_dataset_upload['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset_upload.loc[combined_dataset_upload['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset_upload.loc[combined_dataset_upload['Adcal_DD']>=0.1,'EDV.Price']
                # Adcal_DD < 10 then Adcal Price = EDV Price            
                combined_dataset_upload.loc[combined_dataset_upload['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset_upload.loc[combined_dataset_upload['Adcal_DD'] < 0.1, 'EDV.Price']

                # Adcal Price calculation
                combined_dataset.loc[combined_dataset['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'EDV.Price']
                # Adcal_DD < 10 then Adcal Price = EDV Price            
                combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'EDV.Price']

                # Dropping adcal DD as DD1 & DD2 are added
                combined_dataset.drop(["Adcal_DD"], inplace=True, axis=1)
                combined_dataset_upload.drop(["Adcal_DD"], inplace=True, axis=1)

                # Dropping null volumes
                complete_dataset = combined_dataset.copy()
                complete_dataset_upload = combined_dataset_upload.copy()

                complete_dataset = complete_dataset.loc[complete_dataset["Eq.Unit.Sales"].notnull()]

                complete_dataset['Adcal_Price'] = np.log(complete_dataset['Adcal_Price'])
                complete_dataset['Eq.Unit.Sales'] = np.log(complete_dataset['Eq.Unit.Sales'])
                complete_dataset_upload['Adcal_Price'] = np.log(complete_dataset_upload['Adcal_Price'])

                # TEST PERIOD LOOP (futuristic test period)                
                Test_week_list = {"Post_Launch" : upload_data["Week"].unique()}
                Test_periods = ["Post_Launch"]
                
                for Test_period in Test_periods:                    
                    #Filtering test weeks
                    Test_weeks = Test_week_list[Test_period]

                    Test_week_list_qtr_weeks = []
                    
                    # Selecting required quarter-weeks for existing product
                    for quarters in upload_data['Test_period'].str[4:].unique():    
                        Test_week_list_qtr_weeks.extend(Test_week_list_qtr[quarters])

                    # Dividing train & test data
                    complete_train_data_set = complete_dataset.loc[~complete_dataset["Week"].isin(Test_weeks)].reset_index(drop=True)
                    # Data for existing product
                    complete_test_data_set_exist = complete_dataset.loc[(complete_dataset.Product == product_fil) & (complete_dataset["Week"].isin(Test_week_list_qtr_weeks))].reset_index(drop=True) # CHECK upload
                    # Data for new product
                    complete_test_data_set = complete_dataset_upload.loc[complete_dataset_upload["Week"].isin(Test_weeks)].reset_index(drop=True)

                    # Dropping unneccessary columns from train dataset
                    train_data_set = complete_train_data_set.copy()

                    train_data_brand = train_data_set.copy()
                    train_data_brand.drop(["Week","Week.Name","Banner","Product",'Channel',"EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)

                    # Filtering for existing product
                    test_data_set = complete_test_data_set.copy()
                    test_data_set_exist = complete_test_data_set_exist.copy()

                    # New Category
                    if i == 1:
                        for cols in upload_data.Category.unique():                            
                            test_data_set[cols] = 1
                            
                    # New Pack Subtype
                    if i == 1:
                        for cols in upload_data['Pack.Subtype'].unique():                            
                            test_data_set[cols] = 1
                    
                    # New Pack Content
                    if i == 1:
                        for cols in upload_data['PACK_CONTENT'].unique():                            
                            test_data_set[cols] = 1
                    
                    if product_fil == 'TCCC CORE POWER 414 ML BTTL':
                        test_data_set = test_data_set.drop_duplicates()
                        test_data_set_exist = test_data_set_exist.drop_duplicates()
                        
                    test_data_brand = test_data_set.copy()
                    test_data_brand_exist = test_data_set_exist.copy()

                    if i == 1 :
                        # For New SIZE
                        test_data_brand['SIZE_ML'] = upload_data.SIZE_ML.unique()[0]

                        # For New Count
                        test_data_brand['COUNT'] = upload_data.COUNT.unique()[0]

                    # Dropping unneccessary columns from test dataset
                    test_data_brand.drop(["Week","Week.Name","Banner","Product",'Channel',"EDV.Price"], inplace=True, axis=1)  #CHANGE 101: Integration of uplaoded data with tool
                    test_data_brand_exist.drop(["Week","Week.Name","Banner","Product",'Channel',"EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)

                    #Creating test and train data - independent & dependent columns
                    X_train = train_data_brand
                    y_train = train_data_set["Eq.Unit.Sales"]
                    X_test = test_data_brand
                    X_test_exist = test_data_brand_exist

                    # Appending test data
                    test_data_set["Region"] = Region_key
                    test_data_set["Test_period"] = Test_period
                    test_data_set['iter'] = i

                    # In case the product is not present in certain region-quarter
                    if len(test_data_set) == 0:
                        continue

                    Test_Data = Test_Data.append(test_data_set, ignore_index = True)

                    #Model training
                    rf = RandomForestRegressor(n_jobs=4,random_state=0)
                    rf.fit(X_train, y_train)

                    #Model predictions
                    predictions_rf = rf.predict(X_test)                
                    predictions_rf_exist = rf.predict(X_test_exist)                

                    #Storing test results
                    result = X_test.copy()

                    result["Predictions_rf"] = predictions_rf
                    result['Predictions_rf'] = np.exp(result['Predictions_rf'])    
                    result["Product"] = test_data_set["Product"]
                    result["Region"] = Region_key
                    result["Test_period"] = Test_period
                    result["Week"] = test_data_set["Week"]
                    result["DD_1"] = test_data_set['DD_1']
                    result["DD_2"] = test_data_set['DD_2']
                    result["Banner"] = test_data_set['Banner']
                    result["Channel"] = test_data_set['Channel']
                    result["Adcal_Price"] = test_data_set["Adcal_Price"]
                    result['Adcal_Price'] = np.exp(result['Adcal_Price'])

                    result_exist = X_test_exist.copy()
            
                    result_exist["Adcal_Price_exist"] = test_data_set_exist["Adcal_Price"].values
                    result_exist['Adcal_Price_exist'] = np.exp(result_exist["Adcal_Price_exist"])
                    result_exist['Predictions_rf_exist'] = predictions_rf_exist
                    result_exist['Predictions_rf_exist'] = np.exp(result_exist['Predictions_rf_exist'])   
                    result_exist['Week'] = test_data_set_exist['Week']
                    result_exist['Banner'] = test_data_set_exist['Banner']
                    result_exist['Product'] = test_data_set_exist['Product']
                    result_exist['Channel'] = test_data_set_exist['Channel']
                    result_exist['Region'] = Region_key
                    result_exist['Test_period'] = Test_period

                    result['iter'] = i            
 
                    result = result[["Region","Product",'Channel',"Banner","Test_period","Week","Predictions_rf","DD_1","DD_2","Adcal_Price",'iter']]
                    result_exist = result_exist[["Region","Product",'Channel',"Banner","Test_period","Week","Predictions_rf_exist","Adcal_Price_exist"]]
                    if i == 1:
                        result['Volume_RU'] = result['Predictions_rf']
                        result_exist['Volume_RU_exist'] = result_exist['Predictions_rf_exist']

                    result['Sales_derived'] = result['Predictions_rf'] * result['Adcal_Price']
                    result_exist['Sales_derived_exist'] = result_exist['Predictions_rf_exist'] * result_exist['Adcal_Price_exist']

                    # Appending results to the final results dataframe            
                    Test_results = Test_results.append(result, ignore_index=True)
                    Test_results_exist2 = Test_results_exist2.append(result_exist, ignore_index = True)


        clear_output()

    else:
        print("Product is not present in that Region/Channel for selected period")
                
            
########################################################################################################################
                                    # SIMULATION (WITHOUT UPLOADED DATA)
########################################################################################################################


elif toggle_upload.value == 'No':        

    #Model training and prediction
    category_check_list=[]

    # In case no product is present in selected region-channel
    if len(qtr_edv_baseline) != 0 :
        display(Markdown('<div><div class="loader"></div><h2> &nbsp;Simulating New Product Results</h2></div>'))
        size_exist = Volume_dataset_all_reg[Volume_dataset_all_reg.Product == product_fil]['SIZE_ML'].unique()[0]
        count_exist = Volume_dataset_all_reg[Volume_dataset_all_reg.Product == product_fil]['COUNT'].unique()[0]
        cat_exist = Volume_dataset_all_reg[Volume_dataset_all_reg.Product == product_fil]['Category'].unique()[0]
        packsubtype_exist = Volume_dataset_all_reg[Volume_dataset_all_reg.Product == product_fil]['Pack.Subtype'].unique()[0]
        packcontent_exist = Volume_dataset_all_reg[Volume_dataset_all_reg.Product == product_fil]['PACK_CONTENT'].unique()[0]

        if toggle_np2.value == 'Yes':
            iter_change_list = [1,2]
        else:
            iter_change_list = [1]

        for i in iter_change_list:

            for product_fil in sorted(qtr_df_final['Existing Model Pack'].unique()): # Looping through each existing product

                ################# PROMOTION CHANGE VARIABLES #################
                if (promo_change_toggle.value == 'Yes (All Banners)') | (promo_change_toggle.value == 'Yes (Specific Banners)'):

                    if i == 1:
                        change_mp = middle_change.children[1].value
                    elif i == 2:
                        change_mp = middle_change2.children[1].value

                    if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
                        if len(prod_ensemble_list) != 3: # if 3 products are not selected
                            clear_output()
                            print('Select 3 products as existing model pack')
                        else:
                        
                            if product_fil == edv_disc_join['Existing Model Pack'].unique()[0]:
                                change_mp = middle_change.children[1].value
                            elif product_fil == edv_disc_join['Existing Model Pack'].unique()[1]:
                                change_mp = middle_change2.children[1].value
                            elif product_fil == edv_disc_join['Existing Model Pack'].unique()[2]:
                                change_mp = middle_change3.children[1].value

                    if i == 1:
                        change_fp = front_change.children[1].value
                    elif i == 2:
                        change_fp = front_change2.children[1].value

                    if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
                        if len(prod_ensemble_list) != 3: # if 3 products are not selected
                            clear_output()
                            print('Select 3 products as existing model pack')
                        else:

                            if product_fil == edv_disc_join['Existing Model Pack'].unique()[0]:
                                change_fp = front_change.children[1].value
                            elif product_fil == edv_disc_join['Existing Model Pack'].unique()[1]:
                                change_fp = front_change2.children[1].value
                            elif product_fil == edv_disc_join['Existing Model Pack'].unique()[2]:
                                change_fp = front_change3.children[1].value

                    if i == 1:
                        change_bp = back_change.children[1].value
                    elif i == 2:
                        change_bp = back_change2.children[1].value

                    if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
                        if len(prod_ensemble_list) != 3: # if 3 products are not selected
                            clear_output()
                            print('Select 3 products as existing model pack')
                        else:
                            if product_fil == edv_disc_join['Existing Model Pack'].unique()[0]:
                                change_bp = back_change.children[1].value
                            elif product_fil == edv_disc_join['Existing Model Pack'].unique()[1]:
                                change_bp = back_change2.children[1].value
                            elif product_fil == edv_disc_join['Existing Model Pack'].unique()[2]:
                                change_bp = back_change3.children[1].value

                ################# #Model training and prediction #################

                #Model training and prediction
                for Region_key in Region_List:                 
                    # In case the product is not present at that region
                    if len(Volume_dataset_all_reg[(Volume_dataset_all_reg.Product == product_fil) & (Volume_dataset_all_reg.Region == Region_key)]) == 0:
                        continue

                    category_check_list.extend(list(Volume_dataset_all_reg[(Volume_dataset_all_reg["Region"] == Region_key) & (Volume_dataset_all_reg["Product"] == product_fil)].Category.unique()))

                    Volume_dataset = Volume_dataset_all_reg.loc[(Volume_dataset_all_reg["Region"] == Region_key)
                                                               &(Volume_dataset_all_reg["Channel"].isin(channel_list))]
                    Volume_dataset['Pantry2'] = Volume_dataset['Pantry2'].fillna(0)   
                    required_columns = ['Week','Week.Name','Banner','Product','Channel','Pack.Subtype','PACK_CONTENT','Adcal_Price','EDV.Price','Adcal_DD','Eq.Unit.Sales','Front.Page','Middle.Page','Back.Page','seasonality_index','SIZE_ML','COUNT','No_of_brands','No_of_flavors','No_of_sweetners','No_of_types','Category','Pantry1','Pantry2','Holiday.Week','Pre.Holiday.Week','Post.Holiday.Week']
                    combined_dataset = Volume_dataset[required_columns]

                    combined_dataset = combined_dataset.loc[((combined_dataset["Week"] >= "2017-01") & 
                                                             (combined_dataset["Week"] <= "2019-52"))]

                    combined_dataset["Christmas_flag"] = [1 if x=="Christmas" else 0 for x in combined_dataset["Week.Name"]]
                    combined_dataset["Easter_flag"] = [1 if x=="Easter" else 0 for x in combined_dataset["Week.Name"]]

                    ## CHECKING if banner dummies are not present in required region, channel then introduce banner dummies
                    banner_dummies = pd.get_dummies(combined_dataset.Banner)              

                    # Uploaded data banner dummies
                    banner_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key].Banner)                                 
                    # If all banners are not present for the training data
                    c = [i for i in list(banner_dummies_check.columns) if i not in list(banner_dummies.columns)]
                    if len(c) != 0:
                        banner_dummies[c] = 0

                    # Reordering columns                
                    banner_dummies = banner_dummies.reindex(sorted(banner_dummies.columns), axis=1)                
                    combined_dataset = pd.concat([combined_dataset, banner_dummies], axis=1)

                    ## Not adding Product dummies since for New Product Simulator other Product dummies are not significant
                    combined_dataset = combined_dataset.loc[combined_dataset["Eq.Unit.Sales"].notnull()]

                    combined_dataset.sort_values('Week',inplace = True)
                    combined_dataset = pd.merge(combined_dataset,new_prod_stage,on = ['Product','Week'],how = 'left')
                    combined_dataset['Intial_weeks'] = combined_dataset['Intial_weeks'].fillna('Stabilization')
                    combined_dataset.drop(columns='Unnamed: 0', inplace=True)

                    #Adding dummies for category
                    category_dummies = pd.get_dummies(combined_dataset['Category'])

                    # Adding dummies for product attribute - Category
                    pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
                    # Uploaded data category dummies
                    category_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['Category'])                                 
                    # If all pack subtypes are not present for the training data
                    c = [i for i in list(category_dummies_check.columns) if i not in list(category_dummies.columns)]
                    if len(c) != 0:
                        category_dummies[c] = 0

                    # Reordering columns                
                    category_dummies = category_dummies.reindex(sorted(category_dummies.columns), axis=1)                
                    combined_dataset = pd.concat([combined_dataset, category_dummies], axis=1)
                    combined_dataset.drop(["Category"], inplace=True, axis=1)

                    # Adding dummies for product attribute - pack subtype
                    pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
                    # Uploaded data pack subtype dummies
                    pack_subtypes_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['Pack.Subtype'])                                 
                    # If all pack subtypes are not present for the training data
                    c = [i for i in list(pack_subtypes_dummies_check.columns) if i not in list(pack_subtypes_dummies.columns)]
                    if len(c) != 0:
                        pack_subtypes_dummies[c] = 0

                    # Reordering columns                
                    pack_subtypes_dummies = pack_subtypes_dummies.reindex(sorted(pack_subtypes_dummies.columns), axis=1)                
                    combined_dataset = pd.concat([combined_dataset, pack_subtypes_dummies], axis=1)
                    combined_dataset.drop(["Pack.Subtype"], inplace=True, axis=1)

                    # Adding pack content dummies
                    pack_content_dummies = pd.get_dummies(combined_dataset['PACK_CONTENT'])
                    # Uploaded data pack content dummies
                    pack_content_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['PACK_CONTENT'])                                 
                    # If all pack contents are not present for the training data
                    c = [i for i in list(pack_content_dummies_check.columns) if i not in list(pack_content_dummies.columns)]
                    if len(c) != 0:
                        pack_content_dummies[c] = 0

                    # Reordering columns                
                    pack_content_dummies = pack_content_dummies.reindex(sorted(pack_content_dummies.columns), axis=1)                
                    combined_dataset = pd.concat([combined_dataset, pack_content_dummies], axis=1)
                    combined_dataset.drop(["PACK_CONTENT"], inplace=True, axis=1)
                    
                    combined_dataset_test = combined_dataset.copy()

                    # Product stage dummies
                    stage_dummies = pd.get_dummies(combined_dataset['Intial_weeks'])
                    combined_dataset = pd.concat([combined_dataset, stage_dummies], axis=1)
                    combined_dataset.drop(["Intial_weeks"], inplace=True, axis=1)

                    # Changing product stage as per start date provided by the user
                    prod_stage_data = prod_stage_check(product_fil)            

                    # Getting updated product stages
                    combined_dataset_test_check = combined_dataset_test[combined_dataset_test.Product == product_fil].merge(prod_stage_data[['Week','Initial_weeks1']], how = 'left')
                    combined_dataset_test_check['Initial_weeks1'] = combined_dataset_test_check['Initial_weeks1'].fillna(combined_dataset_test_check['Intial_weeks'])
                    combined_dataset_test_check.drop(columns = {'Intial_weeks'}, inplace = True)
                    combined_dataset_test_check.rename(columns = {'Initial_weeks1':'Intial_weeks'}, inplace = True)

                    combined_dataset_test = combined_dataset_test_check.copy()

                    # Product stage dummies
                    stage_dummies_test = pd.get_dummies(combined_dataset_test['Intial_weeks'])

                    # If all product stages are not present for the uploaded data
                    c = [i for i in list(stage_dummies.columns) if i not in list(stage_dummies_test.columns)]
                    if len(c) != 0 :
                        stage_dummies_test[c] = 0

                    # Reordering columns                
                    stage_dummies_test = stage_dummies_test.reindex(sorted(stage_dummies_test.columns), axis=1)
                    combined_dataset_test = pd.concat([combined_dataset_test, stage_dummies_test], axis=1)
                    combined_dataset_test.drop(["Intial_weeks"], inplace=True, axis=1)

                    # In case the user doesn't want to upload adcal data
                    if toggle_upload.value != 'Yes':                        
                        if 'All' in list(test_period.value):
                            # Discount change
                            if i == 1:
                                if product_fil == sorted(qtr_df_final['Existing Model Pack'].unique())[0]:
                                    combined_dataset_test['Adcal_DD'] = (1+disc_change_fil)*combined_dataset_test['Adcal_DD']
                                    combined_dataset_test.loc[:,'EDV.Price'] = (1+base_price_change_fil)*combined_dataset_test.loc[:,'EDV.Price']

                                if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
                                    if len(prod_ensemble_list) != 3: # if 3 products are not selected
                                        clear_output()
                                        print('Select 3 products as existing model pack')
                                    else:

                                        if product_fil == sorted(qtr_df_final['Existing Model Pack'].unique())[1]:
                                            combined_dataset_test['Adcal_DD'] = (1+disc_change_fil2)*combined_dataset_test['Adcal_DD']
                                            combined_dataset_test.loc[:,'EDV.Price'] = (1+base_price_change_fil2)*combined_dataset_test.loc[:,'EDV.Price']
                                        if product_fil == sorted(qtr_df_final['Existing Model Pack'].unique())[2]:
                                            combined_dataset_test['Adcal_DD'] = (1+disc_change_fil3)*combined_dataset_test['Adcal_DD']
                                            combined_dataset_test.loc[:,'EDV.Price'] = (1+base_price_change_fil3)*combined_dataset_test.loc[:,'EDV.Price']


                            elif i == 2:
                                combined_dataset_test['Adcal_DD'] = (1+disc_change_fil2)*combined_dataset_test['Adcal_DD']
                                combined_dataset_test.loc[:,'EDV.Price'] = (1+base_price_change_fil2)*combined_dataset_test.loc[:,'EDV.Price']          
                        else:                
                            if i == 1:
                                if product_fil == sorted(qtr_df_final['Existing Model Pack'].unique())[0]:                                
                                    combined_dataset_test.loc[combined_dataset_test['Test_period'].isin(test_period_fil),'Adcal_DD'] = (1+disc_change_fil)*combined_dataset_test[combined_dataset_test['Test_period'].isin(test_period_fil)]['Adcal_DD']                                 
                                    combined_dataset_test.loc[(combined_dataset_test['Test_period'].isin(test_period_fil)),'EDV.Price'] = (1+base_price_change_fil)*combined_dataset_test[(combined_dataset_test['Test_period'].isin(test_period_fil))]['EDV.Price']

                                if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
                                    if len(prod_ensemble_list) != 3: # if 3 products are not selected
                                        clear_output()
                                        print('Select 3 products as existing model pack')
                                    else:
                                    
                                        if product_fil == sorted(qtr_df_final['Existing Model Pack'].unique())[1]:                                
                                            combined_dataset_test.loc[combined_dataset_test['Test_period'].isin(test_period_fi2),'Adcal_DD'] = (1+disc_change_fil2)*combined_dataset_test[combined_dataset_test['Test_period'].isin(test_period_fil2)]['Adcal_DD']                                 
                                            combined_dataset_test.loc[(combined_dataset_test['Test_period'].isin(test_period_fil2)),'EDV.Price'] = (1+base_price_change_fil2)*combined_dataset_test[(combined_dataset_test['Test_period'].isin(test_period_fil2))]['EDV.Price']
                                        if product_fil == sorted(qtr_df_final['Existing Model Pack'].unique())[2]:                                
                                            combined_dataset_test.loc[combined_dataset_test['Test_period'].isin(test_period_fil3),'Adcal_DD'] = (1+disc_change_fil3)*combined_dataset_test[combined_dataset_test['Test_period'].isin(test_period_fil3)]['Adcal_DD']                                 
                                            combined_dataset_test.loc[(combined_dataset_test['Test_period'].isin(test_period_fil3)),'EDV.Price'] = (1+base_price_change_fil3)*combined_dataset_test[(combined_dataset_test['Test_period'].isin(test_period_fil3))]['EDV.Price']

                            elif i == 2:
                                # Discount change
                                combined_dataset_test.loc[combined_dataset_test['Test_period'].isin(test_period_fil2),'Adcal_DD'] = (1+disc_change_fil2)*combined_dataset_test[combined_dataset_test['Test_period'].isin(test_period_fil2)]['Adcal_DD']             
                                # Baseline pricing
                                combined_dataset_test.loc[(combined_dataset_test['Test_period'].isin(test_period_fil2)),'EDV.Price'] = (1+base_price_change_fil2)*combined_dataset_test[(combined_dataset_test['Test_period'].isin(test_period_fil2))]['EDV.Price']

                    #Adding discount depth columns
                    combined_dataset["DD_1"] = [1 if (0.25> x >0.1)
                                          else 0 for x in combined_dataset["Adcal_DD"]]
                    combined_dataset["DD_2"] = [1 if (x>=0.25)
                                                     else 0 for x in combined_dataset["Adcal_DD"]]

                    combined_dataset_test["DD_1"] = [1 if (0.25> x >0.1)
                                          else 0 for x in combined_dataset_test["Adcal_DD"]]
                    combined_dataset_test["DD_2"] = [1 if (x>=0.25)
                                                     else 0 for x in combined_dataset_test["Adcal_DD"]]            

                    # Adcal Price calculation
                    # In case the discounts go over 100%, limit to 100%
                    combined_dataset_test.loc[combined_dataset_test.Adcal_DD >= 1, 'Adcal_DD'] = 0.99
                    combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset_test.loc[combined_dataset_test['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset_test.loc[combined_dataset_test['Adcal_DD']>=0.1,'EDV.Price']
                    # Adcal_DD < 10 then Adcal Price = EDV Price            
                    combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] < 0.1, 'EDV.Price']

                    # Adcal Price calculation
                    combined_dataset.loc[combined_dataset['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'EDV.Price']
                    # Adcal_DD < 10 then Adcal Price = EDV Price            
                    combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'EDV.Price']

                    # Dropping adcal DD as DD1 & DD2 are added
                    combined_dataset.drop(["Adcal_DD"], inplace=True, axis=1)
                    combined_dataset_test.drop(["Adcal_DD"], inplace=True, axis=1)

                    # Dropping null volumes
                    complete_dataset = combined_dataset.copy()
                    complete_dataset_test = combined_dataset_test.copy()

                    complete_dataset = complete_dataset.loc[complete_dataset["Eq.Unit.Sales"].notnull()]
                    complete_dataset_test = complete_dataset_test.loc[complete_dataset_test["Eq.Unit.Sales"].notnull()]

                    complete_dataset['Adcal_Price'] = np.log(complete_dataset['Adcal_Price'])
                    complete_dataset['Eq.Unit.Sales'] = np.log(complete_dataset['Eq.Unit.Sales'])

                    complete_dataset_test['Adcal_Price'] = np.log(complete_dataset_test['Adcal_Price'])
                    complete_dataset_test['Eq.Unit.Sales'] = np.log(complete_dataset_test['Eq.Unit.Sales'])

                    # TEST PERIOD LOOP
                    Test_periods = ['2019Q1','2019Q2','2019Q3','2019Q4']

                    for Test_period in Test_periods:
                        if (Test_period>end_week.children[8].value) | (Test_period > Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Product == product_fil, 'Test_period'].max()):
                            continue
                        if (Test_period<start_week.children[8].value) | (Test_period < Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Product == product_fil, 'Test_period'].min()):
                            continue                

                        #Filtering test weeks
                        Test_weeks = Test_week_list[Test_period]

                        # Dividing train & test data
                        complete_train_data_set = complete_dataset.loc[~complete_dataset["Week"].isin(Test_weeks)].reset_index(drop=True)
                        # Data for existing product
                        complete_test_data_set_exist = complete_dataset.loc[complete_dataset["Week"].isin(Test_weeks)].reset_index(drop=True)
                        # Data for new product
                        complete_test_data_set = complete_dataset_test.loc[complete_dataset_test["Week"].isin(Test_weeks)].reset_index(drop=True)

                        # Dropping unneccessary columns from train dataset
                        train_data_set = complete_train_data_set.copy()

                        train_data_brand = train_data_set.copy()
                        train_data_brand.drop(["Week","Week.Name","Banner","Product",'Channel',"EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)

                        # Filtering for existing product
                        test_data_set = complete_test_data_set[complete_test_data_set.Product == product_fil].copy()
                        test_data_set_exist = complete_test_data_set_exist[complete_test_data_set_exist.Product == product_fil].copy()

                        # In case the user doesn't want to upload adcal data
                        if toggle_upload.value != 'Yes':

                            #################### INCREASING/DECREASING PROMOTIONS FOR BANNER WEEKS COMBINATIONS ####################

                            if (promo_change_toggle.value == 'Yes (All Banners)') | (promo_change_toggle.value == 'Yes (Specific Banners)'):
                                if (front_change.children[1].value != 0) | (middle_change.children[1].value != 0) | (back_change.children[1].value != 0) | (front_change2.children[1].value != 0) | (middle_change2.children[1].value != 0) | (back_change2.children[1].value != 0) | (front_change3.children[1].value != 0) | (middle_change3.children[1].value != 0) | (back_change3.children[1].value != 0) : 

                                    ## FRONT PAGE PROMO CHANGES
                                    # New Product 1
                                    if i == 1:
                                        if change_fp > 0:
                                            # Adding additional promotions in the required banner-weeks
                                            for week,banner in zip(ban_week_fp_copy[ban_week_fp_copy.Product == product_fil]['Week'], ban_week_fp_copy[ban_week_fp_copy.Product == product_fil]['Banner']):
                                                test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Front.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Front.Page'] + 1
                                        elif change_fp < 0:
                                            for week,banner in zip(ban_week_fp_copy[ban_week_fp_copy.Product == product_fil]['Week'], ban_week_fp_copy[ban_week_fp_copy.Product == product_fil]['Banner']):                            
                                                test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Front.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Front.Page'] - 1

                                    elif i == 2:
                                        if change_fp > 0:
                                            # Adding additional promotions in the required banner-weeks
                                            for week,banner in zip(ban_week_fp2['Week'], ban_week_fp2['Banner']):                            
                                                test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Front.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Front.Page'] + 1
                                        elif front_change2.children[1].value < 0:
                                            for week,banner in zip(ban_week_fp2['Week'], ban_week_fp2['Banner']):                            
                                                test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Front.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Front.Page'] - 1

                                    ## MIDDLE PAGE PROMO CHANGES
                                    if i == 1:
                                        if change_mp > 0:
                                            # Adding additional promotions in the required banner-weeks
                                            for week,banner in zip(ban_week_mp_copy[ban_week_mp_copy.Product == product_fil]['Week'], ban_week_mp_copy[ban_week_mp_copy.Product == product_fil]['Banner']):                            
                                                test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Middle.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Middle.Page'] + 1
                                        elif change_mp < 0:
                                            for week,banner in zip(ban_week_mp_copy[ban_week_mp_copy.Product == product_fil]['Week'], ban_week_mp_copy[ban_week_mp_copy.Product == product_fil]['Banner']):                            
                                                test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Middle.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Middle.Page'] - 1
                                    elif i == 2:
                                        if change_mp > 0:
                                            # Adding additional promotions in the required banner-weeks
                                            for week,banner in zip(ban_week_mp2['Week'], ban_week_mp2['Banner']):                            
                                                test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Middle.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Middle.Page'] + 1
                                        elif change_mp < 0:
                                            for week,banner in zip(ban_week_mp2['Week'], ban_week_mp2['Banner']):                            
                                                test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Middle.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Middle.Page'] - 1

                                    ## BACK PAGE PROMO CHANGES
                                    if i == 1:
                                        if change_bp > 0:
                                            # Adding additional promotions in the required banner-weeks
                                            for week,banner in zip(ban_week_bp_copy[ban_week_bp_copy.Product == product_fil]['Week'], ban_week_bp_copy[ban_week_bp_copy.Product == product_fil]['Banner']):                            
                                                test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Back.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Back.Page'] + 1
                                        elif change_bp < 0:
                                            for week,banner in zip(ban_week_bp_copy[ban_week_bp_copy.Product == product_fil]['Week'], ban_week_bp_copy[ban_week_bp_copy.Product == product_fil]['Banner']):                            
                                                test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Back.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Back.Page'] - 1
                                    elif i == 2:
                                        if change_bp > 0:
                                            # Adding additional promotions in the required banner-weeks
                                            for week,banner in zip(ban_week_bp2['Week'], ban_week_bp2['Banner']):                            
                                                test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Back.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Back.Page'] + 1
                                        elif change_bp < 0:
                                            for week,banner in zip(ban_week_bp2['Week'], ban_week_bp2['Banner']):                            
                                                test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Back.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Back.Page'] - 1

                        # New Category
                        if i == 1:
                            for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)].Category.unique():
                                if (cols == category_drop.value):                      
                                    test_data_set[cols] = 1
                                else:
                                    test_data_set[cols] = 0
                        if i == 2:
                            for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)].Category.unique():
                                if (cols == category_drop2.value):                      
                                    test_data_set[cols] = 1
                                else:
                                    test_data_set[cols] = 0

                        # New Pack Subtype
                        if i == 1:
                            for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['Pack.Subtype'].unique():
                                if (cols == pack_subtype_drop.value):                      
                                    test_data_set[cols] = 1
                                else:
                                    test_data_set[cols] = 0
                        if i == 2:
                            for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['Pack.Subtype'].unique():
                                if (cols == pack_subtype_drop2.value):                      
                                    test_data_set[cols] = 1
                                else:
                                    test_data_set[cols] = 0
                        # New Pack Content
                        if i == 1:
                            for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['PACK_CONTENT'].unique():
                                if (cols == pack_content_drop.value):                      
                                    test_data_set[cols] = 1
                                else:
                                    test_data_set[cols] = 0
                        if i == 2:
                            for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['PACK_CONTENT'].unique():
                                if (cols == pack_content_drop2.value):                      
                                    test_data_set[cols] = 1
                                else:
                                    test_data_set[cols] = 0

                        if product_fil == 'TCCC CORE POWER 414 ML BTTL':
                            test_data_set = test_data_set.drop_duplicates()
                            test_data_set_exist = test_data_set_exist.drop_duplicates()

                        test_data_brand = test_data_set.copy()
                        test_data_brand_exist = test_data_set_exist.copy()

                        if i == 1 :
                            # For New SIZE
                            test_data_brand['SIZE_ML'] = size.value

                            # For New Count
                            test_data_brand['COUNT'] = count.value
                        elif i == 2:
                            # For New SIZE
                            test_data_brand['SIZE_ML'] = size2.value

                            # For New Count
                            test_data_brand['COUNT'] = count2.value

                        # Dropping unneccessary columns from test dataset
                        test_data_brand.drop(["Week","Week.Name","Banner","Product",'Channel',"EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)
                        test_data_brand_exist.drop(["Week","Week.Name","Banner","Product",'Channel',"EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)

                        #Creating test and train data - independent & dependent columns
                        X_train = train_data_brand
                        y_train = train_data_set["Eq.Unit.Sales"]
                        X_test = test_data_brand
                        X_test_exist = test_data_brand_exist
                        y_test = test_data_set["Eq.Unit.Sales"]

                        # Appending test data
                        test_data_set["Region"] = Region_key
                        test_data_set["Test_period"] = Test_period
                        test_data_set['iter'] = i

                        # In case the product is not present in certain region-quarter
                        if len(test_data_set) == 0:
                            continue

                        Test_Data = Test_Data.append(test_data_set, ignore_index = True)
                    
                        if((Region_key == 'EAST') and (Test_period == '2019Q1')):
                                rf = model_object_EAST_2019Q1_v2
                        elif((Region_key == 'EAST') and (Test_period == '2019Q2')):
                                rf = model_object_EAST_2019Q2_v2
                        elif((Region_key == 'EAST') and (Test_period == '2019Q3')):
                                rf = model_object_EAST_2019Q3_v2
                        elif((Region_key == 'EAST') and (Test_period == '2019Q4')):
                                rf = model_object_EAST_2019Q4_v2
                        elif((Region_key == 'ONTARIO') and (Test_period == '2019Q1')):
                                rf = model_object_ONTARIO_2019Q1_v2
                        elif((Region_key == 'ONTARIO') and (Test_period == '2019Q2')):
                                rf = model_object_ONTARIO_2019Q2_v2
                        elif((Region_key == 'ONTARIO') and (Test_period == '2019Q3')):
                                rf = model_object_ONTARIO_2019Q3_v2
                        elif((Region_key == 'ONTARIO') and (Test_period == '2019Q4')):
                                rf = model_object_ONTARIO_2019Q4_v2
                        elif((Region_key == 'QUEBEC') and (Test_period == '2019Q1')):
                                rf = model_object_QUEBEC_2019Q1_v2
                        elif((Region_key == 'QUEBEC') and (Test_period == '2019Q2')):
                                rf = model_object_QUEBEC_2019Q2_v2
                        elif((Region_key == 'QUEBEC') and (Test_period == '2019Q3')):
                                rf = model_object_QUEBEC_2019Q3_v2
                        elif((Region_key == 'QUEBEC') and (Test_period == '2019Q4')):
                                rf = model_object_QUEBEC_2019Q4_v2
                        elif((Region_key == 'WEST') and (Test_period == '2019Q1')):
                                rf = model_object_WEST_2019Q1_v2
                        elif((Region_key == 'WEST') and (Test_period == '2019Q2')):
                                rf = model_object_WEST_2019Q2_v2
                        elif((Region_key == 'WEST') and (Test_period == '2019Q3')):
                                rf = model_object_WEST_2019Q3_v2
                        else:
                                rf = model_object_WEST_2019Q4_v2

                        #Model predictions
                        predictions_rf = rf.predict(X_test)                
                        predictions_rf_exist = rf.predict(X_test_exist)                

                        #Storing test results
                        result = X_test.copy()
                        result_exist = X_test_exist.copy()

                        result["Predictions_rf"] = predictions_rf
                        result_exist["Predictions_rf"] = predictions_rf_exist

                        result['Predictions_rf'] = np.exp(result['Predictions_rf'])

                        result["Actuals"] = y_test
                        result['Actuals'] = np.exp(result['Actuals'])                    

                        #Calculating APE for the models
                        result["ape_rf"] = (result["Predictions_rf"]-result["Actuals"]).abs()

                        result["Product"] = test_data_set["Product"]
                        result["Region"] = Region_key
                        result["Test_period"] = Test_period
                        result["Week"] = test_data_set["Week"]
                        result["DD_1"] = test_data_set['DD_1']
                        result["DD_2"] = test_data_set['DD_2']
                        result["Banner"] = test_data_set['Banner']
                        result["Channel"] = test_data_set['Channel']
                        ##Columns for Revenue                    
                        result["Adcal_Price"] = test_data_set["Adcal_Price"]
                        result['Adcal_Price'] = np.exp(result['Adcal_Price'])                    

                        result["Adcal_Price_exist"] = test_data_set_exist["Adcal_Price"].values
                        result['Adcal_Price_exist'] = np.exp(result['Adcal_Price_exist'])                    

                        result['Predictions_rf_exist'] = predictions_rf_exist
                        result['Predictions_rf_exist'] = np.exp(result['Predictions_rf_exist'])
                        result['iter'] = i                                                                        

                        result = result[["Region","Product",'Channel',"Banner","Test_period","Week","Actuals","ape_rf","Predictions_rf","Predictions_rf_exist","Adcal_Price_exist","DD_1","DD_2","Adcal_Price",'iter']]

                        if i == 1:
                            result['Volume_RU'] = result['Predictions_rf']
                            result['Volume_RU_exist'] = result['Predictions_rf_exist']
                        elif i == 2:
                            result['Volume_RU'] = result['Predictions_rf']
                            result['Volume_RU_exist'] = result['Predictions_rf_exist']

                        result['Sales_derived'] = result['Predictions_rf'] * result['Adcal_Price']
                        result['Sales_derived_exist'] = result['Predictions_rf_exist'] * result['Adcal_Price_exist']

                        # Appending results to the final results dataframe            
                        Test_results = Test_results.append(result, ignore_index=True)


        if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach
            if len(prod_ensemble_list) != 3: # if 3 products are not selected
                clear_output()
                print('Select 3 products as existing model pack')
            else:

                # Similarity score of each existing product
                sim_score_p1 = score_data[score_data.Product == sorted(qtr_df_final['Existing Model Pack'].unique())[0]]['Similarity_Score'].values[0]
                sim_score_p2 = score_data[score_data.Product == sorted(qtr_df_final['Existing Model Pack'].unique())[1]]['Similarity_Score'].values[0]
                sim_score_p3 = score_data[score_data.Product == sorted(qtr_df_final['Existing Model Pack'].unique())[2]]['Similarity_Score'].values[0]

                # Filtering test results for different products
                Test_results1 = Test_results[Test_results.Product == sorted(qtr_df_final['Existing Model Pack'].unique())[0]]
                Test_results2 = Test_results[Test_results.Product == sorted(qtr_df_final['Existing Model Pack'].unique())[1]]
                Test_results3 = Test_results[Test_results.Product == sorted(qtr_df_final['Existing Model Pack'].unique())[2]]

                # Renaming test results columns
                Test_results2.rename(columns = {'Predictions_rf':'Predictions_rf2','Predictions_rf_exist':'Predictions_rf_exist2','Sales_derived':'Sales_derived2','Sales_derived_exist':'Sales_derived_exist2'}, inplace = True)
                Test_results3.rename(columns = {'Predictions_rf':'Predictions_rf3','Predictions_rf_exist':'Predictions_rf_exist3','Sales_derived':'Sales_derived3','Sales_derived_exist':'Sales_derived_exist3'}, inplace = True)

                # Merging for all 3 existing products
                Test_results_check = Test_results1.merge(Test_results2[['Banner','Region','Channel','Week','Test_period','Predictions_rf2','Predictions_rf_exist2','Sales_derived2','Sales_derived_exist2']], how = 'outer', on = ['Region','Channel','Banner','Week','Test_period']).merge(Test_results3[['Banner','Region','Channel','Week','Test_period','Predictions_rf3','Predictions_rf_exist3','Sales_derived3','Sales_derived_exist3']], how = 'outer', on = ['Region','Channel','Banner','Week','Test_period'])
                # New Product
                Test_results_check['Product'] = 'New'

                # Fill na as 0
                Test_results_check = Test_results_check.fillna(0)

                if toggle_manual_simscore.value == 'Manual':
                    # Getting average predictions
                    Test_results_check['Predictions_rf'] = (Test_results_check['Predictions_rf'] + Test_results_check['Predictions_rf2'] + Test_results_check['Predictions_rf3'])/3
                    Test_results_check['Predictions_rf_exist'] = (Test_results_check['Predictions_rf_exist']+ Test_results_check['Predictions_rf_exist2'] + Test_results_check['Predictions_rf_exist3'])/3
                    Test_results_check['Sales_derived'] = (Test_results_check['Sales_derived'] + Test_results_check['Sales_derived2'] + Test_results_check['Sales_derived3'])/3
                    Test_results_check['Sales_derived_exist'] = (Test_results_check['Sales_derived_exist']  + Test_results_check['Sales_derived_exist2'] + Test_results_check['Sales_derived_exist3'] )/3
                else:
                    # Getting weighted average predictions
                    Test_results_check['Predictions_rf'] = (Test_results_check['Predictions_rf'] * sim_score_p1 + Test_results_check['Predictions_rf2'] * sim_score_p2 + Test_results_check['Predictions_rf3'] * sim_score_p3)/(sim_score_p1 + sim_score_p2 + sim_score_p3)
                    Test_results_check['Predictions_rf_exist'] = (Test_results_check['Predictions_rf_exist'] * sim_score_p1 + Test_results_check['Predictions_rf_exist2'] * sim_score_p2 + Test_results_check['Predictions_rf_exist3'] * sim_score_p3)/(sim_score_p1 + sim_score_p2 + sim_score_p3)
                    Test_results_check['Sales_derived'] = (Test_results_check['Sales_derived'] * sim_score_p1 + Test_results_check['Sales_derived2'] * sim_score_p2 + Test_results_check['Sales_derived3'] * sim_score_p3)/(sim_score_p1 + sim_score_p2 + sim_score_p3)
                    Test_results_check['Sales_derived_exist'] = (Test_results_check['Sales_derived_exist'] * sim_score_p1 + Test_results_check['Sales_derived_exist2'] * sim_score_p2 + Test_results_check['Sales_derived_exist3'] * sim_score_p3)/(sim_score_p1 + sim_score_p2 + sim_score_p3)

                Test_results = Test_results_check[['Region', 'Product', 'Channel', 'Banner', 'Test_period', 'Week',
                               'Actuals', 'ape_rf', 'Predictions_rf', 'Predictions_rf_exist',
                               'Adcal_Price_exist', 'DD_1', 'DD_2', 'Adcal_Price', 'iter', 'Volume_RU',
                               'Volume_RU_exist', 'Sales_derived', 'Sales_derived_exist']].copy()                    

        clear_output()

    else:
        print("Product is not present in that Region/Channel for selected period")   
        
        
else:
    print(colored('Please upload the file to proceed', 'red',attrs=['bold']))

In [177]:
# Checking category, pack subtype, pack content of existing product matches with new product
cat_exist = Volume_dataset_all_reg[Volume_dataset_all_reg.Product == product_fil]['Category'].unique()[0]
packsubtype_exist = Volume_dataset_all_reg[Volume_dataset_all_reg.Product == product_fil]['Pack.Subtype'].unique()[0]
packcontent_exist = Volume_dataset_all_reg[Volume_dataset_all_reg.Product == product_fil]['PACK_CONTENT'].unique()[0]
test_data_set_cr_2 = pd.DataFrame()
test_data_set_cr_ontario = pd.DataFrame()
test_data_set_cr_east = pd.DataFrame()
test_data_set_cr_west = pd.DataFrame()
test_data_set_cr_quebec = pd.DataFrame()

#################################################################################################################### 
# Cannibalisation Code: Step 1 - New Product Predictions WITH UPLOADED DATA
#################################################################################################################### 

# Product_fil assigned as most similar product in ensemble approach
if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach    
    score_data_cann = similarity_score_calculation()
    score_data_cann = score_data_cann[score_data_cann.Product.isin(prod_ensemble_list)].sort_values('Similarity_Score', ascending = False)
    product_fil = score_data_cann.loc[:,'Product'][:1].values[0]
    
# In case the user uploads the data
if (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:

    # Defining model train and test periods    
    launch_date = upload_data['Week'].min()
    Test_week_list = {"Post_launch" : upload_data.loc[(upload_data.Week <= upload_data.Week.max()) & (upload_data["Week"]>= launch_date),"Week"].unique() }    
    # For step2 : to predict during 2019
    Test_week_list_2019 = {"Post_launch" : Volume_dataset_all_reg.loc[(Volume_dataset_all_reg.Week <= '2019-52') & (Volume_dataset_all_reg["Week"]>= '2019-01'),"Week"].unique() }    
    Test_periods = ["Post_launch"]

    #Defining empty dataframes for model training
    combined_dataset = pd.DataFrame({'Week': pd.Series([], dtype='object')})
    Test_results_np_cr = pd.DataFrame({'Region': pd.Series([], dtype='object')})
    Test_Data = pd.DataFrame()
    complete_train_data_set_aa = pd.DataFrame()

    # Model training and prediction
    category_check_list=[]

    # In case no product is present in selected region-channel
    if len(qtr_edv_baseline) != 0 :
        display(Markdown('<div><div class="loader"></div><h2> &nbsp;Measuring Cannibalization</h2></div>'))
        size_exist = Volume_dataset_all_reg[Volume_dataset_all_reg.Product == product_fil]['SIZE_ML'].unique()[0]
        count_exist = Volume_dataset_all_reg[Volume_dataset_all_reg.Product == product_fil]['COUNT'].unique()[0]
        
        # Run for 1 iteration
        iter_change_list = [1]

        for i in iter_change_list:
            channel_list = upload_data.Channel.unique()
            #Model training and prediction
            for Region_key in upload_data.Region.unique(): 
                # In case the product is not present at that region
                if len(upload_data[(upload_data.Region == Region_key)]) == 0:
                    continue

                Volume_dataset = Volume_dataset_all_reg.loc[(Volume_dataset_all_reg["Region"] == Region_key)
                                                           &(Volume_dataset_all_reg["Channel"].isin(channel_list))]
    
                Volume_dataset_upload = upload_data.loc[(upload_data["Region"] == Region_key)
                                           &(upload_data["Channel"].isin(channel_list))]
                
                Volume_dataset['Pantry2'] = Volume_dataset['Pantry2'].fillna(0)
                Volume_dataset_upload['Pantry2'] = Volume_dataset_upload['Pantry2'].fillna(0)
            
                required_columns_upload = ['Week','Week.Name','Banner','Product','Channel','Pack.Subtype','PACK_CONTENT','Adcal_Price','EDV.Price','Adcal_DD','Front.Page','Middle.Page','Back.Page','seasonality_index','SIZE_ML','COUNT','Category','Pantry1','Pantry2','Holiday.Week','Pre.Holiday.Week','Post.Holiday.Week'] 
                required_columns = ['Week','Week.Name','Banner','Product','Channel','Pack.Subtype','PACK_CONTENT','Adcal_Price','EDV.Price','Adcal_DD','Eq.Unit.Sales','Front.Page','Middle.Page','Back.Page','seasonality_index','SIZE_ML','COUNT','Category','Pantry1','Pantry2','Holiday.Week','Pre.Holiday.Week','Post.Holiday.Week'] 
                combined_dataset = Volume_dataset[required_columns] 
                combined_dataset_upload = Volume_dataset_upload[required_columns_upload] 

                combined_dataset = combined_dataset.loc[((combined_dataset["Week"] >= "2017-01") & 
                                                         (combined_dataset["Week"] <= "2019-52"))]
                
                combined_dataset["Christmas_flag"] = [1 if x=="Christmas" else 0 for x in combined_dataset["Week.Name"]]
                combined_dataset["Easter_flag"] = [1 if x=="Easter" else 0 for x in combined_dataset["Week.Name"]]
                
                combined_dataset_upload["Christmas_flag"] = [1 if x=="Christmas" else 0 for x in combined_dataset_upload["Week.Name"]]
                combined_dataset_upload["Easter_flag"] = [1 if x=="Easter" else 0 for x in combined_dataset_upload["Week.Name"]]                

                banner_dummies = pd.get_dummies(combined_dataset.Banner) 
                banner_dummies = banner_dummies.reindex(sorted(banner_dummies.columns), axis=1) #sorting banner columns
                combined_dataset = pd.concat([combined_dataset, banner_dummies], axis=1)
                
                # Uploaded data banner dummies
                banner_dummies_upload = pd.get_dummies(combined_dataset_upload.Banner)                                 
                # If all banners are not present for the uploaded data
                c = [i for i in list(banner_dummies.columns) if i not in list(banner_dummies_upload.columns)]
                if len(c) != 0:
                    banner_dummies_upload[c] = 0
                # Reordering columns                
                banner_dummies_upload = banner_dummies_upload.reindex(sorted(banner_dummies_upload.columns), axis=1)                
                combined_dataset_upload = pd.concat([combined_dataset_upload, banner_dummies_upload], axis=1)
                
                ## Not adding Product dummies since for New Product Simulator other Product dummies are not significant

                combined_dataset = combined_dataset.loc[combined_dataset["Eq.Unit.Sales"].notnull()]            

                combined_dataset.sort_values('Week',inplace = True)
                combined_dataset = pd.merge(combined_dataset,new_prod_stage,on = ['Product','Week'],how = 'left')
                combined_dataset['Intial_weeks'] = combined_dataset['Intial_weeks'].fillna('Stabilization')
                combined_dataset.drop(columns='Unnamed: 0', inplace=True)

                combined_dataset_upload.sort_values('Week',inplace = True)
                combined_dataset_upload = pd.merge(combined_dataset_upload,prod_stage_check_upload(),on = ['Product','Week'],how = 'left')
                combined_dataset_upload['Intial_weeks'] = combined_dataset_upload['Intial_weeks'].fillna('Stabilization')                
                
                #Adding dummies for category
                category_dummies = pd.get_dummies(combined_dataset['Category'])
                combined_dataset = pd.concat([combined_dataset, category_dummies], axis=1) 
                combined_dataset.drop(["Category"], inplace=True, axis=1)

                cat_col = category_dummies.columns #need all category dummy in the test data
                
                #Adding dummies for category UPLOADED DDATA
                combined_dataset_upload[cat_col] = 0
                
                #Adding dummies for product attribute - pack subtype
                pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
                combined_dataset = pd.concat([combined_dataset, pack_subtypes_dummies], axis=1)
                combined_dataset.drop(["Pack.Subtype"], inplace=True, axis=1)

                pack_subtypes_col = pack_subtypes_dummies.columns
                
                #Adding dummies for packsubtype UPLOADED DDATA
                combined_dataset_upload[pack_subtypes_col] = 0
                
                # Adding pack content dummies
                pack_content_dummies = pd.get_dummies(combined_dataset['PACK_CONTENT'])
                combined_dataset = pd.concat([combined_dataset, pack_content_dummies], axis=1)
                combined_dataset.drop(["PACK_CONTENT"], inplace=True, axis=1)

                pack_content_col = pack_content_dummies.columns
                
                #Adding dummies for packsubtype UPLOADED DDATA
                combined_dataset_upload[pack_content_col] = 0
                
                combined_dataset_upload.drop(["Pack.Subtype"], inplace=True, axis=1)
                combined_dataset_upload.drop(["PACK_CONTENT"], inplace=True, axis=1)
                combined_dataset_upload.drop(["Category"], inplace=True, axis=1)
                
                # Product stage dummies
                stage_dummies = pd.get_dummies(combined_dataset['Intial_weeks'])
                combined_dataset = pd.concat([combined_dataset, stage_dummies], axis=1)
                combined_dataset.drop(["Intial_weeks"], inplace=True, axis=1)

                # Product stage dummies
                stage_dummies_upload = pd.get_dummies(combined_dataset_upload['Intial_weeks'])
                
                # If all product stages are not present for the uploaded data
                c = [i for i in list(stage_dummies.columns) if i not in list(stage_dummies_upload.columns)]
                if len(c) != 0 :
                    stage_dummies_upload[c] = 0
                # Reordering columns                
                stage_dummies_upload = stage_dummies_upload.reindex(sorted(stage_dummies_upload.columns), axis=1)
                combined_dataset_upload = pd.concat([combined_dataset_upload, stage_dummies_upload], axis=1)
                combined_dataset_upload.drop(["Intial_weeks"], inplace=True, axis=1)

                #Adding discount depth columns
                combined_dataset["DD_1"] = [1 if (0.25> x >0.1)
                                      else 0 for x in combined_dataset["Adcal_DD"]]
                combined_dataset["DD_2"] = [1 if (x>=0.25)
                                                 else 0 for x in combined_dataset["Adcal_DD"]]

                combined_dataset_upload["DD_1"] = [1 if (0.25> x >0.1)
                                      else 0 for x in combined_dataset_upload["Adcal_DD"]]
                combined_dataset_upload["DD_2"] = [1 if (x>=0.25)
                                                 else 0 for x in combined_dataset_upload["Adcal_DD"]]            

                # Adcal Price calculation                
                combined_dataset_upload.loc[combined_dataset_upload['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset_upload.loc[combined_dataset_upload['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset_upload.loc[combined_dataset_upload['Adcal_DD']>=0.1,'EDV.Price']
                # Adcal_DD < 10 then Adcal Price = EDV Price            
                combined_dataset_upload.loc[combined_dataset_upload['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset_upload.loc[combined_dataset_upload['Adcal_DD'] < 0.1, 'EDV.Price']

                # Adcal Price calculation
                combined_dataset.loc[combined_dataset['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'EDV.Price']
                # Adcal_DD < 10 then Adcal Price = EDV Price            
                combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'EDV.Price']

                # Dropping adcal DD as DD1 & DD2 are added
                combined_dataset.drop(["Adcal_DD"], inplace=True, axis=1)
                combined_dataset_upload.drop(["Adcal_DD"], inplace=True, axis=1)

                # Dropping null volumes
                complete_dataset = combined_dataset.copy()
                complete_dataset_upload = combined_dataset_upload.copy()

                complete_dataset = complete_dataset.loc[complete_dataset["Eq.Unit.Sales"].notnull()]

                complete_dataset['Adcal_Price'] = np.log(complete_dataset['Adcal_Price'])
                complete_dataset['Eq.Unit.Sales'] = np.log(complete_dataset['Eq.Unit.Sales'])

                complete_dataset_upload['Adcal_Price'] = np.log(complete_dataset_upload['Adcal_Price'])
                
                for Test_period in Test_periods: 

                    #Filtering test weeks
                    Test_weeks = Test_week_list[Test_period]                                
                    
                    # Dividing train & test data
                    complete_train_data_set = complete_dataset.loc[~complete_dataset["Week"].isin(Test_weeks)].reset_index(drop=True)
                   # Data for new product
                    complete_test_data_set = complete_dataset_upload.loc[complete_dataset_upload["Week"].isin(Test_weeks)].reset_index(drop=True)

                    # Dropping unneccessary columns from train dataset
                    train_data_set = complete_train_data_set.copy()

                    train_data_brand = train_data_set.copy()
                    train_data_brand.drop(["Week","Week.Name","Banner","Product",'Channel',"EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)

                    # Filtering for existing product
                    test_data_set = complete_test_data_set.copy()

                    # New Category
                    if i == 1:
                        for cols in upload_data.Category.unique():                            
                            test_data_set[cols] = 1
                            
                    # New Pack Subtype
                    if i == 1:
                        for cols in upload_data['Pack.Subtype'].unique():                            
                            test_data_set[cols] = 1
                    
                    # New Pack Content
                    if i == 1:
                        for cols in upload_data['PACK_CONTENT'].unique():                            
                            test_data_set[cols] = 1
                    
                    if product_fil == 'TCCC CORE POWER 414 ML BTTL':
                        test_data_set = test_data_set.drop_duplicates()
                    
                    test_data_brand = test_data_set.copy()

                    if i == 1 :
                        # For New SIZE
                        test_data_brand['SIZE_ML'] = upload_data.SIZE_ML.unique()[0]

                        # For New Count
                        test_data_brand['COUNT'] = upload_data.COUNT.unique()[0]

                    # Dropping unneccessary columns from test dataset
                    test_data_brand.drop(["Week","Week.Name","Banner","Product",'Channel',"EDV.Price"], inplace=True, axis=1)

                    #Creating test and train data - independent & dependent columns
                    X_train = train_data_brand
                    y_train = train_data_set["Eq.Unit.Sales"]
                    X_test = test_data_brand

                    # Need this dataframe to copy new product features to step 3 of cannibalization
                    test_data_set_cr_1 = test_data_set.copy() 
                    if Region_key == 'ONTARIO':                        
                        test_data_set_cr_1['Region'] = Region_key                                         
                        test_data_set_cr_ontario = test_data_set_cr_ontario.append(test_data_set_cr_1, ignore_index = True)

                    if Region_key == 'EAST':                        
                        test_data_set_cr_1['Region'] = Region_key                                         
                        test_data_set_cr_east = test_data_set_cr_east.append(test_data_set_cr_1, ignore_index = True)

                    if Region_key == 'WEST':                        
                        test_data_set_cr_1['Region'] = Region_key                                       
                        test_data_set_cr_west = test_data_set_cr_west.append(test_data_set_cr_1, ignore_index = True)

                    if Region_key == 'QUEBEC':                        
                        test_data_set_cr_1['Region'] = Region_key                                
                        test_data_set_cr_quebec = test_data_set_cr_quebec.append(test_data_set_cr_1, ignore_index = True)
                    
                    # Appending test data
                    test_data_set["Region"] = Region_key
                    test_data_set["Test_period"] = Test_period
                    test_data_set['iter'] = i

                    # In case the product is not present in certain region-quarter
                    if len(test_data_set) == 0:
                        continue

                    #Model training
                    rf = RandomForestRegressor(n_jobs=4,random_state=0)
                    rf.fit(X_train, y_train)

                    #Model predictions
                    predictions_rf = rf.predict(X_test)                

                    #Storing test results
                    result = X_test.copy()

                    result["Predictions_rf"] = predictions_rf

                    result['Predictions_rf'] = np.exp(result['Predictions_rf'])                    

                    #Calculating APE for the models

                    result["Product"] = test_data_set["Product"]
                    result["Region"] = Region_key
                    result["Test_period"] = Test_period
                    result["Week"] = test_data_set["Week"]
                    result["DD_1"] = test_data_set['DD_1']
                    result["DD_2"] = test_data_set['DD_2']
                    result["Banner"] = test_data_set['Banner']
                    result["Channel"] = test_data_set['Channel']
                    ##Columns for Revenue                    
                    result["Adcal_Price"] = test_data_set["Adcal_Price"]
                    result['Adcal_Price'] = np.exp(result['Adcal_Price'])

                    result['iter'] = i            

                    result = result[["Region","Product",'Channel',"Banner","Test_period","Week","Predictions_rf","DD_1","DD_2","Adcal_Price",'iter']]

                    # Appending results to the final results dataframe            
                    Test_results_np_cr = Test_results_np_cr.append(result, ignore_index=True)


        clear_output()

    else:
        print("Product is not present in that Region/Channel for selected period")
   

 #################################################################################################################### 
# Cannibalisation Code: Step 1 - New Product Predictions WITHOUT UPLOADED DATA
#################################################################################################################### 

# In case the user doesn't upload the data
elif (toggle_upload.value == 'No'):

    # Defining model train and test periods    
    launch_date = Volume_dataset_all_reg[(Volume_dataset_all_reg.Product == product_fil) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value)]['Week'].min()

    if launch_date < '2019-01':
        launch_date = '2019-01'

    Test_week_list = {
         "Post_launch" : Volume_dataset_all_reg.loc[(Volume_dataset_all_reg.Test_period <= end_week.children[8].value) & ((Volume_dataset_all_reg["Week"]>= launch_date)),"Week"].unique() }

    Test_periods = ["Post_launch"]

    #Defining empty dataframes for model training
    combined_dataset = pd.DataFrame({'Week': pd.Series([], dtype='object')})
    Test_results_np_cr = pd.DataFrame({'Region': pd.Series([], dtype='object')})
    Test_Data = pd.DataFrame()
    complete_train_data_set_aa = pd.DataFrame()

    # Model training and prediction
    category_check_list=[]

    # In case no product is present in selected region-channel
    if len(qtr_edv_baseline) != 0 :
        display(Markdown('<div><div class="loader"></div><h2> &nbsp;Measuring Cannibalization</h2></div>'))
        size_exist = Volume_dataset_all_reg[Volume_dataset_all_reg.Product == product_fil]['SIZE_ML'].unique()[0]
        count_exist = Volume_dataset_all_reg[Volume_dataset_all_reg.Product == product_fil]['COUNT'].unique()[0]

        if toggle_np2.value == 'Yes':
            iter_change_list = [1,2]
        else:
            iter_change_list = [1]

        for i in iter_change_list:        

            # Model training and prediction
            for Region_key in Region_List:        

                # In case the product is not present at that region
                if len(Volume_dataset_all_reg[(Volume_dataset_all_reg.Product == product_fil) & (Volume_dataset_all_reg.Region == Region_key)]) == 0:
                    continue

                category_check_list.extend(list(Volume_dataset_all_reg[(Volume_dataset_all_reg["Region"] == Region_key) & (Volume_dataset_all_reg["Product"] == product_fil)].Category.unique()))

                Volume_dataset = Volume_dataset_all_reg.loc[(Volume_dataset_all_reg["Region"] == Region_key)
                                                           &(Volume_dataset_all_reg["Channel"].isin(channel_list))]
                Volume_dataset['Pantry2'] = Volume_dataset['Pantry2'].fillna(0)
                required_columns = ['Week','Week.Name','Banner','Product','Channel','Pack.Subtype','PACK_CONTENT','Adcal_Price','EDV.Price','Adcal_DD','Eq.Unit.Sales','Front.Page','Middle.Page','Back.Page','seasonality_index','SIZE_ML','COUNT','No_of_brands','No_of_flavors','No_of_sweetners','No_of_types','Category','Pantry1','Pantry2','Holiday.Week','Pre.Holiday.Week','Post.Holiday.Week']
                combined_dataset = Volume_dataset[required_columns]

                combined_dataset = combined_dataset.loc[((combined_dataset["Week"] >= "2017-01") & 
                                                         (combined_dataset["Week"] <= "2019-52"))]

                combined_dataset["Christmas_flag"] = [1 if x=="Christmas" else 0 for x in combined_dataset["Week.Name"]]
                combined_dataset["Easter_flag"] = [1 if x=="Easter" else 0 for x in combined_dataset["Week.Name"]]

                ## CHECKING if banner dummies are not present in required region, channel then introduce banner dummies
                banner_dummies = pd.get_dummies(combined_dataset.Banner)              

                # Uploaded data banner dummies
                banner_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key].Banner)                                 
                # If all banners are not present for the training data
                c = [i for i in list(banner_dummies_check.columns) if i not in list(banner_dummies.columns)]
                if len(c) != 0:
                    banner_dummies[c] = 0

                # Reordering columns                
                banner_dummies = banner_dummies.reindex(sorted(banner_dummies.columns), axis=1)                
                combined_dataset = pd.concat([combined_dataset, banner_dummies], axis=1)

                ## Not adding Product dummies since for New Product Simulator other Product dummies are not significant
                combined_dataset = combined_dataset.loc[combined_dataset["Eq.Unit.Sales"].notnull()]

                combined_dataset.sort_values('Week',inplace = True)
                combined_dataset = pd.merge(combined_dataset,new_prod_stage,on = ['Product','Week'],how = 'left')
                combined_dataset['Intial_weeks'] = combined_dataset['Intial_weeks'].fillna('Stabilization')
                combined_dataset.drop(columns='Unnamed: 0', inplace=True)

                #Adding dummies for category
                category_dummies = pd.get_dummies(combined_dataset['Category'])

                # Adding dummies for product attribute - Category
                pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
                # Uploaded data category dummies
                category_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['Category'])                                 
                # If all pack subtypes are not present for the training data
                c = [i for i in list(category_dummies_check.columns) if i not in list(category_dummies.columns)]
                if len(c) != 0:
                    category_dummies[c] = 0

                # Reordering columns                
                category_dummies = category_dummies.reindex(sorted(category_dummies.columns), axis=1)                
                combined_dataset = pd.concat([combined_dataset, category_dummies], axis=1)
                combined_dataset.drop(["Category"], inplace=True, axis=1)

                # Adding dummies for product attribute - pack subtype
                pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
                # Uploaded data pack subtype dummies
                pack_subtypes_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['Pack.Subtype'])                                 
                # If all pack subtypes are not present for the training data
                c = [i for i in list(pack_subtypes_dummies_check.columns) if i not in list(pack_subtypes_dummies.columns)]
                if len(c) != 0:
                    pack_subtypes_dummies[c] = 0

                # Reordering columns                
                pack_subtypes_dummies = pack_subtypes_dummies.reindex(sorted(pack_subtypes_dummies.columns), axis=1)                
                combined_dataset = pd.concat([combined_dataset, pack_subtypes_dummies], axis=1)
                combined_dataset.drop(["Pack.Subtype"], inplace=True, axis=1)

                # Adding pack content dummies
                pack_content_dummies = pd.get_dummies(combined_dataset['PACK_CONTENT'])
                # Uploaded data pack content dummies
                pack_content_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['PACK_CONTENT'])                                 
                # If all pack contents are not present for the training data
                c = [i for i in list(pack_content_dummies_check.columns) if i not in list(pack_content_dummies.columns)]
                if len(c) != 0:
                    pack_content_dummies[c] = 0

                # Reordering columns                
                pack_content_dummies = pack_content_dummies.reindex(sorted(pack_content_dummies.columns), axis=1)                
                combined_dataset = pd.concat([combined_dataset, pack_content_dummies], axis=1)
                combined_dataset.drop(["PACK_CONTENT"], inplace=True, axis=1)

                combined_dataset_test = combined_dataset.copy()

                # Product stage dummies
                stage_dummies = pd.get_dummies(combined_dataset['Intial_weeks'])
                combined_dataset = pd.concat([combined_dataset, stage_dummies], axis=1)
                combined_dataset.drop(["Intial_weeks"], inplace=True, axis=1)

                # Changing product stage as per start date provided by the user
                prod_stage_data = prod_stage_check(product_fil)            

                # Getting updated product stages
                combined_dataset_test_check = combined_dataset_test[combined_dataset_test.Product == product_fil].merge(prod_stage_data[['Week','Initial_weeks1']], how = 'left')
                combined_dataset_test_check['Initial_weeks1'] = combined_dataset_test_check['Initial_weeks1'].fillna(combined_dataset_test_check['Intial_weeks'])
                combined_dataset_test_check.drop(columns = {'Intial_weeks'}, inplace = True)
                combined_dataset_test_check.rename(columns = {'Initial_weeks1':'Intial_weeks'}, inplace = True)

                combined_dataset_test = combined_dataset_test_check.copy()

                # Product stage dummies
                stage_dummies_test = pd.get_dummies(combined_dataset_test['Intial_weeks'])
                
                # If all product stages are not present for the uploaded data
                c = [i for i in list(stage_dummies.columns) if i not in list(stage_dummies_test.columns)]
                if len(c) != 0 :
                    stage_dummies_test[c] = 0
                    
                # Reordering columns                
                stage_dummies_test = stage_dummies_test.reindex(sorted(stage_dummies_test.columns), axis=1)
                combined_dataset_test = pd.concat([combined_dataset_test, stage_dummies_test], axis=1)
                combined_dataset_test.drop(["Intial_weeks"], inplace=True, axis=1)

                # In case the user doesn't want to upload adcal data
                if toggle_upload.value != 'Yes':

                    # CHANGES : For adcal discounts
                    if 'All' in list(test_period.value):                
                        # Discount change
                        if i == 1:                
                            combined_dataset_test['Adcal_DD'] = (1+disc_change_fil)*combined_dataset_test['Adcal_DD']
                            combined_dataset_test.loc[:,'EDV.Price'] = (1+base_price_change_fil)*combined_dataset_test.loc[:,'EDV.Price']
                        elif i == 2:
                            combined_dataset_test['Adcal_DD'] = (1+disc_change_fil2)*combined_dataset_test['Adcal_DD']
                            combined_dataset_test.loc[:,'EDV.Price'] = (1+base_price_change_fil2)*combined_dataset_test.loc[:,'EDV.Price']          
                    else:                
                        if i == 1:                
                            # Discount change
                            combined_dataset_test.loc[combined_dataset_test['Test_period'].isin(test_period_fil),'Adcal_DD'] = (1+disc_change_fil)*combined_dataset_test[combined_dataset_test['Test_period'].isin(test_period_fil)]['Adcal_DD'] 
                            # Baseline pricing
                            combined_dataset_test.loc[(combined_dataset_test['Test_period'].isin(test_period_fil)),'EDV.Price'] = (1+base_price_change_fil)*combined_dataset_test[(combined_dataset_test['Test_period'].isin(test_period_fil))]['EDV.Price']
                        elif i == 2:
                            # Discount change
                            combined_dataset_test.loc[combined_dataset_test['Test_period'].isin(test_period_fil2),'Adcal_DD'] = (1+disc_change_fil2)*combined_dataset_test[combined_dataset_test['Test_period'].isin(test_period_fil2)]['Adcal_DD']             
                            # Baseline pricing
                            combined_dataset_test.loc[(combined_dataset_test['Test_period'].isin(test_period_fil2)),'EDV.Price'] = (1+base_price_change_fil2)*combined_dataset_test[(combined_dataset_test['Test_period'].isin(test_period_fil2))]['EDV.Price']

                #Adding discount depth columns
                combined_dataset["DD_1"] = [1 if (0.25> x >0.1)
                                      else 0 for x in combined_dataset["Adcal_DD"]]
                combined_dataset["DD_2"] = [1 if (x>=0.25)
                                                 else 0 for x in combined_dataset["Adcal_DD"]]

                combined_dataset_test["DD_1"] = [1 if (0.25> x >0.1)
                                      else 0 for x in combined_dataset_test["Adcal_DD"]]
                combined_dataset_test["DD_2"] = [1 if (x>=0.25)
                                                 else 0 for x in combined_dataset_test["Adcal_DD"]]            

                # Adcal Price calculation
                # In case the discounts go over 100%, limit to 100%
                combined_dataset_test.loc[combined_dataset_test.Adcal_DD >= 1, 'Adcal_DD'] = 0.99
                
                combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset_test.loc[combined_dataset_test['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset_test.loc[combined_dataset_test['Adcal_DD']>=0.1,'EDV.Price']
                # Adcal_DD < 10 then Adcal Price = EDV Price            
                combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset_test.loc[combined_dataset_test['Adcal_DD'] < 0.1, 'EDV.Price']

                # Adcal Price calculation
                combined_dataset.loc[combined_dataset['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'EDV.Price']
                # Adcal_DD < 10 then Adcal Price = EDV Price            
                combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'EDV.Price']

                # Dropping adcal DD as DD1 & DD2 are added
                combined_dataset.drop(["Adcal_DD"], inplace=True, axis=1)
                combined_dataset_test.drop(["Adcal_DD"], inplace=True, axis=1)

                # Dropping null volumes
                complete_dataset = combined_dataset.copy()
                complete_dataset_test = combined_dataset_test.copy()

                complete_dataset = complete_dataset.loc[complete_dataset["Eq.Unit.Sales"].notnull()]
                complete_dataset_test = complete_dataset_test.loc[complete_dataset_test["Eq.Unit.Sales"].notnull()]

                complete_dataset['Adcal_Price'] = np.log(complete_dataset['Adcal_Price'])
                complete_dataset['Eq.Unit.Sales'] = np.log(complete_dataset['Eq.Unit.Sales'])

                complete_dataset_test['Adcal_Price'] = np.log(complete_dataset_test['Adcal_Price'])
                complete_dataset_test['Eq.Unit.Sales'] = np.log(complete_dataset_test['Eq.Unit.Sales'])

                # TEST PERIOD LOOP            
                for Test_period in Test_periods: 

                    #Filtering test weeks
                    Test_weeks = Test_week_list[Test_period]                                

                    # Dividing train & test data                    
                    complete_train_data_set = complete_dataset.loc[~(complete_dataset["Week"].isin(Test_weeks))].reset_index(drop=True)

                    # Data for new product
                    complete_test_data_set = complete_dataset_test.loc[complete_dataset_test["Week"].isin(Test_weeks)].reset_index(drop=True)

                    # Dropping unneccessary columns from train dataset
                    train_data_set = complete_train_data_set.copy()

                    train_data_brand = train_data_set.copy()
                    train_data_brand.drop(["Week","Week.Name","Banner","Product",'Channel',"EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)

                    # Filtering for existing product
                    test_data_set = complete_test_data_set[complete_test_data_set.Product == product_fil].copy()
                    # In case the user doesn't want to upload adcal data
                    if toggle_upload.value != 'Yes':

                        #################### INCREASING/DECREASING PROMOTIONS FOR BANNER WEEKS COMBINATIONS ####################

                        if (promo_change_toggle.value == 'Yes (All Banners)') | (promo_change_toggle.value == 'Yes (Specific Banners)'):
                            if (front_change.children[1].value != 0) | (middle_change.children[1].value != 0) | (back_change.children[1].value != 0) | (front_change2.children[1].value != 0) | (middle_change2.children[1].value != 0) | (back_change2.children[1].value != 0) | (front_change3.children[1].value != 0) | (middle_change3.children[1].value != 0) | (back_change3.children[1].value != 0) : 

                                ## FRONT PAGE PROMO CHANGES
                                # New Product 1
                                if i == 1:
                                    if front_change.children[1].value > 0:
                                        # Adding additional promotions in the required banner-weeks
                                        for week,banner in zip(ban_week_fp['Week'], ban_week_fp['Banner']):                            
                                            test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Front.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Front.Page'] + 1
                                    elif front_change.children[1].value < 0:
                                        for week,banner in zip(ban_week_fp['Week'], ban_week_fp['Banner']):                            
                                            test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Front.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Front.Page'] - 1

                                elif i == 2:
                                    if front_change2.children[1].value > 0:
                                        # Adding additional promotions in the required banner-weeks
                                        for week,banner in zip(ban_week_fp2['Week'], ban_week_fp2['Banner']):                            
                                            test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Front.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Front.Page'] + 1
                                    elif front_change2.children[1].value < 0:
                                        for week,banner in zip(ban_week_fp2['Week'], ban_week_fp2['Banner']):                            
                                            test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Front.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Front.Page'] - 1

                                ## MIDDLE PAGE PROMO CHANGES
                                if i == 1:
                                    if middle_change.children[1].value > 0:
                                        # Adding additional promotions in the required banner-weeks
                                        for week,banner in zip(ban_week_mp['Week'], ban_week_mp['Banner']):                            
                                            test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Middle.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Middle.Page'] + 1
                                    elif middle_change.children[1].value < 0:
                                        for week,banner in zip(ban_week_mp['Week'], ban_week_mp['Banner']):                            
                                            test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Middle.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Middle.Page'] - 1
                                elif i == 2:
                                    if middle_change2.children[1].value > 0:
                                        # Adding additional promotions in the required banner-weeks
                                        for week,banner in zip(ban_week_mp2['Week'], ban_week_mp2['Banner']):                            
                                            test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Middle.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Middle.Page'] + 1
                                    elif middle_change2.children[1].value < 0:
                                        for week,banner in zip(ban_week_mp2['Week'], ban_week_mp2['Banner']):                            
                                            test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Middle.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Middle.Page'] - 1

                                ## BACK PAGE PROMO CHANGES
                                if i == 1:
                                    if back_change.children[1].value > 0:
                                        # Adding additional promotions in the required banner-weeks
                                        for week,banner in zip(ban_week_bp['Week'], ban_week_bp['Banner']):                            
                                            test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Back.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Back.Page'] + 1
                                    elif back_change.children[1].value < 0:
                                        for week,banner in zip(ban_week_bp['Week'], ban_week_bp['Banner']):                            
                                            test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Back.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Back.Page'] - 1
                                elif i == 2:
                                    if back_change2.children[1].value > 0:
                                        # Adding additional promotions in the required banner-weeks
                                        for week,banner in zip(ban_week_bp2['Week'], ban_week_bp2['Banner']):                            
                                            test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Back.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Back.Page'] + 1
                                    elif back_change2.children[1].value < 0:
                                        for week,banner in zip(ban_week_bp2['Week'], ban_week_bp2['Banner']):                            
                                            test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Back.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Back.Page'] - 1

                    # New Category
                    if i == 1:
                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)].Category.unique():
                            if (cols == category_drop.value):                      
                                test_data_set[cols] = 1
                            else:
                                test_data_set[cols] = 0
                    if i == 2:
                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)].Category.unique():
                            if (cols == category_drop2.value):                      
                                test_data_set[cols] = 1
                            else:
                                test_data_set[cols] = 0

                    # New Pack Subtype
                    if i == 1:
                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['Pack.Subtype'].unique():
                            if (cols == pack_subtype_drop.value):                      
                                test_data_set[cols] = 1
                            else:
                                test_data_set[cols] = 0
                    if i == 2:
                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['Pack.Subtype'].unique():
                            if (cols == pack_subtype_drop2.value):                      
                                test_data_set[cols] = 1
                            else:
                                test_data_set[cols] = 0
                    # New Pack Content
                    if i == 1:
                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['PACK_CONTENT'].unique():
                            if (cols == pack_content_drop.value):                      
                                test_data_set[cols] = 1
                            else:
                                test_data_set[cols] = 0
                    if i == 2:
                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['PACK_CONTENT'].unique():
                            if (cols == pack_content_drop2.value):                      
                                test_data_set[cols] = 1
                            else:
                                test_data_set[cols] = 0

                    if product_fil == 'TCCC CORE POWER 414 ML BTTL':
                        test_data_set = test_data_set.drop_duplicates()
                                
                    test_data_brand = test_data_set.copy()                

                    if i == 1 :
                        # For New SIZE
                        test_data_brand['SIZE_ML'] = size.value

                        # For New Count
                        test_data_brand['COUNT'] = count.value
                    elif i == 2:
                        # For New SIZE
                        test_data_brand['SIZE_ML'] = size2.value

                        # For New Count
                        test_data_brand['COUNT'] = count2.value

                    # Dropping unneccessary columns from test dataset
                    test_data_brand.drop(["Week","Week.Name","Banner","Product",'Channel',"EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)

                    #Creating test and train data - independent & dependent columns
                    X_train = train_data_brand
                    y_train = train_data_set["Eq.Unit.Sales"]
                    X_test = test_data_brand
                    y_test = test_data_set["Eq.Unit.Sales"]
                                        
                    # Need this dataframe to copy new product features to step 3 of cannibalization
                    test_data_set_cr_1 = test_data_set.copy() 
                    if Region_key == 'ONTARIO':                        
                        test_data_set_cr_1['Region'] = Region_key
                        test_data_set_cr_1['iter'] = i                   
                        test_data_set_cr_ontario = test_data_set_cr_ontario.append(test_data_set_cr_1, ignore_index = True)

                    if Region_key == 'EAST':                        
                        test_data_set_cr_1['Region'] = Region_key
                        test_data_set_cr_1['iter'] = i                   
                        test_data_set_cr_east = test_data_set_cr_east.append(test_data_set_cr_1, ignore_index = True)

                    if Region_key == 'WEST':                        
                        test_data_set_cr_1['Region'] = Region_key
                        test_data_set_cr_1['iter'] = i                   
                        test_data_set_cr_west = test_data_set_cr_west.append(test_data_set_cr_1, ignore_index = True)

                    if Region_key == 'QUEBEC':                        
                        test_data_set_cr_1['Region'] = Region_key
                        test_data_set_cr_1['iter'] = i                   
                        test_data_set_cr_quebec = test_data_set_cr_quebec.append(test_data_set_cr_1, ignore_index = True)
                        
                    # Appending test data
                    test_data_set["Region"] = Region_key
                    test_data_set["Test_period"] = Test_period
                    test_data_set['iter'] = i

                    Test_Data = Test_Data.append(test_data_set, ignore_index = True)
                    # In case the product is not present in certain region-quarter
                    if len(test_data_set) == 0:
                        continue

                    #Model training
                    rf = RandomForestRegressor(n_jobs=4,random_state=0)
                    rf.fit(X_train, y_train)

                    #Model predictions
                    predictions_rf = rf.predict(X_test)                

                    #Storing test results
                    result = X_test.copy()

                    result["Predictions_rf"] = predictions_rf
                    result["Actuals"] = y_test
                    
                    result['Predictions_rf'] = np.exp(result['Predictions_rf'])                    
                    result['Actuals'] = np.exp(result['Actuals'])                    

                    #Calculating APE for the models
                    result["ape_rf"] = (result["Predictions_rf"]-result["Actuals"]).abs()

                    result["Product"] = test_data_set["Product"]
                    result["Region"] = Region_key
                    result["Test_period"] = Test_period
                    result["Week"] = test_data_set["Week"]
                    result["DD_1"] = test_data_set['DD_1']
                    result["DD_2"] = test_data_set['DD_2']
                    result["Banner"] = test_data_set['Banner']
                    result["Channel"] = test_data_set['Channel']
                    ##Columns for Revenue                    
                    result["Adcal_Price"] = test_data_set["Adcal_Price"]
                    result["Adcal_Price"] = np.exp(result["Adcal_Price"])
                    
                    result['iter'] = i            

                    result = result[["Region","Product",'Channel',"Banner","Test_period","Week","Actuals","ape_rf","Predictions_rf","DD_1","DD_2","Adcal_Price",'iter']]

                    # Appending results to the final results dataframe            
                    Test_results_np_cr = Test_results_np_cr.append(result, ignore_index=True)


        clear_output()

    else:
        print("Product is not present in that Region/Channel for selected period")

#################################################################################################################### 
# Cannibalisation Code: Step 2 - # Existing Product predictions without NP
#################################################################################################################### 

#Defining empty dataframes for model training
combined_dataset = pd.DataFrame({'Week': pd.Series([], dtype='object')})
Test_results_exist_wo_cr = pd.DataFrame({'Region': pd.Series([], dtype='object')})
Test_Data = pd.DataFrame()

#Model training and prediction
category_check_list=[]

# In case no product is present in selected region-channel
if len(qtr_edv_baseline) != 0 :
    display(Markdown('<div><div class="loader"></div><h2> &nbsp;Measuring Cannibalization</h2></div>'))
    size_exist = Volume_dataset_all_reg[Volume_dataset_all_reg.Product == product_fil]['SIZE_ML'].unique()[0]
    count_exist = Volume_dataset_all_reg[Volume_dataset_all_reg.Product == product_fil]['COUNT'].unique()[0]

    if (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:
        Region_List = upload_data.Region.unique()
        channel_list = upload_data.Channel.unique()
    
    # Model training and prediction
    for Region_key in Region_List:        
        # In case the product is not present at that region
        if len(Volume_dataset_all_reg[(Volume_dataset_all_reg.Product == product_fil) & (Volume_dataset_all_reg.Region == Region_key)]) == 0:
            continue

        category_check_list.extend(list(Volume_dataset_all_reg[(Volume_dataset_all_reg["Region"] == Region_key) & (Volume_dataset_all_reg["Product"] == product_fil)].Category.unique()))

        Volume_dataset = Volume_dataset_all_reg.loc[(Volume_dataset_all_reg["Region"] == Region_key)
                                                   &(Volume_dataset_all_reg["Channel"].isin(channel_list))]
        Volume_dataset['Pantry2'] = Volume_dataset['Pantry2'].fillna(0)
        if (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:
            required_columns = ['Week','Week.Name','Banner','Product','Channel','Pack.Subtype','PACK_CONTENT','Adcal_Price','EDV.Price','Adcal_DD','Eq.Unit.Sales','Front.Page','Middle.Page','Back.Page','seasonality_index','SIZE_ML','COUNT','Category','Pantry1','Pantry2','Holiday.Week','Pre.Holiday.Week','Post.Holiday.Week'] 
        elif (toggle_upload.value == 'No'): # Including number of brand, flavor, sweetener, type columns
            required_columns = ['Week','Week.Name','Banner','Product','Channel','Pack.Subtype','PACK_CONTENT','Adcal_Price','EDV.Price','Adcal_DD','Eq.Unit.Sales','Front.Page','Middle.Page','Back.Page','seasonality_index','SIZE_ML','COUNT','No_of_brands','No_of_flavors','No_of_sweetners','No_of_types','Category','Pantry1','Pantry2','Holiday.Week','Pre.Holiday.Week','Post.Holiday.Week']
        combined_dataset = Volume_dataset[required_columns]

        combined_dataset = combined_dataset.loc[((combined_dataset["Week"] >= "2017-01") & 
                                                 (combined_dataset["Week"] <= "2019-52"))]

        combined_dataset["Christmas_flag"] = [1 if x=="Christmas" else 0 for x in combined_dataset["Week.Name"]]
        combined_dataset["Easter_flag"] = [1 if x=="Easter" else 0 for x in combined_dataset["Week.Name"]]

        ## CHECKING if banner dummies are not present in required region, channel then introduce banner dummies
        banner_dummies = pd.get_dummies(combined_dataset.Banner)              

        # Uploaded data banner dummies
        banner_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key].Banner)                                 
        # If all banners are not present for the training data
        c = [i for i in list(banner_dummies_check.columns) if i not in list(banner_dummies.columns)]
        if len(c) != 0:
            banner_dummies[c] = 0

        # Reordering columns                
        banner_dummies = banner_dummies.reindex(sorted(banner_dummies.columns), axis=1)                
        combined_dataset = pd.concat([combined_dataset, banner_dummies], axis=1)

        ## Not adding Product dummies since for New Product Simulator other Product dummies are not significant
        combined_dataset = combined_dataset.loc[combined_dataset["Eq.Unit.Sales"].notnull()]

        combined_dataset.sort_values('Week',inplace = True)
        combined_dataset = pd.merge(combined_dataset,new_prod_stage,on = ['Product','Week'],how = 'left')
        combined_dataset['Intial_weeks'] = combined_dataset['Intial_weeks'].fillna('Stabilization')
        combined_dataset.drop(columns='Unnamed: 0', inplace=True)

        #Adding dummies for category
        category_dummies = pd.get_dummies(combined_dataset['Category'])

        # Adding dummies for product attribute - Category
        pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
        # Uploaded data category dummies
        category_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['Category'])                                 
        # If all pack subtypes are not present for the training data
        c = [i for i in list(category_dummies_check.columns) if i not in list(category_dummies.columns)]
        if len(c) != 0:
            category_dummies[c] = 0

        # Reordering columns                
        category_dummies = category_dummies.reindex(sorted(category_dummies.columns), axis=1)                
        combined_dataset = pd.concat([combined_dataset, category_dummies], axis=1)
        combined_dataset.drop(["Category"], inplace=True, axis=1)

        # Adding dummies for product attribute - pack subtype
        pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
        # Uploaded data pack subtype dummies
        pack_subtypes_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['Pack.Subtype'])                                 
        # If all pack subtypes are not present for the training data
        c = [i for i in list(pack_subtypes_dummies_check.columns) if i not in list(pack_subtypes_dummies.columns)]
        if len(c) != 0:
            pack_subtypes_dummies[c] = 0

        # Reordering columns                
        pack_subtypes_dummies = pack_subtypes_dummies.reindex(sorted(pack_subtypes_dummies.columns), axis=1)                
        combined_dataset = pd.concat([combined_dataset, pack_subtypes_dummies], axis=1)
        combined_dataset.drop(["Pack.Subtype"], inplace=True, axis=1)

        # Adding pack content dummies
        pack_content_dummies = pd.get_dummies(combined_dataset['PACK_CONTENT'])
        # Uploaded data pack content dummies
        pack_content_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['PACK_CONTENT'])                                 
        # If all pack contents are not present for the training data
        c = [i for i in list(pack_content_dummies_check.columns) if i not in list(pack_content_dummies.columns)]
        if len(c) != 0:
            pack_content_dummies[c] = 0

        # Reordering columns                
        pack_content_dummies = pack_content_dummies.reindex(sorted(pack_content_dummies.columns), axis=1)                
        combined_dataset = pd.concat([combined_dataset, pack_content_dummies], axis=1)
        combined_dataset.drop(["PACK_CONTENT"], inplace=True, axis=1)

        # Product stage dummies
        stage_dummies = pd.get_dummies(combined_dataset['Intial_weeks'])
        combined_dataset = pd.concat([combined_dataset, stage_dummies], axis=1)
        combined_dataset.drop(["Intial_weeks"], inplace=True, axis=1)


        #Adding discount depth columns
        combined_dataset["DD_1"] = [1 if (0.25> x >0.1)
                              else 0 for x in combined_dataset["Adcal_DD"]]
        combined_dataset["DD_2"] = [1 if (x>=0.25)
                                         else 0 for x in combined_dataset["Adcal_DD"]]
        # Adcal Price calculation
        combined_dataset.loc[combined_dataset['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'EDV.Price']
        # Adcal_DD < 10 then Adcal Price = EDV Price            
        combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'EDV.Price']

        # Dropping adcal DD as DD1 & DD2 are added
        combined_dataset.drop(["Adcal_DD"], inplace=True, axis=1)

        # Dropping null volumes
        complete_dataset = combined_dataset.copy()        

        complete_dataset = complete_dataset.loc[complete_dataset["Eq.Unit.Sales"].notnull()]        

        complete_dataset['Adcal_Price'] = np.log(complete_dataset['Adcal_Price'])
        complete_dataset['Eq.Unit.Sales'] = np.log(complete_dataset['Eq.Unit.Sales'])
        
        # TEST PERIOD LOOP
        for Test_period in Test_periods:

            if launch_date < Max_POS:
                Test_weeks = Test_week_list[Test_period]
            else:
                Test_weeks = Test_week_list_2019[Test_period] # 2019 weeks as test data
                
            # Dividing train & test data
            complete_train_data_set = complete_dataset.loc[(complete_dataset.Week <= max(Test_weeks)) & ~(complete_dataset["Week"].isin(Test_weeks))].reset_index(drop=True)            
            complete_test_data_set = complete_dataset.loc[complete_dataset["Week"].isin(Test_weeks)].reset_index(drop=True)

            # Dropping unneccessary columns from train dataset
            train_data_set = complete_train_data_set.copy()

            train_data_brand = train_data_set.copy()
            train_data_brand.drop(["Week","Week.Name","Banner","Product",'Channel',"EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)
          
            test_data_set = complete_test_data_set.copy()
            test_data_brand = test_data_set.copy()            

            # Dropping unneccessary columns from test dataset
            test_data_brand.drop(["Week","Week.Name","Banner","Product",'Channel',"EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)            

            #Creating test and train data - independent & dependent columns
            X_train = train_data_brand
            y_train = train_data_set["Eq.Unit.Sales"]
            X_test = test_data_brand     
            y_test = test_data_set["Eq.Unit.Sales"]

            # Appending test data
            test_data_set["Region"] = Region_key
            test_data_set["Test_period"] = Test_period
            test_data_set['iter'] = i

            Test_Data = Test_Data.append(test_data_set, ignore_index = True)
            # In case the product is not present in certain region-quarter
            if len(test_data_set) == 0:
                continue

            #Model training
            rf = RandomForestRegressor(n_jobs=4,random_state=0)
            rf.fit(X_train, y_train)

            #Model predictions
            predictions_rf = rf.predict(X_test)                       

            #Storing test results
            result = X_test.copy()            

            result["Predictions_rf"] = predictions_rf            
            result["Actuals"] = y_test

            result['Predictions_rf'] = np.exp(result['Predictions_rf'])                    
            result['Actuals'] = np.exp(result['Actuals'])                    
            
            #Calculating APE for the models
            result["ape_rf"] = (result["Predictions_rf"]-result["Actuals"]).abs()

            result["Product"] = test_data_set["Product"]
            result["Region"] = Region_key
            result["Test_period"] = Test_period
            result["Week"] = test_data_set["Week"]
            result["DD_1"] = test_data_set['DD_1']
            result["DD_2"] = test_data_set['DD_2']
            result["Banner"] = test_data_set['Banner']
            result["Channel"] = test_data_set['Channel']
            ##Columns for Revenue            
            result["Adcal_Price"] = test_data_set["Adcal_Price"]                               
            result["Adcal_Price"] = np.exp(result["Adcal_Price"])
            
            result = result[["Region","Product",'Channel',"Banner","Test_period","Week","Actuals","ape_rf","Predictions_rf","DD_1","DD_2","Adcal_Price"]]

            # Appending results to the final results dataframe            
            Test_results_exist_wo_cr = Test_results_exist_wo_cr.append(result, ignore_index=True)            

    clear_output()

else:
    print("Product is not present in that Region/Channel for selected period")

    
#################################################################################################################### 
# Cannibalisation Code: Step 3 - # Exist product with NP (WITH UPLOADED DATA)
#################################################################################################################### 

# In case the user uploads the data
if (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:

    #Defining empty dataframes for model training
    combined_dataset = pd.DataFrame({'Week': pd.Series([], dtype='object')})
    Test_results_exist_with_cr = pd.DataFrame({'Region': pd.Series([], dtype='object')})
    Test_Data = pd.DataFrame()

    #Model training and prediction
    category_check_list=[]

    # In case no product is present in selected region-channel
    if len(qtr_edv_baseline) != 0 :
        display(Markdown('<div><div class="loader"></div><h2> &nbsp;Measuring Cannibalization</h2></div>'))
        size_exist = Volume_dataset_all_reg[Volume_dataset_all_reg.Product == product_fil]['SIZE_ML'].unique()[0]
        count_exist = Volume_dataset_all_reg[Volume_dataset_all_reg.Product == product_fil]['COUNT'].unique()[0]

        iter_change_list = [1]

        for i in iter_change_list:
            channel_list = upload_data.Channel.unique()

            # Model training and prediction
            for Region_key in upload_data.Region.unique():                        
                # In case the product is not present at that region
                if len(upload_data[(upload_data.Region == Region_key)]) == 0:
                    continue

                category_check_list.extend(list(Volume_dataset_all_reg[(Volume_dataset_all_reg["Region"] == Region_key) & (Volume_dataset_all_reg["Product"] == product_fil)].Category.unique()))

                Volume_dataset = Volume_dataset_all_reg.loc[(Volume_dataset_all_reg["Region"] == Region_key)
                                                           &(Volume_dataset_all_reg["Channel"].isin(channel_list))]

                Volume_dataset_upload = upload_data.loc[(upload_data["Region"] == Region_key)
                                           &(upload_data["Channel"].isin(channel_list))]

                Volume_dataset['Pantry2'] = Volume_dataset['Pantry2'].fillna(0)
                Volume_dataset_upload['Pantry2'] = Volume_dataset_upload['Pantry2'].fillna(0)

                required_columns_upload = ['Week','Week.Name','Banner','Product','Channel','Pack.Subtype','PACK_CONTENT','Adcal_Price','EDV.Price','Adcal_DD','Front.Page','Middle.Page','Back.Page','seasonality_index','SIZE_ML','COUNT','Category','Pantry1','Pantry2','Holiday.Week','Pre.Holiday.Week','Post.Holiday.Week'] 
                required_columns = ['Week','Week.Name','Banner','Product','Channel','Pack.Subtype','PACK_CONTENT','Adcal_Price','EDV.Price','Adcal_DD','Eq.Unit.Sales','Front.Page','Middle.Page','Back.Page','seasonality_index','SIZE_ML','COUNT','Category','Pantry1','Pantry2','Holiday.Week','Pre.Holiday.Week','Post.Holiday.Week'] 
                combined_dataset = Volume_dataset[required_columns] 
                combined_dataset_upload = Volume_dataset_upload[required_columns_upload] 

                combined_dataset = combined_dataset.loc[((combined_dataset["Week"] >= "2017-01") & 
                                                         (combined_dataset["Week"] <= "2019-52"))]

                combined_dataset["Christmas_flag"] = [1 if x=="Christmas" else 0 for x in combined_dataset["Week.Name"]]
                combined_dataset["Easter_flag"] = [1 if x=="Easter" else 0 for x in combined_dataset["Week.Name"]]

                combined_dataset_upload["Christmas_flag"] = [1 if x=="Christmas" else 0 for x in combined_dataset_upload["Week.Name"]]
                combined_dataset_upload["Easter_flag"] = [1 if x=="Easter" else 0 for x in combined_dataset_upload["Week.Name"]]                

                banner_dummies = pd.get_dummies(combined_dataset.Banner) 
                banner_dummies = banner_dummies.reindex(sorted(banner_dummies.columns), axis=1) #sorting banner columns
                combined_dataset = pd.concat([combined_dataset, banner_dummies], axis=1)

                # Uploaded data banner dummies
                banner_dummies_upload = pd.get_dummies(combined_dataset_upload.Banner)                                 
                # If all banners are not present for the uploaded data
                c = [i for i in list(banner_dummies.columns) if i not in list(banner_dummies_upload.columns)]
                if len(c) != 0:
                    banner_dummies_upload[c] = 0
                # Reordering columns                
                banner_dummies_upload = banner_dummies_upload.reindex(sorted(banner_dummies_upload.columns), axis=1)                
                combined_dataset_upload = pd.concat([combined_dataset_upload, banner_dummies_upload], axis=1)

                ## Not adding Product dummies since for New Product Simulator other Product dummies are not significant

                combined_dataset = combined_dataset.loc[combined_dataset["Eq.Unit.Sales"].notnull()]

                combined_dataset.sort_values('Week',inplace = True)
                combined_dataset = pd.merge(combined_dataset,new_prod_stage,on = ['Product','Week'],how = 'left')
                combined_dataset['Intial_weeks'] = combined_dataset['Intial_weeks'].fillna('Stabilization')
                combined_dataset.drop(columns='Unnamed: 0', inplace=True)

                combined_dataset_upload.sort_values('Week',inplace = True)
                combined_dataset_upload = pd.merge(combined_dataset_upload,prod_stage_check_upload(),on = ['Product','Week'],how = 'left')
                combined_dataset_upload['Intial_weeks'] = combined_dataset_upload['Intial_weeks'].fillna('Stabilization')                

                #Adding dummies for category
                category_dummies = pd.get_dummies(combined_dataset['Category'])
                combined_dataset = pd.concat([combined_dataset, category_dummies], axis=1) 
                combined_dataset.drop(["Category"], inplace=True, axis=1)

                cat_col = category_dummies.columns #need all category dummy in the test data

                #Adding dummies for category UPLOADED DDATA
                combined_dataset_upload[cat_col] = 0

                #Adding dummies for product attribute - pack subtype
                pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
                combined_dataset = pd.concat([combined_dataset, pack_subtypes_dummies], axis=1)
                combined_dataset.drop(["Pack.Subtype"], inplace=True, axis=1)

                pack_subtypes_col = pack_subtypes_dummies.columns

                #Adding dummies for packsubtype UPLOADED DDATA
                combined_dataset_upload[pack_subtypes_col] = 0

                # Adding pack content dummies
                pack_content_dummies = pd.get_dummies(combined_dataset['PACK_CONTENT'])
                combined_dataset = pd.concat([combined_dataset, pack_content_dummies], axis=1)
                combined_dataset.drop(["PACK_CONTENT"], inplace=True, axis=1)

                pack_content_col = pack_content_dummies.columns

                #Adding dummies for packsubtype UPLOADED DDATA
                combined_dataset_upload[pack_content_col] = 0

                combined_dataset_upload.drop(["Pack.Subtype"], inplace=True, axis=1)
                combined_dataset_upload.drop(["PACK_CONTENT"], inplace=True, axis=1)
                combined_dataset_upload.drop(["Category"], inplace=True, axis=1)

                # Product stage dummies
                stage_dummies = pd.get_dummies(combined_dataset['Intial_weeks'])
                combined_dataset = pd.concat([combined_dataset, stage_dummies], axis=1)
                combined_dataset.drop(["Intial_weeks"], inplace=True, axis=1)

                # Product stage dummies
                stage_dummies_upload = pd.get_dummies(combined_dataset_upload['Intial_weeks'])

                # If all product stages are not present for the uploaded data
                c = [i for i in list(stage_dummies.columns) if i not in list(stage_dummies_upload.columns)]
                if len(c) != 0 :
                    stage_dummies_upload[c] = 0
                # Reordering columns                
                stage_dummies_upload = stage_dummies_upload.reindex(sorted(stage_dummies_upload.columns), axis=1)
                combined_dataset_upload = pd.concat([combined_dataset_upload, stage_dummies_upload], axis=1)
                combined_dataset_upload.drop(["Intial_weeks"], inplace=True, axis=1)

                #Adding discount depth columns
                combined_dataset["DD_1"] = [1 if (0.25> x >0.1)
                                      else 0 for x in combined_dataset["Adcal_DD"]]
                combined_dataset["DD_2"] = [1 if (x>=0.25)
                                                 else 0 for x in combined_dataset["Adcal_DD"]]

                combined_dataset_upload["DD_1"] = [1 if (0.25> x >0.1)
                                      else 0 for x in combined_dataset_upload["Adcal_DD"]]
                combined_dataset_upload["DD_2"] = [1 if (x>=0.25)
                                                 else 0 for x in combined_dataset_upload["Adcal_DD"]]            

                # Adcal Price calculation
                combined_dataset_upload.loc[combined_dataset_upload['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset_upload.loc[combined_dataset_upload['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset_upload.loc[combined_dataset_upload['Adcal_DD']>=0.1,'EDV.Price']
                # Adcal_DD < 10 then Adcal Price = EDV Price            
                combined_dataset_upload.loc[combined_dataset_upload['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset_upload.loc[combined_dataset_upload['Adcal_DD'] < 0.1, 'EDV.Price']

                # Adcal Price calculation
                combined_dataset.loc[combined_dataset['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'EDV.Price']
                # Adcal_DD < 10 then Adcal Price = EDV Price            
                combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'EDV.Price']

                # Dropping adcal DD as DD1 & DD2 are added
                combined_dataset.drop(["Adcal_DD"], inplace=True, axis=1)
                combined_dataset_upload.drop(["Adcal_DD"], inplace=True, axis=1)

                # Dropping null volumes
                complete_dataset = combined_dataset.copy()
                complete_dataset_upload = combined_dataset_upload.copy()

                complete_dataset = complete_dataset.loc[complete_dataset["Eq.Unit.Sales"].notnull()]

                # Get new product prediction for selected region
                if Region_key == 'ONTARIO':                    
                    complete_dataset_prod = test_data_set_cr_ontario.copy() 
                    complete_dataset_prod['Adcal_Price'] = np.exp(complete_dataset_prod['Adcal_Price'])                    

                if Region_key == 'WEST':                    
                    complete_dataset_prod = test_data_set_cr_west.copy() 
                    complete_dataset_prod['Adcal_Price'] = np.exp(complete_dataset_prod['Adcal_Price'])                   

                if Region_key == 'QUEBEC':                    
                    complete_dataset_prod = test_data_set_cr_quebec.copy() 
                    complete_dataset_prod['Adcal_Price'] = np.exp(complete_dataset_prod['Adcal_Price'])                    

                if Region_key == 'EAST':                    
                    complete_dataset_prod = test_data_set_cr_east.copy() 
                    complete_dataset_prod['Adcal_Price'] = np.exp(complete_dataset_prod['Adcal_Price'])                   
                    
                # Dropping columns as they are not required in complete dataset
                complete_dataset_prod = complete_dataset_prod.drop(['Region'],axis = 1)
                    
                # Adding predicted data of new product in training            
                Test_results_np_cr_v2 = Test_results_np_cr[(Test_results_np_cr["Region"]==Region_key) & (Test_results_np_cr["iter"]==i)]            
                # Merge data to replace eq.unit,sales with predictions for new product
                complete_dataset2= pd.merge(Test_results_np_cr_v2[['Week','Product','Channel','Banner','Predictions_rf']], complete_dataset_prod[complete_dataset_prod.Week <= max(Test_weeks)], 
                                   left_on=['Week','Product','Channel','Banner'], 
                                   right_on=['Week','Product','Channel','Banner'],how='outer') 

                complete_dataset2['Product'] = 'New'

                # Replacing eq unit sales with new product prediction # Week filter given for 2017-2018 products
                complete_dataset_prod['Eq.Unit.Sales'] = complete_dataset2.loc[(complete_dataset2.Product == 'New') & (complete_dataset2.Week >= launch_date), 'Predictions_rf'].values
                complete_dataset_prod['Product'] = 'New'

                complete_dataset = complete_dataset.append(complete_dataset_prod, ignore_index = True)

                complete_dataset['Eq.Unit.Sales'] = complete_dataset['Eq.Unit.Sales'].round(4)

                complete_dataset['Adcal_Price'] = np.log(complete_dataset['Adcal_Price'])
                complete_dataset['Eq.Unit.Sales'] = np.log(complete_dataset['Eq.Unit.Sales'])
                
                # TEST PERIOD LOOP
                for Test_period in Test_periods:

                    #Filtering test weeks
                    Test_weeks = Test_week_list[Test_period]

                    if launch_date < Max_POS:
                        Test_weeks = Test_week_list[Test_period]
                    else:
                        Test_weeks = Test_week_list_2019[Test_period] # 2019 weeks as test data
                    
                    # Dividing train & test data
                    complete_train_data_set = complete_dataset.loc[(complete_dataset.Week <= max(Test_weeks)) & ~(complete_dataset["Week"].isin(Test_weeks))].reset_index(drop=True)
                    # Including new product in training
                    complete_train_data_set = complete_train_data_set.append(complete_dataset[complete_dataset.Product == 'New'], ignore_index = True)

                    # Test data
                    complete_test_data_set = complete_dataset.loc[complete_dataset["Week"].isin(Test_weeks)].reset_index(drop=True)
                    complete_test_data_set = complete_test_data_set.append(complete_dataset[complete_dataset.Product == 'New'], ignore_index = True)

                    # Dropping unneccessary columns from train dataset
                    train_data_set = complete_train_data_set.copy()

                    train_data_brand = train_data_set.copy()
                    train_data_brand.drop(["Week","Week.Name","Banner","Product",'Channel',"EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)

                    test_data_set = complete_test_data_set.copy()
                    test_data_brand = test_data_set.copy()

                    # Dropping unneccessary columns from test dataset
                    test_data_brand.drop(["Week","Week.Name","Banner","Product",'Channel',"EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)            
                    
                    #Creating test and train data - independent & dependent columns
                    X_train = train_data_brand
                    X_train['Adcal_Price'].fillna(X_train['Adcal_Price'].max(), inplace = True)
                    y_train = train_data_set["Eq.Unit.Sales"]
                    X_test = test_data_brand  
                    X_test['Adcal_Price'].fillna(X_test['Adcal_Price'].max(), inplace = True)
                    y_test = test_data_set["Eq.Unit.Sales"]

                    # Appending test data
                    test_data_set["Region"] = Region_key
                    test_data_set["Test_period"] = Test_period
                    test_data_set['iter'] = i
                    # In case the product is not present in certain region-quarter
                    if len(test_data_set) == 0:
                        continue

                    Test_Data = Test_Data.append(test_data_set, ignore_index = True)                
                    #Model training
                    rf = RandomForestRegressor(n_jobs=4,random_state=0)
                    rf.fit(X_train, y_train)

                    #Model predictions
                    predictions_rf = rf.predict(X_test)                       

                    #Storing test results
                    result = X_test.copy()            

                    result["Predictions_rf"] = predictions_rf            
                    result["Actuals"] = y_test
                    
                    result['Predictions_rf'] = np.exp(result['Predictions_rf'])                    
                    result['Actuals'] = np.exp(result['Actuals'])                                        

                    #Calculating APE for the models
                    result["ape_rf"] = (result["Predictions_rf"]-result["Actuals"]).abs()

                    result["Product"] = test_data_set["Product"]
                    result["Region"] = Region_key
                    result["Test_period"] = Test_period
                    result["Week"] = test_data_set["Week"]
                    result["DD_1"] = test_data_set['DD_1']
                    result["DD_2"] = test_data_set['DD_2']
                    result["Banner"] = test_data_set['Banner']
                    result["Channel"] = test_data_set['Channel']
                    ##Columns for Revenue                    
                    result["Adcal_Price"] = test_data_set["Adcal_Price"] 
                    result["Adcal_Price"] = np.exp(result["Adcal_Price"])
                    
                    result['iter'] = i

                    result = result[["Region","Product",'Channel','iter',"Banner","Test_period","Week","Actuals","ape_rf","Predictions_rf","DD_1","DD_2","Adcal_Price"]]

                    # Appending results to the final results dataframe            
                    Test_results_exist_with_cr = Test_results_exist_with_cr.append(result, ignore_index=True)

# In case the user doesn't uplaod data
elif (toggle_upload.value == 'No') :
    
    #################################################################################################################### 
                    # Cannibalisation Code: Step 3 - # Exist product with NP (WITHOUT UPLOADED DATA)
    #################################################################################################################### 

    #Defining empty dataframes for model training
    combined_dataset = pd.DataFrame({'Week': pd.Series([], dtype='object')})
    Test_results_exist_with_cr = pd.DataFrame({'Region': pd.Series([], dtype='object')})
    Test_Data = pd.DataFrame()

    #Model training and prediction
    category_check_list=[]

    # In case no product is present in selected region-channel
    if len(qtr_edv_baseline) != 0 :
        display(Markdown('<div><div class="loader"></div><h2> &nbsp;Measuring Cannibalization</h2></div>'))
    #     display(Markdown('<div><div class="loader"></div><h2> &nbsp;Simulating New Product Results</h2></div>'))
        size_exist = Volume_dataset_all_reg[Volume_dataset_all_reg.Product == product_fil]['SIZE_ML'].unique()[0]
        count_exist = Volume_dataset_all_reg[Volume_dataset_all_reg.Product == product_fil]['COUNT'].unique()[0]

        if toggle_np2.value == 'Yes':
            iter_change_list = [1,2]
        else:
            iter_change_list = [1]

        for i in iter_change_list:


            # Model training and prediction
            for Region_key in Region_List:                 
                # In case the product is not present at that region
                if len(Volume_dataset_all_reg[(Volume_dataset_all_reg.Product == product_fil) & (Volume_dataset_all_reg.Region == Region_key)]) == 0:
                    continue

                category_check_list.extend(list(Volume_dataset_all_reg[(Volume_dataset_all_reg["Region"] == Region_key) & (Volume_dataset_all_reg["Product"] == product_fil)].Category.unique()))

                Volume_dataset = Volume_dataset_all_reg.loc[(Volume_dataset_all_reg["Region"] == Region_key)
                                                           &(Volume_dataset_all_reg["Channel"].isin(channel_list))]
                Volume_dataset['Pantry2'] = Volume_dataset['Pantry2'].fillna(0)
                required_columns = ['Week','Week.Name','Banner','Product','Channel','Pack.Subtype','PACK_CONTENT','Adcal_Price','EDV.Price','Adcal_DD','Eq.Unit.Sales','Front.Page','Middle.Page','Back.Page','seasonality_index','SIZE_ML','COUNT','No_of_brands','No_of_flavors','No_of_sweetners','No_of_types','Category','Pantry1','Pantry2','Holiday.Week','Pre.Holiday.Week','Post.Holiday.Week']
                combined_dataset = Volume_dataset[required_columns]

                combined_dataset = combined_dataset.loc[((combined_dataset["Week"] >= "2017-01") & 
                                                         (combined_dataset["Week"] <= "2019-52"))]

                combined_dataset["Christmas_flag"] = [1 if x=="Christmas" else 0 for x in combined_dataset["Week.Name"]]
                combined_dataset["Easter_flag"] = [1 if x=="Easter" else 0 for x in combined_dataset["Week.Name"]]

                ## CHECKING if banner dummies are not present in required region, channel then introduce banner dummies
                banner_dummies = pd.get_dummies(combined_dataset.Banner)              

                # Uploaded data banner dummies
                banner_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key].Banner)                                 
                # If all banners are not present for the training data
                c = [i for i in list(banner_dummies_check.columns) if i not in list(banner_dummies.columns)]
                if len(c) != 0:
                    banner_dummies[c] = 0

                # Reordering columns                
                banner_dummies = banner_dummies.reindex(sorted(banner_dummies.columns), axis=1)                
                combined_dataset = pd.concat([combined_dataset, banner_dummies], axis=1)

                ## Not adding Product dummies since for New Product Simulator other Product dummies are not significant
                combined_dataset = combined_dataset.loc[combined_dataset["Eq.Unit.Sales"].notnull()]

                combined_dataset.sort_values('Week',inplace = True)
                combined_dataset = pd.merge(combined_dataset,new_prod_stage,on = ['Product','Week'],how = 'left')
                combined_dataset['Intial_weeks'] = combined_dataset['Intial_weeks'].fillna('Stabilization')
                combined_dataset.drop(columns='Unnamed: 0', inplace=True)

                #Adding dummies for category
                category_dummies = pd.get_dummies(combined_dataset['Category'])

                # Adding dummies for product attribute - Category
                pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
                # Uploaded data category dummies
                category_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['Category'])                                 
                # If all pack subtypes are not present for the training data
                c = [i for i in list(category_dummies_check.columns) if i not in list(category_dummies.columns)]
                if len(c) != 0:
                    category_dummies[c] = 0

                # Reordering columns                
                category_dummies = category_dummies.reindex(sorted(category_dummies.columns), axis=1)                
                combined_dataset = pd.concat([combined_dataset, category_dummies], axis=1)
                combined_dataset.drop(["Category"], inplace=True, axis=1)

                # Adding dummies for product attribute - pack subtype
                pack_subtypes_dummies = pd.get_dummies(combined_dataset['Pack.Subtype'])
                # Uploaded data pack subtype dummies
                pack_subtypes_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['Pack.Subtype'])                                 
                # If all pack subtypes are not present for the training data
                c = [i for i in list(pack_subtypes_dummies_check.columns) if i not in list(pack_subtypes_dummies.columns)]
                if len(c) != 0:
                    pack_subtypes_dummies[c] = 0

                # Reordering columns                
                pack_subtypes_dummies = pack_subtypes_dummies.reindex(sorted(pack_subtypes_dummies.columns), axis=1)                
                combined_dataset = pd.concat([combined_dataset, pack_subtypes_dummies], axis=1)
                combined_dataset.drop(["Pack.Subtype"], inplace=True, axis=1)

                # Adding pack content dummies
                pack_content_dummies = pd.get_dummies(combined_dataset['PACK_CONTENT'])
                # Uploaded data pack content dummies
                pack_content_dummies_check = pd.get_dummies(Volume_dataset_all_reg[Volume_dataset_all_reg.Region == Region_key]['PACK_CONTENT'])                                 
                # If all pack contents are not present for the training data
                c = [i for i in list(pack_content_dummies_check.columns) if i not in list(pack_content_dummies.columns)]
                if len(c) != 0:
                    pack_content_dummies[c] = 0

                # Reordering columns                
                pack_content_dummies = pack_content_dummies.reindex(sorted(pack_content_dummies.columns), axis=1)                
                combined_dataset = pd.concat([combined_dataset, pack_content_dummies], axis=1)
                combined_dataset.drop(["PACK_CONTENT"], inplace=True, axis=1)
                
                # Product stage dummies
                stage_dummies = pd.get_dummies(combined_dataset['Intial_weeks'])
                combined_dataset = pd.concat([combined_dataset, stage_dummies], axis=1)
                combined_dataset.drop(["Intial_weeks"], inplace=True, axis=1)

                #Adding discount depth columns
                combined_dataset["DD_1"] = [1 if (0.25> x >0.1)
                                      else 0 for x in combined_dataset["Adcal_DD"]]
                combined_dataset["DD_2"] = [1 if (x>=0.25)
                                                 else 0 for x in combined_dataset["Adcal_DD"]]
                # Adcal Price calculation
                combined_dataset.loc[combined_dataset['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset.loc[combined_dataset['Adcal_DD']>=0.1,'EDV.Price']
                # Adcal_DD < 10 then Adcal Price = EDV Price            
                combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset.loc[combined_dataset['Adcal_DD'] < 0.1, 'EDV.Price']

                # Dropping adcal DD as DD1 & DD2 are added
                combined_dataset.drop(["Adcal_DD"], inplace=True, axis=1)

                # Dropping null volumes
                complete_dataset = combined_dataset.copy()        

                complete_dataset = complete_dataset.loc[complete_dataset["Eq.Unit.Sales"].notnull()]        
                
                # Get new product prediction for selected region
                if Region_key == 'ONTARIO':                    
                    complete_dataset_prod = test_data_set_cr_ontario[test_data_set_cr_ontario.iter == i].copy() 
                    complete_dataset_prod['Adcal_Price'] = np.exp(complete_dataset_prod['Adcal_Price'])
                    complete_dataset_prod['Eq.Unit.Sales'] = np.exp(complete_dataset_prod['Eq.Unit.Sales'])

                if Region_key == 'WEST':                    
                    complete_dataset_prod = test_data_set_cr_west[test_data_set_cr_west.iter == i].copy() 
                    complete_dataset_prod['Adcal_Price'] = np.exp(complete_dataset_prod['Adcal_Price'])
                    complete_dataset_prod['Eq.Unit.Sales'] = np.exp(complete_dataset_prod['Eq.Unit.Sales'])                                

                if Region_key == 'QUEBEC':                    
                    complete_dataset_prod = test_data_set_cr_quebec[test_data_set_cr_quebec.iter == i].copy() 
                    complete_dataset_prod['Adcal_Price'] = np.exp(complete_dataset_prod['Adcal_Price'])
                    complete_dataset_prod['Eq.Unit.Sales'] = np.exp(complete_dataset_prod['Eq.Unit.Sales'])

                if Region_key == 'EAST':                    
                    complete_dataset_prod = test_data_set_cr_east[test_data_set_cr_east.iter == i].copy() 
                    complete_dataset_prod['Adcal_Price'] = np.exp(complete_dataset_prod['Adcal_Price'])
                    complete_dataset_prod['Eq.Unit.Sales'] = np.exp(complete_dataset_prod['Eq.Unit.Sales'])

                complete_dataset_prod['SIZE_ML'] = size.value
                complete_dataset_prod['COUNT'] = count.value
                # Dropping columns as they are not required in complete dataset
                complete_dataset_prod = complete_dataset_prod.drop(['Region','iter'],axis = 1)

                # Adding predicted data of new product in training            
                Test_results_np_cr_v2 = Test_results_np_cr[(Test_results_np_cr["Region"]==Region_key) & (Test_results_np_cr["iter"]==i)]            
                # Merge data to replace eq.unit,sales with predictions for new product
                complete_dataset2= pd.merge(Test_results_np_cr_v2[['Week','Product','Channel','Banner','Predictions_rf']], complete_dataset_prod[complete_dataset_prod.Week <= max(Test_weeks)], 
                                   left_on=['Week','Product','Channel','Banner'], 
                                   right_on=['Week','Product','Channel','Banner'],how='outer') 

                complete_dataset2['Product'] = 'New'

                # Replacing eq unit sales with new product prediction # Week filter given for 2017-2018 products
                complete_dataset_prod['Eq.Unit.Sales'] = complete_dataset2.loc[(complete_dataset2.Product == 'New') & (complete_dataset2.Week >= launch_date), 'Predictions_rf'].values
                complete_dataset_prod['Product'] = 'New'

                complete_dataset = complete_dataset.append(complete_dataset_prod, ignore_index = True)

                complete_dataset['Eq.Unit.Sales'] = complete_dataset['Eq.Unit.Sales'].round(4)

                complete_dataset['Adcal_Price'] = np.log(complete_dataset['Adcal_Price'])
                complete_dataset['Eq.Unit.Sales'] = np.log(complete_dataset['Eq.Unit.Sales'])
                
                # TEST PERIOD LOOP
                for Test_period in Test_periods:

                    #Filtering test weeks
                    Test_weeks = Test_week_list[Test_period]

                    # Dividing train & test data
                    complete_train_data_set = complete_dataset.loc[(complete_dataset.Week <= max(Test_weeks)) & ~(complete_dataset["Week"].isin(Test_weeks))].reset_index(drop=True)
                    # Including new product in training
                    complete_train_data_set = complete_train_data_set.append(complete_dataset[complete_dataset.Product == 'New'], ignore_index = True)

                    # Test data
                    complete_test_data_set = complete_dataset.loc[complete_dataset["Week"].isin(Test_weeks)].reset_index(drop=True)

                    # Dropping unneccessary columns from train dataset
                    train_data_set = complete_train_data_set.copy()

                    train_data_brand = train_data_set.copy()
                    train_data_brand.drop(["Week","Week.Name","Banner","Product",'Channel',"EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)

                    test_data_set = complete_test_data_set.copy()
                    test_data_brand = test_data_set.copy()            

                    # Dropping unneccessary columns from test dataset
                    test_data_brand.drop(["Week","Week.Name","Banner","Product",'Channel',"EDV.Price","Eq.Unit.Sales"], inplace=True, axis=1)            

                    #Creating test and train data - independent & dependent columns
                    X_train = train_data_brand
                    y_train = train_data_set["Eq.Unit.Sales"]
                    X_test = test_data_brand     
                    y_test = test_data_set["Eq.Unit.Sales"]

                    # Appending test data
                    test_data_set["Region"] = Region_key
                    test_data_set["Test_period"] = Test_period
                    test_data_set['iter'] = i
                    # In case the product is not present in certain region-quarter
                    if len(test_data_set) == 0:
                        continue

                    Test_Data = Test_Data.append(test_data_set, ignore_index = True)                
                    #Model training
                    rf = RandomForestRegressor(n_jobs=4,random_state=0)
                    rf.fit(X_train, y_train)

                    #Model predictions
                    predictions_rf = rf.predict(X_test)                       

                    #Storing test results
                    result = X_test.copy()            

                    result["Predictions_rf"] = predictions_rf            
                    result["Actuals"] = y_test

                    result['Predictions_rf'] = np.exp(result['Predictions_rf'])                    
                    result['Actuals'] = np.exp(result['Actuals'])                                        

                    #Calculating APE for the models
                    result["ape_rf"] = (result["Predictions_rf"]-result["Actuals"]).abs()

                    result["Product"] = test_data_set["Product"]
                    result["Region"] = Region_key
                    result["Test_period"] = Test_period
                    result["Week"] = test_data_set["Week"]
                    result["DD_1"] = test_data_set['DD_1']
                    result["DD_2"] = test_data_set['DD_2']
                    result["Banner"] = test_data_set['Banner']
                    result["Channel"] = test_data_set['Channel']
                    ##Columns for Revenue                    
                    result["Adcal_Price"] = test_data_set["Adcal_Price"] 
                    result["Adcal_Price"] = np.exp(result["Adcal_Price"])
                    result['iter'] = i

                    result = result[["Region","Product",'Channel','iter',"Banner","Test_period","Week","Actuals","ape_rf","Predictions_rf","DD_1","DD_2","Adcal_Price"]]

                    # Appending results to the final results dataframe            
                    Test_results_exist_with_cr = Test_results_exist_with_cr.append(result, ignore_index=True)


################################################################################################################## 
                    # Collate all 3 step results
################################################################################################################## 

if (toggle_np2.value == 'Yes') & (toggle_upload.value != 'Yes'):
    iter_change_list = [1,2]
else:
    iter_change_list = [1]

iteration_dataset_all_v2 = pd.DataFrame()    
iteration_dataset_all_v3 = pd.DataFrame()

if launch_date > Max_POS: 
    launch_date = '2019-01' # To include new product in step 2 otherwise it gets filtered out
    
for i in iter_change_list:    

    # Input data for CR calculation
    Model_1_data = Test_results_np_cr[Test_results_np_cr.iter == i]
    Model_2_data = Test_results_exist_wo_cr
    Model_3_data = Test_results_exist_with_cr[Test_results_exist_with_cr.iter == i]    

    # Getting new product volume at region channel level
    Model_1_data = Model_1_data[(Model_1_data["Week"]>= launch_date)]
    Model_1_summary = Model_1_data.groupby(['Region','Channel','Product'],as_index=False)['Predictions_rf'].sum()
    Model_1_summary.columns = ['Region', 'Channel', 'Product', 'New Product Volume']
    Model_1_summary['Product'] = 'New'
    
    # Same category products as affected products
    Affected_products_list = Volume_dataset_all_reg[Volume_dataset_all_reg.Category == category_drop.value].Product.unique()
    if (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:
        Affected_products_list = np.append(Affected_products_list,'New')
        
    # Getting existing product volumes from model 2 at region channel level
    Model_2_data=Model_2_data[(Model_2_data["Week"]>= launch_date)]
    Model_2_data=Model_2_data[Model_2_data["Product"].isin(Affected_products_list)]
    Model_2_summary= Model_2_data.groupby(['Region','Channel','Product'],as_index=False)['Predictions_rf'].sum()
    Model_2_summary.columns= ['Region', 'Channel', 'Product', 'Existing Product Vol Without']

    # Getting existing product volumes from model 3 at region channel level
    Model_3_data=Model_3_data[(Model_3_data["Week"]>= launch_date)]
    
    Model_3_data=Model_3_data[Model_3_data["Product"].isin(Affected_products_list)]
    Model_3_summary= Model_3_data.groupby(['Region','Channel','Product'],as_index=False)['Predictions_rf'].sum()
    Model_3_summary.columns= ['Region', 'Channel', 'Product', 'Existing Product Vol With']

    # Merging data
    all_data=  pd.merge(Model_1_summary, Model_2_summary, on=["Region","Channel","Product"],how="outer")
    all_data=pd.merge(all_data, Model_3_summary, on=["Region","Channel","Product"],how="outer")

    ################################################################################################################## 
                        # Capping, scaling for new product cannibalization
    ################################################################################################################## 

    iteration_dataset_all = pd.DataFrame()

    All_result = pd.DataFrame({'Region': pd.Series([], dtype='object'),
                             'Channel': pd.Series([], dtype='object'),
                              'New_Product': pd.Series([], dtype='object'),
                              'New Product Volume': pd.Series([], dtype='object'),
                              'Can_sum_neg': pd.Series([], dtype='object'),
                               'CR_negatives': pd.Series([], dtype='object')
                              })

    lower_thresh = -0.20
    upper_thresh = 0.26

    new_product_list = [product_fil]
    for p in new_product_list:
        input_data = all_data
        new_product_data=input_data[input_data["Product"]=='New']
        existing_product_data=input_data    

        existing_product_data=existing_product_data[["Region","Channel","Product","Existing Product Vol Without","Existing Product Vol With"]]
        existing_product_data.columns=['Region', 'Channel', 'Existing_Product', 'Existing Product Vol Without','Existing Product Vol With']

        new_product_data=new_product_data[["Region","Channel","Product","New Product Volume"]]
        new_product_data.columns=["Region","Channel","New_Product","New Product Volume"]

        working_dataset= pd.merge(existing_product_data,new_product_data,left_on=['Region','Channel'],right_on=['Region','Channel'])
        working_dataset["Cann Vol"]= working_dataset["Existing Product Vol With"]-working_dataset["Existing Product Vol Without"]
        working_dataset["Cann Vol%"]=working_dataset["Cann Vol"]/working_dataset["New Product Volume"]
        working_dataset["lower_thresh"]=lower_thresh
        working_dataset["upper_thresh"]=upper_thresh
        working_dataset = working_dataset[working_dataset['Existing_Product'] != 'New']
        region_list= working_dataset[["Region"]].drop_duplicates()["Region"].tolist()
        channel_list_cr= working_dataset[["Channel"]].drop_duplicates()["Channel"].tolist()

        for k in region_list:           
            for j in channel_list_cr:                
                iteration_dataset= working_dataset[(working_dataset["Region"]==k) & (working_dataset["Channel"]==j)]           
                iteration_dataset["Neg_max"]=iteration_dataset["Cann Vol%"].min() 
                temp_neg=iteration_dataset[iteration_dataset['Cann Vol%']<0]
                iteration_dataset["Neg_min"]=temp_neg['Cann Vol%'].max()
                iteration_dataset["Pos_max"]=iteration_dataset["Cann Vol%"].max() 
                temp_pos=iteration_dataset[iteration_dataset['Cann Vol%']>0]
                iteration_dataset["Pos_min"]=temp_pos['Cann Vol%'].min()

                # Calculating positive in range
                temp_pin=iteration_dataset[(iteration_dataset['Cann Vol%']>0)&(iteration_dataset['Cann Vol%']<upper_thresh)]['Cann Vol%'].max()
                temp_nin=iteration_dataset[(iteration_dataset['Cann Vol%']<0)&(iteration_dataset['Cann Vol%']>lower_thresh)]['Cann Vol%'].min()
                iteration_dataset["Positive_in_range"]=temp_pin
                iteration_dataset["Negative_in_range"]=temp_nin
                iteration_dataset1=iteration_dataset
                iteration_dataset1['Negative_in_range'] = iteration_dataset1['Negative_in_range'].replace(np.nan, -0.0001)
                iteration_dataset1["Recalculated_Can%"]= np.where(iteration_dataset1["Cann Vol%"]<0,
                                                                  np.where(iteration_dataset1["Cann Vol%"]<lower_thresh,
                                                                        (iteration_dataset1["lower_thresh"]-iteration_dataset1["Negative_in_range"])
                                                                           *(iteration_dataset1["Cann Vol%"]-iteration_dataset1["Neg_min"])/
                                                                           (iteration_dataset1["Neg_max"]-iteration_dataset1["Neg_min"])+iteration_dataset1["Negative_in_range"]
                                                                           ,iteration_dataset1["Cann Vol%"]),
                                                                  np.where(iteration_dataset1["Cann Vol%"]>upper_thresh,
                                                                          (iteration_dataset1["upper_thresh"]-iteration_dataset1["Positive_in_range"])*
                                                                          (iteration_dataset1["Cann Vol%"]-iteration_dataset1["Pos_min"])/
                                                                           (iteration_dataset1["Pos_max"]-iteration_dataset1["Pos_min"])+iteration_dataset1["Positive_in_range"],
                                                                  iteration_dataset1["Cann Vol%"])
                                                                 )
                iteration_dataset1['Recalculated_Can%'] = iteration_dataset1['Recalculated_Can%'].replace(np.nan, lower_thresh)

                iteration_dataset1["Updated_Can_Vol"]=iteration_dataset1["New Product Volume"]*iteration_dataset1["Recalculated_Can%"]
                iteration_dataset1["Can_sum_neg"]=iteration_dataset1[iteration_dataset1["Updated_Can_Vol"]<0]["Updated_Can_Vol"].sum()    
                iteration_dataset1["CR_negatives"]=iteration_dataset1["Can_sum_neg"]/iteration_dataset1["New Product Volume"]    

                results= iteration_dataset1[["Region","Channel","New_Product","New Product Volume","Can_sum_neg","CR_negatives",]]
                results=results.drop_duplicates().reset_index(drop=True)
                iteration_dataset_all=iteration_dataset_all.append(iteration_dataset1)
                All_result=All_result.append(results)        

    if len(iteration_dataset_all) == 0: # in case the user gave to use 2nd new product & upload adcal data feature, this will remove the 2nd iteration & just keep the uploaded adcal data comparison
        continue
    # Region-channel CR view
    iteration_dataset_all_v2_iter = iteration_dataset_all[iteration_dataset_all.Updated_Can_Vol < 0].copy()
    iteration_dataset_all_v2_iter = iteration_dataset_all_v2_iter.groupby(['Region','Channel','New_Product']).agg({'New Product Volume':'sum','Updated_Can_Vol':'sum','Existing_Product':'nunique'}).reset_index()
    iteration_dataset_all_v2_iter['New_Product_Volume'] = iteration_dataset_all_v2_iter['New Product Volume']/iteration_dataset_all_v2_iter['Existing_Product']
    iteration_dataset_all_v2_iter.drop(columns = {'New Product Volume'}, inplace = True)
    iteration_dataset_all_v2_iter.drop(columns = {'Existing_Product'}, inplace = True)
    iteration_dataset_all_v2_iter.drop(columns = {'New_Product'}, inplace = True)
    iteration_dataset_all_v2_iter.rename(columns = {'Updated_Can_Vol':'Displaced_Volume'}, inplace = True)
    iteration_dataset_all_v2_iter = iteration_dataset_all_v2_iter[['Region','Channel','New_Product_Volume','Displaced_Volume']]
    iteration_dataset_all_v2_iter['iter'] = i    
    iteration_dataset_all_v2 = iteration_dataset_all_v2.append(iteration_dataset_all_v2_iter, ignore_index = True)
        
    # Affected product cannibalization distribution
    iteration_dataset_all_v3_iter = iteration_dataset_all[iteration_dataset_all.Updated_Can_Vol < 0]
    iteration_dataset_all_v3_iter = iteration_dataset_all_v3_iter.groupby(['Region','Existing_Product']).agg({'Existing Product Vol Without':'sum','Updated_Can_Vol':'sum'}).reset_index()
    iteration_dataset_all_v3_iter['% CR'] = iteration_dataset_all_v3_iter['Updated_Can_Vol']/iteration_dataset_all_v3_iter['Existing Product Vol Without']                
    iteration_dataset_all_v3_iter['iter'] = i
    iteration_dataset_all_v3 = iteration_dataset_all_v3.append(iteration_dataset_all_v3_iter, ignore_index = True)

    clear_output()

# Checking overall cannibalised units (limiting it to 95%)
iteration_dataset_all_v2['Displaced_Volume'] = np.where(iteration_dataset_all_v2['Displaced_Volume']/iteration_dataset_all_v2['New_Product_Volume'] < -0.95, -0.95*iteration_dataset_all_v2['New_Product_Volume'], iteration_dataset_all_v2['Displaced_Volume'])    

In [178]:
if len(qtr_edv_baseline) != 0 :
    
    # In case the user selects 2 new product for simulation    
    if (toggle_np2.value == 'Yes') & (toggle_upload.value != 'Yes'):        
        ########### TOTAL VOLUME ###########
        iteration_dataset_all_v2_result = iteration_dataset_all_v2.groupby('iter')['Displaced_Volume','New_Product_Volume'].sum().reset_index()
        iteration_dataset_all_v2_result['CR'] = iteration_dataset_all_v2_result['Displaced_Volume']/iteration_dataset_all_v2_result['New_Product_Volume']        

        # Results    
        old_vol = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)]['Predictions_rf_exist'].sum()/2
        new_vol = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].Predictions_rf.sum()
        new_vol2 = Test_results[Test_results.iter == 2][(Test_results[Test_results.iter == 2].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 2].Test_period <= end_week.children[8].value)].Predictions_rf.sum()
        # initialize list of lists
        vol_data = [['Product', old_vol,new_vol,new_vol2]]

        # Create the pandas DataFrame
        df = pd.DataFrame(vol_data, columns = ['Volume','Existing','New Product 1','New Product 2'])

        kpi1 = {}
        kpi1['Volume RU Change in % (New Product 1)'] = round(100*(new_vol - old_vol)/old_vol,2)
        kpi1['Volume RU Change in % (New Product 2)'] = round(100*(new_vol2 - old_vol)/old_vol,2)
        kpi_df1 = pd.DataFrame(kpi1, index=[''])

         ########### TOTAL REVENUE ###########
        # Results    
        old_revenue = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)]['Sales_derived_exist'].sum()/2
        new_revenue = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].Sales_derived.sum()
        new_revenue2 = Test_results[Test_results.iter == 2][(Test_results[Test_results.iter == 2].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 2].Test_period <= end_week.children[8].value)].Sales_derived.sum()
        # initialize list of lists
        revenue_data = [['Product', old_revenue,new_revenue,new_revenue2]]

        # Create the pandas DataFrame
        df = pd.DataFrame(revenue_data, columns = ['Revenue','Existing','New Product 1','New Product 2'])

        kpi2 = {}
        kpi2['Revenue Change in % (New Product 1)'] = round(100*(new_revenue - old_revenue)/old_revenue,2)
        kpi2['Revenue Change in % (New Product 2)'] = round(100*(new_revenue2 - old_revenue)/old_revenue,2)
        kpi_df2 = pd.DataFrame(kpi2, index=[''])

        kpi_df3 = iteration_dataset_all_v2_result[['iter','CR']]

        if len(kpi_df3) > 0:
            kpi_df3_final_1 = kpi_df3[kpi_df3.iter == 1].reset_index()
            kpi_df3_final_2 = kpi_df3[kpi_df3.iter == 2].reset_index()

            del kpi_df3_final_1['index']
            del kpi_df3_final_2['index']

            kpi_df3_final_1.drop(columns = {'iter'}, inplace = True)
            kpi_df3_final_2.drop(columns = {'iter'}, inplace = True)
            kpi_df3_final_1.rename(columns={'CR':'Cannibalization Rate in %'}, index={0:''}, inplace=True)
            kpi_df3_final_2.rename(columns={'CR':'Cannibalization Rate in %'}, index={0:''}, inplace=True)    

            kpi_df3_final_1['Cannibalization Rate in %'] = 100*kpi_df3_final_1['Cannibalization Rate in %']
            kpi_df3_final_2['Cannibalization Rate in %'] = 100*kpi_df3_final_2['Cannibalization Rate in %']

            kpi_df3_final_1['Cannibalization Rate in %'] = kpi_df3_final_1['Cannibalization Rate in %'].round(2)
            kpi_df3_final_2['Cannibalization Rate in %'] = kpi_df3_final_2['Cannibalization Rate in %'].round(2)
            
            kpi_df3_final_1.rename(columns={'Cannibalization Rate in %':'Cannibalization Rate in % (New Product 1)'}, index={0:''}, inplace=True)
            kpi_df3_final_2.rename(columns={'Cannibalization Rate in %':'Cannibalization Rate in % (New Product 2)'}, index={0:''}, inplace=True)    
            
        else:
            kpi_df3_dict = {}
            kpi_df3_dict['Existing Product Category CR in %'] = 'CR not available for selected Product Category'
            kpi_df3_final = pd.DataFrame(kpi_df3_dict, index=[''])
            kpi_df3_final.rename(columns = {'Existing Product Category CR in %':'Cannibalization Rate in %'}, inplace = True)
            kpi_df3_final['Cannibalization Rate in %'] = 100*iteration_dataset_all_v2['Displaced_Volume'].sum()/iteration_dataset_all_v2['New_Product_Volume'].sum()
            kpi_df3_final['Cannibalization Rate in %'] = kpi_df3_final['Cannibalization Rate in %'].round(2)
    
    # In case the user selects 1 new product for simulation    
    else:

        ########### TOTAL VOLUME ###########

        # Results 
        # In case the user doesn't upload the data
        if toggle_upload.value != 'Yes':
            old_vol = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)]['Predictions_rf_exist'].sum()
            new_vol = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].Predictions_rf.sum()
        elif (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:
            old_vol = Test_results_exist2['Predictions_rf_exist'].sum()
            new_vol = Test_results.Predictions_rf.sum()

        # initialize list of lists
        vol_data = [['Product', old_vol,new_vol]]

        # Create the pandas DataFrame
        df = pd.DataFrame(vol_data, columns = ['Volume','Existing','New'])

        kpi1 = {}
        kpi1['Volume RU Change in %'] = round(100*(new_vol - old_vol)/old_vol,2)
        kpi_df1 = pd.DataFrame(kpi1, index=[''])

         ########### TOTAL REVENUE ###########
        # Results  
        # In case the user doesn't upload the data
        if toggle_upload.value != 'Yes':
            old_revenue = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)]['Sales_derived_exist'].sum()
            new_revenue = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].Sales_derived.sum()
        elif (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:
            old_revenue = Test_results_exist2['Sales_derived_exist'].sum()
            new_revenue = Test_results.Sales_derived.sum()            
            
        # initialize list of lists
        revenue_data = [['Product', old_revenue,new_revenue]]

        # Create the pandas DataFrame
        df = pd.DataFrame(revenue_data, columns = ['Revenue','Existing','New'])

        kpi2 = {}
        kpi2['Revenue Change in %'] = round(100*(new_revenue - old_revenue)/old_revenue,2)
        kpi_df2 = pd.DataFrame(kpi2, index=[''])

        # initialize CR dataframe
        kpi_df3 = {}
        kpi_df3['Cannibalization Rate in %'] = 100*iteration_dataset_all_v2['Displaced_Volume'].sum()/iteration_dataset_all_v2['New_Product_Volume'].sum()
        kpi_df3_final = pd.DataFrame(kpi_df3, index=[''])
        kpi_df3_final        
        kpi_df3_final['Cannibalization Rate in %'] = kpi_df3_final['Cannibalization Rate in %'].round(2)

else:    
    clear_output()
    print('Product is not present in that Region/Channel for selected period')        
    
    

In [179]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 2px  black solid !important;
  color: black !important;
}
</style>

In [212]:
if len(qtr_edv_baseline) != 0 :
    if (toggle_np2.value == 'Yes') & (toggle_upload.value != 'Yes'):        

        if kpi_df3_final_1['Cannibalization Rate in % (New Product 1)'].values < -95:
            kpi_df3_final_1['Cannibalization Rate in % (New Product 1)'] = -95
        if kpi_df3_final_2['Cannibalization Rate in % (New Product 2)'].values < -95:
            kpi_df3_final_2['Cannibalization Rate in % (New Product 2)'] = -95    

        final_df = pd.concat([kpi_df1,kpi_df2, kpi_df3_final_1,kpi_df3_final_2], axis=1)
        print('New Product 1 - ', 'Count: ', int(count.value), ' | Size: '  , int(size.value), ' | Category: ', category_drop.value, ' | Pack Subtype: ', pack_subtype_drop.value, ' | Pack Content: ', pack_content_drop.value)

        final_df1 = final_df[['Volume RU Change in % (New Product 1)','Revenue Change in % (New Product 1)','Cannibalization Rate in % (New Product 1)']]

        display(HTML(final_df1.to_html(index=False)))

        print('New Product 2 - ', 'Count: ', int(count2.value), ' | Size: '  , int(size2.value), ' | Category: ', category_drop2.value, ' | Pack Subtype: ', pack_subtype_drop2.value, ' | Pack Content: ', pack_content_drop2.value)
        final_df2 = final_df[['Volume RU Change in % (New Product 2)','Revenue Change in % (New Product 2)','Cannibalization Rate in % (New Product 2)']]
        display(HTML(final_df2.to_html(index=False)))
    
    else:
        
        final_df = pd.concat([kpi_df1,kpi_df2, kpi_df3_final], axis=1)
        display(HTML(final_df.to_html(index=False)))

else:    
    clear_output()
    print('Product is not present in that Region/Channel for selected period')

In [213]:
if len(qtr_edv_baseline) != 0:
    
    # When CR is present across region-channel
    if len(kpi_df3) > 0:

        # In case the user selects 2 new product for simulation    
        if (toggle_np2.value == 'Yes') & (toggle_upload.value != 'Yes'):

            plt.rcParams.update({'font.size': 15})
            ########### TOTAL VOLUME ###########

            # Results    
                        
            old_vol = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)]['Predictions_rf_exist'].sum()/2
            new_vol = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].Predictions_rf.sum()
            new_vol2 = Test_results[Test_results.iter == 2][(Test_results[Test_results.iter == 2].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 2].Test_period <= end_week.children[8].value)].Predictions_rf.sum()
            # initialize list of lists
            vol_data = [['Product', old_vol,new_vol,new_vol2]]

            # Create the pandas DataFrame
            df = pd.DataFrame(vol_data, columns = ['Volume RU','Existing','New Product 1','New Product 2'])

            df['Existing'] = df['Existing'].astype('int64')
            df['Existing_formatted'] = df['Existing'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(round(x,2)).format('{:,}'))

            df['New Product 1_overall'] = df['New Product 1']
            df['New Product 1'] = df['New Product 1']*(-1*(final_df['Cannibalization Rate in % (New Product 1)'].values[0]/100))        
            df['New_formatted 1'] = df['New Product 1'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(round(x,2)).format('{:,}'))
            df['New_formatted 1_overall'] = df['New Product 1_overall'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(x).format('{:,}'))


            df['New Product 1_inc'] = new_vol*(1+final_df['Cannibalization Rate in % (New Product 1)'].values[0]/100)
            df['New_formatted 1_inc'] = df['New Product 1_inc'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(round(x,2)).format('{:,}'))        

            df['New Product 2_overall'] = df['New Product 2']
            df['New Product 2'] = df['New Product 2']*(-1*(final_df['Cannibalization Rate in % (New Product 2)'].values[0]/100))
            df['New_formatted 2'] = df['New Product 2'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(round(x,2)).format('{:,}'))
            df['New_formatted 2_overall'] = df['New Product 2_overall'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(round(x,2)).format('{:,}'))

            df['New Product 2_inc'] = new_vol2*(1+final_df['Cannibalization Rate in % (New Product 2)'].values[0]/100)
            df['New_formatted 2_inc'] = df['New Product 2_inc'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(round(x,2)).format('{:,}'))        


            ########### TOTAL REVENUE ###########

            # Results    
            old_revenue = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)]['Sales_derived_exist'].sum()/2
            new_revenue = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].Sales_derived.sum()
            new_revenue2 = Test_results[Test_results.iter == 2][(Test_results[Test_results.iter == 2].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 2].Test_period <= end_week.children[8].value)].Sales_derived.sum()
            # initialize list of lists
            revenue_data = [['Product', old_revenue,new_revenue,new_revenue2]]

            # Create the pandas DataFrame
            df_rev = pd.DataFrame(revenue_data, columns = ['Revenue','Existing','New Product 1','New Product 2'])

            df_rev['Existing'] = df_rev['Existing'].astype('int64')
            df_rev['Existing_formatted'] = df_rev['Existing'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(x).format('{:,}'))

            df_rev['New Product 1_overall'] = df_rev['New Product 1']
            df_rev['New Product 1'] = df_rev['New Product 1']*(-1*(final_df['Cannibalization Rate in % (New Product 1)'].values[0]/100))
            df_rev['New_formatted 1'] = df_rev['New Product 1'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(x).format('{:,}'))
            df_rev['New_formatted 1_overall'] = df_rev['New Product 1_overall'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(x).format('{:,}'))

            df_rev['New Product 1_inc'] = new_revenue*(1+final_df['Cannibalization Rate in % (New Product 1)'].values[0]/100)
            df_rev['New_formatted 1_inc'] = df_rev['New Product 1_inc'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(x).format('{:,}'))

            df_rev['New Product 2_overall'] = df_rev['New Product 2']
            df_rev['New Product 2'] = df_rev['New Product 2']*(-1*(final_df['Cannibalization Rate in % (New Product 2)'].values[0]/100))
            df_rev['New_formatted 2'] = df_rev['New Product 2'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(x).format('{:,}'))
            df_rev['New_formatted 2_overall'] = df_rev['New Product 2_overall'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(x).format('{:,}'))

            df_rev['New Product 2_inc'] = new_revenue2*(1+final_df['Cannibalization Rate in % (New Product 2)'].values[0]/100)
            df_rev['New_formatted 2_inc'] = df_rev['New Product 2_inc'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(x).format('{:,}'))        

            ###############PLOTS###################

            x1 = np.arange(len([df.columns[0]]))  # the label locations
            x2 = np.arange(len([df_rev.columns[0]]))  # the label locations    
            width = 300  # the width of the bars

            fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize = (20,8), tight_layout = True)
            rects1 = ax1.bar(x1 - width/3, list(df['Existing']), width = 80, label='Existing')
            rects2 = ax1.bar(x1, list(df['New Product 1']), width = 80, label='New Product 1 Cannibalised Units')
            rects2_overall = ax1.bar(x1, list(df['New Product 1_overall']),  width =0)
            rects2_inc = ax1.bar(x1, list(df['New Product 1_inc']), bottom = list(df['New Product 1']),width = 80, label='New Product 1 Incremental Units')
            rects2_2 = ax1.bar(x1 + width/3, list(df['New Product 2']), width = 80, label='New Product 2 Cannibalised Units')
            rects2_2_overall = ax1.bar(x1 + width/3, list(df['New Product 2_overall']),  width =0)
            rects2_2_inc = ax1.bar(x1 + width/3, list(df['New Product 2_inc']), bottom = list(df['New Product 2']), width = 80, label='New Product 2 Incremental Units')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax1.set_ylabel('Volume RU', fontsize=20)
            ax1.set_title('Existing vs New Product Volume RU (Overall)', fontsize=25)
            ax1.set_xticks(x1)
            ax1.set_xticklabels([df.columns[0]], fontsize=20)
            ax1.legend(loc = 'upper left',bbox_to_anchor=(0, -0.05))

            ax1.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

            ax1.bar_label(rects1, padding=5, label_type = 'edge', labels=list(df['Existing_formatted']))
            ax1.bar_label(rects2, padding=5, label_type = 'center', labels=[df['New_formatted 1'].values[0] + " " + "(" + str(int(-1*final_df['Cannibalization Rate in % (New Product 1)'].values[0].round(0))) + "%" ")"])
            ax1.bar_label(rects2_overall, padding=5, label_type = 'edge', labels=list(df['New_formatted 1_overall']))
            ax1.bar_label(rects2_inc, padding=5, label_type = 'center', labels=[df['New_formatted 1_inc'].values[0] + " " +  "(" + str(int(100 + final_df['Cannibalization Rate in % (New Product 1)'].values[0].round(0))) + "%" ")"])
            ax1.bar_label(rects2_2, padding=5, label_type = 'center', labels=[df['New_formatted 2'].values[0] + " " + "(" + str(int(-1*final_df['Cannibalization Rate in % (New Product 2)'].values[0].round(0))) + "%" ")"])
            ax1.bar_label(rects2_2_overall, padding=5, label_type = 'edge', labels=list(df['New_formatted 2_overall']))
            ax1.bar_label(rects2_2_inc, padding=5, label_type = 'center', labels=[df['New_formatted 2_inc'].values[0] + " " + "(" + str(int(100 + final_df['Cannibalization Rate in % (New Product 2)'].values[0].round(0))) + "%" ")"])

            rects3 = ax2.bar(x2 - width/3., list(df_rev['Existing']), width=80, label='Existing')
            rects4 = ax2.bar(x2, list(df_rev['New Product 1']), width=80, label='New Product 1 Cannibalised Revenue')
            rects4_overall = ax2.bar(x2, list(df_rev['New Product 1_overall']), width=0)
            rects4_inc = ax2.bar(x2, list(df_rev['New Product 1_inc']), bottom = list(df_rev['New Product 1']),width=80, label='New Product 1 Incremental Revenue')
            rects4_2 = ax2.bar(x2 + width/3., list(df_rev['New Product 2']), width=80, label='New Product 2 Cannibalised Revenue')
            rects4_2_overall = ax2.bar(x2 + width/3, list(df_rev['New Product 2_overall']), width=0)
            rects4_2_inc = ax2.bar(x2 + width/3., list(df_rev['New Product 2_inc']), bottom = list(df_rev['New Product 2']),width=80, label='New Product 2 Incremental Revenue')


            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax2.set_ylabel('Revenue', fontsize=20)
            ax2.set_title('Existing vs New Product Revenue (Overall)', fontsize=25)
            ax2.set_xticks(x2)
            ax2.set_xticklabels([df_rev.columns[0]], fontsize=20)
            ax2.legend(loc = 'upper left',bbox_to_anchor=(0, -0.05))

            ax2.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

            ax2.bar_label(rects3, padding=5, label_type = 'edge', labels=list(df_rev['Existing_formatted']))
            ax2.bar_label(rects4, padding=5, label_type = 'center', labels=[df_rev['New_formatted 1'].values[0] + " " + "(" + str(int(-1*final_df['Cannibalization Rate in % (New Product 1)'].values[0].round(0))) + "%" ")"])
            ax2.bar_label(rects4_overall, padding=5, label_type = 'edge', labels=list(df_rev['New_formatted 1_overall']))
            ax2.bar_label(rects4_inc, padding=5, label_type = 'center', labels=[df_rev['New_formatted 1_inc'].values[0] + " " + "(" + str(int(100 + final_df['Cannibalization Rate in % (New Product 1)'].values[0].round(0))) + "%" ")"])
            ax2.bar_label(rects4_2, padding=5, label_type = 'center', labels=[df_rev['New_formatted 2'].values[0] + " " + "(" + str(int(-1*final_df['Cannibalization Rate in % (New Product 2)'].values[0].round(0))) + "%" ")"])
            ax2.bar_label(rects4_2_overall, padding=5, label_type = 'edge', labels=list(df_rev['New_formatted 2_overall']))
            ax2.bar_label(rects4_2_inc, padding=5, label_type = 'center', labels=[df_rev['New_formatted 2_inc'].values[0] + " " + "(" + str(int(100 + final_df['Cannibalization Rate in % (New Product 2)'].values[0].round(0))) + "%" ")"])


            plt.show()

        # In case the user selects 1 new product for simulation            
        else:
            plt.rcParams.update({'font.size': 15})
            ########### TOTAL VOLUME ###########

            # Results   
            
            # In case the user doesn't upload the data
            if toggle_upload.value != 'Yes':            
                old_vol = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)]['Predictions_rf_exist'].sum()
                new_vol = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].Predictions_rf.sum()
            elif (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:
                old_vol = Test_results_exist2['Predictions_rf_exist'].sum()
                new_vol = Test_results.Predictions_rf.sum()
                
            # initialize list of lists
            vol_data = [['Product', old_vol,new_vol]]

            # Create the pandas DataFrame
            df = pd.DataFrame(vol_data, columns = ['Volume RU','Existing','New'])

            df['Existing'] = df['Existing'].astype('int64')
            df['Existing_formatted'] = df['Existing'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(round(x,2)).format('{:,}'))

            df['New_overall'] = df['New']
            df['New'] = df['New']*(-1*(final_df['Cannibalization Rate in %'].values[0]/100))
            df['New_formatted'] = df['New'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
            df['New_formatted_overall'] = df['New_overall'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(round(x,2)).format('{:,}'))

            df['New_Inc'] = new_vol*(1+final_df['Cannibalization Rate in %'].values[0]/100)
            df['New_formatted_Inc'] = df['New_Inc'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(round(x,2)).format('{:,}'))


            ########### TOTAL REVENUE ###########

            # Results  
            # In case the user doesn't upload the data
            if toggle_upload.value != 'Yes':
                old_revenue = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)]['Sales_derived_exist'].sum()
                new_revenue = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].Sales_derived.sum()
            elif (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:
                old_revenue = Test_results_exist2['Sales_derived_exist'].sum()
                new_revenue = Test_results.Sales_derived.sum()            
            
            # initialize list of lists
            revenue_data = [['Product', old_revenue,new_revenue]]

            # Create the pandas DataFrame
            df_rev = pd.DataFrame(revenue_data, columns = ['Revenue','Existing','New'])

            df_rev['Existing'] = df_rev['Existing'].astype('int64')
            df_rev['Existing_formatted'] = df_rev['Existing'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(x).format('{:,}'))

            df_rev['New_overall'] = df_rev['New']
            df_rev['New'] = df_rev['New']*(-1*(final_df['Cannibalization Rate in %'].values[0]/100))
            df_rev['New_formatted'] = df_rev['New'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(x).format('{:,}'))
            df_rev['New_formatted_overall'] = df_rev['New_overall'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(x).format('{:,}'))

            df_rev['New_Inc'] = new_revenue*(1+final_df['Cannibalization Rate in %'].values[0]/100)
            df_rev['New_formatted_Inc'] = df_rev['New_Inc'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(x).format('{:,}'))

            ###############PLOTS###################

            x1 = np.arange(len([df.columns[0]]))  # the label locations
            x2 = np.arange(len([df_rev.columns[0]]))  # the label locations
            width = 300  # the width of the bars

            fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize = (20,8), tight_layout = True)
            rects1 = ax1.bar(x1 - width/2, list(df['Existing']), width = 80, label='Existing')
            rects2 = ax1.bar(x1 + width/2, list(df['New']), width = 80, label='Cannibalised Units')
            rects2_overall = ax1.bar(x1 + width/2, list(df['New_overall']), width = 0)
            rects2_inc = ax1.bar(x1 + width/2, list(df['New_Inc']), bottom =  list(df['New']),width = 80, label='Incremental Units')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax1.set_ylabel('Volume RU', fontsize=20)
            ax1.set_title('Existing vs New Product Volume RU (Overall)', fontsize=25)
            ax1.set_xticks(x1)
            ax1.set_xticklabels([df.columns[0]], fontsize=20)
            ax1.legend()

            ax1.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

            ax1.bar_label(rects1, padding=5, label_type = 'edge', labels=list(df['Existing_formatted']))
            ax1.bar_label(rects2, padding=5, label_type = 'center', labels=[df['New_formatted'].values[0] + " " + "(" + str(int(-1*final_df['Cannibalization Rate in %'].values[0].round(0))) + "%" ")"])
            ax1.bar_label(rects2_overall, padding=5, label_type = 'edge', labels=list(df['New_formatted_overall']))
            ax1.bar_label(rects2_inc, padding=5, label_type = 'center', labels=[df['New_formatted_Inc'].values[0] + " " + "(" + str(int(100+final_df['Cannibalization Rate in %'].values[0].round(0))) + "%" ")"])

            rects3 = ax2.bar(x2 - width/2., list(df_rev['Existing']), width=80, label='Existing')
            rects4 = ax2.bar(x2 + width/2., list(df_rev['New']), width=80, label='Cannibalised Revenue')
            rects4_overall = ax2.bar(x2 + width/2., list(df_rev['New_overall']), width=0)
            rects4_inc = ax2.bar(x2 + width/2., list(df_rev['New_Inc']), bottom =  list(df_rev['New']), width=80, label='Incremental Revenue')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax2.set_ylabel('Revenue', fontsize=20)
            ax2.set_title('Existing vs New Product Revenue (Overall)', fontsize=25)
            ax2.set_xticks(x2)
            ax2.set_xticklabels([df_rev.columns[0]], fontsize=20)
            ax2.legend()

            ax2.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

            ax2.bar_label(rects3, padding=5, label_type = 'edge', labels=list(df_rev['Existing_formatted']))
            ax2.bar_label(rects4, padding=5, label_type = 'center', labels=[df_rev['New_formatted'].values[0] +" " + "(" + str(int(-1*final_df['Cannibalization Rate in %'].values[0].round(0))) + "%" ")"])
            ax2.bar_label(rects4_overall, padding=5, label_type = 'edge', labels=list(df_rev['New_formatted_overall']))
            ax2.bar_label(rects4_inc, padding=5, label_type = 'center', labels=[df_rev['New_formatted_Inc'].values[0] + " " + "(" + str(int(100+final_df['Cannibalization Rate in %'].values[0].round(0))) + "%" ")"])

            plt.show()

    # When CR is not present across region-channel            
    else:
                
        # In case the user selects 2 new product for simulation    
        if (toggle_np2.value == 'Yes') & (toggle_upload.value != 'Yes'):

            plt.rcParams.update({'font.size': 15})
            ########### TOTAL VOLUME ###########

            # Results    
            old_vol = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)]['Predictions_rf_exist'].sum()/2
            new_vol = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].Predictions_rf.sum()
            new_vol2 = Test_results[Test_results.iter == 2][(Test_results[Test_results.iter == 2].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 2].Test_period <= end_week.children[8].value)].Predictions_rf.sum()
            # initialize list of lists
            vol_data = [['Product', old_vol,new_vol,new_vol2]]

            # Create the pandas DataFrame
            df = pd.DataFrame(vol_data, columns = ['Volume RU','Existing','New Product 1','New Product 2'])

            df['Existing'] = df['Existing'].astype('int64')
            df['Existing_formatted'] = df['Existing'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(x).format('{:,}'))

            df['New Product 1'] = df['New Product 1'].astype('int64')
            df['New_formatted 1'] = df['New Product 1'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(x).format('{:,}'))

            df['New Product 2'] = df['New Product 2'].astype('int64')
            df['New_formatted 2'] = df['New Product 2'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(x).format('{:,}'))


            ########### TOTAL REVENUE ###########

            # Results    
            old_revenue = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)]['Sales_derived_exist'].sum()/2
            new_revenue = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].Sales_derived.sum()
            new_revenue2 = Test_results[Test_results.iter == 2][(Test_results[Test_results.iter == 2].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 2].Test_period <= end_week.children[8].value)].Sales_derived.sum()
            # initialize list of lists
            revenue_data = [['Product', old_revenue,new_revenue,new_revenue2]]

            # Create the pandas DataFrame
            df_rev = pd.DataFrame(revenue_data, columns = ['Revenue','Existing','New Product 1','New Product 2'])

            df_rev['Existing'] = df_rev['Existing'].astype('int64')
            df_rev['Existing_formatted'] = df_rev['Existing'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(x).format('{:,}'))

            df_rev['New Product 1'] = df_rev['New Product 1'].astype('int64')
            df_rev['New_formatted 1'] = df_rev['New Product 1'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(x).format('{:,}'))

            df_rev['New Product 2'] = df_rev['New Product 2'].astype('int64')
            df_rev['New_formatted 2'] = df_rev['New Product 2'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(x).format('{:,}'))

            ###############PLOTS###################

            x1 = np.arange(len([df.columns[0]]))  # the label locations
            x2 = np.arange(len([df_rev.columns[0]]))  # the label locations    
            width = 300  # the width of the bars

            fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize = (20,8), tight_layout = True)
            rects1 = ax1.bar(x1 - width/3, list(df['Existing']), width = 80, label='Existing')
            rects2 = ax1.bar(x1, list(df['New Product 1']), width = 80, label='New Product 1')
            rects2_2 = ax1.bar(x1 + width/3, list(df['New Product 2']), width = 80, label='New Product 2')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax1.set_ylabel('Volume RU', fontsize=20)
            ax1.set_title('Existing vs New Product Volume RU (Overall)', fontsize=25)
            ax1.set_xticks(x1)
            ax1.set_xticklabels([df.columns[0]], fontsize=20)
            ax1.legend(loc = 'lower right')

            ax1.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

            ax1.bar_label(rects1, padding=5, labels=list(df['Existing_formatted']))
            ax1.bar_label(rects2, padding=5, labels=list(df['New_formatted 1']))
            ax1.bar_label(rects2_2, padding=5, labels=list(df['New_formatted 2']))

            rects3 = ax2.bar(x2 - width/3., list(df_rev['Existing']), width=80, label='Existing')
            rects4 = ax2.bar(x2, list(df_rev['New Product 1']), width=80, label='New Product 1')
            rects4_2 = ax2.bar(x2 + width/3., list(df_rev['New Product 2']), width=80, label='New Product 2')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax2.set_ylabel('Revenue', fontsize=20)
            ax2.set_title('Existing vs New Product Revenue (Overall)', fontsize=25)
            ax2.set_xticks(x2)
            ax2.set_xticklabels([df_rev.columns[0]], fontsize=20)
            ax2.legend(loc = 'lower right')

            ax2.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

            ax2.bar_label(rects3, padding=5, labels=list(df_rev['Existing_formatted']))
            ax2.bar_label(rects4, padding=5, labels=list(df_rev['New_formatted 1']))
            ax2.bar_label(rects4_2, padding=5, labels=list(df_rev['New_formatted 2']))

            plt.show()

        # In case the user selects 1 new product for simulation            
        else:
            plt.rcParams.update({'font.size': 15})
            ########### TOTAL VOLUME ###########

            # In case the user doesn't upload the data
            if toggle_upload.value != 'Yes':            
                old_vol = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)]['Predictions_rf_exist'].sum()
                new_vol = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].Predictions_rf.sum()
            elif (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:
                old_vol = Test_results['Predictions_rf_exist'].sum()
                new_vol = Test_results.Predictions_rf.sum()

            # initialize list of lists
            vol_data = [['Product', old_vol,new_vol]]

            # Create the pandas DataFrame
            df = pd.DataFrame(vol_data, columns = ['Volume RU','Existing','New'])

            df['Existing'] = df['Existing'].astype('int64')
            df['Existing_formatted'] = df['Existing'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))

            df['New'] = df['New'].astype('int64')
            df['New_formatted'] = df['New'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))


            ########### TOTAL REVENUE ###########

            # In case the user doesn't upload the data
            if toggle_upload.value != 'Yes':
                old_revenue = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)]['Sales_derived_exist'].sum()
                new_revenue = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].Sales_derived.sum()
            elif (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:
                old_revenue = Test_results['Sales_derived_exist'].sum()
                new_revenue = Test_results.Sales_derived.sum()            
            # initialize list of lists
            revenue_data = [['Product', old_revenue,new_revenue]]

            # Create the pandas DataFrame
            df_rev = pd.DataFrame(revenue_data, columns = ['Revenue','Existing','New'])

            df_rev['Existing'] = df_rev['Existing'].astype('int64')
            df_rev['Existing_formatted'] = df_rev['Existing'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(x).format('{:,}'))

            df_rev['New'] = df_rev['New'].astype('int64')
            df_rev['New_formatted'] = df_rev['New'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000,2))+'k' if x > 1000 else str(x).format('{:,}'))

            ###############PLOTS###################

            x1 = np.arange(len([df.columns[0]]))  # the label locations
            x2 = np.arange(len([df_rev.columns[0]]))  # the label locations
            width = 300  # the width of the bars

            fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize = (20,8), tight_layout = True)
            rects1 = ax1.bar(x1 - width/2, list(df['Existing']), width = 80, label='Existing')
            rects2 = ax1.bar(x1 + width/2, list(df['New']), width = 80, label='New')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax1.set_ylabel('Volume RU', fontsize=20)
            ax1.set_title('Existing vs New Product Volume RU (Overall)', fontsize=25)
            ax1.set_xticks(x1)
            ax1.set_xticklabels([df.columns[0]], fontsize=20)
            ax1.legend()

            ax1.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

            ax1.bar_label(rects1, padding=5, labels=list(df['Existing_formatted']))
            ax1.bar_label(rects2, padding=5, labels=list(df['New_formatted']))

            rects3 = ax2.bar(x2 - width/2., list(df_rev['Existing']), width=80, label='Existing')
            rects4 = ax2.bar(x2 + width/2., list(df_rev['New']), width=80, label='New')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax2.set_ylabel('Revenue', fontsize=20)
            ax2.set_title('Existing vs New Product Revenue (Overall)', fontsize=25)
            ax2.set_xticks(x2)
            ax2.set_xticklabels([df_rev.columns[0]], fontsize=20)
            ax2.legend()

            ax2.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

            ax2.bar_label(rects3, padding=5, labels=list(df_rev['Existing_formatted']))
            ax2.bar_label(rects4, padding=5, labels=list(df_rev['New_formatted']))

            plt.show()

        

else:    
    clear_output()
    print('Product is not present in that Region/Channel for selected period')    
    


In [182]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [214]:
plt.rcParams.update({'font.size': 15})


########### REGION VOLUME ###########
region_2 = widgets.Dropdown(options=list(np.sort(Volume_dataset_all_reg.Region.unique())) + ['All'], value=region.value, description='Region', layout = layout3)

region_drop = region_2

##############################################################################################################################
#  Getting required data
##############################################################################################################################

if len(qtr_edv_baseline) != 0 :
    # In case the user selects 2 new product for simulation    
    if (toggle_np2.value == 'Yes') & (toggle_upload.value != 'Yes'):
        
        # Results    
        old_reg = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby('Region')['Predictions_rf_exist'].sum().reset_index()
        new_reg = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby('Region').Predictions_rf.sum().reset_index()
        new_reg.rename(columns = {'Predictions_rf':'Predictions_rf_np1'}, inplace = True)
        new_reg_2 = Test_results[Test_results.iter == 2][(Test_results[Test_results.iter == 2].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 2].Test_period <= end_week.children[8].value)].groupby('Region').Predictions_rf.sum().reset_index()
        new_reg_2.rename(columns = {'Predictions_rf':'Predictions_rf_np2'}, inplace = True)

        # merge w.r.t. region
        tot_vol = old_reg.merge(new_reg).merge(new_reg_2)
        tot_vol = tot_vol.rename(columns = {'Predictions_rf_exist':'Existing','Predictions_rf_np1' : 'New Product 1','Predictions_rf_np2' : 'New Product 2'})

        ########### REGION REVENUE ###########

        # Results    
        old_reg_revenue = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby('Region')['Sales_derived_exist'].sum().reset_index()
        new_reg_revenue = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby('Region').Sales_derived.sum().reset_index()
        new_reg_revenue.rename(columns = {'Sales_derived':'Sales_derived_np1'}, inplace = True)
        new_reg_revenue_2 = Test_results[Test_results.iter == 2][(Test_results[Test_results.iter == 2].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 2].Test_period <= end_week.children[8].value)].groupby('Region').Sales_derived.sum().reset_index()
        new_reg_revenue_2.rename(columns = {'Sales_derived':'Sales_derived_np2'}, inplace = True)

        merge_data_region = new_reg_revenue.merge(new_reg_revenue_2, on = 'Region', how = 'outer').sort_values('Region').merge(old_reg_revenue, on = 'Region', how = 'outer').sort_values('Region')
        merge_data_region = merge_data_region.rename(columns = {'Sales_derived_np1':'New Product 1','Sales_derived_np2':'New Product 2','Sales_derived_exist':'Existing'})

        ########### Channel VOLUME ###########

        # Results    
        old_chan_volume = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby(['Channel','Region'])['Predictions_rf_exist'].sum().reset_index()
        new_chan_volume = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby(['Channel','Region']).Predictions_rf.sum().reset_index()
        new_chan_volume.rename(columns = {'Predictions_rf':'Predictions_rf_np1'}, inplace = True)
        
        new_chan_volume2 = Test_results[Test_results.iter == 2][(Test_results[Test_results.iter == 2].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 2].Test_period <= end_week.children[8].value)].groupby(['Channel','Region']).Predictions_rf.sum().reset_index()
        new_chan_volume2.rename(columns = {'Predictions_rf':'Predictions_rf_np2'}, inplace = True)
        
        merge_data_channel = new_chan_volume.merge(new_chan_volume2, on = ['Region','Channel'], how = 'outer').sort_values('Channel').merge(old_chan_volume, on = ['Region','Channel'], how = 'outer').sort_values('Channel')
        merge_data_channel = merge_data_channel.rename(columns = {'Predictions_rf_exist':'Existing','Predictions_rf_np1' : 'New Product 1','Predictions_rf_np2' : 'New Product 2'})

        ########### Channel REVENUE ###########

        # Results    
        old_chan_revenue = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby(['Channel','Region'])['Sales_derived_exist'].sum().reset_index()
        new_chan_revenue = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby(['Channel','Region']).Sales_derived.sum().reset_index()
        new_chan_revenue.rename(columns = {'Sales_derived':'Sales_derived_np1'}, inplace = True)
        
        new_chan_revenue2 = Test_results[Test_results.iter == 2][(Test_results[Test_results.iter == 2].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 2].Test_period <= end_week.children[8].value)].groupby(['Channel','Region']).Sales_derived.sum().reset_index()
        new_chan_revenue2.rename(columns = {'Sales_derived':'Sales_derived_np2'}, inplace = True)
        
        merge_data_channel_revenue = new_chan_revenue.merge(new_chan_revenue2, on = ['Region','Channel'], how = 'outer').sort_values('Channel').merge(old_chan_revenue, on = ['Region','Channel'], how = 'outer').sort_values('Channel')
        merge_data_channel_revenue = merge_data_channel_revenue.rename(columns = {'Sales_derived_np1':'New Product 1','Sales_derived_np2':'New Product 2','Sales_derived_exist':'Existing'})
        
        ########### QUARTER VOLUME ###########

        # Results    
        ref_product_quarter = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby(['Test_period','Region'])['Predictions_rf_exist'].sum().reset_index()
        new_product_quarter = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby(['Test_period','Region']).Predictions_rf.sum().reset_index()
        new_product_quarter.rename(columns = {'Predictions_rf':'Predictions_rf_np1'}, inplace = True)
        new_product_quarter_2 = Test_results[Test_results.iter == 2][(Test_results[Test_results.iter == 2].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 2].Test_period <= end_week.children[8].value)].groupby(['Test_period','Region']).Predictions_rf.sum().reset_index()
        new_product_quarter_2.rename(columns = {'Predictions_rf':'Predictions_rf_np2'}, inplace = True)

        merge_data_quarter = new_product_quarter.merge(new_product_quarter_2, on = ['Region','Test_period'], how = 'outer').sort_values(['Region','Test_period']).merge(ref_product_quarter, on = ['Region','Test_period'], how = 'outer').sort_values(['Region','Test_period'])
        merge_data_quarter = merge_data_quarter.rename(columns = {'Predictions_rf_np1':'New Product 1','Predictions_rf_np2':'New Product 2','Predictions_rf_exist':'Existing'})

        merge_data_quarter = merge_data_quarter.rename(columns = {'Test_period':'Year Quarter'})

        ########### QUARTER REVENUE ###########

        # Results    
        ref_product_quarter_revenue = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby(['Test_period','Region'])['Sales_derived_exist'].sum().reset_index()
        new_product_quarter_revenue = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby(['Test_period','Region']).Sales_derived.sum().reset_index()
        new_product_quarter_revenue.rename(columns = {'Sales_derived':'Sales_derived_np1'}, inplace = True)

        new_product_quarter_revenue_2 = Test_results[Test_results.iter == 2][(Test_results[Test_results.iter == 2].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 2].Test_period <= end_week.children[8].value)].groupby(['Test_period','Region']).Sales_derived.sum().reset_index()
        new_product_quarter_revenue_2.rename(columns = {'Sales_derived':'Sales_derived_np2'}, inplace = True)

        merge_data_quarter_revenue = new_product_quarter_revenue.merge(new_product_quarter_revenue_2, on = ['Region','Test_period'], how = 'outer').sort_values(['Region','Test_period']).merge(ref_product_quarter_revenue, on = ['Region','Test_period'], how = 'outer').sort_values(['Region','Test_period'])
        merge_data_quarter_revenue = merge_data_quarter_revenue.rename(columns = {'Sales_derived_np1':'New Product 1','Sales_derived_np2':'New Product 2','Sales_derived_exist':'Existing'})

        merge_data_quarter_revenue = merge_data_quarter_revenue.rename(columns = {'Test_period':'Year Quarter'})


        ############################# HOLIDAY VOLUME #############################

        # Holiday weeks
        holiday_weeks = Volume_dataset[(Volume_dataset.Product == product_fil) & (Volume_dataset.Week >= '2019-01') & (Volume_dataset.Week <= '2020-52')][['Week','Pre.Holiday.Week', 'Holiday.Week', 'Post.Holiday.Week']].drop_duplicates().sort_values('Week')
        pre_holiday_weeks = list(holiday_weeks[holiday_weeks['Pre.Holiday.Week'] == 1].Week)
        during_holiday_weeks = list(holiday_weeks[holiday_weeks['Holiday.Week'] == 1].Week)
        post_holiday_weeks = list(holiday_weeks[holiday_weeks['Post.Holiday.Week'] == 1].Week)

        # Pre Results    
        pre_holiday_vol_old = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Week.isin(pre_holiday_weeks))&(Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby('Region')['Predictions_rf_exist'].sum().reset_index()
        # Product 1
        pre_holiday_vol_new = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Week.isin(pre_holiday_weeks)) & (Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby('Region').Predictions_rf.sum().reset_index()
        pre_holiday_vol_new.rename(columns = {'Predictions_rf':'Predictions_rf_np1'}, inplace = True)
        # Product 2
        pre_holiday_vol_new_2 = Test_results[Test_results.iter == 2][(Test_results[Test_results.iter == 2].Week.isin(pre_holiday_weeks)) & (Test_results[Test_results.iter == 2].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 2].Test_period <= end_week.children[8].value)].groupby('Region').Predictions_rf.sum().reset_index()
        pre_holiday_vol_new_2.rename(columns = {'Predictions_rf':'Predictions_rf_np2'}, inplace = True)

        pre_merge = pre_holiday_vol_old.merge(pre_holiday_vol_new).merge(pre_holiday_vol_new_2)
        pre_merge = pre_merge.rename(columns = {'Predictions_rf_exist':'Existing','Predictions_rf_np1':'New Product 1','Predictions_rf_np2':'New Product 2'})

        # During Results    
        during_holiday_vol_old = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Week.isin(during_holiday_weeks))&(Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby('Region')['Predictions_rf_exist'].sum().reset_index()
        # Product 1
        during_holiday_vol_new = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Week.isin(during_holiday_weeks)) & (Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby('Region').Predictions_rf.sum().reset_index()
        during_holiday_vol_new.rename(columns = {'Predictions_rf':'Predictions_rf_np1'}, inplace = True)
        # Product 2
        during_holiday_vol_new_2 = Test_results[Test_results.iter == 2][(Test_results[Test_results.iter == 2].Week.isin(during_holiday_weeks)) & (Test_results[Test_results.iter == 2].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 2].Test_period <= end_week.children[8].value)].groupby('Region').Predictions_rf.sum().reset_index()
        during_holiday_vol_new_2.rename(columns = {'Predictions_rf':'Predictions_rf_np2'}, inplace = True)

        during_merge = during_holiday_vol_old.merge(during_holiday_vol_new).merge(during_holiday_vol_new_2)
        during_merge = during_merge.rename(columns = {'Predictions_rf_exist':'Existing','Predictions_rf_np1':'New Product 1','Predictions_rf_np2':'New Product 2'})

        # Post Results    
        post_holiday_vol_old = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Week.isin(post_holiday_weeks))&(Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby('Region')['Predictions_rf_exist'].sum().reset_index()
        # Product 1
        post_holiday_vol_new = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Week.isin(post_holiday_weeks)) & (Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby('Region').Predictions_rf.sum().reset_index()
        post_holiday_vol_new.rename(columns = {'Predictions_rf':'Predictions_rf_np1'}, inplace = True)
        # Product 2
        post_holiday_vol_new_2 = Test_results[Test_results.iter == 2][(Test_results[Test_results.iter == 2].Week.isin(post_holiday_weeks)) & (Test_results[Test_results.iter == 2].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 2].Test_period <= end_week.children[8].value)].groupby('Region').Predictions_rf.sum().reset_index()
        post_holiday_vol_new_2.rename(columns = {'Predictions_rf':'Predictions_rf_np2'}, inplace = True)

        post_merge = post_holiday_vol_old.merge(post_holiday_vol_new).merge(post_holiday_vol_new_2)
        post_merge = post_merge.rename(columns = {'Predictions_rf_exist':'Existing','Predictions_rf_np1':'New Product 1','Predictions_rf_np2':'New Product 2'})

        # Non holiday results
        no_holiday_vol_old = Test_results[Test_results.iter == 1][~(Test_results[Test_results.iter == 1].Week.isin(post_holiday_weeks)) & ~((Test_results[Test_results.iter == 1].Week.isin(during_holiday_weeks))) & ~(Test_results[Test_results.iter == 1].Week.isin(pre_holiday_weeks)) & (Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby('Region')['Predictions_rf_exist'].sum().reset_index()
        # Product 1
        no_holiday_vol_new = Test_results[Test_results.iter == 1][~(Test_results[Test_results.iter == 1].Week.isin(post_holiday_weeks)) & ~(Test_results[Test_results.iter == 1].Week.isin(during_holiday_weeks)) & ~(Test_results[Test_results.iter == 1].Week.isin(pre_holiday_weeks)) & (Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby('Region').Predictions_rf.sum().reset_index()
        no_holiday_vol_new.rename(columns = {'Predictions_rf':'Predictions_rf_np1'}, inplace = True)
        # Product 2
        no_holiday_vol_new_2 = Test_results[Test_results.iter == 2][~(Test_results[Test_results.iter == 2].Week.isin(post_holiday_weeks)) & ~(Test_results[Test_results.iter == 2].Week.isin(during_holiday_weeks)) & ~(Test_results[Test_results.iter == 2].Week.isin(pre_holiday_weeks)) & (Test_results[Test_results.iter == 2].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 2].Test_period <= end_week.children[8].value)].groupby('Region').Predictions_rf.sum().reset_index()
        no_holiday_vol_new_2.rename(columns = {'Predictions_rf':'Predictions_rf_np2'}, inplace = True)

        no_merge = no_holiday_vol_old.merge(no_holiday_vol_new).merge(no_holiday_vol_new_2)
        no_merge = no_merge.rename(columns = {'Predictions_rf_exist':'Existing','Predictions_rf_np1':'New Product 1','Predictions_rf_np2':'New Product 2'})

        # All holiday merge
        pre_merge['Holiday'] = 'Pre'
        during_merge['Holiday'] = 'During'
        post_merge['Holiday'] = 'Post'

        no_merge['Holiday'] = 'No Holiday'
        hol_merge_1 = pre_merge.append(during_merge, ignore_index = True)
       
        hol_merge_11 = hol_merge_1.append(post_merge, ignore_index = True)

        hol_merge_2 = hol_merge_11.append(no_merge, ignore_index = True)
        hol_merge_2 = hol_merge_2.sort_values(['Region'])


        ######################## HOLIDAY REVENUE #############################
        # Pre Results    
        pre_holiday_rev_old = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Week.isin(pre_holiday_weeks))&(Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby('Region')['Sales_derived_exist'].sum().reset_index()
        pre_holiday_rev_new = Test_results[Test_results.iter == 1][Test_results[Test_results.iter == 1].Week.isin(pre_holiday_weeks) & (Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby('Region').Sales_derived.sum().reset_index()
        pre_holiday_rev_new.rename(columns = {'Sales_derived':'Sales_derived_np1'}, inplace = True)
        pre_holiday_rev_new_2 = Test_results[Test_results.iter == 2][Test_results[Test_results.iter == 2].Week.isin(pre_holiday_weeks) & (Test_results[Test_results.iter == 2].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 2].Test_period <= end_week.children[8].value)].groupby('Region').Sales_derived.sum().reset_index()
        pre_holiday_rev_new_2.rename(columns = {'Sales_derived':'Sales_derived_np2'}, inplace = True)

        pre_merge_rev = pre_holiday_rev_old.merge(pre_holiday_rev_new).merge(pre_holiday_rev_new_2)
        pre_merge_rev = pre_merge_rev.rename(columns = {'Sales_derived_np1':'New Product 1','Sales_derived_np2':'New Product 2','Sales_derived_exist':'Existing'})

        # During Results    
        during_holiday_rev_old = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Week.isin(during_holiday_weeks))&(Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby('Region')['Sales_derived_exist'].sum().reset_index()
        during_holiday_rev_new = Test_results[Test_results.iter == 1][Test_results[Test_results.iter == 1].Week.isin(during_holiday_weeks) & (Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby('Region').Sales_derived.sum().reset_index()
        during_holiday_rev_new.rename(columns = {'Sales_derived':'Sales_derived_np1'}, inplace = True)
        during_holiday_rev_new_2 = Test_results[Test_results.iter == 2][Test_results[Test_results.iter == 2].Week.isin(during_holiday_weeks) & (Test_results[Test_results.iter == 2].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 2].Test_period <= end_week.children[8].value)].groupby('Region').Sales_derived.sum().reset_index()
        during_holiday_rev_new_2.rename(columns = {'Sales_derived':'Sales_derived_np2'}, inplace = True)

        during_merge_rev = during_holiday_rev_old.merge(during_holiday_rev_new).merge(during_holiday_rev_new_2)
        during_merge_rev = during_merge_rev.rename(columns = {'Sales_derived_np1':'New Product 1','Sales_derived_np2':'New Product 2','Sales_derived_exist':'Existing'})

        # Post Results    
        post_holiday_rev_old = Test_results[Test_results.iter == 1][(Test_results[Test_results.iter == 1].Week.isin(post_holiday_weeks))&(Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby('Region')['Sales_derived_exist'].sum().reset_index()
        post_holiday_rev_new = Test_results[Test_results.iter == 1][Test_results[Test_results.iter == 1].Week.isin(post_holiday_weeks) & (Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby('Region').Sales_derived.sum().reset_index()
        post_holiday_rev_new.rename(columns = {'Sales_derived':'Sales_derived_np1'}, inplace = True)
        post_holiday_rev_new_2 = Test_results[Test_results.iter == 2][Test_results[Test_results.iter == 2].Week.isin(post_holiday_weeks) & (Test_results[Test_results.iter == 2].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 2].Test_period <= end_week.children[8].value)].groupby('Region').Sales_derived.sum().reset_index()
        post_holiday_rev_new_2.rename(columns = {'Sales_derived':'Sales_derived_np2'}, inplace = True)

        post_merge_rev = post_holiday_rev_old.merge(post_holiday_rev_new).merge(post_holiday_rev_new_2)
        post_merge_rev = post_merge_rev.rename(columns = {'Sales_derived_np1':'New Product 1','Sales_derived_np2':'New Product 2','Sales_derived_exist':'Existing'})

        # Non holiday results
        no_holiday_rev_old = Test_results[Test_results.iter == 1][~(Test_results[Test_results.iter == 1].Week.isin(post_holiday_weeks)) & ~((Test_results[Test_results.iter == 1].Week.isin(during_holiday_weeks))) & ~(Test_results[Test_results.iter == 1].Week.isin(pre_holiday_weeks)) & (Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby('Region')['Sales_derived_exist'].sum().reset_index()
        no_holiday_rev_new = Test_results[Test_results.iter == 1][~(Test_results[Test_results.iter == 1].Week.isin(post_holiday_weeks)) & ~(Test_results[Test_results.iter == 1].Week.isin(during_holiday_weeks)) & ~(Test_results[Test_results.iter == 1].Week.isin(pre_holiday_weeks)) & (Test_results[Test_results.iter == 1].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 1].Test_period <= end_week.children[8].value)].groupby('Region').Sales_derived.sum().reset_index()
        no_holiday_rev_new.rename(columns = {'Sales_derived':'Sales_derived_np1'}, inplace = True)

        no_holiday_rev_new_2 = Test_results[Test_results.iter == 2][~(Test_results[Test_results.iter == 2].Week.isin(post_holiday_weeks)) & ~(Test_results[Test_results.iter == 2].Week.isin(during_holiday_weeks)) & ~(Test_results[Test_results.iter == 2].Week.isin(pre_holiday_weeks)) & (Test_results[Test_results.iter == 2].Test_period >= start_week.children[8].value) & (Test_results[Test_results.iter == 2].Test_period <= end_week.children[8].value)].groupby('Region').Sales_derived.sum().reset_index()
        no_holiday_rev_new_2.rename(columns = {'Sales_derived':'Sales_derived_np2'}, inplace = True)

        no_merge_rev = no_holiday_rev_old.merge(no_holiday_rev_new).merge(no_holiday_rev_new_2)
        no_merge_rev = no_merge_rev.rename(columns = {'Sales_derived_np1':'New Product 1','Sales_derived_np2':'New Product 2','Sales_derived_exist':'Existing'})

        # All holiday merge
        pre_merge_rev['Holiday'] = 'Pre'
        during_merge_rev['Holiday'] = 'During'
        post_merge_rev['Holiday'] = 'Post'
       
        no_merge_rev['Holiday'] = 'No Holiday'
        hol_merge_1_rev = pre_merge_rev.append(during_merge_rev, ignore_index = True)
       
        hol_merge_2_rev_1 = hol_merge_1_rev.append(post_merge_rev, ignore_index = True)
       
        hol_merge_2_rev = hol_merge_2_rev_1.append(no_merge_rev, ignore_index = True)
        hol_merge_2_rev = hol_merge_2_rev.sort_values(['Region'])
        
        ######################## CR ACROSS AFFECTED PRODUCTS #############################9
   
    # In case only 1 new product is selected
    else:

        # In case the user doesn't upload the data
        if toggle_upload.value != 'Yes':            
            old_reg = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby('Region')['Predictions_rf_exist'].sum().reset_index()
            new_reg = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby('Region').Predictions_rf.sum().reset_index()        
        elif (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:
            old_reg = Test_results_exist2.groupby('Region')['Predictions_rf_exist'].sum().reset_index()
            new_reg = Test_results.groupby('Region').Predictions_rf.sum().reset_index()   
                

        # merge w.r.t. region
        tot_vol = old_reg.merge(new_reg)
        tot_vol = tot_vol.rename(columns = {'Predictions_rf_exist':'Existing','Predictions_rf' : 'New'})

        ########### REGION REVENUE ###########

        # Results    
        
        # In case the user doesn't upload the data
        if toggle_upload.value != 'Yes':            
            old_reg_revenue = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby('Region')['Sales_derived_exist'].sum().reset_index()
            new_reg_revenue = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby('Region').Sales_derived.sum().reset_index()

        elif (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:
            old_reg_revenue = Test_results_exist2.groupby('Region')['Sales_derived_exist'].sum().reset_index()
            new_reg_revenue = Test_results.groupby('Region').Sales_derived.sum().reset_index()
            
            
        merge_data_region = new_reg_revenue.merge(old_reg_revenue, on = 'Region', how = 'outer').sort_values('Region')
        merge_data_region = merge_data_region.rename(columns = {'Sales_derived':'New','Sales_derived_exist':'Existing'})

        ########### Channel VOLUME ###########

        # In case the user doesn't upload the data
        if toggle_upload.value != 'Yes':            

            # Results    
            old_chan_volume = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby(['Channel','Region'])['Predictions_rf_exist'].sum().reset_index()
            new_chan_volume = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby(['Channel','Region']).Predictions_rf.sum().reset_index()
        elif (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:
            old_chan_volume = Test_results_exist2.groupby(['Channel','Region'])['Predictions_rf_exist'].sum().reset_index()
            new_chan_volume = Test_results.groupby(['Channel','Region']).Predictions_rf.sum().reset_index()

        merge_data_channel = new_chan_volume.merge(old_chan_volume, on = ['Region','Channel'], how = 'outer').sort_values('Channel')
        merge_data_channel = merge_data_channel.rename(columns = {'Predictions_rf':'New','Predictions_rf_exist':'Existing'})

        ########### Channel REVENUE ###########
        # In case the user doesn't upload the data
        if toggle_upload.value != 'Yes':            
            # Results    
            old_chan_revenue = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby(['Channel','Region'])['Sales_derived_exist'].sum().reset_index()
            new_chan_revenue = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby(['Channel','Region']).Sales_derived.sum().reset_index()
        elif (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:
            old_chan_revenue = Test_results_exist2.groupby(['Channel','Region'])['Sales_derived_exist'].sum().reset_index()
            new_chan_revenue = Test_results.groupby(['Channel','Region']).Sales_derived.sum().reset_index()

        merge_data_channel_revenue = new_chan_revenue.merge(old_chan_revenue, on = ['Region','Channel'], how = 'outer').sort_values('Channel')
        merge_data_channel_revenue = merge_data_channel_revenue.rename(columns = {'Sales_derived':'New','Sales_derived_exist':'Existing'})
        
        ########### QUARTER VOLUME ###########

        # In case the user doesn't upload the data
        if toggle_upload.value != 'Yes':            
            # Results    
            ref_product_quarter = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby(['Test_period','Region'])['Predictions_rf_exist'].sum().reset_index()
            new_product_quarter = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby(['Test_period','Region']).Predictions_rf.sum().reset_index()
        elif (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:
            ref_product_quarter = Test_results_exist2.groupby(['Test_period','Region'])['Predictions_rf_exist'].sum().reset_index()
            new_product_quarter = Test_results.groupby(['Test_period','Region']).Predictions_rf.sum().reset_index()

        merge_data_quarter = new_product_quarter.merge(ref_product_quarter, on = ['Region','Test_period'], how = 'outer').sort_values(['Region','Test_period'])
        merge_data_quarter = merge_data_quarter.rename(columns = {'Predictions_rf':'New','Predictions_rf_exist':'Existing'})

        merge_data_quarter = merge_data_quarter.rename(columns = {'Test_period':'Year Quarter'})

        ########### QUARTER REVENUE ###########
        # In case the user doesn't upload the data
        if toggle_upload.value != 'Yes':            
            # Results    
            ref_product_quarter_revenue = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby(['Test_period','Region'])['Sales_derived_exist'].sum().reset_index()
            new_product_quarter_revenue = Test_results[(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby(['Test_period','Region']).Sales_derived.sum().reset_index()
        elif (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:
            ref_product_quarter_revenue = Test_results_exist2.groupby(['Test_period','Region'])['Sales_derived_exist'].sum().reset_index()
            new_product_quarter_revenue = Test_results.groupby(['Test_period','Region']).Sales_derived.sum().reset_index()

        merge_data_quarter_revenue = new_product_quarter_revenue.merge(ref_product_quarter_revenue, on = ['Region','Test_period'], how = 'outer').sort_values(['Region','Test_period'])
        merge_data_quarter_revenue = merge_data_quarter_revenue.rename(columns = {'Sales_derived':'New','Sales_derived_exist':'Existing'})

        merge_data_quarter_revenue = merge_data_quarter_revenue.rename(columns = {'Test_period':'Year Quarter'})


        ############################# HOLIDAY VOLUME #############################

        # Holiday weeks
        holiday_weeks = Volume_dataset[(Volume_dataset.Product == product_fil) & (Volume_dataset.Week >= '2019-01') & (Volume_dataset.Week <= '2020-52')][['Week','Pre.Holiday.Week', 'Holiday.Week', 'Post.Holiday.Week']].drop_duplicates().sort_values('Week')
        pre_holiday_weeks = list(holiday_weeks[holiday_weeks['Pre.Holiday.Week'] == 1].Week)
        during_holiday_weeks = list(holiday_weeks[holiday_weeks['Holiday.Week'] == 1].Week)
        post_holiday_weeks = list(holiday_weeks[holiday_weeks['Post.Holiday.Week'] == 1].Week)

        if (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:
            # Holiday weeks of uploaded data
            holiday_weeks_exist = upload_data[['Week','Pre.Holiday.Week', 'Holiday.Week', 'Post.Holiday.Week']].drop_duplicates().sort_values('Week')
            pre_holiday_weeks_exist = list(holiday_weeks_exist[holiday_weeks_exist['Pre.Holiday.Week'] == 1].Week)
            during_holiday_weeks_exist = list(holiday_weeks_exist[holiday_weeks_exist['Holiday.Week'] == 1].Week)
            post_holiday_weeks_exist = list(holiday_weeks_exist[holiday_weeks_exist['Post.Holiday.Week'] == 1].Week)

        # Pre Results    
        # In case the user doesn't upload the data
        if toggle_upload.value != 'Yes':
            pre_holiday_vol_old = Test_results[(Test_results.Week.isin(pre_holiday_weeks))&(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby('Region')['Predictions_rf_exist'].sum().reset_index()
            pre_holiday_vol_new = Test_results[(Test_results.Week.isin(pre_holiday_weeks)) & (Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby('Region').Predictions_rf.sum().reset_index()
        elif (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:
            pre_holiday_vol_old = Test_results_exist2[(Test_results_exist2.Week.isin(pre_holiday_weeks))].groupby('Region')['Predictions_rf_exist'].sum().reset_index()
            pre_holiday_vol_new = Test_results[(Test_results.Week.isin(pre_holiday_weeks_exist))].groupby('Region').Predictions_rf.sum().reset_index()

        pre_merge = pre_holiday_vol_old.merge(pre_holiday_vol_new)
        pre_merge = pre_merge.rename(columns = {'Predictions_rf_exist':'Existing','Predictions_rf':'New'})

        # During Results   
        # In case the user doesn't upload the data
        if toggle_upload.value != 'Yes':          
            during_holiday_vol_old = Test_results[(Test_results.Week.isin(during_holiday_weeks))&(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby('Region')['Predictions_rf_exist'].sum().reset_index()
            during_holiday_vol_new = Test_results[(Test_results.Week.isin(during_holiday_weeks)) & (Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby('Region').Predictions_rf.sum().reset_index()
        elif (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:
            during_holiday_vol_old = Test_results_exist2[(Test_results_exist2.Week.isin(during_holiday_weeks))].groupby('Region')['Predictions_rf_exist'].sum().reset_index()
            during_holiday_vol_new = Test_results[(Test_results.Week.isin(during_holiday_weeks_exist))].groupby('Region').Predictions_rf.sum().reset_index()

        during_merge = during_holiday_vol_old.merge(during_holiday_vol_new)
        during_merge = during_merge.rename(columns = {'Predictions_rf_exist':'Existing','Predictions_rf':'New'})

        # Post Results   
        # In case the user doesn't upload the data
        if toggle_upload.value != 'Yes':           
            post_holiday_vol_old = Test_results[(Test_results.Week.isin(post_holiday_weeks))&(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby('Region')['Predictions_rf_exist'].sum().reset_index()
            post_holiday_vol_new = Test_results[(Test_results.Week.isin(post_holiday_weeks)) & (Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby('Region').Predictions_rf.sum().reset_index()
        elif (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:
            post_holiday_vol_old = Test_results_exist2[(Test_results_exist2.Week.isin(post_holiday_weeks))].groupby('Region')['Predictions_rf_exist'].sum().reset_index()
            post_holiday_vol_new = Test_results[(Test_results.Week.isin(post_holiday_weeks_exist))].groupby('Region').Predictions_rf.sum().reset_index()


        post_merge = post_holiday_vol_old.merge(post_holiday_vol_new)
        post_merge = post_merge.rename(columns = {'Predictions_rf_exist':'Existing','Predictions_rf':'New'})

        # Non holiday results
        # In case the user doesn't upload the data
        if toggle_upload.value != 'Yes':         
            no_holiday_vol_old = Test_results[~(Test_results.Week.isin(post_holiday_weeks)) & ~(Test_results.Week.isin(during_holiday_weeks)) & ~(Test_results.Week.isin(pre_holiday_weeks)) & (Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby('Region')['Predictions_rf_exist'].sum().reset_index()
            no_holiday_vol_new = Test_results[~(Test_results.Week.isin(post_holiday_weeks)) & ~(Test_results.Week.isin(during_holiday_weeks)) & ~(Test_results.Week.isin(pre_holiday_weeks)) & (Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby('Region').Predictions_rf.sum().reset_index()
        elif (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:
            no_holiday_vol_old = Test_results_exist2[~(Test_results_exist2.Week.isin(post_holiday_weeks)) & ~(Test_results_exist2.Week.isin(during_holiday_weeks)) & ~(Test_results_exist2.Week.isin(pre_holiday_weeks))].groupby('Region')['Predictions_rf_exist'].sum().reset_index()
            no_holiday_vol_new = Test_results[~(Test_results.Week.isin(post_holiday_weeks_exist)) & ~(Test_results.Week.isin(during_holiday_weeks_exist)) & ~(Test_results.Week.isin(pre_holiday_weeks_exist))].groupby('Region').Predictions_rf.sum().reset_index()

        no_merge = no_holiday_vol_old.merge(no_holiday_vol_new)
        no_merge = no_merge.rename(columns = {'Predictions_rf_exist':'Existing','Predictions_rf':'New'})

        # All holiday merge
        pre_merge['Holiday'] = 'Pre'
        during_merge['Holiday'] = 'During'
        post_merge['Holiday'] = 'Post'
       
        no_merge['Holiday'] = 'No Holiday'
        hol_merge_1 = pre_merge.append(during_merge, ignore_index = True)
       
        hol_merge_11 = hol_merge_1.append(post_merge, ignore_index = True)
       
        hol_merge_2 = hol_merge_11.append(no_merge, ignore_index = True)
        hol_merge_2 = hol_merge_2.sort_values(['Region'])


        ######################## HOLIDAY REVENUE #############################
        # Pre Results    
        # In case the user doesn't upload the dat
        if toggle_upload.value != 'Yes':      

            pre_holiday_rev_old = Test_results[(Test_results.Week.isin(pre_holiday_weeks))&(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby('Region')['Sales_derived_exist'].sum().reset_index()
            pre_holiday_rev_new = Test_results[Test_results.Week.isin(pre_holiday_weeks) & (Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby('Region').Sales_derived.sum().reset_index()
        elif (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:
            pre_holiday_rev_old = Test_results_exist2[Test_results_exist2.Week.isin(pre_holiday_weeks)].groupby('Region')['Sales_derived_exist'].sum().reset_index()
            pre_holiday_rev_new = Test_results[Test_results.Week.isin(pre_holiday_weeks_exist)].groupby('Region').Sales_derived.sum().reset_index()

        pre_merge_rev = pre_holiday_rev_old.merge(pre_holiday_rev_new)
        pre_merge_rev = pre_merge_rev.rename(columns = {'Sales_derived':'New','Sales_derived_exist':'Existing'})

        # During Results  
        # In case the user doesn't upload the data
        if toggle_upload.value != 'Yes':
            during_holiday_rev_old = Test_results[(Test_results.Week.isin(during_holiday_weeks))&(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby('Region')['Sales_derived_exist'].sum().reset_index()
            during_holiday_rev_new = Test_results[Test_results.Week.isin(during_holiday_weeks) & (Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby('Region').Sales_derived.sum().reset_index()
        elif (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:
            during_holiday_rev_old = Test_results_exist2[Test_results_exist2.Week.isin(during_holiday_weeks)].groupby('Region')['Sales_derived_exist'].sum().reset_index()
            during_holiday_rev_new = Test_results[Test_results.Week.isin(during_holiday_weeks_exist)].groupby('Region').Sales_derived.sum().reset_index()

        during_merge_rev = during_holiday_rev_old.merge(during_holiday_rev_new)

        during_merge_rev = during_merge_rev.rename(columns = {'Sales_derived':'New','Sales_derived_exist':'Existing'})

        # Post Results
        # In case the user doesn't upload the data
        if toggle_upload.value != 'Yes':        
            post_holiday_rev_old = Test_results[(Test_results.Week.isin(post_holiday_weeks))&(Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby('Region')['Sales_derived_exist'].sum().reset_index()
            post_holiday_rev_new = Test_results[(Test_results.Week.isin(post_holiday_weeks)) & (Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby('Region').Sales_derived.sum().reset_index()
        elif (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:
            post_holiday_rev_old = Test_results_exist2[(Test_results_exist2.Week.isin(post_holiday_weeks))].groupby('Region')['Sales_derived_exist'].sum().reset_index()
            post_holiday_rev_new = Test_results[Test_results.Week.isin(post_holiday_weeks_exist)].groupby('Region').Sales_derived.sum().reset_index()

        post_merge_rev = post_holiday_rev_old.merge(post_holiday_rev_new)
        post_merge_rev = post_merge_rev.rename(columns = {'Sales_derived':'New','Sales_derived_exist':'Existing'})

        # Non holiday results
        # In case the user doesn't upload the data
        if toggle_upload.value != 'Yes':        
            no_holiday_rev_old = Test_results[~(Test_results.Week.isin(post_holiday_weeks)) & ~((Test_results.Week.isin(during_holiday_weeks))) & ~(Test_results.Week.isin(pre_holiday_weeks)) & (Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby('Region')['Sales_derived_exist'].sum().reset_index()
            no_holiday_rev_new = Test_results[~(Test_results.Week.isin(post_holiday_weeks)) & ~(Test_results.Week.isin(during_holiday_weeks)) & ~(Test_results.Week.isin(pre_holiday_weeks)) & (Test_results.Test_period >= start_week.children[8].value) & (Test_results.Test_period <= end_week.children[8].value)].groupby('Region').Sales_derived.sum().reset_index()
        elif (toggle_upload.value == 'Yes') & (len(uploader.value)) != 0:

            no_holiday_rev_old = Test_results_exist2[~(Test_results_exist2.Week.isin(post_holiday_weeks)) & ~((Test_results_exist2.Week.isin(during_holiday_weeks))) & ~(Test_results_exist2.Week.isin(pre_holiday_weeks))].groupby('Region')['Sales_derived_exist'].sum().reset_index()
            no_holiday_rev_new = Test_results[~(Test_results.Week.isin(post_holiday_weeks_exist)) & ~(Test_results.Week.isin(during_holiday_weeks_exist)) & ~(Test_results.Week.isin(pre_holiday_weeks_exist))].groupby('Region').Sales_derived.sum().reset_index()

        no_merge_rev = no_holiday_rev_old.merge(no_holiday_rev_new)
        no_merge_rev = no_merge_rev.rename(columns = {'Sales_derived':'New','Sales_derived_exist':'Existing'})

        # All holiday merge
        pre_merge_rev['Holiday'] = 'Pre'
        during_merge_rev['Holiday'] = 'During'
        post_merge_rev['Holiday'] = 'Post'
        
        no_merge_rev['Holiday'] = 'No Holiday'
        hol_merge_1_rev = pre_merge_rev.append(during_merge_rev, ignore_index = True)
       
        hol_merge_2_rev_1 = hol_merge_1_rev.append(post_merge_rev, ignore_index = True)
       
        hol_merge_2_rev = hol_merge_2_rev_1.append(no_merge_rev, ignore_index = True)
        hol_merge_2_rev = hol_merge_2_rev.sort_values(['Region'])
                        
else:    
    clear_output()
    print('Product is not present in that Region/Channel for selected period')   

    
##############################################################################################################################
#  Plotting Charts
##############################################################################################################################

# In case the user selects 2 new product for simulation    
if (toggle_np2.value == 'Yes') & (toggle_upload.value != 'Yes'):

    def plotit(Region):

        if region_drop.value != 'All':
            Region_List_filter = [region_drop.value]
        else:
            Region_List_filter = Volume_dataset_all_reg.Region.unique()
        
        tot_vol2 = tot_vol[tot_vol.Region.isin(Region_List_filter)]
        merge_data_region2 = merge_data_region[merge_data_region.Region.isin(Region_List_filter)]
        merge_data_channel2 = merge_data_channel[merge_data_channel.Region.isin(Region_List_filter)]
        merge_data_channel_revenue2 = merge_data_channel_revenue[merge_data_channel_revenue.Region.isin(Region_List_filter)]
        merge_data_quarter2 = merge_data_quarter[merge_data_quarter.Region.isin(Region_List_filter)]  
        merge_data_quarter_revenue2 = merge_data_quarter_revenue[merge_data_quarter_revenue.Region.isin(Region_List_filter)]
        hol_merge_2_copy = hol_merge_2[hol_merge_2.Region.isin(Region_List_filter)]    
        hol_merge_2_rev_copy = hol_merge_2_rev[hol_merge_2_rev.Region.isin(Region_List_filter)] 
    
        cr_dist_prod_copy = iteration_dataset_all_v3[iteration_dataset_all_v3.Region.isin(Region_List_filter)].copy()
        if len(Region_List_filter) > 1:
            cr_dist_prod_copy = iteration_dataset_all_v3[iteration_dataset_all_v3.Region.isin(data_all.Region.unique())].groupby(['Existing_Product','iter']).agg({'Existing Product Vol Without':'sum','Updated_Can_Vol':'sum'}).reset_index()
            cr_dist_prod_copy['% CR'] = cr_dist_prod_copy['Updated_Can_Vol']/cr_dist_prod_copy['Existing Product Vol Without']
    

        tot_vol2['Existing'] = tot_vol2['Existing'].astype('int64')
        tot_vol2['Existing_formatted'] = tot_vol2['Existing'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        tot_vol2['New Product 1'] = tot_vol2['New Product 1'].astype('int64')
        tot_vol2['New Product 2'] = tot_vol2['New Product 2'].astype('int64')
        tot_vol2['New_formatted 1'] = tot_vol2['New Product 1'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        tot_vol2['New_formatted 2'] = tot_vol2['New Product 2'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))

        merge_data_region2['Existing'] = merge_data_region2['Existing'].astype('int64')
        merge_data_region2['Existing_formatted'] = merge_data_region2['Existing'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        merge_data_region2['New Product 1'] = merge_data_region2['New Product 1'].astype('int64')
        merge_data_region2['New Product 2'] = merge_data_region2['New Product 2'].astype('int64')
        merge_data_region2['New_formatted 1'] = merge_data_region2['New Product 1'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        merge_data_region2['New_formatted 2'] = merge_data_region2['New Product 2'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))

        merge_data_channel2 = merge_data_channel2.groupby('Channel').sum().reset_index()
        merge_data_channel2['Existing'] = merge_data_channel2['Existing'].astype('int64')
        merge_data_channel2['Existing_formatted'] = merge_data_channel2['Existing'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        merge_data_channel2['New Product 1'] = merge_data_channel2['New Product 1'].astype('int64')
        merge_data_channel2['New Product 2'] = merge_data_channel2['New Product 2'].astype('int64')
        merge_data_channel2['New_formatted 1'] = merge_data_channel2['New Product 1'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        merge_data_channel2['New_formatted 2'] = merge_data_channel2['New Product 2'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        
        merge_data_channel_revenue2 = merge_data_channel_revenue2.groupby('Channel').sum().reset_index()
        merge_data_channel_revenue2['Existing'] = merge_data_channel_revenue2['Existing'].astype('int64')
        merge_data_channel_revenue2['Existing_formatted'] = merge_data_channel_revenue2['Existing'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        merge_data_channel_revenue2['New Product 1'] = merge_data_channel_revenue2['New Product 1'].astype('int64')
        merge_data_channel_revenue2['New Product 2'] = merge_data_channel_revenue2['New Product 2'].astype('int64')
        merge_data_channel_revenue2['New_formatted 1'] = merge_data_channel_revenue2['New Product 1'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        merge_data_channel_revenue2['New_formatted 2'] = merge_data_channel_revenue2['New Product 2'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        
        merge_data_quarter2 = merge_data_quarter2.groupby('Year Quarter').sum().reset_index()
        merge_data_quarter2['Existing'] = merge_data_quarter2['Existing'].astype('int64')
        merge_data_quarter2['Existing_formatted'] = merge_data_quarter2['Existing'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        merge_data_quarter2['New Product 1'] = merge_data_quarter2['New Product 1'].astype('int64')
        merge_data_quarter2['New Product 2'] = merge_data_quarter2['New Product 2'].astype('int64')
        merge_data_quarter2['New_formatted 1'] = merge_data_quarter2['New Product 1'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        merge_data_quarter2['New_formatted 2'] = merge_data_quarter2['New Product 2'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))

        merge_data_quarter_revenue2 = merge_data_quarter_revenue2.groupby('Year Quarter').sum().reset_index()
        merge_data_quarter_revenue2['Existing'] = merge_data_quarter_revenue2['Existing'].astype('int64')
        merge_data_quarter_revenue2['Existing_formatted'] = merge_data_quarter_revenue2['Existing'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        merge_data_quarter_revenue2['New Product 1'] = merge_data_quarter_revenue2['New Product 1'].astype('int64')
        merge_data_quarter_revenue2['New Product 2'] = merge_data_quarter_revenue2['New Product 2'].astype('int64')
        merge_data_quarter_revenue2['New_formatted 1'] = merge_data_quarter_revenue2['New Product 1'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        merge_data_quarter_revenue2['New_formatted 2'] = merge_data_quarter_revenue2['New Product 2'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))

        hol_merge_2_copy = hol_merge_2_copy.groupby('Holiday').sum().reset_index()
        hol_merge_2_copy['Existing'] = hol_merge_2_copy['Existing'].astype('int64')
        hol_merge_2_copy['Existing_formatted'] = hol_merge_2_copy['Existing'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        hol_merge_2_copy['New Product 1'] = hol_merge_2_copy['New Product 1'].astype('int64')
        hol_merge_2_copy['New Product 2'] = hol_merge_2_copy['New Product 2'].astype('int64')
        hol_merge_2_copy['New_formatted 1'] = hol_merge_2_copy['New Product 1'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        hol_merge_2_copy['New_formatted 2'] = hol_merge_2_copy['New Product 2'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        hol_merge_2_copy = hol_merge_2_copy.reindex([3,0,2,1])

        hol_merge_2_rev_copy = hol_merge_2_rev_copy.groupby('Holiday').sum().reset_index()
        hol_merge_2_rev_copy['Existing'] = hol_merge_2_rev_copy['Existing'].astype('int64')
        hol_merge_2_rev_copy['Existing_formatted'] = hol_merge_2_rev_copy['Existing'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        hol_merge_2_rev_copy['New Product 1'] = hol_merge_2_rev_copy['New Product 1'].astype('int64')
        hol_merge_2_rev_copy['New Product 2'] = hol_merge_2_rev_copy['New Product 2'].astype('int64')
        hol_merge_2_rev_copy['New_formatted 1'] = hol_merge_2_rev_copy['New Product 1'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        hol_merge_2_rev_copy['New_formatted 2'] = hol_merge_2_rev_copy['New Product 2'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        hol_merge_2_rev_copy = hol_merge_2_rev_copy.reindex([3,0,2,1])

        # CR distribution across affected products ## ITER 1##
        cr_dist_prod_copy_1 = cr_dist_prod_copy[cr_dist_prod_copy.iter == 1]
        cr_dist_prod_copy_1 = cr_dist_prod_copy_1.sort_values('Updated_Can_Vol').reset_index(drop=True)
        cr_dist_prod_copy_1['% CR'] = 100*cr_dist_prod_copy_1['% CR']
        cr_dist_prod_copy_1['% CR'] = cr_dist_prod_copy_1['% CR'].round(1)
        cr_dist_prod_copy_1['% CR'] = -1*cr_dist_prod_copy_1['% CR'] 
        cr_dist_prod_copy_1['Updated_Can_Vol'] = -1*cr_dist_prod_copy_1['Updated_Can_Vol']
        cr_dist_prod_copy_1['CR_Vol_formatted'] = cr_dist_prod_copy_1['Updated_Can_Vol'].astype('int').apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))

        cr_dist_prod_copy_2 = cr_dist_prod_copy[cr_dist_prod_copy.iter == 2]
        cr_dist_prod_copy_2 = cr_dist_prod_copy_2.sort_values('Updated_Can_Vol').reset_index(drop=True)
        cr_dist_prod_copy_2['% CR'] = 100*cr_dist_prod_copy_2['% CR']
        cr_dist_prod_copy_2['% CR'] = cr_dist_prod_copy_2['% CR'].round(1)
        cr_dist_prod_copy_2['% CR'] = -1*cr_dist_prod_copy_2['% CR'] 
        cr_dist_prod_copy_2['Updated_Can_Vol'] = -1*cr_dist_prod_copy_2['Updated_Can_Vol']
        cr_dist_prod_copy_2['CR_Vol_formatted'] = cr_dist_prod_copy_2['Updated_Can_Vol'].astype('int').apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        
        
        # Plot it (only if there's data to plot)
        if len(tot_vol2) > 0:        
            ############# REGION PLOTS ##################
            if region_drop.value == 'All':
                width = 0.78
            else:
                width = 300
                
            x1 = np.arange(len(list(tot_vol2['Region'])))  # the label locations
            x2 = np.arange(len(list(merge_data_region2['Region'])))  # the label locations
            width = width # the width of the bars

            fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize = (20,8), tight_layout = True)
            rects1 = ax1.bar(x1 - width/3, list(tot_vol2['Existing']), width = width/3, label='Existing')
            rects2 = ax1.bar(x1, list(tot_vol2['New Product 1']), width = width/3, label='New Product 1')
            rects2_2 = ax1.bar(x1 + width/3, list(tot_vol2['New Product 2']), width = width/3, label='New Product 2')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax1.set_ylabel('Volume RU', fontsize=20)
            ax1.set_title('Existing vs New Product Volume RU (Region)', fontsize=25)
            ax1.set_xticks(x1)
            ax1.set_xticklabels(list(tot_vol2['Region']), fontsize=20)
            ax1.legend(loc = 'lower right')

            ax1.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

            ax1.bar_label(rects1, padding=5, labels=list(tot_vol2['Existing_formatted']))
            ax1.bar_label(rects2, padding=5, labels=list(tot_vol2['New_formatted 1']))
            ax1.bar_label(rects2_2, padding=5, labels=list(tot_vol2['New_formatted 2']))

            rects3 = ax2.bar(x2 - width/3, list(merge_data_region2['Existing']), width =width/3, label='Existing')
            rects4 = ax2.bar(x2, list(merge_data_region2['New Product 1']), width = width/3, label='New Product 1')
            rects4_2 = ax2.bar(x2 + width/3, list(merge_data_region2['New Product 2']), width = width/3, label='New Product 2')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax2.set_ylabel('Revenue', fontsize=20)
            ax2.set_title('Existing vs New Product Revenue (Region)', fontsize=25)
            ax2.set_xticks(x2)
            ax2.set_xticklabels(list(merge_data_region2['Region']), fontsize=20)
            ax2.legend(loc = 'lower right')

            ax2.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

            ax2.bar_label(rects3, padding=5, labels=list(merge_data_region2['Existing_formatted']))
            ax2.bar_label(rects4, padding=5, labels=list(merge_data_region2['New_formatted 1']))
            ax2.bar_label(rects4_2, padding=5, labels=list(merge_data_region2['New_formatted 2']))

            plt.show()

            ################## CHANNEL PLOTS ################
            x1 = np.arange(len(list(merge_data_channel2['Channel'])))  # the label locations
            x2 = np.arange(len(list(merge_data_channel_revenue2['Channel'])))  # the label locations
            width = 0.78  # the width of the bars

            fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize = (20,8), tight_layout = True)
            rects1 = ax1.bar(x1 - width/3, list(merge_data_channel2['Existing']), width = width/3, label='Existing')
            rects2 = ax1.bar(x1, list(merge_data_channel2['New Product 1']), width = width/3, label='New Product 1')
            rects2_2 = ax1.bar(x1 + width/3, list(merge_data_channel2['New Product 2']), width = width/3, label='New Product 2')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax1.set_ylabel('Volume RU', fontsize=20)
            ax1.set_title('Existing vs New Product Volume RU (Channel)', fontsize=25)
            ax1.set_xticks(x1)
            ax1.set_xticklabels(list(merge_data_channel2['Channel']), fontsize=20)
            ax1.legend(loc = 'lower left')

            ax1.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

            ax1.bar_label(rects1, padding=5, labels=list(merge_data_channel2['Existing_formatted']))
            ax1.bar_label(rects2, padding=5, labels=list(merge_data_channel2['New_formatted 1']))
            ax1.bar_label(rects2_2, padding=5, labels=list(merge_data_channel2['New_formatted 2']))

            rects3 = ax2.bar(x2 - width/3, list(merge_data_channel_revenue2['Existing']), width = width/3, label='Existing')
            rects4 = ax2.bar(x2, list(merge_data_channel_revenue2['New Product 1']), width =width/3, label='New Product 1')
            rects4_2 = ax2.bar(x2 + width/3, list(merge_data_channel_revenue2['New Product 2']), width = width/3, label='New Product 2')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax2.set_ylabel('Revenue', fontsize=20)
            ax2.set_title('Existing vs New Product Revenue (Channel)', fontsize=25)
            ax2.set_xticks(x2)
            ax2.set_xticklabels(list(merge_data_channel_revenue2['Channel']), fontsize=20)
            ax2.legend(loc = 'lower left')

            ax2.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

            ax2.bar_label(rects3, padding=5, labels=list(merge_data_channel_revenue2['Existing_formatted']))
            ax2.bar_label(rects4, padding=5, labels=list(merge_data_channel_revenue2['New_formatted 1']))
            ax2.bar_label(rects4_2, padding=5, labels=list(merge_data_channel_revenue2['New_formatted 2']))

            plt.show()
            
            ################## QUARTER PLOTS ################
            x1 = np.arange(len(list(merge_data_quarter2['Year Quarter'])))  # the label locations
            x2 = np.arange(len(list(merge_data_quarter_revenue2['Year Quarter'])))  # the label locations
            width = 0.78  # the width of the bars

            fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize = (20,8), tight_layout = True)
            rects1 = ax1.bar(x1 - width/3, list(merge_data_quarter2['Existing']), width = width/3, label='Existing')
            rects2 = ax1.bar(x1, list(merge_data_quarter2['New Product 1']), width = width/3, label='New Product 1')
            rects2_2 = ax1.bar(x1 + width/3, list(merge_data_quarter2['New Product 2']), width = width/3, label='New Product 2')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax1.set_ylabel('Volume RU', fontsize=20)
            ax1.set_title('Existing vs New Product Volume RU (All Quarters)', fontsize=25)
            ax1.set_xticks(x1)
            ax1.set_xticklabels(list(merge_data_quarter2['Year Quarter']), fontsize=20)
            ax1.legend(loc = 'lower right')

            ax1.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

            ax1.bar_label(rects1, padding=5, labels=list(merge_data_quarter2['Existing_formatted']))
            ax1.bar_label(rects2, padding=5, labels=list(merge_data_quarter2['New_formatted 1']))
            ax1.bar_label(rects2_2, padding=5, labels=list(merge_data_quarter2['New_formatted 2']))

            rects3 = ax2.bar(x2 - width/3, list(merge_data_quarter_revenue2['Existing']), width = width/3, label='Existing')
            rects4 = ax2.bar(x2, list(merge_data_quarter_revenue2['New Product 1']), width =width/3, label='New Product 1')
            rects4_2 = ax2.bar(x2 + width/3, list(merge_data_quarter_revenue2['New Product 2']), width = width/3, label='New Product 2')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax2.set_ylabel('Revenue', fontsize=20)
            ax2.set_title('Existing vs New Product Revenue (All Quarters)', fontsize=25)
            ax2.set_xticks(x2)
            ax2.set_xticklabels(list(merge_data_quarter_revenue2['Year Quarter']), fontsize=20)
            ax2.legend(loc = 'lower right')

            ax2.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

            ax2.bar_label(rects3, padding=5, labels=list(merge_data_quarter_revenue2['Existing_formatted']))
            ax2.bar_label(rects4, padding=5, labels=list(merge_data_quarter_revenue2['New_formatted 1']))
            ax2.bar_label(rects4_2, padding=5, labels=list(merge_data_quarter_revenue2['New_formatted 2']))

            plt.show()

            ################## HOLIDAY PLOTS ################
            x1 = np.arange(len(list(hol_merge_2_copy['Holiday'])))  # the label locations
            x2 = np.arange(len(list(hol_merge_2_rev_copy['Holiday'])))  # the label locations
            width = 0.78  # the width of the bars

            fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize = (20,8), tight_layout = True)
            rects1 = ax1.bar(x1 - width/3, list(hol_merge_2_copy['Existing']), width = width/3, label='Existing')
            rects2 = ax1.bar(x1, list(hol_merge_2_copy['New Product 1']), width = width/3, label='New Product 1')
            rects2_2 = ax1.bar(x1 + width/3, list(hol_merge_2_copy['New Product 2']), width = width/3, label='New Product 2')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax1.set_ylabel('Volume RU', fontsize=20)
            ax1.set_title('Existing vs New Product Volume RU (Holidays)', fontsize=25)
            ax1.set_xticks(x1)
            ax1.set_xticklabels(list(hol_merge_2_copy['Holiday']), fontsize=20)
            ax1.legend()

            ax1.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

            ax1.bar_label(rects1, padding=5, labels=list(hol_merge_2_copy['Existing_formatted']))
            ax1.bar_label(rects2, padding=5, labels=list(hol_merge_2_copy['New_formatted 1']))
            ax1.bar_label(rects2_2, padding=5, labels=list(hol_merge_2_copy['New_formatted 2']))

            rects3 = ax2.bar(x2 - width/3, list(hol_merge_2_rev_copy['Existing']), width = width/3, label='Existing')
            rects4 = ax2.bar(x2, list(hol_merge_2_rev_copy['New Product 1']), width = width/3, label='New Product 1')
            rects4_2 = ax2.bar(x2 + width/3, list(hol_merge_2_rev_copy['New Product 2']), width =width/3, label='New Product 2')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax2.set_ylabel('Revenue', fontsize=20)
            ax2.set_title('Existing vs New Product Revenue (Holidays)', fontsize=25)
            ax2.set_xticks(x2)
            ax2.set_xticklabels(list(hol_merge_2_rev_copy['Holiday']), fontsize=20)
            ax2.legend()

            ax2.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

            ax2.bar_label(rects3, padding=5, labels=list(hol_merge_2_rev_copy['Existing_formatted']))
            ax2.bar_label(rects4, padding=5, labels=list(hol_merge_2_rev_copy['New_formatted 1']))
            ax2.bar_label(rects4_2, padding=5, labels=list(hol_merge_2_rev_copy['New_formatted 2']))

            plt.show()
            
            # When CR is present across region-channel ## ITER 1##
            if len(kpi_df3) > 0:

                ################## CR AFFECTED PRODUCTS PLOTS 2 ##################                
                x1 = np.arange(len(list(cr_dist_prod_copy_1['Existing_Product'])))  # the label locations            
                width = 0.35  # the width of the bars

                fig, (ax1) = plt.subplots(nrows=1, ncols=1, figsize = (25,8))
                ax2 = ax1.twinx()
                rects1 = ax1.bar(x1, list(cr_dist_prod_copy_1['Updated_Can_Vol']), width,label='Cannibalised Units')
                rects2 = ax2.plot(x1, list(cr_dist_prod_copy_1['% CR']), width, linewidth=4, color = '#F97306',label='% Cannibalisation Rate', marker='o')

                # Add some text for labels, title and custom x-axis tick labels, etc.
                ax1.set_ylabel('Cannibalised Units', fontsize=20)
                ax2.set_ylabel('% Cannibalisation Rate', fontsize=20)
                ax1.set_title('% Cannibalization Rate across affected products (New Product 1)', fontsize=25)
                ax1.set_xticks(x1)
                ax1.set_xticklabels(list(cr_dist_prod_copy_1['Existing_Product']), fontsize=15,rotation=45, ha='right')
                ax1.legend(loc = 'upper left',bbox_to_anchor=(0.5, -0.7))
                ax2.legend(loc = 'upper right',bbox_to_anchor=(0.5, -0.7))

                ax1.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())
                ax1.bar_label(rects1, padding=5, labels=list(cr_dist_prod_copy_1['CR_Vol_formatted']),label_type = 'center')
                
                for i,j in cr_dist_prod_copy_1['% CR'].items():
                    ax2.annotate(str(j) + "%", xy=(i, j))

                plt.show()


            # When CR is present across region-channel ## ITER 2##
            if len(kpi_df3) > 0:

                ################## CR AFFECTED PRODUCTS PLOTS 2 ##################                
                x1 = np.arange(len(list(cr_dist_prod_copy_2['Existing_Product'])))  # the label locations            
                width = 0.35  # the width of the bars

                fig, (ax1) = plt.subplots(nrows=1, ncols=1, figsize = (25,8))
                ax2 = ax1.twinx()
                rects1 = ax1.bar(x1, list(cr_dist_prod_copy_2['Updated_Can_Vol']), width,label='Cannibalised Units')
                rects2 = ax2.plot(x1, list(cr_dist_prod_copy_2['% CR']), width, linewidth=4, color = '#F97306',label='% Cannibalisation Rate', marker='o')

                # Add some text for labels, title and custom x-axis tick labels, etc.
                ax1.set_ylabel('Cannibalised Units', fontsize=20)
                ax2.set_ylabel('% Cannibalisation Rate', fontsize=20)
                ax1.set_title('% Cannibalization Rate across affected products (New Product 2)', fontsize=25)
                ax1.set_xticks(x1)
                ax1.set_xticklabels(list(cr_dist_prod_copy_2['Existing_Product']), fontsize=15,rotation=45, ha='right')
                ax1.legend(loc = 'upper left',bbox_to_anchor=(0.5, -0.7))
                ax2.legend(loc = 'upper right',bbox_to_anchor=(0.5, -0.7))

                ax1.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())
                ax1.bar_label(rects1, padding=5, labels=list(cr_dist_prod_copy_2['CR_Vol_formatted']), label_type = 'center')
                
                for i,j in cr_dist_prod_copy_2['% CR'].items():
                    ax2.annotate(str(j) + "%", xy=(i, j))

                plt.show()
                print("Note1: Cannibalization is measured across the same category products")
                print("Note2: For ease of showing numbers, cannibalisation rate is shown positive, in reality it is negative cannibalisation")

        else:
            print("No data to show for current selection")
 
# In case the user selects 1 new product for simulation    
elif (toggle_np2.value == 'No'):
        
    def plotit(Region):

        if region_drop.value != 'All':
            Region_List_filter = [region_drop.value]
        else:
            Region_List_filter = Volume_dataset_all_reg.Region.unique()

        tot_vol2 = tot_vol[tot_vol.Region.isin(Region_List_filter)]
        merge_data_region2 = merge_data_region[merge_data_region.Region.isin(Region_List_filter)]
        merge_data_channel2 = merge_data_channel[merge_data_channel.Region.isin(Region_List_filter)]
        merge_data_channel_revenue2 = merge_data_channel_revenue[merge_data_channel_revenue.Region.isin(Region_List_filter)]
        merge_data_quarter2 = merge_data_quarter[merge_data_quarter.Region.isin(Region_List_filter)]  
        merge_data_quarter_revenue2 = merge_data_quarter_revenue[merge_data_quarter_revenue.Region.isin(Region_List_filter)]
        hol_merge_2_copy = hol_merge_2[hol_merge_2.Region.isin(Region_List_filter)]    
        hol_merge_2_rev_copy = hol_merge_2_rev[hol_merge_2_rev.Region.isin(Region_List_filter)] 
        
        cr_dist_prod_copy = iteration_dataset_all_v3[iteration_dataset_all_v3.Region.isin(Region_List_filter)].copy()
        if len(Region_List_filter) > 1:
            cr_dist_prod_copy = iteration_dataset_all_v3[iteration_dataset_all_v3.Region.isin(data_all.Region.unique())].groupby(['Existing_Product','iter']).agg({'Existing Product Vol Without':'sum','Updated_Can_Vol':'sum'}).reset_index()
            cr_dist_prod_copy['% CR'] = cr_dist_prod_copy['Updated_Can_Vol']/cr_dist_prod_copy['Existing Product Vol Without']
        
        
        tot_vol2['Existing'] = tot_vol2['Existing'].astype('int64')
        tot_vol2['Existing_formatted'] = tot_vol2['Existing'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        tot_vol2['New'] = tot_vol2['New'].astype('int64')
        tot_vol2['New_formatted'] = tot_vol2['New'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))

        merge_data_region2['Existing'] = merge_data_region2['Existing'].astype('int64')
        merge_data_region2['Existing_formatted'] = merge_data_region2['Existing'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        merge_data_region2['New'] = merge_data_region2['New'].astype('int64')
        merge_data_region2['New_formatted'] = merge_data_region2['New'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))

        merge_data_channel2 = merge_data_channel2.groupby('Channel').sum().reset_index()
        merge_data_channel2['Existing'] = merge_data_channel2['Existing'].astype('int64')
        merge_data_channel2['Existing_formatted'] = merge_data_channel2['Existing'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        merge_data_channel2['New'] = merge_data_channel2['New'].astype('int64')
        merge_data_channel2['New_formatted'] = merge_data_channel2['New'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))        
        
        merge_data_channel_revenue2 = merge_data_channel_revenue2.groupby('Channel').sum().reset_index()
        merge_data_channel_revenue2['Existing'] = merge_data_channel_revenue2['Existing'].astype('int64')
        merge_data_channel_revenue2['Existing_formatted'] = merge_data_channel_revenue2['Existing'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        merge_data_channel_revenue2['New'] = merge_data_channel_revenue2['New'].astype('int64')
        merge_data_channel_revenue2['New_formatted'] = merge_data_channel_revenue2['New'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))

        merge_data_quarter2 = merge_data_quarter2.groupby('Year Quarter').sum().reset_index()
        merge_data_quarter2['Existing'] = merge_data_quarter2['Existing'].astype('int64')
        merge_data_quarter2['Existing_formatted'] = merge_data_quarter2['Existing'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        merge_data_quarter2['New'] = merge_data_quarter2['New'].astype('int64')
        merge_data_quarter2['New_formatted'] = merge_data_quarter2['New'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))

        merge_data_quarter_revenue2 = merge_data_quarter_revenue2.groupby('Year Quarter').sum().reset_index()
        merge_data_quarter_revenue2['Existing'] = merge_data_quarter_revenue2['Existing'].astype('int64')
        merge_data_quarter_revenue2['Existing_formatted'] = merge_data_quarter_revenue2['Existing'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        merge_data_quarter_revenue2['New'] = merge_data_quarter_revenue2['New'].astype('int64')
        merge_data_quarter_revenue2['New_formatted'] = merge_data_quarter_revenue2['New'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))

        hol_merge_2_copy = hol_merge_2_copy.groupby('Holiday').sum().reset_index()
        hol_merge_2_copy['Existing'] = hol_merge_2_copy['Existing'].astype('int64')
        hol_merge_2_copy['Existing_formatted'] = hol_merge_2_copy['Existing'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        hol_merge_2_copy['New'] = hol_merge_2_copy['New'].astype('int64')
        hol_merge_2_copy['New_formatted'] = hol_merge_2_copy['New'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        hol_merge_2_copy = hol_merge_2_copy.reindex([3,0,2,1])

        hol_merge_2_rev_copy = hol_merge_2_rev_copy.groupby('Holiday').sum().reset_index()
        hol_merge_2_rev_copy['Existing'] = hol_merge_2_rev_copy['Existing'].astype('int64')
        hol_merge_2_rev_copy['Existing_formatted'] = hol_merge_2_rev_copy['Existing'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        hol_merge_2_rev_copy['New'] = hol_merge_2_rev_copy['New'].astype('int64')
        hol_merge_2_rev_copy['New_formatted'] = hol_merge_2_rev_copy['New'].apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))
        hol_merge_2_rev_copy = hol_merge_2_rev_copy.reindex([3,0,2,1])

        # CR distribution across affected products
        cr_dist_prod_copy = cr_dist_prod_copy.sort_values('Updated_Can_Vol').reset_index(drop=True)
        cr_dist_prod_copy['% CR'] = 100*cr_dist_prod_copy['% CR']
        cr_dist_prod_copy['% CR'] = cr_dist_prod_copy['% CR'].round(1)
        cr_dist_prod_copy['% CR'] = -1*cr_dist_prod_copy['% CR'] 
        cr_dist_prod_copy['Updated_Can_Vol'] = -1*cr_dist_prod_copy['Updated_Can_Vol']
        cr_dist_prod_copy['CR_Vol_formatted'] = cr_dist_prod_copy['Updated_Can_Vol'].astype('int').apply(lambda x: str(round(x/1000000,2))+'M' if x >= 1000000 else str(round(x/1000))+'k' if x > 1000 else str(x).format('{:,}'))

        # Plot it (only if there's data to plot)
        if len(tot_vol2) > 0:        
            ############# REGION PLOTS ##################
            if region_drop.value == 'All':
                width = 0.35
            else:
                width = 300

            x1 = np.arange(len(list(tot_vol2['Region'])))  # the label locations
            x2 = np.arange(len(list(merge_data_region2['Region'])))  # the label locations
            width = width  # the width of the bars

            fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize = (20,8), tight_layout = True)
            rects1 = ax1.bar(x1 - width/2, list(tot_vol2['Existing']), width, label='Existing')
            rects2 = ax1.bar(x1 + width/2, list(tot_vol2['New']), width, label='New')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax1.set_ylabel('Volume RU', fontsize=20)
            ax1.set_title('Existing vs New Product Volume RU (Region)', fontsize=25)
            ax1.set_xticks(x1)
            ax1.set_xticklabels(list(tot_vol2['Region']), fontsize=20)
            ax1.legend()

            ax1.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

            ax1.bar_label(rects1, padding=5, labels=list(tot_vol2['Existing_formatted']))
            ax1.bar_label(rects2, padding=5, labels=list(tot_vol2['New_formatted']))

            rects3 = ax2.bar(x2 - width/2, list(merge_data_region2['Existing']), width, label='Existing')
            rects4 = ax2.bar(x2 + width/2, list(merge_data_region2['New']), width, label='New')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax2.set_ylabel('Revenue', fontsize=20)
            ax2.set_title('Existing vs New Product Revenue (Region)', fontsize=25)
            ax2.set_xticks(x2)
            ax2.set_xticklabels(list(merge_data_region2['Region']), fontsize=20)
            ax2.legend()

            ax2.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

            ax2.bar_label(rects3, padding=5, labels=list(merge_data_region2['Existing_formatted']))
            ax2.bar_label(rects4, padding=5, labels=list(merge_data_region2['New_formatted']))

            plt.show()
           
            ################## CHANNEL PLOTS ################
            x1 = np.arange(len(list(merge_data_channel2['Channel'])))  # the label locations
            x2 = np.arange(len(list(merge_data_channel_revenue2['Channel'])))  # the label locations
            width = 0.35  # the width of the bars

            fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize = (20,8), tight_layout = True)
            rects1 = ax1.bar(x1 - width/2, list(merge_data_channel2['Existing']), width, label='Existing')
            rects2 = ax1.bar(x1 + width/2, list(merge_data_channel2['New']), width, label='New')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax1.set_ylabel('Volume RU', fontsize=20)
            ax1.set_title('Existing vs New Product Volume RU (Channel)', fontsize=25)
            ax1.set_xticks(x1)
            ax1.set_xticklabels(list(merge_data_channel2['Channel']), fontsize=20)
            ax1.legend()

            ax1.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

            ax1.bar_label(rects1, padding=5, labels=list(merge_data_channel2['Existing_formatted']))
            ax1.bar_label(rects2, padding=5, labels=list(merge_data_channel2['New_formatted']))

            rects3 = ax2.bar(x2 - width/2, list(merge_data_channel_revenue2['Existing']), width, label='Existing')
            rects4 = ax2.bar(x2 + width/2, list(merge_data_channel_revenue2['New']), width, label='New')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax2.set_ylabel('Revenue', fontsize=20)
            ax2.set_title('Existing vs New Product Revenue (Channel)', fontsize=25)
            ax2.set_xticks(x2)
            ax2.set_xticklabels(list(merge_data_channel_revenue2['Channel']), fontsize=20)
            ax2.legend()

            ax2.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

            ax2.bar_label(rects3, padding=5, labels=list(merge_data_channel_revenue2['Existing_formatted']))
            ax2.bar_label(rects4, padding=5, labels=list(merge_data_channel_revenue2['New_formatted']))

            plt.show()
        
            ################## QUARTER PLOTS ################
            x1 = np.arange(len(list(merge_data_quarter2['Year Quarter'])))  # the label locations
            x2 = np.arange(len(list(merge_data_quarter_revenue2['Year Quarter'])))  # the label locations
            width = 0.35  # the width of the bars

            fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize = (20,8), tight_layout = True)
            rects1 = ax1.bar(x1 - width/2, list(merge_data_quarter2['Existing']), width, label='Existing')
            rects2 = ax1.bar(x1 + width/2, list(merge_data_quarter2['New']), width, label='New')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax1.set_ylabel('Volume RU', fontsize=20)
            ax1.set_title('Existing vs New Product Volume RU (All Quarters)', fontsize=25)
            ax1.set_xticks(x1)
            ax1.set_xticklabels(list(merge_data_quarter2['Year Quarter']), fontsize=20)
            ax1.legend()

            ax1.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

            ax1.bar_label(rects1, padding=5, labels=list(merge_data_quarter2['Existing_formatted']))
            ax1.bar_label(rects2, padding=5, labels=list(merge_data_quarter2['New_formatted']))

            rects3 = ax2.bar(x2 - width/2, list(merge_data_quarter_revenue2['Existing']), width, label='Existing')
            rects4 = ax2.bar(x2 + width/2, list(merge_data_quarter_revenue2['New']), width, label='New')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax2.set_ylabel('Revenue', fontsize=20)
            ax2.set_title('Existing vs New Product Revenue (All Quarters)', fontsize=25)
            ax2.set_xticks(x2)
            ax2.set_xticklabels(list(merge_data_quarter_revenue2['Year Quarter']), fontsize=20)
            ax2.legend()

            ax2.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

            ax2.bar_label(rects3, padding=5, labels=list(merge_data_quarter_revenue2['Existing_formatted']))
            ax2.bar_label(rects4, padding=5, labels=list(merge_data_quarter_revenue2['New_formatted']))

            plt.show()

            ################## HOLIDAY PLOTS ################
            x1 = np.arange(len(list(hol_merge_2_copy['Holiday'])))  # the label locations
            x2 = np.arange(len(list(hol_merge_2_rev_copy['Holiday'])))  # the label locations
            width = 0.35  # the width of the bars

            fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize = (20,8), tight_layout = True)
            rects1 = ax1.bar(x1 - width/2, list(hol_merge_2_copy['Existing']), width, label='Existing')
            rects2 = ax1.bar(x1 + width/2, list(hol_merge_2_copy['New']), width, label='New')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax1.set_ylabel('Volume RU', fontsize=20)
            ax1.set_title('Existing vs New Product Volume RU (Holidays)', fontsize=25)
            ax1.set_xticks(x1)
            ax1.set_xticklabels(list(hol_merge_2_copy['Holiday']), fontsize=20)
            ax1.legend()

            ax1.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

            ax1.bar_label(rects1, padding=5, labels=list(hol_merge_2_copy['Existing_formatted']))
            ax1.bar_label(rects2, padding=5, labels=list(hol_merge_2_copy['New_formatted']))

            rects3 = ax2.bar(x2 - width/2, list(hol_merge_2_rev_copy['Existing']), width, label='Existing')
            rects4 = ax2.bar(x2 + width/2, list(hol_merge_2_rev_copy['New']), width, label='New')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax2.set_ylabel('Revenue', fontsize=20)
            ax2.set_title('Existing vs New Product Revenue (Holidays)', fontsize=25)
            ax2.set_xticks(x2)
            ax2.set_xticklabels(list(hol_merge_2_rev_copy['Holiday']), fontsize=20)
            ax2.legend()

            ax2.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

            ax2.bar_label(rects3, padding=5, labels=list(hol_merge_2_rev_copy['Existing_formatted']))
            ax2.bar_label(rects4, padding=5, labels=list(hol_merge_2_rev_copy['New_formatted']))
            plt.show()
           
            # When CR is present across region-channel
            if len(kpi_df3) > 0:

                ################## CR AFFECTED PRODUCTS PLOTS 2 ##################                
                x1 = np.arange(len(list(cr_dist_prod_copy['Existing_Product'])))  # the label locations            
                width = 0.35  # the width of the bars

                fig, (ax1) = plt.subplots(nrows=1, ncols=1, figsize = (25,8))
                ax2 = ax1.twinx()
                rects1 = ax1.bar(x1, list(cr_dist_prod_copy['Updated_Can_Vol']), width,label='Cannibalised Units')
                rects2 = ax2.plot(x1, list(cr_dist_prod_copy['% CR']), linewidth=4, color = '#F97306',label='% Cannibalisation Rate', marker='o')

                # Add some text for labels, title and custom x-axis tick labels, etc.
                ax1.set_ylabel('Cannibalised Units', fontsize=20)
                ax2.set_ylabel('% Cannibalisation Rate', fontsize=20)
                ax1.set_title('% Cannibalization Rate across affected products', fontsize=25)
                ax1.set_xticks(x1)
                ax1.set_xticklabels(list(cr_dist_prod_copy['Existing_Product']), fontsize=15,rotation=45, ha='right')
                ax1.legend(loc = 'upper left',bbox_to_anchor=(0.5, -0.7))
                ax2.legend(loc = 'upper right',bbox_to_anchor=(0.5, -0.7))

                ax1.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())

                ax1.bar_label(rects1, padding=5, labels=list(cr_dist_prod_copy['CR_Vol_formatted']),label_type = 'center')
                for i,j in cr_dist_prod_copy['% CR'].items():
                    ax2.annotate(str(j) + "%", xy=(i, j))
                print("Note1: Cannibalization is measured across the same category products")
                print("Note2: For ease of showing numbers, cannibalisation rate is shown positive, in reality it is negative cannibalisation")
                plt.show()
            
        else:
            print("No data to show for current selection")
        
        
if len(qtr_edv_baseline) != 0 :
    display(interactive(plotit, Region=region_drop))
else:
    print("Product is not present in that Region/Channel for selected period")

__Cannibalization View__

In [215]:
plt.rcParams.update({'font.size': 15})

# Region CR dropdown
region_CR = widgets.Dropdown(options=['All','EAST','ONTARIO','QUEBEC','WEST'] , description='Region')
# Channel CR dropdown
channel_CR = widgets.Dropdown(options=['All','CONVENTIONAL','DISCOUNT','DRUG','WHOLESALE'] , description='Channel')

region_drop_CR = region_CR
channel_drop_CR = channel_CR

# In case of 1 new products    
if len(iter_change_list) == 1:

    def plotit_CR(R, C):
        if((channel_drop_CR.value == 'All') and (region_drop_CR.value == 'All')):
            final_df = iteration_dataset_all_v2.copy()
            final_df['CR'] = (final_df['Displaced_Volume']/final_df['New_Product_Volume'])*100
            final_df['CR'] = final_df['CR'].round(decimals=2)
            region_cr_final_data = iteration_dataset_all_v2.groupby(['Region']).sum().reset_index()
            region_cr_final_data['CR'] = (region_cr_final_data['Displaced_Volume']/region_cr_final_data['New_Product_Volume'])*100
            region_cr_final_data['CR'] = region_cr_final_data['CR'].round(decimals=2)
            channel_cr_final_data = iteration_dataset_all_v2.groupby(['Channel']).sum().reset_index()
            channel_cr_final_data['CR'] = (channel_cr_final_data['Displaced_Volume']/channel_cr_final_data['New_Product_Volume'])*100
            channel_cr_final_data['CR'] = channel_cr_final_data['CR'].round(decimals=2)
            
            ####################### PLOTS ###################
            x1 = np.arange(len(list(region_cr_final_data['Region'])))  # the label locations
            x2 = np.arange(len(list(channel_cr_final_data['Channel'])))  # the label locations
            width = 0.35  # the width of the bars

            fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize = (20,8), tight_layout = True)
            rects1 = ax1.bar(x1 - width/2, list(region_cr_final_data['CR']), width, label='Region')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax1.set_ylabel('Cannibalization Rate', fontsize=20)
            ax1.set_title('Region Cannibalization Rate in %', fontsize=25)
            ax1.set_xticks(x1)
            ax1.set_xticklabels(list(region_cr_final_data['Region']), horizontalalignment="right", fontsize=20)        

            ax1.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

            ax1.bar_label(rects1, padding=5, labels= list(region_cr_final_data['CR']))

            rects2 = ax2.bar(x2 - width/2,  list(channel_cr_final_data['CR']), width, label='Channel')

            # # Add some text for labels, title and custom x-axis tick labels, etc.
            ax2.set_ylabel('Cannibalization Rate', fontsize=20)
            ax2.set_title('Channel Cannibalization Rate in %', fontsize=25)
            ax2.set_xticks(x2)
            ax2.set_xticklabels(list(channel_cr_final_data['Channel']), horizontalalignment="right", fontsize=20)           

            ax2.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

            ax2.bar_label(rects2, padding=5, labels= list(channel_cr_final_data['CR']))
            plt.show()

        elif region_drop_CR.value == 'All':
            final_df = iteration_dataset_all_v2[(iteration_dataset_all_v2['Channel']==channel_drop_CR.value)]
            final_df['CR'] = (final_df['Displaced_Volume']/final_df['New_Product_Volume'])*100
            final_df['CR'] = final_df['CR'].round(decimals=2)
            region_cr_final_data = final_df[['Region','New_Product_Volume','Displaced_Volume']]
            region_cr_final_data['CR'] = (region_cr_final_data['Displaced_Volume']/region_cr_final_data['New_Product_Volume'])*100
            region_cr_final_data['CR'] = region_cr_final_data['CR'].round(decimals=2)
            channel_cr_final_data = final_df.groupby(['Channel']).sum().reset_index()
            channel_cr_final_data['CR'] = (channel_cr_final_data['Displaced_Volume']/channel_cr_final_data['New_Product_Volume'])*100
            channel_cr_final_data['CR'] = channel_cr_final_data['CR'].round(decimals=2)

            ####################### PLOTS ###################
            x1 = np.arange(len(list(region_cr_final_data['Region'])))  # the label locations
            x2 = np.arange(len(list(channel_cr_final_data['Channel'])))  # the label locations
            width = 0.35  # the width of the bars

            fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize = (20,8), tight_layout = True)
            rects1 = ax1.bar(x1 - width/2, list(region_cr_final_data['CR']), width, label='Region')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax1.set_ylabel('Cannibalization Rate', fontsize=20)
            ax1.set_title('Region Cannibalization Rate in %', fontsize=25)
            ax1.set_xticks(x1)
            ax1.set_xticklabels(list(region_cr_final_data['Region']), horizontalalignment="right", fontsize=20)           

            ax1.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

            ax1.bar_label(rects1, padding=5, labels= list(region_cr_final_data['CR']))

            rects2 = ax2.bar(x2 - width/2,  list(channel_cr_final_data['CR']), width, label='Channel')

            # # Add some text for labels, title and custom x-axis tick labels, etc.
            ax2.set_ylabel('Cannibalization Rate', fontsize=20)
            ax2.set_title('Channel Cannibalization Rate in %', fontsize=25)
            ax2.set_xticks(x2)
            ax2.set_xticklabels(list(channel_cr_final_data['Channel']), horizontalalignment="right", fontsize=20)            

            ax2.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

            ax2.bar_label(rects2, padding=5, labels= list(channel_cr_final_data['CR']))
            plt.show()

        elif channel_drop_CR.value == 'All':
            final_df = iteration_dataset_all_v2[(iteration_dataset_all_v2['Region']==region_drop_CR.value)]
            final_df['CR'] = (final_df['Displaced_Volume']/final_df['New_Product_Volume'])*100
            final_df['CR'] = final_df['CR'].round(decimals=2)
            region_cr_final_data = final_df.groupby(['Region']).sum().reset_index()
            region_cr_final_data['CR'] = (region_cr_final_data['Displaced_Volume']/region_cr_final_data['New_Product_Volume'])*100
            region_cr_final_data['CR'] = region_cr_final_data['CR'].round(decimals=2)
            channel_cr_final_data = final_df[['Channel','New_Product_Volume','Displaced_Volume']]
            channel_cr_final_data['CR'] = (channel_cr_final_data['Displaced_Volume']/channel_cr_final_data['New_Product_Volume'])*100
            channel_cr_final_data['CR'] = channel_cr_final_data['CR'].round(decimals=2)
            
            ####################### PLOTS ###################
            x1 = np.arange(len(list(region_cr_final_data['Region'])))  # the label locations
            x2 = np.arange(len(list(channel_cr_final_data['Channel'])))  # the label locations
            width = 0.35  # the width of the bars

            fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize = (20,8), tight_layout = True)
            rects1 = ax1.bar(x1 - width/2, list(region_cr_final_data['CR']), width, label='Region')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax1.set_ylabel('Cannibalization Rate', fontsize=20)
            ax1.set_title('Region Cannibalization Rate in %', fontsize=25)
            ax1.set_xticks(x1)
            ax1.set_xticklabels(list(region_cr_final_data['Region']), horizontalalignment="right", fontsize=20)           

            ax1.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

            ax1.bar_label(rects1, padding=5, labels= list(region_cr_final_data['CR']))

            rects2 = ax2.bar(x2 - width/2,  list(channel_cr_final_data['CR']), width, label='Channel')

            # # Add some text for labels, title and custom x-axis tick labels, etc.
            ax2.set_ylabel('Cannibalization Rate', fontsize=20)
            ax2.set_title('Channel Cannibalization Rate in %', fontsize=25)
            ax2.set_xticks(x2)
            ax2.set_xticklabels(list(channel_cr_final_data['Channel']), horizontalalignment="right", fontsize=20)

            ax2.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

            ax2.bar_label(rects2, padding=5, labels= list(channel_cr_final_data['CR']))
            plt.show()

        else:
            final_df = iteration_dataset_all_v2[(iteration_dataset_all_v2['Region']==region_drop_CR.value)
                           &(iteration_dataset_all_v2['Channel']==channel_drop_CR.value)]
            final_df['CR'] = (final_df['Displaced_Volume']/final_df['New_Product_Volume'])*100
            final_df['CR'] = final_df['CR'].round(decimals=2)
            region_cr_final_data = final_df[['Region','New_Product_Volume','Displaced_Volume']]
            region_cr_final_data['CR'] = (region_cr_final_data['Displaced_Volume']/region_cr_final_data['New_Product_Volume'])*100
            region_cr_final_data['CR'] = region_cr_final_data['CR'].round(decimals=2)
            channel_cr_final_data = final_df[['Channel','New_Product_Volume','Displaced_Volume']]
            channel_cr_final_data['CR'] = (channel_cr_final_data['Displaced_Volume']/channel_cr_final_data['New_Product_Volume'])*100
            channel_cr_final_data['CR'] = channel_cr_final_data['CR'].round(decimals=2)

            ####################### PLOTS ###################
            x1 = np.arange(len(list(region_cr_final_data['Region'])))  # the label locations
            x2 = np.arange(len(list(channel_cr_final_data['Channel'])))  # the label locations
            width = 0.35  # the width of the bars

            fig, (ax1) = plt.subplots(nrows=1, ncols=1, figsize = (8,6))
            rects1 = ax1.bar(x1 - width/2, list(region_cr_final_data['CR']), width, label='Region')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax1.set_ylabel('Cannibalization Rate', fontsize=20)
            ax1.set_title('Region-Channel Cannibalization Rate in %', fontsize=25)
            ax1.set_xticks(x1)
            ax1.set_xticklabels(list(region_cr_final_data['Region']+'-'+channel_cr_final_data['Channel']), horizontalalignment="right", fontsize=20)


            ax1.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

            ax1.bar_label(rects1, labels= list(region_cr_final_data['CR']))
            plt.show()

    interactive_plot = interactive(plotit_CR, R=region_drop_CR, C=channel_drop_CR)
    display(interactive_plot)
    
# In case of 2 new products    
elif len(iter_change_list) == 2:
    
    def plotit_CR(R, C):
        # All regions, all channel
        if((channel_drop_CR.value == 'All') and (region_drop_CR.value == 'All')):
            final_df_1 = iteration_dataset_all_v2[iteration_dataset_all_v2.iter == 1].copy()
            final_df_2 = iteration_dataset_all_v2[iteration_dataset_all_v2.iter == 2].copy()
            final_df_1['CR'] = (final_df_1['Displaced_Volume']/final_df_1['New_Product_Volume'])*100
            final_df_2['CR'] = (final_df_2['Displaced_Volume']/final_df_2['New_Product_Volume'])*100
            final_df_1['CR'] = final_df_1['CR'].round(decimals=2)
            final_df_2['CR'] = final_df_2['CR'].round(decimals=2)
            region_cr_final_data_1 = iteration_dataset_all_v2[iteration_dataset_all_v2.iter == 1].groupby(['Region']).sum().reset_index()
            region_cr_final_data_2 = iteration_dataset_all_v2[iteration_dataset_all_v2.iter == 2].groupby(['Region']).sum().reset_index()
            region_cr_final_data_1['CR'] = (region_cr_final_data_1['Displaced_Volume']/region_cr_final_data_1['New_Product_Volume'])*100
            region_cr_final_data_2['CR'] = (region_cr_final_data_2['Displaced_Volume']/region_cr_final_data_2['New_Product_Volume'])*100
            region_cr_final_data_1['CR'] = region_cr_final_data_1['CR'].round(decimals=2)
            region_cr_final_data_2['CR'] = region_cr_final_data_2['CR'].round(decimals=2)
            channel_cr_final_data_1 = iteration_dataset_all_v2[iteration_dataset_all_v2.iter == 1].groupby(['Channel']).sum().reset_index()
            channel_cr_final_data_2 = iteration_dataset_all_v2[iteration_dataset_all_v2.iter == 2].groupby(['Channel']).sum().reset_index()
            channel_cr_final_data_1['CR'] = (channel_cr_final_data_1['Displaced_Volume']/channel_cr_final_data_1['New_Product_Volume'])*100
            channel_cr_final_data_2['CR'] = (channel_cr_final_data_2['Displaced_Volume']/channel_cr_final_data_2['New_Product_Volume'])*100
            channel_cr_final_data_1['CR'] = channel_cr_final_data_1['CR'].round(decimals=2)
            channel_cr_final_data_2['CR'] = channel_cr_final_data_2['CR'].round(decimals=2)

            channel_cr_final_data_1 = channel_cr_final_data_1.sort_values('Channel')
            channel_cr_final_data_2 = channel_cr_final_data_2.sort_values('Channel')
            
            region_cr_final_data_1 = region_cr_final_data_1.sort_values('Region')
            region_cr_final_data_2 = region_cr_final_data_2.sort_values('Region')

            ####################### PLOTS ################### ITER 1 & ITER 2
            x1 = np.arange(len(list(region_cr_final_data_1['Region'])))  # the label locations
            x2 = np.arange(len(list(channel_cr_final_data_1['Channel'])))  # the label locations
            width = 0.35  # the width of the bars

            fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize = (20,8), tight_layout = True)
            rects1 = ax1.bar(x1 - width/2, list(region_cr_final_data_1['CR']), width, label='New Product 1')
            rects2 = ax1.bar(x1 + width/2, list(region_cr_final_data_2['CR']), width, label='New Product 2')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax1.set_ylabel('Cannibalization Rate', fontsize=20)
            ax1.set_title('Region Cannibalization Rate in %', fontsize=25)
            ax1.set_xticks(x1)
            ax1.set_xticklabels(list(region_cr_final_data_1['Region']), horizontalalignment="right", fontsize=20)
            ax1.legend()

            ax1.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

            ax1.bar_label(rects1, padding=5, labels= list(region_cr_final_data_1['CR']))
            ax1.bar_label(rects2, padding=5, labels= list(region_cr_final_data_2['CR']))

            rects3 = ax2.bar(x2 - width/2,  list(channel_cr_final_data_1['CR']), width, label='New Product 1')
            rects4 = ax2.bar(x2 + width/2,  list(channel_cr_final_data_2['CR']), width, label='New Product 2')

            # # Add some text for labels, title and custom x-axis tick labels, etc.
            ax2.set_ylabel('Cannibalization Rate', fontsize=20)
            ax2.set_title('Channel Cannibalization Rate in %', fontsize=25)
            ax2.set_xticks(x2)
            ax2.set_xticklabels(list(channel_cr_final_data_1['Channel']), horizontalalignment="right", fontsize=20)
            ax2.legend()

            ax2.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

            ax2.bar_label(rects3, padding=5, labels= list(channel_cr_final_data_1['CR']))
            ax2.bar_label(rects4, padding=5, labels= list(channel_cr_final_data_2['CR']))

            plt.show()
            
        # All region, one channel
        elif region_drop_CR.value == 'All':
            final_df_1 = iteration_dataset_all_v2[(iteration_dataset_all_v2.iter == 1) & (iteration_dataset_all_v2['Channel']==channel_drop_CR.value)]
            final_df_2 = iteration_dataset_all_v2[(iteration_dataset_all_v2.iter == 2) & (iteration_dataset_all_v2['Channel']==channel_drop_CR.value)]
            final_df_1['CR'] = (final_df_1['Displaced_Volume']/final_df_1['New_Product_Volume'])*100
            final_df_2['CR'] = (final_df_2['Displaced_Volume']/final_df_1['New_Product_Volume'])*100
            final_df_1['CR'] = final_df_1['CR'].round(decimals=2)
            region_cr_final_data_1 = final_df_1[['Region','New_Product_Volume','Displaced_Volume']]
            region_cr_final_data_2 = final_df_2[['Region','New_Product_Volume','Displaced_Volume']]
            region_cr_final_data_1['CR'] = (region_cr_final_data_1['Displaced_Volume']/region_cr_final_data_1['New_Product_Volume'])*100
            region_cr_final_data_2['CR'] = (region_cr_final_data_2['Displaced_Volume']/region_cr_final_data_2['New_Product_Volume'])*100
            
            region_cr_final_data_1['CR'] = region_cr_final_data_1['CR'].round(decimals=2)
            region_cr_final_data_2['CR'] = region_cr_final_data_2['CR'].round(decimals=2)
            
            channel_cr_final_data_1 = final_df_1.groupby(['Channel']).sum().reset_index()
            channel_cr_final_data_2 = final_df_2.groupby(['Channel']).sum().reset_index()
            
            channel_cr_final_data_1['CR'] = (channel_cr_final_data_1['Displaced_Volume']/channel_cr_final_data_1['New_Product_Volume'])*100
            channel_cr_final_data_2['CR'] = (channel_cr_final_data_2['Displaced_Volume']/channel_cr_final_data_2['New_Product_Volume'])*100
            
            channel_cr_final_data_1['CR'] = channel_cr_final_data_1['CR'].round(decimals=2)
            channel_cr_final_data_2['CR'] = channel_cr_final_data_2['CR'].round(decimals=2)

            channel_cr_final_data_1 = channel_cr_final_data_1.sort_values('Channel')
            channel_cr_final_data_2 = channel_cr_final_data_2.sort_values('Channel')
            
            region_cr_final_data_1 = region_cr_final_data_1.sort_values('Region')
            region_cr_final_data_2 = region_cr_final_data_2.sort_values('Region')
            
            ####################### PLOTS ################### ITER 1 & ITER 2
            x1 = np.arange(len(list(region_cr_final_data_1['Region'])))  # the label locations
            x2 = np.arange(len(list(channel_cr_final_data_1['Channel'])))  # the label locations
            width = 0.35  # the width of the bars

            fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize = (20,8), tight_layout = True)
            rects1 = ax1.bar(x1 - width/2, list(region_cr_final_data_1['CR']), width, label='New Product 1')
            rects2 = ax1.bar(x1 + width/2, list(region_cr_final_data_2['CR']), width, label='New Product 2')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax1.set_ylabel('Cannibalization Rate', fontsize=20)
            ax1.set_title('Region Cannibalization Rate in %', fontsize=25)
            ax1.set_xticks(x1)
            ax1.set_xticklabels(list(region_cr_final_data_1['Region']), horizontalalignment="right", fontsize=20)
            ax1.legend()

            ax1.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

            ax1.bar_label(rects1, padding=5, labels= list(region_cr_final_data_1['CR']))
            ax1.bar_label(rects2, padding=5, labels= list(region_cr_final_data_2['CR']))

            rects3 = ax2.bar(x2 - width/2,  list(channel_cr_final_data_1['CR']), width, label='New Product 1')
            rects4 = ax2.bar(x2 + width/2,  list(channel_cr_final_data_2['CR']), width, label='New Product 2')

            # # Add some text for labels, title and custom x-axis tick labels, etc.
            ax2.set_ylabel('Cannibalization Rate', fontsize=20)
            ax2.set_title('Channel Cannibalization Rate in %', fontsize=25)
            ax2.set_xticks(x2)
            ax2.set_xticklabels(list(channel_cr_final_data_1['Channel']), horizontalalignment="right", fontsize=20)
            ax2.legend()

            ax2.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

            ax2.bar_label(rects3, padding=5, labels= list(channel_cr_final_data_1['CR']))
            ax2.bar_label(rects4, padding=5, labels= list(channel_cr_final_data_2['CR']))

            plt.show()
        
        # All channel one region
        elif channel_drop_CR.value == 'All':
            final_df_1 = iteration_dataset_all_v2[(iteration_dataset_all_v2.iter == 1) & (iteration_dataset_all_v2['Region']==region_drop_CR.value)]
            final_df_1['CR'] = (final_df_1['Displaced_Volume']/final_df_1['New_Product_Volume'])*100
            final_df_1['CR'] = final_df_1['CR'].round(decimals=2)
            region_cr_final_data_1 = final_df_1.groupby(['Region']).sum().reset_index()
            region_cr_final_data_1['CR'] = (region_cr_final_data_1['Displaced_Volume']/region_cr_final_data_1['New_Product_Volume'])*100
            region_cr_final_data_1['CR'] = region_cr_final_data_1['CR'].round(decimals=2)
            region_cr_final_data_1 = region_cr_final_data_1.sort_values('Region')
            channel_cr_final_data_1 = final_df_1[['Channel','New_Product_Volume','Displaced_Volume']]
            channel_cr_final_data_1['CR'] = (channel_cr_final_data_1['Displaced_Volume']/channel_cr_final_data_1['New_Product_Volume'])*100
            channel_cr_final_data_1['CR'] = channel_cr_final_data_1['CR'].round(decimals=2)
            channel_cr_final_data_1 = channel_cr_final_data_1.sort_values('Channel')

            final_df_2 = iteration_dataset_all_v2[(iteration_dataset_all_v2.iter == 2) & (iteration_dataset_all_v2['Region']==region_drop_CR.value)]
            final_df_2['CR'] = (final_df_2['Displaced_Volume']/final_df_2['New_Product_Volume'])*100
            final_df_2['CR'] = final_df_2['CR'].round(decimals=2)
            region_cr_final_data_2 = final_df_2.groupby(['Region']).sum().reset_index()
            region_cr_final_data_2['CR'] = (region_cr_final_data_2['Displaced_Volume']/region_cr_final_data_2['New_Product_Volume'])*100
            region_cr_final_data_2['CR'] = region_cr_final_data_2['CR'].round(decimals=2)
            region_cr_final_data_2 = region_cr_final_data_2.sort_values('Region')
            channel_cr_final_data_2 = final_df_2[['Channel','New_Product_Volume','Displaced_Volume']]
            channel_cr_final_data_2['CR'] = (channel_cr_final_data_2['Displaced_Volume']/channel_cr_final_data_2['New_Product_Volume'])*100
            channel_cr_final_data_2['CR'] = channel_cr_final_data_2['CR'].round(decimals=2)            
            channel_cr_final_data_2 = channel_cr_final_data_2.sort_values('Channel')

            ####################### PLOTS ################### ITER 1 & ITER 2
            x1 = np.arange(len(list(region_cr_final_data_1['Region'])))  # the label locations
            x2 = np.arange(len(list(channel_cr_final_data_1['Channel'])))  # the label locations
            width = 0.35  # the width of the bars

            fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize = (20,8), tight_layout = True)
            rects1 = ax1.bar(x1 - width/2, list(region_cr_final_data_1['CR']), width, label='New Product 1')
            rects2 = ax1.bar(x1 + width/2, list(region_cr_final_data_2['CR']), width, label='New Product 2')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax1.set_ylabel('Cannibalization Rate', fontsize=20)
            ax1.set_title('Region Cannibalization Rate in %', fontsize=25)
            ax1.set_xticks(x1)
            ax1.set_xticklabels(list(region_cr_final_data_1['Region']), horizontalalignment="right", fontsize=20)
            ax1.legend()

            ax1.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

            ax1.bar_label(rects1, padding=5, labels= list(region_cr_final_data_1['CR']))
            ax1.bar_label(rects2, padding=5, labels= list(region_cr_final_data_2['CR']))

            rects3 = ax2.bar(x2 - width/2,  list(channel_cr_final_data_1['CR']), width, label='New Product 1')
            rects4 = ax2.bar(x2 + width/2,  list(channel_cr_final_data_2['CR']), width, label='New Product 2')

            # # Add some text for labels, title and custom x-axis tick labels, etc.
            ax2.set_ylabel('Cannibalization Rate', fontsize=20)
            ax2.set_title('Channel Cannibalization Rate in %', fontsize=25)
            ax2.set_xticks(x2)
            ax2.set_xticklabels(list(channel_cr_final_data_1['Channel']), horizontalalignment="right", fontsize=20)
            ax2.legend()

            ax2.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

            ax2.bar_label(rects3, padding=5, labels= list(channel_cr_final_data_1['CR']))
            ax2.bar_label(rects4, padding=5, labels= list(channel_cr_final_data_2['CR']))

            plt.show()
        
        # One region, one channel
        else:
            final_df_1 = iteration_dataset_all_v2[(iteration_dataset_all_v2.iter == 1) & (iteration_dataset_all_v2['Region']==region_drop_CR.value)
                           &(iteration_dataset_all_v2['Channel']==channel_drop_CR.value)]
            final_df_2 = iteration_dataset_all_v2[(iteration_dataset_all_v2.iter == 2) & (iteration_dataset_all_v2['Region']==region_drop_CR.value)
                           &(iteration_dataset_all_v2['Channel']==channel_drop_CR.value)]
            final_df_1['CR'] = (final_df_1['Displaced_Volume']/final_df_1['New_Product_Volume'])*100
            final_df_1['CR'] = final_df_1['CR'].round(decimals=2)
            region_cr_final_data_1 = final_df_1[['Region','New_Product_Volume','Displaced_Volume']]
            region_cr_final_data_1['CR'] = (region_cr_final_data_1['Displaced_Volume']/region_cr_final_data_1['New_Product_Volume'])*100
            region_cr_final_data_1['CR'] = region_cr_final_data_1['CR'].round(decimals=2)
            channel_cr_final_data_1 = final_df_1[['Channel','New_Product_Volume','Displaced_Volume']]
            channel_cr_final_data_1['CR'] = (channel_cr_final_data_1['Displaced_Volume']/channel_cr_final_data_1['New_Product_Volume'])*100
            channel_cr_final_data_1['CR'] = channel_cr_final_data_1['CR'].round(decimals=2)

            final_df_2['CR'] = (final_df_2['Displaced_Volume']/final_df_2['New_Product_Volume'])*100
            final_df_2['CR'] = final_df_2['CR'].round(decimals=2)
            region_cr_final_data_2 = final_df_2[['Region','New_Product_Volume','Displaced_Volume']]
            region_cr_final_data_2['CR'] = (region_cr_final_data_2['Displaced_Volume']/region_cr_final_data_2['New_Product_Volume'])*100
            region_cr_final_data_2['CR'] = region_cr_final_data_2['CR'].round(decimals=2)
            channel_cr_final_data_2 = final_df_2[['Channel','New_Product_Volume','Displaced_Volume']]
            channel_cr_final_data_2['CR'] = (channel_cr_final_data_2['Displaced_Volume']/channel_cr_final_data_2['New_Product_Volume'])*100
            channel_cr_final_data_2['CR'] = channel_cr_final_data_2['CR'].round(decimals=2)
            
            channel_cr_final_data_1 = channel_cr_final_data_1.sort_values('Channel')
            channel_cr_final_data_2 = channel_cr_final_data_2.sort_values('Channel')
            
            region_cr_final_data_1 = region_cr_final_data_1.sort_values('Region')
            region_cr_final_data_2 = region_cr_final_data_2.sort_values('Region')
            
            ####################### PLOTS ################### ITER 1 & ITER 2
            x1 = np.arange(len(list(region_cr_final_data_1['Region'])))  # the label locations
            x2 = np.arange(len(list(channel_cr_final_data_1['Channel'])))  # the label locations
            width = 0.35  # the width of the bars

            fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize = (20,8), tight_layout = True)
            rects1 = ax1.bar(x1 - width/2, list(region_cr_final_data_1['CR']), width, label='New Product 1')
            rects2 = ax1.bar(x1 + width/2, list(region_cr_final_data_2['CR']), width, label='New Product 2')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax1.set_ylabel('Cannibalization Rate', fontsize=20)
            ax1.set_title('Region Cannibalization Rate in %', fontsize=25)
            ax1.set_xticks(x1)
            ax1.set_xticklabels(list(region_cr_final_data_1['Region']), horizontalalignment="right", fontsize=20)
            ax1.legend()

            ax1.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

            ax1.bar_label(rects1, padding=5, labels= list(region_cr_final_data_1['CR']))
            ax1.bar_label(rects2, padding=5, labels= list(region_cr_final_data_2['CR']))

            rects3 = ax2.bar(x2 - width/2,  list(channel_cr_final_data_1['CR']), width, label='New Product 1')
            rects4 = ax2.bar(x2 + width/2,  list(channel_cr_final_data_2['CR']), width, label='New Product 2')

            # # Add some text for labels, title and custom x-axis tick labels, etc.
            ax2.set_ylabel('Cannibalization Rate', fontsize=20)
            ax2.set_title('Channel Cannibalization Rate in', fontsize=25)
            ax2.set_xticks(x2)
            ax2.set_xticklabels(list(channel_cr_final_data_1['Channel']), horizontalalignment="right", fontsize=20)
            ax2.legend()

            ax2.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

            ax2.bar_label(rects3, padding=5, labels= list(channel_cr_final_data_1['CR']))
            ax2.bar_label(rects4, padding=5, labels= list(channel_cr_final_data_2['CR']))

            plt.show()
            

    interactive_plot = interactive(plotit_CR, R=region_drop_CR, C=channel_drop_CR)
    display(interactive_plot)


__Net Gain due to Promotion of New Product__

In [216]:
# Product_fil assigned as most similar product in ensemble approach
if toggle_ensemble_normal.value == 'Multiple Product': # In case of ensemble approach    
    
    if len(prod_ensemble_list) != 3: # if 3 products are not selected
        clear_output()
        print('Select 3 products as existing model pack')
    else:
        score_data_cann = score_data_cann[score_data_cann.Product.isin(prod_ensemble_list)].sort_values('Similarity_Score', ascending = False)
        product_fil = score_data_cann.loc[:,'Product'][:1].values[0]


# In case the user doesn't upload the data or is not an ensemle approach
if (toggle_upload.value != 'Yes'):
    # In case no product is present in selected region-channel
    if len(qtr_edv_baseline) != 0 :
        display(Markdown('<div><div class="loader"></div><h2> &nbsp;Measuring Net Gain due to Promotions/Discounts</h2></div>'))

        #Defining empty dataframes for model training
        combined_dataset = pd.DataFrame({'Week': pd.Series([], dtype='object')})
        Test_results_net = pd.DataFrame({'Region': pd.Series([], dtype='object')})
        Test_results_net_cann = pd.DataFrame({'Region': pd.Series([], dtype='object')})
        Test_Data = pd.DataFrame()

        # Defining test week list
        Test_week_list = {"2019Q1" : ["2019-01","2019-02","2019-03","2019-04","2019-05","2019-06","2019-07","2019-08","2019-09","2019-10","2019-11","2019-12","2019-13"],
             "2019Q2" : ["2019-14","2019-15","2019-16","2019-17","2019-18","2019-19","2019-20","2019-21","2019-22","2019-23","2019-24","2019-25","2019-26"],
             "2019Q3" : ["2019-27","2019-28","2019-29","2019-30","2019-31","2019-32","2019-33","2019-34","2019-35","2019-36","2019-37","2019-38","2019-39"],
             "2019Q4" : ["2019-40","2019-41","2019-42","2019-43","2019-44","2019-45","2019-46","2019-47","2019-48","2019-49","2019-50","2019-51","2019-52"],

        # CHECK : ADDING 2020 weeks                  
             "2020Q1" : ["2020-01","2020-02","2020-03","2020-04","2020-05","2020-06","2020-07","2020-08","2020-09","2020-10","2020-11","2020-12","2020-13"],
             "2020Q2" : ["2020-14","2020-15","2020-16","2020-17","2020-18","2020-19","2020-20","2020-21","2020-22","2020-23","2020-24","2020-25","2020-26"],
             "2020Q3" : ["2020-27","2020-28","2020-29","2020-30","2020-31","2020-32","2020-33","2020-34","2020-35","2020-36","2020-37","2020-38","2020-39"],
             "2020Q4" : ["2020-40","2020-41","2020-42","2020-43","2020-44","2020-45","2020-46","2020-47","2020-48","2020-49","2020-50","2020-51","2020-52"],
             "fulltrain" : []}

        if toggle_np2.value == 'Yes':
            iter_change_list = [1,2]
        else:
            iter_change_list = [1]

        for i in iter_change_list:
                        
            #Model training and prediction
            for Region_key in Region_List:
        
                Volume_dataset = Volume_dataset_all_reg.loc[(Volume_dataset_all_reg["Region"] == Region_key)
                                                           &(Volume_dataset_all_reg["Channel"].isin(channel_list))]
                Volume_dataset['Pantry2'] = Volume_dataset['Pantry2'].fillna(0)
                brand_mapping = Volume_dataset_all_reg[(Volume_dataset_all_reg["Region"] == Region_key)
                                                      &(Volume_dataset_all_reg["Channel"].isin(channel_list))][['Banner','Product','brand',"Category","Pack.Subtype","Brand"]].drop_duplicates()
                sel_brands = Volume_dataset_all_reg.loc[((Volume_dataset_all_reg["Week"]> "2019-01")
                                                        &(Volume_dataset_all_reg["Region"]== Region_key)
                                                        &(Volume_dataset_all_reg["Channel"].isin(channel_list))
                                                        &(Volume_dataset_all_reg["Eq.Unit.Sales"].notnull())), "brand"].unique()            

                #################### ALL REGION COMBINED DATASET ####################
                
                if(Region_key == 'EAST'):
                    combined_dataset_adcal_temp = combined_dataset_adcal_temp_EAST.copy()
                    combined_dataset_edv_temp = combined_dataset_edv_temp_EAST.copy()
                    combined_dataset_other_promo_temp = combined_dataset_other_promo_temp_EAST.copy()
                    combined_dataset_self_promo_temp = combined_dataset_self_promo_temp_EAST.copy()

                elif(Region_key == 'ONTARIO'):
                    combined_dataset_adcal_temp = combined_dataset_adcal_temp_ONTARIO.copy()
                    combined_dataset_edv_temp = combined_dataset_edv_temp_ONTARIO.copy()
                    combined_dataset_other_promo_temp = combined_dataset_other_promo_temp_ONTARIO.copy()
                    combined_dataset_self_promo_temp = combined_dataset_self_promo_temp_ONTARIO.copy()

                elif(Region_key == 'QUEBEC'):
                    combined_dataset_adcal_temp = combined_dataset_adcal_temp_QUEBEC.copy()
                    combined_dataset_edv_temp = combined_dataset_edv_temp_QUEBEC.copy()
                    combined_dataset_other_promo_temp = combined_dataset_other_promo_temp_QUEBEC.copy()
                    combined_dataset_self_promo_temp = combined_dataset_self_promo_temp_QUEBEC.copy()

                else:
                    combined_dataset_adcal_temp = combined_dataset_adcal_temp_WEST.copy()
                    combined_dataset_edv_temp = combined_dataset_edv_temp_WEST.copy()
                    combined_dataset_other_promo_temp = combined_dataset_other_promo_temp_WEST.copy()
                    combined_dataset_self_promo_temp = combined_dataset_self_promo_temp_WEST.copy()
    
                # Get category columns
                combined_dataset_adcal_temp = combined_dataset_adcal_temp.merge(Volume_dataset_all_reg[['Product','Category']].drop_duplicates(), how = 'left')
                combined_dataset_edv_temp = combined_dataset_edv_temp.merge(Volume_dataset_all_reg[['Product','Category']].drop_duplicates(), how = 'left')
                combined_dataset_other_promo_temp = combined_dataset_other_promo_temp.merge(Volume_dataset_all_reg[['Product','Category']].drop_duplicates(), how = 'left')
                combined_dataset_self_promo_temp = combined_dataset_self_promo_temp.merge(Volume_dataset_all_reg[['Product','Category']].drop_duplicates(), how = 'left')

                # 4 scenarios
                combined_dataset_adcal = combined_dataset_adcal_temp.copy()
                combined_dataset_edv = combined_dataset_edv_temp[combined_dataset_edv_temp.Product == product_fil].copy()
                combined_dataset_other_promo = combined_dataset_other_promo_temp[combined_dataset_other_promo_temp.Product == product_fil].copy()
                combined_dataset_self_promo = combined_dataset_self_promo_temp[combined_dataset_self_promo_temp.Product == product_fil].copy()
                
            #######################################################################################################################
                                        # 1. Adcal promo Data
            #######################################################################################################################

                banner_dummies = pd.get_dummies(combined_dataset_adcal.Banner)
                banner_dummies_common = banner_dummies.copy()
                combined_dataset_adcal = pd.concat([combined_dataset_adcal, banner_dummies], axis=1)

                ## Not adding Product dummies since for New Product Simulator other Product dummies are not significant

                combined_dataset_adcal = combined_dataset_adcal.loc[combined_dataset_adcal["Eq.Unit.Sales"].notnull()]

                combined_dataset_adcal.sort_values('Week',inplace = True)
                combined_dataset_adcal = pd.merge(combined_dataset_adcal,new_prod_stage,on = ['Product','Week'],how = 'left')
                combined_dataset_adcal['Intial_weeks'] = combined_dataset_adcal['Intial_weeks'].fillna('Stabilization')
                combined_dataset_adcal.drop(columns='Unnamed: 0', inplace=True)

                #Adding dummies for category
                category_dummies = pd.get_dummies(combined_dataset_adcal['Category'])
                category_dummies_common = category_dummies.copy()
                combined_dataset_adcal = pd.concat([combined_dataset_adcal, category_dummies], axis=1)
                combined_dataset_adcal.drop(["Category"], inplace=True, axis=1)

                #Adding dummies for product attribute - pack subtype
                pack_subtypes_dummies = pd.get_dummies(combined_dataset_adcal['Pack.Subtype'])
                pack_subtypes_dummies_common = pack_subtypes_dummies.copy()
                combined_dataset_adcal = pd.concat([combined_dataset_adcal, pack_subtypes_dummies], axis=1)
                combined_dataset_adcal.drop(["Pack.Subtype"], inplace=True, axis=1)

                # Adding pack content dummies
                pack_content_dummies = pd.get_dummies(combined_dataset_adcal['PACK_CONTENT'])
                pack_content_dummies_common = pack_content_dummies.copy()
                combined_dataset_adcal = pd.concat([combined_dataset_adcal, pack_content_dummies], axis=1)
                combined_dataset_adcal.drop(["PACK_CONTENT"], inplace=True, axis=1)

                combined_dataset_adcal_test = combined_dataset_adcal.copy()

                # Product stage dummies
                stage_dummies = pd.get_dummies(combined_dataset_adcal['Intial_weeks'])
                combined_dataset_adcal = pd.concat([combined_dataset_adcal, stage_dummies], axis=1)
                combined_dataset_adcal.drop(["Intial_weeks"], inplace=True, axis=1)

                # Changing product stage as per start date provided by the user
                prod_stage_data = prod_stage_check(product_fil)            

                # Getting updated product stages
                combined_dataset_test_check = combined_dataset_adcal_test[combined_dataset_adcal_test.Product == product_fil].merge(prod_stage_data[['Week','Initial_weeks1']], how = 'left')
                combined_dataset_test_check['Initial_weeks1'] = combined_dataset_test_check['Initial_weeks1'].fillna(combined_dataset_test_check['Intial_weeks'])
                combined_dataset_test_check.drop(columns = {'Intial_weeks'}, inplace = True)
                combined_dataset_test_check.rename(columns = {'Initial_weeks1':'Intial_weeks'}, inplace = True)

                combined_dataset_adcal_test = combined_dataset_test_check.copy()

                # Product stage dummies
                stage_dummies_test = pd.get_dummies(combined_dataset_adcal_test['Intial_weeks'])

                # If all product stages are not present for the uploaded data
                c = [i for i in list(stage_dummies.columns) if i not in list(stage_dummies_test.columns)]
                if len(c) != 0 :
                    stage_dummies_test[c] = 0

                # Reordering columns                
                stage_dummies_test = stage_dummies_test.reindex(sorted(stage_dummies_test.columns), axis=1)
                combined_dataset_adcal_test = pd.concat([combined_dataset_adcal_test, stage_dummies_test], axis=1)
                combined_dataset_adcal_test.drop(["Intial_weeks"], inplace=True, axis=1)

                # In case the user doesn't want to upload adcal data
                if toggle_upload.value != 'Yes':
                    # CHANGES : For adcal discounts
                    if 'All' in list(test_period.value):
                        # Discount change
                        if i == 1:                
                            combined_dataset_adcal_test['Adcal_DD'] = (1+disc_change_fil)*combined_dataset_adcal_test['Adcal_DD']
                            combined_dataset_adcal_test.loc[:,'EDV.Price'] = (1+base_price_change_fil)*combined_dataset_adcal_test.loc[:,'EDV.Price']
                        elif i == 2:
                            combined_dataset_adcal_test['Adcal_DD'] = (1+disc_change_fil2)*combined_dataset_adcal_test['Adcal_DD']
                            combined_dataset_adcal_test.loc[:,'EDV.Price'] = (1+base_price_change_fil2)*combined_dataset_adcal_test.loc[:,'EDV.Price']          
                    else:                
                        if i == 1:                
                            # Discount change
                            combined_dataset_adcal_test.loc[combined_dataset_adcal_test['Test_period'].isin(test_period_fil),'Adcal_DD'] = (1+disc_change_fil)*combined_dataset_adcal_test[combined_dataset_adcal_test['Test_period'].isin(test_period_fil)]['Adcal_DD'] 
                            # Baseline pricing
                            combined_dataset_adcal_test.loc[(combined_dataset_adcal_test['Test_period'].isin(test_period_fil)),'EDV.Price'] = (1+base_price_change_fil)*combined_dataset_adcal_test[(combined_dataset_adcal_test['Test_period'].isin(test_period_fil))]['EDV.Price']
                        elif i == 2:
                            # Discount change
                            combined_dataset_adcal_test.loc[combined_dataset_adcal_test['Test_period'].isin(test_period_fil2),'Adcal_DD'] = (1+disc_change_fil2)*combined_dataset_adcal_test[combined_dataset_adcal_test['Test_period'].isin(test_period_fil2)]['Adcal_DD']             
                            # Baseline pricing
                            combined_dataset_adcal_test.loc[(combined_dataset_adcal_test['Test_period'].isin(test_period_fil2)),'EDV.Price'] = (1+base_price_change_fil2)*combined_dataset_adcal_test[(combined_dataset_adcal_test['Test_period'].isin(test_period_fil2))]['EDV.Price']

                #Adding discount depth columns
                combined_dataset_adcal["DD_1"] = [1 if (0.25> x >0.1)
                                      else 0 for x in combined_dataset_adcal["Adcal_DD"]]
                combined_dataset_adcal["DD_2"] = [1 if (x>=0.25)
                                                 else 0 for x in combined_dataset_adcal["Adcal_DD"]]

                combined_dataset_adcal_test["DD_1"] = [1 if (0.25> x >0.1)
                                      else 0 for x in combined_dataset_adcal_test["Adcal_DD"]]
                combined_dataset_adcal_test["DD_2"] = [1 if (x>=0.25)
                                                 else 0 for x in combined_dataset_adcal_test["Adcal_DD"]]            

                # Adcal Price calculation
                # In case the discounts go over 100%, limit to 100%
                combined_dataset_adcal_test.loc[combined_dataset_adcal_test.Adcal_DD >= 1, 'Adcal_DD'] = 0.99
                combined_dataset_adcal_test.loc[combined_dataset_adcal_test['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset_adcal_test.loc[combined_dataset_adcal_test['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset_adcal_test.loc[combined_dataset_adcal_test['Adcal_DD']>=0.1,'EDV.Price']
                # Adcal_DD < 10 then Adcal Price = EDV Price            
                combined_dataset_adcal_test.loc[combined_dataset_adcal_test['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset_adcal_test.loc[combined_dataset_adcal_test['Adcal_DD'] < 0.1, 'EDV.Price']

                # Adcal Price calculation
                combined_dataset_adcal.loc[combined_dataset_adcal['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset_adcal.loc[combined_dataset_adcal['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset_adcal.loc[combined_dataset_adcal['Adcal_DD']>=0.1,'EDV.Price']
                # Adcal_DD < 10 then Adcal Price = EDV Price            
                combined_dataset_adcal.loc[combined_dataset_adcal['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset_adcal.loc[combined_dataset_adcal['Adcal_DD'] < 0.1, 'EDV.Price']

                # Dropping adcal DD as DD1 & DD2 are added
                combined_dataset_adcal.drop(["Adcal_DD"], inplace=True, axis=1)
                combined_dataset_adcal_test.drop(["Adcal_DD"], inplace=True, axis=1)

                # Dropping null volumes
                complete_dataset_adcal_test = combined_dataset_adcal_test.copy()
                complete_dataset_adcal_test = complete_dataset_adcal_test.loc[complete_dataset_adcal_test["Eq.Unit.Sales"].notnull()]

                complete_dataset_adcal = combined_dataset_adcal.copy()
                complete_dataset_adcal = complete_dataset_adcal.loc[complete_dataset_adcal["Eq.Unit.Sales"].notnull()]

                complete_dataset_adcal = complete_dataset_adcal.loc[((complete_dataset_adcal["Week"] >= "2017-01") & 
                                                         (complete_dataset_adcal["Week"] <= "2019-52"))]

                complete_dataset_adcal_test = complete_dataset_adcal_test.loc[((complete_dataset_adcal_test["Week"] >= "2017-01") & 
                                                         (complete_dataset_adcal_test["Week"] <= "2019-52"))]

                complete_dataset_adcal['Adcal_Price'] = np.log(complete_dataset_adcal['Adcal_Price'])              

                complete_dataset_adcal_test['Adcal_Price'] = np.log(complete_dataset_adcal_test['Adcal_Price'])
                complete_dataset_adcal_test.sort_values(by=['Banner','Product','Week'], inplace=True)           


                #######################################################################################################################
                                        # 2. EDV promo Data
                #######################################################################################################################

                banner_dummies = pd.get_dummies(combined_dataset_edv.Banner)
                # If all category dummies are not present for the uploaded data
                c = [i for i in list(banner_dummies_common.columns) if i not in list(banner_dummies.columns)]
                if len(c) != 0 :
                    banner_dummies[c] = 0

                # Reordering columns                
                banner_dummies = banner_dummies.reindex(sorted(banner_dummies.columns), axis=1)
                combined_dataset_edv = pd.concat([combined_dataset_edv, banner_dummies], axis=1)

                ## Not adding Product dummies since for New Product Simulator other Product dummies are not significant

                combined_dataset_edv = combined_dataset_edv.loc[combined_dataset_edv["Eq.Unit.Sales"].notnull()]

                combined_dataset_edv.sort_values('Week',inplace = True)
                combined_dataset_edv = pd.merge(combined_dataset_edv,new_prod_stage,on = ['Product','Week'],how = 'left')
                combined_dataset_edv['Intial_weeks'] = combined_dataset_edv['Intial_weeks'].fillna('Stabilization')
                combined_dataset_edv.drop(columns='Unnamed: 0', inplace=True)

                #Adding dummies for category
                category_dummies = pd.get_dummies(combined_dataset_edv['Category'])

                # If all category dummies are not present for the uploaded data
                c = [i for i in list(category_dummies_common.columns) if i not in list(category_dummies.columns)]
                if len(c) != 0 :
                    category_dummies[c] = 0

                # Reordering columns                
                category_dummies = category_dummies.reindex(sorted(category_dummies.columns), axis=1)
                combined_dataset_edv = pd.concat([combined_dataset_edv, category_dummies], axis=1)
                combined_dataset_edv.drop(["Category"], inplace=True, axis=1)

                #Adding dummies for product attribute - pack subtype
                pack_subtypes_dummies = pd.get_dummies(combined_dataset_edv['Pack.Subtype'])

                # If all pack subtype dummies are not present for the uploaded data
                c = [i for i in list(pack_subtypes_dummies_common.columns) if i not in list(pack_subtypes_dummies.columns)]
                if len(c) != 0 :
                    pack_subtypes_dummies[c] = 0

                # Reordering columns                
                pack_subtypes_dummies = pack_subtypes_dummies.reindex(sorted(pack_subtypes_dummies.columns), axis=1)
                combined_dataset_edv = pd.concat([combined_dataset_edv, pack_subtypes_dummies], axis=1)
                combined_dataset_edv.drop(["Pack.Subtype"], inplace=True, axis=1)

                # Adding pack content dummies    
                pack_content_dummies = pd.get_dummies(combined_dataset_edv['PACK_CONTENT'])

                # If all pack content dummies are not present for the uploaded data
                c = [i for i in list(pack_content_dummies_common.columns) if i not in list(pack_content_dummies.columns)]
                if len(c) != 0 :
                    pack_content_dummies[c] = 0

                # Reordering columns                
                pack_content_dummies = pack_content_dummies.reindex(sorted(pack_content_dummies.columns), axis=1)
                combined_dataset_edv = pd.concat([combined_dataset_edv, pack_content_dummies], axis=1)
                combined_dataset_edv.drop(["PACK_CONTENT"], inplace=True, axis=1)

                # Changing product stage as per start date provided by the user
                prod_stage_data = prod_stage_check(product_fil)            

                # Getting updated product stages
                combined_dataset_test_check = combined_dataset_edv[combined_dataset_edv.Product == product_fil].merge(prod_stage_data[['Week','Initial_weeks1']], how = 'left')
                combined_dataset_test_check['Initial_weeks1'] = combined_dataset_test_check['Initial_weeks1'].fillna(combined_dataset_test_check['Intial_weeks'])
                combined_dataset_test_check.drop(columns = {'Intial_weeks'}, inplace = True)
                combined_dataset_test_check.rename(columns = {'Initial_weeks1':'Intial_weeks'}, inplace = True)

                combined_dataset_edv = combined_dataset_test_check.copy()

                # Product stage dummies
                stage_dummies_test = pd.get_dummies(combined_dataset_edv['Intial_weeks'])

                # If all product stages are not present for the uploaded data
                c = [i for i in list(stage_dummies.columns) if i not in list(stage_dummies_test.columns)]
                if len(c) != 0 :
                    stage_dummies_test[c] = 0

                # Reordering columns                
                stage_dummies_test = stage_dummies_test.reindex(sorted(stage_dummies_test.columns), axis=1)
                combined_dataset_edv = pd.concat([combined_dataset_edv, stage_dummies_test], axis=1)
                combined_dataset_edv.drop(["Intial_weeks"], inplace=True, axis=1)

                # Dropping adcal DD as DD1 & DD2 are added
                combined_dataset_edv.drop(["Adcal_DD"], inplace=True, axis=1)

                complete_dataset_edv = combined_dataset_edv.copy()
                complete_dataset_edv = complete_dataset_edv.loc[complete_dataset_edv["Eq.Unit.Sales"].notnull()]

                complete_dataset_edv = complete_dataset_edv.loc[((complete_dataset_edv["Week"] >= "2017-01") & 
                                                         (complete_dataset_edv["Week"] <= "2019-52"))]
                complete_dataset_edv.sort_values(by=['Banner','Product','Week'], inplace=True)

                #######################################################################################################################
                                        # 3. Other promo Data
                #######################################################################################################################

                banner_dummies = pd.get_dummies(combined_dataset_other_promo.Banner)
                # If all category dummies are not present for the uploaded data
                c = [i for i in list(banner_dummies_common.columns) if i not in list(banner_dummies.columns)]
                if len(c) != 0 :
                    banner_dummies[c] = 0

                # Reordering columns                
                banner_dummies = banner_dummies.reindex(sorted(banner_dummies.columns), axis=1)
                combined_dataset_other_promo = pd.concat([combined_dataset_other_promo, banner_dummies], axis=1)

                ## Not adding Product dummies since for New Product Simulator other Product dummies are not significant

                combined_dataset_other_promo = combined_dataset_other_promo.loc[combined_dataset_other_promo["Eq.Unit.Sales"].notnull()]

                combined_dataset_other_promo.sort_values('Week',inplace = True)
                combined_dataset_other_promo = pd.merge(combined_dataset_other_promo,new_prod_stage,on = ['Product','Week'],how = 'left')
                combined_dataset_other_promo['Intial_weeks'] = combined_dataset_other_promo['Intial_weeks'].fillna('Stabilization')
                combined_dataset_other_promo.drop(columns='Unnamed: 0', inplace=True)

                #Adding dummies for category
                category_dummies = pd.get_dummies(combined_dataset_other_promo['Category'])

                # If all category dummies are not present for the uploaded data
                c = [i for i in list(category_dummies_common.columns) if i not in list(category_dummies.columns)]
                if len(c) != 0 :
                    category_dummies[c] = 0

                # Reordering columns                
                category_dummies = category_dummies.reindex(sorted(category_dummies.columns), axis=1)
                combined_dataset_other_promo = pd.concat([combined_dataset_other_promo, category_dummies], axis=1)
                combined_dataset_other_promo.drop(["Category"], inplace=True, axis=1)

                #Adding dummies for product attribute - pack subtype
                pack_subtypes_dummies = pd.get_dummies(combined_dataset_other_promo['Pack.Subtype'])

                # If all pack subtype dummies are not present for the uploaded data
                c = [i for i in list(pack_subtypes_dummies_common.columns) if i not in list(pack_subtypes_dummies.columns)]
                if len(c) != 0 :
                    pack_subtypes_dummies[c] = 0

                # Reordering columns                
                pack_subtypes_dummies = pack_subtypes_dummies.reindex(sorted(pack_subtypes_dummies.columns), axis=1)
                combined_dataset_other_promo = pd.concat([combined_dataset_other_promo, pack_subtypes_dummies], axis=1)
                combined_dataset_other_promo.drop(["Pack.Subtype"], inplace=True, axis=1)

                # Adding pack content dummies    
                pack_content_dummies = pd.get_dummies(combined_dataset_other_promo['PACK_CONTENT'])

                # If all pack content dummies are not present for the uploaded data
                c = [i for i in list(pack_content_dummies_common.columns) if i not in list(pack_content_dummies.columns)]
                if len(c) != 0 :
                    pack_content_dummies[c] = 0

                # Reordering columns                
                pack_content_dummies = pack_content_dummies.reindex(sorted(pack_content_dummies.columns), axis=1)
                combined_dataset_other_promo = pd.concat([combined_dataset_other_promo, pack_content_dummies], axis=1)
                combined_dataset_other_promo.drop(["PACK_CONTENT"], inplace=True, axis=1)

                # Changing product stage as per start date provided by the user
                prod_stage_data = prod_stage_check(product_fil)            

                # Getting updated product stages
                combined_dataset_test_check = combined_dataset_other_promo[combined_dataset_other_promo.Product == product_fil].merge(prod_stage_data[['Week','Initial_weeks1']], how = 'left')
                combined_dataset_test_check['Initial_weeks1'] = combined_dataset_test_check['Initial_weeks1'].fillna(combined_dataset_test_check['Intial_weeks'])
                combined_dataset_test_check.drop(columns = {'Intial_weeks'}, inplace = True)
                combined_dataset_test_check.rename(columns = {'Initial_weeks1':'Intial_weeks'}, inplace = True)

                combined_dataset_other_promo = combined_dataset_test_check.copy()

                # Product stage dummies
                stage_dummies_test = pd.get_dummies(combined_dataset_other_promo['Intial_weeks'])

                # If all product stages are not present for the uploaded data
                c = [i for i in list(stage_dummies.columns) if i not in list(stage_dummies_test.columns)]
                if len(c) != 0 :
                    stage_dummies_test[c] = 0

                # Reordering columns                
                stage_dummies_test = stage_dummies_test.reindex(sorted(stage_dummies_test.columns), axis=1)
                combined_dataset_other_promo = pd.concat([combined_dataset_other_promo, stage_dummies_test], axis=1)
                combined_dataset_other_promo.drop(["Intial_weeks"], inplace=True, axis=1)

                # Dropping adcal DD as DD1 & DD2 are added
                combined_dataset_other_promo.drop(["Adcal_DD"], inplace=True, axis=1)

                complete_dataset_other_promo = combined_dataset_other_promo.copy()
                complete_dataset_other_promo = complete_dataset_other_promo.loc[complete_dataset_other_promo["Eq.Unit.Sales"].notnull()]

                complete_dataset_other_promo = complete_dataset_other_promo.loc[((complete_dataset_other_promo["Week"] >= "2017-01") & 
                                                         (complete_dataset_other_promo["Week"] <= "2019-52"))]
                complete_dataset_other_promo.sort_values(by=['Banner','Product','Week'], inplace=True)

                #######################################################################################################################
                                        # 4. Self promo Data
                #######################################################################################################################

                banner_dummies = pd.get_dummies(combined_dataset_self_promo.Banner)
                # If all category dummies are not present for the uploaded data
                c = [i for i in list(banner_dummies_common.columns) if i not in list(banner_dummies.columns)]
                if len(c) != 0 :
                    banner_dummies[c] = 0

                # Reordering columns                
                banner_dummies = banner_dummies.reindex(sorted(banner_dummies.columns), axis=1)
                combined_dataset_self_promo = pd.concat([combined_dataset_self_promo, banner_dummies], axis=1)

                ## Not adding Product dummies since for New Product Simulator other Product dummies are not significant

                combined_dataset_self_promo = combined_dataset_self_promo.loc[combined_dataset_self_promo["Eq.Unit.Sales"].notnull()]

                combined_dataset_self_promo.sort_values('Week',inplace = True)
                combined_dataset_self_promo = pd.merge(combined_dataset_self_promo,new_prod_stage,on = ['Product','Week'],how = 'left')
                combined_dataset_self_promo['Intial_weeks'] = combined_dataset_self_promo['Intial_weeks'].fillna('Stabilization')
                combined_dataset_self_promo.drop(columns='Unnamed: 0', inplace=True)

                #Adding dummies for category
                category_dummies = pd.get_dummies(combined_dataset_self_promo['Category'])

                # If all category dummies are not present for the uploaded data
                c = [i for i in list(category_dummies_common.columns) if i not in list(category_dummies.columns)]
                if len(c) != 0 :
                    category_dummies[c] = 0

                # Reordering columns                
                category_dummies = category_dummies.reindex(sorted(category_dummies.columns), axis=1)
                combined_dataset_self_promo = pd.concat([combined_dataset_self_promo, category_dummies], axis=1)
                combined_dataset_self_promo.drop(["Category"], inplace=True, axis=1)

                #Adding dummies for product attribute - pack subtype
                pack_subtypes_dummies = pd.get_dummies(combined_dataset_self_promo['Pack.Subtype'])

                # If all pack subtype dummies are not present for the uploaded data
                c = [i for i in list(pack_subtypes_dummies_common.columns) if i not in list(pack_subtypes_dummies.columns)]
                if len(c) != 0 :
                    pack_subtypes_dummies[c] = 0

                # Reordering columns                
                pack_subtypes_dummies = pack_subtypes_dummies.reindex(sorted(pack_subtypes_dummies.columns), axis=1)
                combined_dataset_self_promo = pd.concat([combined_dataset_self_promo, pack_subtypes_dummies], axis=1)
                combined_dataset_self_promo.drop(["Pack.Subtype"], inplace=True, axis=1)

                # Adding pack content dummies    
                pack_content_dummies = pd.get_dummies(combined_dataset_self_promo['PACK_CONTENT'])

                # If all pack content dummies are not present for the uploaded data
                c = [i for i in list(pack_content_dummies_common.columns) if i not in list(pack_content_dummies.columns)]
                if len(c) != 0 :
                    pack_content_dummies[c] = 0

                # Reordering columns                
                pack_content_dummies = pack_content_dummies.reindex(sorted(pack_content_dummies.columns), axis=1)
                combined_dataset_self_promo = pd.concat([combined_dataset_self_promo, pack_content_dummies], axis=1)
                combined_dataset_self_promo.drop(["PACK_CONTENT"], inplace=True, axis=1)

                # Changing product stage as per start date provided by the user
                prod_stage_data = prod_stage_check(product_fil)            

                # Getting updated product stages
                combined_dataset_test_check = combined_dataset_self_promo[combined_dataset_self_promo.Product == product_fil].merge(prod_stage_data[['Week','Initial_weeks1']], how = 'left')
                combined_dataset_test_check['Initial_weeks1'] = combined_dataset_test_check['Initial_weeks1'].fillna(combined_dataset_test_check['Intial_weeks'])
                combined_dataset_test_check.drop(columns = {'Intial_weeks'}, inplace = True)
                combined_dataset_test_check.rename(columns = {'Initial_weeks1':'Intial_weeks'}, inplace = True)

                combined_dataset_self_promo = combined_dataset_test_check.copy()

                # Product stage dummies
                stage_dummies_test = pd.get_dummies(combined_dataset_self_promo['Intial_weeks'])

                # If all product stages are not present for the uploaded data
                c = [i for i in list(stage_dummies.columns) if i not in list(stage_dummies_test.columns)]
                if len(c) != 0 :
                    stage_dummies_test[c] = 0

                # Reordering columns                
                stage_dummies_test = stage_dummies_test.reindex(sorted(stage_dummies_test.columns), axis=1)
                combined_dataset_self_promo = pd.concat([combined_dataset_self_promo, stage_dummies_test], axis=1)
                combined_dataset_self_promo.drop(["Intial_weeks"], inplace=True, axis=1)

                # In case the user doesn't want to upload adcal data
                if toggle_upload.value != 'Yes':
                    # CHANGES : For adcal discounts
                    if 'All' in list(test_period.value):
                        # Discount change
                        if i == 1:                
                            combined_dataset_self_promo['Adcal_DD'] = (1+disc_change_fil)*combined_dataset_self_promo['Adcal_DD']
                            combined_dataset_self_promo.loc[:,'EDV.Price'] = (1+base_price_change_fil)*combined_dataset_self_promo.loc[:,'EDV.Price']
                        elif i == 2:
                            combined_dataset_self_promo['Adcal_DD'] = (1+disc_change_fil2)*combined_dataset_self_promo['Adcal_DD']
                            combined_dataset_self_promo.loc[:,'EDV.Price'] = (1+base_price_change_fil2)*combined_dataset_self_promo.loc[:,'EDV.Price']          
                    else:                
                        if i == 1:                
                            # Discount change
                            combined_dataset_self_promo.loc[combined_dataset_self_promo['Test_period'].isin(test_period_fil),'Adcal_DD'] = (1+disc_change_fil)*combined_dataset_self_promo[combined_dataset_self_promo['Test_period'].isin(test_period_fil)]['Adcal_DD'] 
                            # Baseline pricing
                            combined_dataset_self_promo.loc[(combined_dataset_self_promo['Test_period'].isin(test_period_fil)),'EDV.Price'] = (1+base_price_change_fil)*combined_dataset_self_promo[(combined_dataset_self_promo['Test_period'].isin(test_period_fil))]['EDV.Price']
                        elif i == 2:
                            # Discount change
                            combined_dataset_self_promo.loc[combined_dataset_self_promo['Test_period'].isin(test_period_fil2),'Adcal_DD'] = (1+disc_change_fil2)*combined_dataset_self_promo[combined_dataset_self_promo['Test_period'].isin(test_period_fil2)]['Adcal_DD']             
                            # Baseline pricing
                            combined_dataset_self_promo.loc[(combined_dataset_self_promo['Test_period'].isin(test_period_fil2)),'EDV.Price'] = (1+base_price_change_fil2)*combined_dataset_self_promo[(combined_dataset_self_promo['Test_period'].isin(test_period_fil2))]['EDV.Price']

                #Adding discount depth columns
                combined_dataset_self_promo["DD_1"] = [1 if (0.25> x >0.1)
                                      else 0 for x in combined_dataset_self_promo["Adcal_DD"]]
                combined_dataset_self_promo["DD_2"] = [1 if (x>=0.25)
                                                 else 0 for x in combined_dataset_self_promo["Adcal_DD"]]            

                # Adcal Price calculation
                # In case the discounts go over 100%, limit to 100%
                combined_dataset_self_promo.loc[combined_dataset_self_promo.Adcal_DD >= 1, 'Adcal_DD'] = 0.99
                combined_dataset_self_promo.loc[combined_dataset_self_promo['Adcal_DD'] >= 0.1, 'Adcal_Price'] = (1-combined_dataset_self_promo.loc[combined_dataset_self_promo['Adcal_DD']>=0.1,'Adcal_DD'])*combined_dataset_self_promo.loc[combined_dataset_self_promo['Adcal_DD']>=0.1,'EDV.Price']
                # Adcal_DD < 10 then Adcal Price = EDV Price            
                combined_dataset_self_promo.loc[combined_dataset_self_promo['Adcal_DD'] < 0.1, 'Adcal_Price'] = combined_dataset_self_promo.loc[combined_dataset_self_promo['Adcal_DD'] < 0.1, 'EDV.Price']

                # Dropping adcal DD as DD1 & DD2 are added
                combined_dataset_self_promo.drop(["Adcal_DD"], inplace=True, axis=1)

                complete_dataset_self_promo = combined_dataset_self_promo.copy()
                complete_dataset_self_promo = complete_dataset_self_promo.loc[complete_dataset_self_promo["Eq.Unit.Sales"].notnull()]

                complete_dataset_self_promo = complete_dataset_self_promo.loc[((complete_dataset_self_promo["Week"] >= "2017-01") & 
                                                         (complete_dataset_self_promo["Week"] <= "2019-52"))]    

                complete_dataset_self_promo['Adcal_Price'] = np.log(complete_dataset_self_promo['Adcal_Price'])
                complete_dataset_self_promo.sort_values(by=['Banner','Product','Week'], inplace=True)           

                Test_periods = ['2019Q1','2019Q2','2019Q3','2019Q4']

                for Test_period in Test_periods:
                    if (Test_period>end_week.children[8].value) | (Test_period > Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Product == product_fil, 'Test_period'].max()):
                        continue
                    if (Test_period<start_week.children[8].value) | (Test_period < Volume_dataset_all_reg.loc[Volume_dataset_all_reg.Product == product_fil, 'Test_period'].min()):
                        continue                

                    #Filtering test weeks
                    Test_weeks = Test_week_list[Test_period]

                    # Dividing train & test data
                    complete_train_data_set = complete_dataset_adcal.loc[~complete_dataset_adcal["Week"].isin(Test_weeks)].reset_index(drop=True)
                    complete_test_data_set_adcal = complete_dataset_adcal_test.loc[complete_dataset_adcal_test["Week"].isin(Test_weeks)].reset_index(drop=True)
                    complete_test_data_set_edv = complete_dataset_edv.loc[complete_dataset_edv["Week"].isin(Test_weeks)].reset_index(drop=True)
                    complete_test_data_set_other_promo = complete_dataset_other_promo.loc[complete_dataset_other_promo["Week"].isin(Test_weeks)].reset_index(drop=True)
                    complete_test_data_set_self_promo = complete_dataset_self_promo.loc[complete_dataset_self_promo["Week"].isin(Test_weeks)].reset_index(drop=True)

                    # Dropping unneccessary columns from train dataset
                    train_data_set = complete_train_data_set.copy()

                    train_data_brand = train_data_set.copy()
                    train_data_brand.drop(["Week","Banner","Product","EDV.Price","Eq.Unit.Sales","Dollar.Sales","brand"], inplace=True, axis=1)

                    # Filtering for New product
                    test_data_set = complete_test_data_set_adcal.copy()
                    test_data_set_edv = complete_test_data_set_edv.copy()
                    test_data_set_other_promo = complete_test_data_set_other_promo.copy()
                    test_data_set_self_promo = complete_test_data_set_self_promo.copy()            

                    # In case the user doesn't want to upload adcal data
                    if toggle_upload.value != 'Yes':

                        #################### INCREASING/DECREASING PROMOTIONS FOR BANNER WEEKS COMBINATIONS ####################

                        if (promo_change_toggle.value == 'Yes (All Banners)') | (promo_change_toggle.value == 'Yes (Specific Banners)'):
                            if (front_change.children[1].value != 0) | (middle_change.children[1].value != 0) | (back_change.children[1].value != 0) | (front_change2.children[1].value != 0) | (middle_change2.children[1].value != 0) | (back_change2.children[1].value != 0) : 

                                ## FRONT PAGE PROMO CHANGES
                                # New Product 1
                                if i == 1:
                                    if front_change.children[1].value > 0:
                                        # Adding additional promotions in the required banner-weeks
                                        for week,banner in zip(ban_week_fp['Week'], ban_week_fp['Banner']):                            
                                            test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Front.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Front.Page'] + 1
                                            test_data_set_self_promo.loc[(test_data_set_self_promo.Week == week) & (test_data_set_self_promo.Banner == banner), 'Front.Page'] = test_data_set_self_promo.loc[(test_data_set_self_promo.Week == week) & (test_data_set_self_promo.Banner == banner), 'Front.Page'] + 1
                                    elif front_change.children[1].value < 0:
                                        for week,banner in zip(ban_week_fp['Week'], ban_week_fp['Banner']):                            
                                            test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Front.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Front.Page'] - 1
                                            test_data_set_self_promo.loc[(test_data_set_self_promo.Week == week) & (test_data_set_self_promo.Banner == banner), 'Front.Page'] = test_data_set_self_promo.loc[(test_data_set_self_promo.Week == week) & (test_data_set_self_promo.Banner == banner), 'Front.Page'] - 1

                                elif i == 2:
                                    if front_change2.children[1].value > 0:
                                        # Adding additional promotions in the required banner-weeks
                                        for week,banner in zip(ban_week_fp2['Week'], ban_week_fp2['Banner']):                            
                                            test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Front.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Front.Page'] + 1
                                            test_data_set_self_promo.loc[(test_data_set_self_promo.Week == week) & (test_data_set_self_promo.Banner == banner), 'Front.Page'] = test_data_set_self_promo.loc[(test_data_set_self_promo.Week == week) & (test_data_set_self_promo.Banner == banner), 'Front.Page'] + 1
                                    elif front_change2.children[1].value < 0:
                                        for week,banner in zip(ban_week_fp2['Week'], ban_week_fp2['Banner']):                            
                                            test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Front.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Front.Page'] - 1
                                            test_data_set_self_promo.loc[(test_data_set_self_promo.Week == week) & (test_data_set_self_promo.Banner == banner), 'Front.Page'] = test_data_set_self_promo.loc[(test_data_set_self_promo.Week == week) & (test_data_set_self_promo.Banner == banner), 'Front.Page'] - 1

                                ## MIDDLE PAGE PROMO CHANGES
                                if i == 1:
                                    if middle_change.children[1].value > 0:
                                        # Adding additional promotions in the required banner-weeks
                                        for week,banner in zip(ban_week_mp['Week'], ban_week_mp['Banner']):                            
                                            test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Middle.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Middle.Page'] + 1
                                            test_data_set_self_promo.loc[(test_data_set_self_promo.Week == week) & (test_data_set_self_promo.Banner == banner), 'Middle.Page'] = test_data_set_self_promo.loc[(test_data_set_self_promo.Week == week) & (test_data_set_self_promo.Banner == banner), 'Middle.Page'] + 1
                                    elif middle_change.children[1].value < 0:
                                        for week,banner in zip(ban_week_mp['Week'], ban_week_mp['Banner']):                            
                                            test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Middle.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Middle.Page'] - 1
                                            test_data_set_self_promo.loc[(test_data_set_self_promo.Week == week) & (test_data_set_self_promo.Banner == banner), 'Middle.Page'] = test_data_set_self_promo.loc[(test_data_set_self_promo.Week == week) & (test_data_set_self_promo.Banner == banner), 'Middle.Page'] - 1
                                elif i == 2:
                                    if middle_change2.children[1].value > 0:
                                        # Adding additional promotions in the required banner-weeks
                                        for week,banner in zip(ban_week_mp2['Week'], ban_week_mp2['Banner']):                            
                                            test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Middle.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Middle.Page'] + 1
                                            test_data_set_self_promo.loc[(test_data_set_self_promo.Week == week) & (test_data_set_self_promo.Banner == banner), 'Middle.Page'] = test_data_set_self_promo.loc[(test_data_set_self_promo.Week == week) & (test_data_set_self_promo.Banner == banner), 'Middle.Page'] + 1
                                    elif middle_change2.children[1].value < 0:
                                        for week,banner in zip(ban_week_mp2['Week'], ban_week_mp2['Banner']):                            
                                            test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Middle.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Middle.Page'] - 1
                                            test_data_set_self_promo.loc[(test_data_set_self_promo.Week == week) & (test_data_set_self_promo.Banner == banner), 'Middle.Page'] = test_data_set_self_promo.loc[(test_data_set_self_promo.Week == week) & (test_data_set_self_promo.Banner == banner), 'Middle.Page'] - 1

                                ## BACK PAGE PROMO CHANGES
                                if i == 1:
                                    if back_change.children[1].value > 0:
                                        # Adding additional promotions in the required banner-weeks
                                        for week,banner in zip(ban_week_bp['Week'], ban_week_bp['Banner']):                            
                                            test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Back.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Back.Page'] + 1
                                            test_data_set_self_promo.loc[(test_data_set_self_promo.Week == week) & (test_data_set_self_promo.Banner == banner), 'Back.Page'] = test_data_set_self_promo.loc[(test_data_set_self_promo.Week == week) & (test_data_set_self_promo.Banner == banner), 'Back.Page'] + 1
                                    elif back_change.children[1].value < 0:
                                        for week,banner in zip(ban_week_bp['Week'], ban_week_bp['Banner']):                            
                                            test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Back.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Back.Page'] - 1
                                            test_data_set_self_promo.loc[(test_data_set_self_promo.Week == week) & (test_data_set_self_promo.Banner == banner), 'Back.Page'] = test_data_set_self_promo.loc[(test_data_set_self_promo.Week == week) & (test_data_set_self_promo.Banner == banner), 'Back.Page'] - 1
                                elif i == 2:
                                    if back_change2.children[1].value > 0:
                                        # Adding additional promotions in the required banner-weeks
                                        for week,banner in zip(ban_week_bp2['Week'], ban_week_bp2['Banner']):                            
                                            test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Back.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Back.Page'] + 1
                                            test_data_set_self_promo.loc[(test_data_set_self_promo.Week == week) & (test_data_set_self_promo.Banner == banner), 'Back.Page'] = test_data_set_self_promo.loc[(test_data_set_self_promo.Week == week) & (test_data_set_self_promo.Banner == banner), 'Back.Page'] + 1
                                    elif back_change2.children[1].value < 0:
                                        for week,banner in zip(ban_week_bp2['Week'], ban_week_bp2['Banner']):                            
                                            test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Back.Page'] = test_data_set.loc[(test_data_set.Week == week) & (test_data_set.Banner == banner), 'Back.Page'] - 1
                                            test_data_set_self_promo.loc[(test_data_set_self_promo.Week == week) & (test_data_set_self_promo.Banner == banner), 'Back.Page'] = test_data_set_self_promo.loc[(test_data_set_self_promo.Week == week) & (test_data_set_self_promo.Banner == banner), 'Back.Page'] - 1


                    # New Category
                    if i == 1:
                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)].Category.unique():
                            if (cols == category_drop.value):                      
                                test_data_set[cols] = 1
                                test_data_set_edv[cols] = 1
                                test_data_set_other_promo[cols] = 1
                                test_data_set_self_promo[cols] = 1
                            else:
                                test_data_set[cols] = 0
                                test_data_set_edv[cols] = 0
                                test_data_set_other_promo[cols] = 0
                                test_data_set_self_promo[cols] = 0

                    if i == 2:
                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)].Category.unique():
                            if (cols == category_drop2.value):                      
                                test_data_set[cols] = 1
                                test_data_set_edv[cols] = 1
                                test_data_set_other_promo[cols] = 1
                                test_data_set_self_promo[cols] = 1

                            else:
                                test_data_set[cols] = 0
                                test_data_set_edv[cols] = 0
                                test_data_set_other_promo[cols] = 0
                                test_data_set_self_promo[cols] = 0

                    # New Pack Subtype
                    if i == 1:
                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['Pack.Subtype'].unique():
                            if (cols == pack_subtype_drop.value):                      
                                test_data_set[cols] = 1
                                test_data_set_edv[cols] = 1
                                test_data_set_other_promo[cols] = 1
                                test_data_set_self_promo[cols] = 1

                            else:
                                test_data_set[cols] = 0
                                test_data_set_edv[cols] = 0
                                test_data_set_other_promo[cols] = 0
                                test_data_set_self_promo[cols] = 0

                    if i == 2:
                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['Pack.Subtype'].unique():
                            if (cols == pack_subtype_drop2.value):                      
                                test_data_set[cols] = 1
                                test_data_set_edv[cols] = 1
                                test_data_set_other_promo[cols] = 1
                                test_data_set_self_promo[cols] = 1

                            else:
                                test_data_set[cols] = 0
                                test_data_set_edv[cols] = 0
                                test_data_set_other_promo[cols] = 0
                                test_data_set_self_promo[cols] = 0

                    # New Pack Content
                    if i == 1:
                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['PACK_CONTENT'].unique():
                            if (cols == pack_content_drop.value):                      
                                test_data_set[cols] = 1
                                test_data_set_edv[cols] = 1
                                test_data_set_other_promo[cols] = 1
                                test_data_set_self_promo[cols] = 1

                            else:
                                test_data_set[cols] = 0
                                test_data_set_edv[cols] = 0
                                test_data_set_other_promo[cols] = 0
                                test_data_set_self_promo[cols] = 0

                    if i == 2:
                        for cols in Volume_dataset_all_reg[(Volume_dataset_all_reg.Region == Region_key) & (Volume_dataset_all_reg.Channel.isin(channel_list)) & (Volume_dataset_all_reg.Test_period >= start_week.children[8].value) & (Volume_dataset_all_reg.Test_period <= end_week.children[8].value)]['PACK_CONTENT'].unique():
                            if (cols == pack_content_drop2.value):                      
                                test_data_set[cols] = 1
                                test_data_set_edv[cols] = 1
                                test_data_set_other_promo[cols] = 1
                                test_data_set_self_promo[cols] = 1

                            else:
                                test_data_set[cols] = 0
                                test_data_set_edv[cols] = 0
                                test_data_set_other_promo[cols] = 0
                                test_data_set_self_promo[cols] = 0

                    if product_fil == 'TCCC CORE POWER 414 ML BTTL':
                        test_data_set = test_data_set.drop_duplicates()
                        test_data_set_edv = test_data_set_edv.drop_duplicates()
                        test_data_set_other_promo = test_data_set_other_promo.drop_duplicates()
                        test_data_set_self_promo = test_data_set_self_promo.drop_duplicates()

                    test_data_brand = test_data_set.copy()
                    test_data_brand_edv = test_data_set_edv.copy()
                    test_data_brand_other_promo = test_data_set_other_promo.copy()
                    test_data_brand_self_promo = test_data_set_self_promo.copy()

                    if i == 1 :
                        # For New SIZE
                        test_data_brand['SIZE_ML'] = size.value
                        test_data_brand_edv['SIZE_ML'] = size.value
                        test_data_brand_other_promo['SIZE_ML'] = size.value
                        test_data_brand_self_promo['SIZE_ML'] = size.value

                        # For New Count
                        test_data_brand['COUNT'] = count.value
                        test_data_brand_edv['COUNT'] = count.value
                        test_data_brand_other_promo['COUNT'] = count.value
                        test_data_brand_self_promo['COUNT'] = count.value

                    elif i == 2:
                        # For New SIZE
                        test_data_brand['SIZE_ML'] = size2.value
                        test_data_brand_edv['SIZE_ML'] = size2.value
                        test_data_brand_other_promo['SIZE_ML'] = size2.value
                        test_data_brand_self_promo['SIZE_ML'] = size2.value


                        # For New Count
                        test_data_brand['COUNT'] = count2.value
                        test_data_brand_edv['COUNT'] = count2.value
                        test_data_brand_other_promo['COUNT'] = count2.value
                        test_data_brand_self_promo['COUNT'] = count2.value

                    if(test_data_brand.shape[0]==0):
                        continue

                    # Dropping unneccessary columns from test dataset
                    test_data_brand.drop(["Week","Banner","Product","EDV.Price","Eq.Unit.Sales","Dollar.Sales","brand"], inplace=True, axis=1)
                    test_data_brand_edv.drop(["Week","Banner","Product","EDV.Price","Eq.Unit.Sales","Dollar.Sales","brand"], inplace=True, axis=1)
                    test_data_brand_other_promo.drop(["Week","Banner","Product","EDV.Price","Eq.Unit.Sales","Dollar.Sales","brand"], inplace=True, axis=1)
                    test_data_brand_self_promo.drop(["Week","Banner","Product","EDV.Price","Eq.Unit.Sales","Dollar.Sales","brand"], inplace=True, axis=1)

                    #Creating test and train data - independent & dependent columns
                    X_train = train_data_brand
                    y_train = train_data_set["Eq.Unit.Sales"]
                    X_test = test_data_brand
                    X_test_edv = test_data_brand_edv
                    X_test_other_promo = test_data_brand_other_promo
                    X_test_self_promo = test_data_brand_self_promo.copy()
                    X_test_dd_promo = test_data_brand_self_promo.copy()
                    X_test_dd_promo['Front.Page'] = 0
                    X_test_dd_promo['Middle.Page'] = 0
                    X_test_dd_promo['Back.Page'] = 0
                    X_test_page_promo = test_data_brand_edv.copy()
                    X_test_page_promo['Front.Page'] = X_test_self_promo['Front.Page']
                    X_test_page_promo['Middle.Page'] = X_test_self_promo['Middle.Page']
                    X_test_page_promo['Back.Page'] = X_test_self_promo['Back.Page']
                    y_test = test_data_set["Eq.Unit.Sales"]
                    

                    # Appending test data
                    test_data_set["Region"] = Region_key
                    test_data_set["Test_period"] = Test_period

                    Test_Data = Test_Data.append(test_data_set, ignore_index = True)

                    #Setting seed
                    random.seed(42)
                    

                    if((Region_key == 'EAST') and (Test_period == '2019Q1')):
                            rf = model_object_EAST_2019Q1
                    elif((Region_key == 'EAST') and (Test_period == '2019Q2')):
                            rf = model_object_EAST_2019Q2
                    elif((Region_key == 'EAST') and (Test_period == '2019Q3')):
                            rf = model_object_EAST_2019Q3
                    elif((Region_key == 'EAST') and (Test_period == '2019Q4')):
                            rf = model_object_EAST_2019Q4
                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q1')):
                            rf = model_object_ONTARIO_2019Q1
                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q2')):
                            rf = model_object_ONTARIO_2019Q2
                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q3')):
                            rf = model_object_ONTARIO_2019Q3
                    elif((Region_key == 'ONTARIO') and (Test_period == '2019Q4')):
                            rf = model_object_ONTARIO_2019Q4
                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q1')):
                            rf = model_object_QUEBEC_2019Q1
                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q2')):
                            rf = model_object_QUEBEC_2019Q2
                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q3')):
                            rf = model_object_QUEBEC_2019Q3
                    elif((Region_key == 'QUEBEC') and (Test_period == '2019Q4')):
                            rf = model_object_QUEBEC_2019Q4
                    elif((Region_key == 'WEST') and (Test_period == '2019Q1')):
                            rf = model_object_WEST_2019Q1
                    elif((Region_key == 'WEST') and (Test_period == '2019Q2')):
                            rf = model_object_WEST_2019Q2
                    elif((Region_key == 'WEST') and (Test_period == '2019Q3')):
                            rf = model_object_WEST_2019Q3
                    else:
                            rf = model_object_WEST_2019Q4

                    #Model predictions
                    predictions_rf_adcal = rf.predict(X_test)
                    predictions_rf_edv = rf.predict(X_test_edv)
                    predictions_rf_other_promo = rf.predict(X_test_other_promo)
                    predictions_rf_self_promo = rf.predict(X_test_self_promo)
                    predictions_rf_dd_promo = rf.predict(X_test_dd_promo)
                    predictions_rf_page_promo = rf.predict(X_test_page_promo)

                    #Storing test results
                    result = X_test.copy()
                    result["Predictions_rf_adcal"] = np.exp(predictions_rf_adcal)
                    result["Predictions_rf_edv"] = np.exp(predictions_rf_edv)
                    result["Predictions_rf_other_promo"] = np.exp(predictions_rf_other_promo)
                    result["Predictions_rf_self_promo"] = np.exp(predictions_rf_self_promo)
                    result["Predictions_rf_dd_promo"] = np.exp(predictions_rf_dd_promo)
                    result["Predictions_rf_page_promo"] = np.exp(predictions_rf_page_promo)
                    result["Actuals"] = np.exp(y_test)

                    #Calculating APE for the models        
                    result["Banner"] = test_data_set["Banner"]
                    result["Product"] = test_data_set["Product"]
                    result["brand"] = test_data_set["brand"]
                    result["Region"] = Region_key
                    result["Test_period"] = Test_period
                    result["Week"] = test_data_set["Week"]
                    result['iter'] = i
                    result['Total_Promo'] = result['Front.Page'] + result['Middle.Page'] + result['Back.Page']
                    result['Total_DD'] = result['DD_1'] + result['DD_2']
                    result['flag'] = result['Total_Promo'] + result['Total_DD']

                    result.loc[result['flag']==2, 'Predictions_rf_page_promo'] = result.loc[result['flag']==2, 'Predictions_rf_edv'] + result.loc[result['flag']==2, 'Predictions_rf_self_promo'] - result.loc[result['flag']==2, 'Predictions_rf_dd_promo']

                    result = result[["Region","Banner","Product","brand","Test_period","Week","Actuals","Predictions_rf_adcal","Predictions_rf_edv","Predictions_rf_other_promo","Predictions_rf_self_promo","Predictions_rf_dd_promo","Predictions_rf_page_promo",'iter','Front.Page','Middle.Page','Back.Page','DD_1','DD_2']]

                    #Appending results to the final results dataframe
                    Test_results_net = Test_results_net.append(result, ignore_index=True) 
                    
                    ## Cann Contibution code ##
                    sel_brands_new_product = list(result['brand'].unique())

                    Affecting_brand_weeks_final = pd.DataFrame()
                    for brand_id in sel_brands_new_product:
                            Affecting_brands = find_affecting_brands(brand_id)
                            Affecting_brand_weeks = Volume_dataset[(Volume_dataset['brand'].isin(Affecting_brands))
                                                          &(Volume_dataset['Adcal_DD']>0.25)
                                                          &(Volume_dataset['brand']!=brand_id)
                                                          &(Volume_dataset['Week'].isin(Test_weeks))].sort_values(by='Week')
                            Affecting_brand_weeks.rename(columns={'brand':'Affecting_brand'}, inplace=True)
                            Affecting_brand_weeks['Affected_brand'] = brand_id
                            Affecting_brand_weeks_final = Affecting_brand_weeks_final.append(Affecting_brand_weeks[['Week','Affected_brand','Affecting_brand']])

                    # Filtering for existing product
                    test_data_set_edv_simulation = pd.merge(test_data_set_edv, Affecting_brand_weeks_final, how='inner', left_on=['Week','brand'], right_on=['Week','Affected_brand'])
                    test_data_brand_edv_simulation = test_data_set_edv_simulation.copy()

                    # # Dropping unneccessary columns from test dataset
                    test_data_brand_edv_simulation.drop(["Week","Banner","Product","EDV.Price","Eq.Unit.Sales","Dollar.Sales","brand","Affected_brand","Affecting_brand"], inplace=True, axis=1)

                    # Filtering for existing product
                    Adcal_data = test_data_set.copy()

                    columns = list(Adcal_data.columns)
                    cross_columns = [s for s in columns if "Cross" in s]
                    cross_columns.append('Week')
                    cross_columns.append('brand')
                    Adcal_data = Adcal_data[cross_columns]
                    Adcal_data.columns = [str(col) + '_Adcal' for col in Adcal_data.columns]

                    test_data_set_edv_simulation_v2 = pd.merge(test_data_set_edv_simulation, Adcal_data, how='inner', left_on=['Week','brand'], right_on=['Week_Adcal','brand_Adcal'])
                    test_data_set_edv_simulation_v2 = test_data_set_edv_simulation_v2.reset_index()

                    index_list = (list(test_data_set_edv_simulation_v2['index']))
                    for index in index_list:
                        brand = int(test_data_set_edv_simulation_v2[test_data_set_edv_simulation_v2['index']==index]['Affecting_brand'].unique()[0])
                        original_cross = 'Cross_'+str(brand)
                        adcal_cross = 'Cross_'+str(brand)+'_Adcal'
                        test_data_set_edv_simulation_v2.loc[(test_data_set_edv_simulation_v2['Affecting_brand']==brand)&(test_data_set_edv_simulation_v2['index']==index), original_cross] = test_data_set_edv_simulation_v2.loc[(test_data_set_edv_simulation_v2['Affecting_brand']==brand)&(test_data_set_edv_simulation_v2['index']==index), adcal_cross]

                    final_columns = list(test_data_set_edv.columns)
                    final_columns.remove('brand')
                    final_columns.append('Affected_brand')
                    final_columns.append('Affecting_brand')

                    test_data_set_edv_simulation_v2 = test_data_set_edv_simulation_v2[final_columns]

                    # # Dropping unneccessary columns from test dataset
                    test_data_brand_edv_simulation = test_data_set_edv_simulation_v2.copy()
                    test_data_brand_edv_simulation.drop(["Week","Banner","Product","EDV.Price","Eq.Unit.Sales","Dollar.Sales","Affected_brand","Affecting_brand"], inplace=True, axis=1)

                    #Creating test and train data - independent & dependent columns
                    X_test_edv_simulation = test_data_brand_edv_simulation

                    #Model predictions
                    predictions_rf_edv_simulation = rf.predict(X_test_edv_simulation)

                    #Storing test results
                    result2 = X_test_edv_simulation.copy()
                    result2["Predictions_rf_edv_simulation"] = np.exp(predictions_rf_edv_simulation)

                    result2["Actuals"] = np.exp(y_test)

                    #Calculating APE for the models
                    result2["Affected_brand"] = test_data_set_edv_simulation_v2["Affected_brand"]
                    result2["Affecting_brand"] = test_data_set_edv_simulation_v2["Affecting_brand"]
                    result2["Banner"] = test_data_set_edv_simulation_v2["Banner"]
                    result2["Product"] = test_data_set_edv_simulation_v2["Product"]
                    result2["Region"] = Region_key
                    result2["Test_period"] = Test_period
                    result2["Week"] = test_data_set_edv_simulation_v2["Week"]
                    result2['iter'] = i

                    result2 = result2[["Region","Banner","Product","Affected_brand","Affecting_brand","Test_period","Week","Actuals","Predictions_rf_edv_simulation","iter"]]

                    Sim_results = pd.merge(result[['Region','brand','Banner','Product','Week','Predictions_rf_edv']], result2[['Region','Affected_brand','Affecting_brand','Banner','Product','Week','Predictions_rf_edv_simulation']], how='inner', on=['Banner','Product','Week'])

                    Sim_results = Sim_results[Sim_results['Predictions_rf_edv'] != Sim_results['Predictions_rf_edv_simulation']]

                    Sim_results['Diff'] = Sim_results['Predictions_rf_edv'] - Sim_results['Predictions_rf_edv_simulation']
                    Sim_results = Sim_results[Sim_results['Diff']>0]
                    Sim_results['iter'] = i

                    #Appending results to the final results dataframe
                    Test_results_net_cann = Test_results_net_cann.append(Sim_results, ignore_index=True)

        clear_output()
        
        # Net gain calculation
        Test_results_net['Promo Gain'] = Test_results_net['Predictions_rf_self_promo'] - Test_results_net['Predictions_rf_edv']
        Test_results_net['Cann Loss'] = Test_results_net['Predictions_rf_other_promo'] - Test_results_net['Predictions_rf_edv']
        Test_results_net['Updated_Cann_loss'] = np.where(Test_results_net['Cann Loss'] > 0, 0, Test_results_net['Cann Loss'])

        # Year
        Test_results_net['Year'] = Test_results_net.Week.str.slice(0,4)
        # Week number
        Test_results_net['Week_no'] = Test_results_net.Week.str.slice(5,7)
        # Converting into year-week-days
        Test_results_net['year_week_ts'] = Test_results_net.apply(lambda row: year_week(row.Year, row.Week_no), axis=1)
        Test_results_net['Month'] = pd.DatetimeIndex(Test_results_net['year_week_ts']).month
        Test_results_net['Year_month'] = Test_results_net['Year'].astype(str) + '-' + Test_results_net['Month'].astype(str)

        # Iteration 1   
        if toggle_np2.value != 'Yes':
            # Calculating net gain on year-month level
            Test_results_net_gain1 = Test_results_net[Test_results_net.iter == 1].groupby(['Year_month','Month'])['Actuals','Predictions_rf_adcal','Predictions_rf_edv','Predictions_rf_other_promo','Predictions_rf_self_promo','Predictions_rf_dd_promo','Predictions_rf_page_promo','Promo Gain','Updated_Cann_loss'].sum().reset_index().sort_values('Month')
            Test_results_net_gain1['Net Gain'] = Test_results_net_gain1['Updated_Cann_loss'] + Test_results_net_gain1['Promo Gain']
            Test_results_net_gain1['% Net Gain'] = 100*Test_results_net_gain1['Net Gain']/Test_results_net_gain1['Predictions_rf_adcal']

            Test_results_net_gain1['Page_Promo_Gain'] = round(Test_results_net_gain1['Predictions_rf_page_promo'] - Test_results_net_gain1['Predictions_rf_edv'], 2)
            Test_results_net_gain1['Price_Promo_Gain'] = round(Test_results_net_gain1['Predictions_rf_dd_promo'] - Test_results_net_gain1['Predictions_rf_edv'], 2)
            Test_results_net_gain1['Page_Promo_Gain'] = Test_results_net_gain1['Page_Promo_Gain'].abs()
            Test_results_net_gain1['Price_Promo_Gain'] = Test_results_net_gain1['Price_Promo_Gain'].abs()
            Test_results_net_gain1['Page Promo Gain %'] = round((Test_results_net_gain1['Page_Promo_Gain']/(Test_results_net_gain1['Page_Promo_Gain']+Test_results_net_gain1['Price_Promo_Gain']))*100,1)
            Test_results_net_gain1['Price Promo Gain %'] = round((Test_results_net_gain1['Price_Promo_Gain']/(Test_results_net_gain1['Page_Promo_Gain']+Test_results_net_gain1['Price_Promo_Gain']))*100,1)

            # Net Gain KPI
            net_gain_kpi = {}
            overall_net_gain_percent_np1 = round((Test_results_net_gain1['Net Gain'].sum()/Test_results_net_gain1['Predictions_rf_adcal'].sum())*100,1)
            net_gain_kpi['New Product Overall Net Gain %'] = overall_net_gain_percent_np1
            net_gain_kpi_df1 = pd.DataFrame(net_gain_kpi, index=[''])
            display(HTML(net_gain_kpi_df1.to_html(index=False)))
            ################## NET GAIN PLOT ##################                
            x1 = np.arange(len(list(Test_results_net_gain1['Year_month'])))  # the label locations            

            fig, (ax1) = plt.subplots(nrows=1, ncols=1, figsize = (25,8))
            rects1 = ax1.plot(x1, list(Test_results_net_gain1['% Net Gain']),label='% Net Gain', marker = 'o')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax1.set_ylabel('Net Gain %')
            ax1.set_xlabel('Year Month')
            ax1.set_title('New Product Net Gain')
            ax1.set_xticks(x1)
            ax1.set_xticklabels(list(Test_results_net_gain1['Year_month']), fontsize=15,rotation=45)
            ax1.legend(loc = 'upper left')

            ax1.get_yaxis().set_major_formatter(mtick.PercentFormatter())

            # ax1.bar_label(rects1, padding=5, labels=list(Test_results_net_gain1['% Net Gain'].round(1)),label_type = 'center')
            for i,j in Test_results_net_gain1['% Net Gain'].reset_index()['% Net Gain'].round(0).items():
                ax1.annotate(str(j) + "%", xy=(i, j))

            plt.show();

            #Promo Gain KPI
            promo_gain_kpi = {}
            promo_gain_kpi['New Product Total Promo Gain'] = human_format(Test_results_net_gain1['Promo Gain'].sum().astype('int'))
            promo_gain_kpi_df1 = pd.DataFrame(promo_gain_kpi, index=[''])
            display(HTML(promo_gain_kpi_df1.to_html(index=False)))

            ## Promo Gain breakdown Plots ##
            fig, ax1 = plt.subplots(figsize=(12,5))
            a1=ax1.bar(Test_results_net_gain1['Year_month'], Test_results_net_gain1['Price Promo Gain %'], color='orange', bottom=Test_results_net_gain1['Page Promo Gain %'], label='New Product 1 Price Promo Gain %')
            a2=ax1.bar(Test_results_net_gain1['Year_month'], Test_results_net_gain1['Page Promo Gain %'], color='yellow', label='New Product 1 Page Promo Gain %')
            for i,j in Test_results_net_gain1['Price Promo Gain %'].reset_index()['Price Promo Gain %'].items():
                ax1.annotate(str(j) + "%", xy=(i, 100-j/2),ha='center')            

            for i,j in Test_results_net_gain1['Page Promo Gain %'].reset_index()['Page Promo Gain %'].items():
                ax1.annotate(str(j) + "%", xy=(i, j/2),ha='center')
            ax1.set_title('New Product Promo Gain breakdown')
            ax1.set_ylabel('Promo Gain breakdown %')
            ax1.set_xlabel('Year Month')
            ax1.set_xticklabels(Test_results_net_gain1['Year_month'], rotation=45, ha='right')
            ax1.set_ylim(0,130)
            ax1.legend(loc='upper right')
            ax1.get_yaxis().set_major_formatter(mtick.PercentFormatter())

            plt.show();

            #Cannibalization loss KPI
            cann_loss_kpi = {}
            cann_loss_kpi['New Product Total Cannibalization loss due to Affecting Products Promotions'] = human_format(Test_results_net_gain1['Updated_Cann_loss'].sum().astype('int'))
            cann_loss_kpi_df1 = pd.DataFrame(cann_loss_kpi, index=[''])
            display(HTML(cann_loss_kpi_df1.to_html(index=False)))

            print("Note: Cannibalization loss is measured due to Affecting Products Promotions")

            ## Cannibalization Contribution
            Sim_results_final_BP1 = pd.merge(Test_results_net_cann[Test_results_net_cann['iter']==1][['Affecting_brand','Diff']], brand_mapping[['brand','Banner','Product']], how='left', left_on='Affecting_brand', right_on='brand')
            cann_contribution1 = Sim_results_final_BP1.groupby('Product')['Diff'].sum().reset_index()
            cann_contribution1['Diff_sum'] = cann_contribution1['Diff'].sum()
            cann_contribution1['Cann_Contribution'] = round((cann_contribution1['Diff'] / cann_contribution1['Diff_sum'])*100,1)
            final_cann_contribution1 = cann_contribution1.sort_values(by='Cann_Contribution', ascending=False).head(10)

            ## Cannibalization Contribution Plots ##
            fig, ax1 =  plt.subplots(figsize=(12,5))
            a1=ax1.bar(final_cann_contribution1['Product'], final_cann_contribution1['Cann_Contribution'], color='dodgerblue')
            for i,j in final_cann_contribution1['Cann_Contribution'].reset_index()['Cann_Contribution'].items():
                ax1.annotate(str(j) + "%", xy=(i, j/2),ha='center')
            ax1.set_title('New Product Cannibalization loss Top 10 Affecting Products Contribution %')
            ax1.set_ylabel('Cannibalization loss Contribution %')
            ax1.set_xlabel('Product')
            ax1.set_xticklabels(final_cann_contribution1['Product'], rotation=45, ha='right')
            ax1.get_yaxis().set_major_formatter(mtick.PercentFormatter())

            plt.show();                        
        
        # Iteration 1 & Iteration 2 combined
        if toggle_np2.value == 'Yes':
            # Calculating net gain on year-month level
            Test_results_net_gain1 = Test_results_net[Test_results_net.iter == 1].groupby(['Year_month','Month'])['Actuals','Predictions_rf_adcal','Predictions_rf_edv','Predictions_rf_other_promo','Predictions_rf_self_promo','Predictions_rf_dd_promo','Predictions_rf_page_promo','Promo Gain','Updated_Cann_loss'].sum().reset_index().sort_values('Month')
            Test_results_net_gain2 = Test_results_net[Test_results_net.iter == 2].groupby(['Year_month','Month'])['Actuals','Predictions_rf_adcal','Predictions_rf_edv','Predictions_rf_other_promo','Predictions_rf_self_promo','Predictions_rf_dd_promo','Predictions_rf_page_promo','Promo Gain','Updated_Cann_loss'].sum().reset_index().sort_values('Month')
            Test_results_net_gain1['Net Gain'] = Test_results_net_gain1['Updated_Cann_loss'] + Test_results_net_gain1['Promo Gain']
            Test_results_net_gain2['Net Gain'] = Test_results_net_gain2['Updated_Cann_loss'] + Test_results_net_gain2['Promo Gain']
            Test_results_net_gain1['% Net Gain'] = 100*Test_results_net_gain1['Net Gain']/Test_results_net_gain1['Predictions_rf_adcal']
            Test_results_net_gain2['% Net Gain'] = 100*Test_results_net_gain2['Net Gain']/Test_results_net_gain2['Predictions_rf_adcal']

            Test_results_net_gain1['Page_Promo_Gain'] = round(Test_results_net_gain1['Predictions_rf_page_promo'] - Test_results_net_gain1['Predictions_rf_edv'], 2)
            Test_results_net_gain1['Price_Promo_Gain'] = round(Test_results_net_gain1['Predictions_rf_dd_promo'] - Test_results_net_gain1['Predictions_rf_edv'], 2)
            Test_results_net_gain1['Page_Promo_Gain'] = Test_results_net_gain1['Page_Promo_Gain'].abs()
            Test_results_net_gain1['Price_Promo_Gain'] = Test_results_net_gain1['Price_Promo_Gain'].abs()
            Test_results_net_gain1['Page Promo Gain %'] = round((Test_results_net_gain1['Page_Promo_Gain']/Test_results_net_gain1['Promo Gain'])*100,1)
            Test_results_net_gain1['Price Promo Gain %'] = round((Test_results_net_gain1['Price_Promo_Gain']/Test_results_net_gain1['Promo Gain'])*100,1)

            Test_results_net_gain2['Page_Promo_Gain'] = round(Test_results_net_gain2['Predictions_rf_page_promo'] - Test_results_net_gain2['Predictions_rf_edv'], 2)
            Test_results_net_gain2['Price_Promo_Gain'] = round(Test_results_net_gain2['Predictions_rf_dd_promo'] - Test_results_net_gain2['Predictions_rf_edv'], 2)
            Test_results_net_gain2['Page_Promo_Gain'] = Test_results_net_gain2['Page_Promo_Gain'].abs()
            Test_results_net_gain2['Price_Promo_Gain'] = Test_results_net_gain2['Price_Promo_Gain'].abs()
            Test_results_net_gain2['Page Promo Gain %'] = round((Test_results_net_gain2['Page_Promo_Gain']/(Test_results_net_gain2['Page_Promo_Gain']+Test_results_net_gain2['Price_Promo_Gain']))*100,1)
            Test_results_net_gain2['Price Promo Gain %'] = round((Test_results_net_gain2['Price_Promo_Gain']/(Test_results_net_gain2['Page_Promo_Gain']+Test_results_net_gain2['Price_Promo_Gain']))*100,1)

            # Net Gain KPI
            net_gain_kpi = {}
            overall_net_gain_percent_np1 = round((Test_results_net_gain1['Net Gain'].sum()/Test_results_net_gain1['Predictions_rf_adcal'].sum())*100,1)
            overall_net_gain_percent_np2 = round((Test_results_net_gain2['Net Gain'].sum()/Test_results_net_gain2['Predictions_rf_adcal'].sum())*100,1)
            net_gain_kpi['New Product 1 Overall Net Gain %'] = overall_net_gain_percent_np1
            net_gain_kpi['New Product 2 Overall Net Gain %'] = overall_net_gain_percent_np2
            net_gain_kpi_df2 = pd.DataFrame(net_gain_kpi, index=[''])
            display(HTML(net_gain_kpi_df2.to_html(index=False)))

            ################## NET GAIN PLOT ##################                
            x1 = np.arange(len(list(Test_results_net_gain1['Year_month'])))  # the label locations            
            x2 = np.arange(len(list(Test_results_net_gain2['Year_month'])))  # the label locations

            fig, (ax1) = plt.subplots(nrows=1, ncols=1, figsize = (25,8))
            rects1 = ax1.plot(x1, list(Test_results_net_gain1['% Net Gain']),label='% Net Gain (New Product 1)', marker = 'o')
            rects2 = ax1.plot(x1, list(Test_results_net_gain2['% Net Gain']),label='% Net Gain (New Product 2)', marker = 'o')

            # Add some text for labels, title and custom x-axis tick labels, etc.
            ax1.set_ylabel('% Net Gain', fontsize=20)
            ax1.set_xlabel('Year Month', fontsize=20)
            ax1.set_title('Net Gain for '+ product_fil, fontsize=25)
            ax1.set_xticks(x1)
            ax1.set_xticklabels(list(Test_results_net_gain2['Year_month']), fontsize=15,rotation=90)
            ax1.legend(loc = 'upper left')

            ax1.get_yaxis().set_major_formatter(mtick.PercentFormatter())

            for i,j in Test_results_net_gain1['% Net Gain'].reset_index()['% Net Gain'].round(0).items():
                ax1.annotate(str(j) + "%", xy=(i, j))
            for i,j in Test_results_net_gain2['% Net Gain'].reset_index()['% Net Gain'].round(0).items():
                ax1.annotate(str(j) + "%", xy=(i, j))

            plt.show();

            #Promo Gain KPI
            promo_gain_kpi = {}
            promo_gain_kpi['New Product 1 Total Promo Gain'] = human_format(Test_results_net_gain1['Promo Gain'].sum().astype('int'))
            promo_gain_kpi['New Product 2 Total Promo Gain'] = human_format(Test_results_net_gain2['Promo Gain'].sum().astype('int'))
            promo_gain_kpi_df2 = pd.DataFrame(promo_gain_kpi, index=[''])
            display(HTML(promo_gain_kpi_df2.to_html(index=False)))

            ## Promo Gain breakdown Plots ##
            fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize = (20,8), tight_layout = True)
            a1=ax1.bar(Test_results_net_gain1['Year_month'], Test_results_net_gain1['Price Promo Gain %'], color='orange', bottom=Test_results_net_gain1['Page Promo Gain %'], label='New Product 1 Price Promo Gain %')
            a2=ax1.bar(Test_results_net_gain1['Year_month'], Test_results_net_gain1['Page Promo Gain %'], color='yellow', label='New Product 1 Page Promo Gain %')
            for i,j in Test_results_net_gain1['Price Promo Gain %'].reset_index()['Price Promo Gain %'].items():
                ax1.annotate(str(j) + "%", xy=(i, 100-j/2),ha='center')            

            for i,j in Test_results_net_gain1['Page Promo Gain %'].reset_index()['Page Promo Gain %'].items():
                ax1.annotate(str(j) + "%", xy=(i, j/2),ha='center')
            ax1.set_title('New Product 1 Promo Gain breakdown')
            ax1.set_ylabel('Promo Gain breakdown %')
            ax1.set_xlabel('Year Month')
            ax1.set_xticklabels(Test_results_net_gain1['Year_month'], rotation=45, ha='right')
            ax1.set_ylim(0,120)
            ax1.legend(loc='upper right')
            ax1.get_yaxis().set_major_formatter(mtick.PercentFormatter())

            a3=ax2.bar(Test_results_net_gain2['Year_month'], Test_results_net_gain2['Price Promo Gain %'], color='orange', bottom=Test_results_net_gain2['Page Promo Gain %'], label='New Product 2 Price Promo Gain %')
            a4=ax2.bar(Test_results_net_gain2['Year_month'], Test_results_net_gain2['Page Promo Gain %'], color='yellow', label='New Product 2 Page Promo Gain %')
            for i,j in Test_results_net_gain2['Price Promo Gain %'].reset_index()['Price Promo Gain %'].items():
                ax2.annotate(str(j) + "%", xy=(i, 100-j/2),ha='center')            

            for i,j in Test_results_net_gain2['Page Promo Gain %'].reset_index()['Page Promo Gain %'].items():
                ax2.annotate(str(j) + "%", xy=(i, j/2),ha='center')
            ax2.set_title('New Product 2 Promo Gain breakdown')
            ax2.set_ylabel('Promo Gain breakdown %')
            ax2.set_xlabel('Year Month')
            ax2.set_xticklabels(Test_results_net_gain2['Year_month'], rotation=45, ha='right')
            ax2.set_ylim(0,120)
            ax2.legend(loc='upper right')
            ax2.get_yaxis().set_major_formatter(mtick.PercentFormatter())

            plt.show();

            ## Cannibalization Contribution ##
            Sim_results_final_BP1 = pd.merge(Test_results_net_cann[Test_results_net_cann['iter']==1][['Affecting_brand','Diff']], brand_mapping[['brand','Banner','Product']], how='left', left_on='Affecting_brand', right_on='brand')
            cann_contribution1 = Sim_results_final_BP1.groupby('Product')['Diff'].sum().reset_index()
            cann_contribution1['Diff_sum'] = cann_contribution1['Diff'].sum()
            cann_contribution1['Cann_Contribution'] = round((cann_contribution1['Diff'] / cann_contribution1['Diff_sum'])*100,1)
            final_cann_contribution1 = cann_contribution1.sort_values(by='Cann_Contribution', ascending=False).head(10)

            #### ## Cannibalization Contribution
            Sim_results_final_BP2 = pd.merge(Test_results_net_cann[Test_results_net_cann['iter']==2][['Affecting_brand','Diff']], brand_mapping[['brand','Banner','Product']], how='left', left_on='Affecting_brand', right_on='brand')
            cann_contribution2 = Sim_results_final_BP2.groupby('Product')['Diff'].sum().reset_index()
            cann_contribution2['Diff_sum'] = cann_contribution2['Diff'].sum()
            cann_contribution2['Cann_Contribution'] = round((cann_contribution2['Diff'] / cann_contribution2['Diff_sum'])*100,1)
            final_cann_contribution2 = cann_contribution2.sort_values(by='Cann_Contribution', ascending=False).head(10)

            #Cannibalization loss KPI
            cann_loss_kpi = {}
            cann_loss_kpi['New Product 1 Total Cannibalization loss due to Affecting Products Promotions'] = human_format(Test_results_net_gain1['Updated_Cann_loss'].sum().astype('int'))
            cann_loss_kpi['New Product 2 Total Cannibalization loss due to Affecting Products Promotions'] = human_format(Test_results_net_gain2['Updated_Cann_loss'].sum().astype('int'))
            cann_loss_kpi_df2 = pd.DataFrame(cann_loss_kpi, index=[''])
            display(HTML(cann_loss_kpi_df2.to_html(index=False)))

            print("Note: Cannibalization loss is measured due to Affecting Products Promotions")

            ## Cannibalization Contribution Plots ##
            fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize = (20,8), tight_layout = True)
            a1=ax1.bar(final_cann_contribution1['Product'], final_cann_contribution1['Cann_Contribution'], color='dodgerblue')
            for i,j in final_cann_contribution1['Cann_Contribution'].reset_index()['Cann_Contribution'].items():
                ax1.annotate(str(j) + "%", xy=(i, j/2),ha='center')
            ax1.set_title('New Product 1 Cannibalization loss Top 10 Affecting Products Contribution %')
            ax1.set_ylabel('Cannibalization loss Contribution %')
            ax1.set_xlabel('Product')
            ax1.set_xticklabels(final_cann_contribution1['Product'], rotation=45, ha='right')
            ax1.get_yaxis().set_major_formatter(mtick.PercentFormatter())

            a2=ax2.bar(final_cann_contribution2['Product'], final_cann_contribution2['Cann_Contribution'], color='dodgerblue')
            for i,j in final_cann_contribution2['Cann_Contribution'].reset_index()['Cann_Contribution'].items():
                ax2.annotate(str(j) + "%", xy=(i, j/2),ha='center')
            ax2.set_title('New Product 2 Cannibalization loss Top 10 Affecting Products Contribution %')
            ax2.set_ylabel('Cannibalization loss Contribution %')
            ax2.set_xlabel('Product')
            ax2.set_xticklabels(final_cann_contribution2['Product'], rotation=45, ha='right')
            ax2.get_yaxis().set_major_formatter(mtick.PercentFormatter())

            plt.show();                      
        
    else:
        print("Product is not present in that Region/Channel for selected period")